In [1]:
import pandas as pd
import requests
import re # for regex in html
from bs4 import BeautifulSoup

%load_ext line_profiler

pd.options.display.max_columns=500
pd.options.display.max_rows=1500
pd.options.display.max_colwidth=1000

data_ = pd.read_json('https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2015.json')

In [12]:
data = data_[data_.armed=="unarmed"].copy()
data = data.reset_index()

sources = pd.DataFrame(data["sources"].apply(lambda x:[y["url"] for y in x]).tolist())

for col in sources.columns:
    data["source_"+str(col)]=sources[col]


failed_urls = []  

def scraper(URL):
    if URL==None:
        pass
        return
    else:
        try:
            r = requests.get(URL)
        except Exception:
            failed_urls.append(URL)
            pass
            return
            
        soup = BeautifulSoup(r.content, "lxml") # Warning message advises specifying BeautifulSoup(YOUR_MARKUP, "lxml")
    
        return soup.prettify()
    
df2015_with_info = data.copy() # for reference, df2015_with_info.shape is (94, 30)


for source_col in ['source_0','source_1','source_2','source_3','source_4']:
    df2015_with_info[source_col] = df2015_with_info[source_col].apply(scraper)

In [16]:
data_with = df2015_with_info

In [15]:
# only one that failed in 2015 data.

data[data['source_2']=='http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html']

,index,age,armed,blurb,city,date,description,editor_note,flee,gender,id,is_body_camera,is_geocoding_exact,is_officer_charged,lat,lon,mental,name,photos,race,sources,state,threat_level_display,videos,weapon,source_0,source_1,source_2,source_3,source_4
20,159,26.0,unarmed,"an unarmed 26-year-old white man, was shot on March 4, 2015, in a house in Deltona, Fla.",Deltona,2015-03-04,A Volusia County sheriff’s deputy was part of a squad serving a search warrant for narcotics at the home when he shot Cruice in the face.,,NOT,M,199,False,True,False,28.887012,-81.254315,False,Derek Cruice,[],W,"[{u'url': u'http://www.washingtonpost.com/news/the-watch/wp/2015/03/06/another-day-another-drug-raid-fatality/', u'source_name': u'Washington Post', u'type': u'NEWS', u'title': u'Another day, another drug raid fatality'}, {u'url': u'http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2015/3/4/man_shot_by_volusia_.html', u'source_name': u'mynews13.com', u'type': u'NEWS', u'title': u'Sheriff's office: Man shot in face by Volusia deputy was unarmed'}, {u'url': u'http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html', u'source_name': u'The Huffington Post', u'type': u'NEWS', u'title': u'Florida Deputy Shoots Unarmed Man While Serving Warrant'}, {u'url': u'http://www.orlandosentinel.com/news/breaking-news/os-deputy-involved-shooting-deltona-20150304-story.html', u'source_name': u'OrlandoSentinel.com', u'type': u'NEWS', u'title': u'FDLE launches investigation after deputy fatally shoots man in Deltona'}]",FL,other,[],None,http://www.washingtonpost.com/news/the-watch/wp/2015/03/06/another-day-another-drug-raid-fatality/,http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2015/3/4/man_shot_by_volusia_.html,http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html,http://www.orlandosentinel.com/news/breaking-news/os-deputy-involved-shooting-deltona-20150304-story.html,None


In [25]:
data_with.loc[20,'source_2']

In [ ]:
missing = ("""<html lang="en" class=" js no-touch history websockets csstransforms3d csstransitions localstorage webworkers filereader"><head><script type="text/javascript" src="https://bam.nr-data.net/1/a2134792db?a=36240457&v=1039.bef6007&to=dFgPTEBWVA1cQEtXVkRcFVdCFl0PTUANVkAYRAlXRQ%3D%3D&rst=65284&ref=http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html&qt=1&ap=263&be=1649&fe=65152&dc=10930&perf=%7B%22timing%22:%7B%22of%22:1498785544142,%22n%22:0,%22f%22:220,%22dn%22:220,%22dne%22:220,%22c%22:220,%22ce%22:220,%22rq%22:223,%22rp%22:1628,%22rpe%22:1635,%22dl%22:1632,%22di%22:10929,%22ds%22:10930,%22de%22:10937,%22dc%22:65150,%22l%22:65151,%22le%22:65249%7D,%22navigation%22:%7B%7D%7D&at=TBUUGghCGgRdWxBaXFkVWxpHShpNG0QBQUdeVABUEAMaIktbCVYRSko%3D&jsonp=NREUM.setToken"></script><script src="http://s.skimresources.com/js/38395X1559799.skimlinks.js" async=""></script><script src="https://js-agent.newrelic.com/nr-1039.min.js"></script><script type="text/javascript" async="" src="http://jsonp.moatads.com/ocr/AOL2/level3/93432943?t=201753024"></script><script async="" src="http://b.scorecardresearch.com/beacon.js"></script><script src="http://static.parsely.com/code/ptrack-v0.9.2-engaged-time-slots-video.js"></script><script async="" src="http://b.scorecardresearch.com/beacon.js"></script><script async="" src="//widget.perfectmarket.com/aol-huffingtonpost/load.js"></script><script charset="UTF-8" type="text/javascript" src="http://cdn.taboola.com/libtrc/impl.260-187-RELEASE.js"></script><script src="http://srv-2017-06-30-01.config.parsely.com/config/huffingtonpost.com"></script>

<script src="http://cdn.pubexchange.com/modules/partner/huffpost_crime" async=""></script><script id="nativeads-plugin" type="text/javascript" async="" src="http://plugin.mediavoice.com/plugin.js"></script><script type="text/javascript" async="" src="//assets.pinterest.com/js/pinit_main.js?0.3816898932071533"></script><script async="" src="http://cdn.taboola.com/libtrc/aol-huffingtonpost/loader.js"></script><script type="text/javascript" async="" src="http://ak.sail-horizon.com/horizon/v1.js"></script><script id="twitter-wjs" async="" src="https://platform.twitter.com/widgets.js"></script><script src="http://connect.facebook.net/signals/config/1100125983380803?v=2.7.17" async=""></script><script src="http://connect.facebook.net/signals/config/1621685564716533?v=2.7.17" async=""></script><script src="http://connect.facebook.net/signals/config/1043018625788392?v=2.7.17" async=""></script><script async="" src="//connect.facebook.net/en_US/fbevents.js"></script><script async="" src="//dc8xl0ndzn2cb.cloudfront.net/sp.js"></script><script id="facebook-jssdk" async="" src="https://connect.facebook.net/en_US/sdk.js#xfbml=1&version=v2.4"></script><script type="text/javascript" async="" src="http://www.zergnet.com/zerg.js?id=49669"></script><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"a2134792db","applicationID":"36240457","transactionName":"dFgPTEBWVA1cQEtXVkRcFVdCFl0PTUANVkAYRAlXRQ==","queueTime":1,"applicationTime":263,"agent":"","atts":"TBUUGghCGgRdWxBaXFkVWxpHShpNG0QBQUdeVABUEAMaIktbCVYRSko="}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn-";a(p,function(e,n){s[n]=o(d+n,!0,"api")}),s.addPageAction=o(d+"addPageAction",!0),s.setCurrentRouteName=o(d+"routeName",!0),n.exports=newrelic,s.interaction=function(){return(new r).get()};var m=r.prototype={createTracer:function(e,n){var t={},r=this,o="function"==typeof n;return i(l+"tracer",[c.now(),e,t],r),function(){if(f.emit((o?"":"no-")+"fn-start",[c.now(),r,o],t),o)try{return n.apply(this,arguments)}finally{f.emit("fn-end",[c.now()],t)}}}};a("setName,setAttribute,save,ignore,onEnd,getContext,end,get".split(","),function(e,n){m[n]=o(l+n)}),newrelic.noticeError=function(e){"string"==typeof e&&(e=new Error(e)),i("err",[e,c.now()])}},{}],2:[function(e,n,t){function r(e,n){var t=[],r="",i=0;for(r in e)o.call(e,r)&&(t[i]=n(r,e[r]),i+=1);return t}var o=Object.prototype.hasOwnProperty;n.exports=r},{}],3:[function(e,n,t){function r(e,n,t){n||(n=0),"undefined"==typeof t&&(t=e?e.length:0);for(var r=-1,o=t-n||0,i=Array(o<0?0:o);++r<o;)i[r]=e[n+r];return i}n.exports=r},{}],4:[function(e,n,t){n.exports={exists:"undefined"!=typeof window.performance&&window.performance.timing&&"undefined"!=typeof window.performance.timing.navigationStart}},{}],ee:[function(e,n,t){function r(){}function o(e){function n(e){return e&&e instanceof r?e:e?f(e,u,i):i()}function t(t,r,o,i){if(!d.aborted||i){e&&e(t,r,o);for(var a=n(o),u=m(t),f=u.length,c=0;c<f;c++)u[c].apply(a,r);var p=s[y[t]];return p&&p.push([b,t,r,a]),a}}function l(e,n){v[e]=m(e).concat(n)}function m(e){return v[e]||[]}function w(e){return p[e]=p[e]||o(t)}function g(e,n){c(e,function(e,t){n=n||"feature",y[t]=n,n in s||(s[n]=[])})}var v={},y={},b={on:l,emit:t,get:w,listeners:m,context:n,buffer:g,abort:a,aborted:!1};return b}function i(){return new r}function a(){(s.api||s.feature)&&(d.aborted=!0,s=d.backlog={})}var u="nr@context",f=e("gos"),c=e(2),s={},p={},d=n.exports=o();d.backlog=s},{}],gos:[function(e,n,t){function r(e,n,t){if(o.call(e,n))return e[n];var r=t();if(Object.defineProperty&&Object.keys)try{return Object.defineProperty(e,n,{value:r,writable:!0,enumerable:!1}),r}catch(i){}return e[n]=r,r}var o=Object.prototype.hasOwnProperty;n.exports=r},{}],handle:[function(e,n,t){function r(e,n,t,r){o.buffer([e],r),o.emit(e,n,t)}var o=e("ee").get("handle");n.exports=r,r.ee=o},{}],id:[function(e,n,t){function r(e){var n=typeof e;return!e||"object"!==n&&"function"!==n?-1:e===window?0:a(e,i,function(){return o++})}var o=1,i="nr@id",a=e("gos");n.exports=r},{}],loader:[function(e,n,t){function r(){if(!x++){var e=h.info=NREUM.info,n=d.getElementsByTagName("script")[0];if(setTimeout(s.abort,3e4),!(e&&e.licenseKey&&e.applicationID&&n))return s.abort();c(y,function(n,t){e[n]||(e[n]=t)}),f("mark",["onload",a()+h.offset],null,"api");var t=d.createElement("script");t.src="https://"+e.agent,n.parentNode.insertBefore(t,n)}}function o(){"complete"===d.readyState&&i()}function i(){f("mark",["domContent",a()+h.offset],null,"api")}function a(){return E.exists&&performance.now?Math.round(performance.now()):(u=Math.max((new Date).getTime(),u))-h.offset}var u=(new Date).getTime(),f=e("handle"),c=e(2),s=e("ee"),p=window,d=p.document,l="addEventListener",m="attachEvent",w=p.XMLHttpRequest,g=w&&w.prototype;NREUM.o={ST:setTimeout,SI:p.setImmediate,CT:clearTimeout,XHR:w,REQ:p.Request,EV:p.Event,PR:p.Promise,MO:p.MutationObserver};var v=""+location,y={beacon:"bam.nr-data.net",errorBeacon:"bam.nr-data.net",agent:"js-agent.newrelic.com/nr-1039.min.js"},b=w&&g&&g[l]&&!/CriOS/.test(navigator.userAgent),h=n.exports={offset:u,now:a,origin:v,features:{},xhrWrappable:b};e(1),d[l]?(d[l]("DOMContentLoaded",i,!1),p[l]("load",r,!1)):(d[m]("onreadystatechange",o),p[m]("onload",r)),f("mark",["firstbyte",u],null,"api");var x=0,E=e(4)},{}]},{},["loader"]);</script>
<meta content="97Mj-wW7u8U1NOnkW1rfH-I1kgO0-a9YsgoesOk41R8" name="google-site-verification">
<meta content="18468761129" property="fb:pages">

<title>Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant | HuffPost</title>
<meta charset="utf-8">
<meta content="width=1000" name="viewport">
<meta content="A Volusia County Sheriff's deputy shot an unarmed man in the face while serving a drug warrant on Wednesday.

Deputies say Derek Cruice, 26, resisted a..." name="description">
<meta content="Derek Cruice,Derek Cruice dead,todd raible,Volusia County Sheriff,todd raible volusia county sheriff,cop shoots pot dealer in face,deputy shoots man in face serving warrant,video,police" name="keywords">
<meta content="Florida Deputy Shoots Unarmed Man While Serving Warrant" property="og:title">
<meta content="article" property="og:type">
<meta content="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" property="og:image">
<meta content="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" property="og:image:url">
<meta content="image/jpeg" property="og:image:type">
<meta content="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html" property="og:url">
<meta content="HuffPost" property="og:site_name">
<meta content="en_US" property="og:locale">
<meta content="A Volusia County Sheriff's deputy shot an unarmed man in the face while serving a drug warrant on Wednesday.

Deputies say Derek Cruice, 26, resisted a..." property="og:description">
<meta content="Derek Cruice,Derek Cruice dead,todd raible,Volusia County Sheriff,todd raible volusia county sheriff,cop shoots pot dealer in face,deputy shoots man in face serving warrant,video,police" property="article:tag">
<meta content="Crime" property="article:section">
<meta content="summary_large_image" name="twitter:card">
<meta content="@HuffPostCrime" name="twitter:site">
<meta content="Florida Deputy Shoots Unarmed Man While Serving Warrant" property="twitter:title">
<meta content="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" property="twitter:image">
<meta content="A Volusia County Sheriff's deputy shot an unarmed man in the face while serving a drug warrant on Wednesday.

Deputies say Derek Cruice, 26, resisted a..." property="twitter:description">
<meta content="AndyBCampbell" property="twitter:creator">
<meta content="2015-03-05 02:57:45 -0500" property="article:published_time">
<meta content="2015-03-05 02:59:01 -0500" property="article:modified_time">
<meta content="huffpostnews:///?editionID=us&entryID=6805902" property="al:ios:url">
<meta content="306621789" property="al:ios:app_store_id">
<meta content="HuffPost" property="al:ios:app_name">
<link href="ios-app://306621789/huffpostnews/?editionID=us&entryID=6805902" rel="alternate">

<meta content="46744042133" property="fb:app_id">
<meta content="text/html" http-equiv="Content-Type">
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible">
<link href="/static/search.xml?title=HuffPost" rel="search" type="application/opensearchdescription+xml">
<link rel="shortcut icon" type="image/x-icon" href="http://s.m.huffpost.com/assets/favicon-2020e123c064086aad150fe909c53771f862d7c76cd62f6146e81d533845fd7d.ico">
<link href="http://m.huffpost.com/us/entry/6805902" media="handheld" rel="alternate">
<link href="http://m.huffpost.com/us/entry/6805902" media="only screen and (max-width: 640px)" rel="alternate">
<link href="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html" rel="canonical">
<style>
html{font-family:sans-serif;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%}body{margin:0}div,dl,dt,dd,ul,ol,li,h1,h2,h3,h4,h5,h6,pre,form,p,blockquote,th,td{margin:0;padding:0}code,kbd,pre,samp{font-family:monospace, serif;font-size:1em}pre{white-space:pre-wrap}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sup{top:-0.5em}sub{bottom:-0.25em}figure{margin:0}fieldset{border:1px solid silver;margin:0 2px;padding:0.35em 0.625em 0.75em}legend{border:0;padding:0}button,input,select,textarea{font-family:inherit;font-size:100%;margin:0}button,input{line-height:normal}button,select{text-transform:none}button,html input[type="button"],input[type="reset"],input[type="submit"]{-webkit-appearance:button;cursor:pointer}button[disabled],html input[disabled]{cursor:default}input[type="checkbox"],input[type="radio"]{-webkit-box-sizing:border-box;box-sizing:border-box;padding:0}input[type="search"]{-webkit-appearance:textfield;-webkit-box-sizing:content-box;box-sizing:content-box}input[type="search"]::-webkit-search-cancel-button,input[type="search"]::-webkit-search-decoration{-webkit-appearance:none}button::-moz-focus-inner,input::-moz-focus-inner{border:0;padding:0}textarea{overflow:auto;vertical-align:top}table{border-collapse:collapse;border-spacing:0}img{border-style:none}svg:not(:root){overflow:hidden}@font-face{font-family:"huffington-icons";font-style:normal;font-weight:normal;src:url(//s.m.huffpost.com/assets/huffington-icons-769c71a5af49aea4984447148fd7b63068992240bfb92a81d4e1b9a996731f85.eot?#iefix) format("embedded-opentype"),url(//s.m.huffpost.com/assets/huffington-icons-80fe0099d468a2b3c698925f5a97f3203dd1f0bad89bf305ec6b12cb4d590c68.woff) format("woff"),url(//s.m.huffpost.com/assets/huffington-icons-20d66e867f9f71de8368b8379941a35e5fbef404feaaf0f7654691209d760bfd.svg#huffington-icons) format("svg")}body{width:100%;-webkit-box-sizing:border-box;box-sizing:border-box}body.mapi{background:#ffffff}body #recaptcha-container{display:none}body.recaptcha-open #recaptcha-container{position:fixed;top:0;bottom:0;left:0;right:0;z-index:9000000;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;background-color:rgba(111,111,111,0.6)}img{max-width:100%;height:auto}center img+em{display:block}html{-webkit-box-sizing:border-box;box-sizing:border-box}*,*:before,*:after{-webkit-box-sizing:inherit;box-sizing:inherit}.fyre{-webkit-box-sizing:initial;box-sizing:initial}.clearfix:after{content:"";display:table;clear:both}.row{width:100%;position:relative}.row:after{content:"";display:table;clear:both}.col{float:left}.col:nth-child(n+2){padding-left:5px}.col-full{padding-left:0;float:left;width:100%}.col-half{padding-left:0;float:left;width:50%}.col-third{padding-left:0;float:left;width:33.33%}.col-two-thirds{padding-left:0;float:left;width:66.66%}.seperator{border-bottom:1px solid #2e7061;padding-left:5px;padding-right:5px}.hidden{display:none !important;visibility:hidden !important}.flex-video{position:relative;padding-bottom:56.55%;height:0;overflow:hidden}.flex-video iframe,.flex-video object,.flex-video embed,.flex-video video{position:absolute;top:0;left:0;width:100%;height:100%}.flex-video.wapo{padding-bottom:50%}.flex-video.vimeo{padding-top:0}.flex-video.msnbc{padding-bottom:91%}.no-scroll{overflow:hidden}.sponsor-container{border:1px solid #dbdbdb;border-left:none;border-right:none}html{font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif}h1,h2,h3,h4,h5{font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;line-height:1.4em;font-weight:normal}ul,ol{padding-top:5px;padding-bottom:5px}blockquote{padding-top:15px;padding-bottom:15px;padding-left:15px;padding-right:15px;margin-top:15px;margin-bottom:15px;color:#444444;border:1px solid #dbdbdb;line-height:1.5em}p{font-size:1.0625rem}a{color:#2e7061}@-webkit-keyframes error-shake{0%{-webkit-transform:translateX(0);transform:translateX(0)}10%{-webkit-transform:translateX(-10px);transform:translateX(-10px)}20%{-webkit-transform:translateX(10px);transform:translateX(10px)}30%{-webkit-transform:translateX(-8px);transform:translateX(-8px)}40%{-webkit-transform:translateX(8px);transform:translateX(8px)}50%{-webkit-transform:translateX(-6px);transform:translateX(-6px)}60%{-webkit-transform:translateX(6px);transform:translateX(6px)}70%{-webkit-transform:translateX(-4px);transform:translateX(-4px)}80%{-webkit-transform:translateX(4px);transform:translateX(4px)}90%{-webkit-transform:translateX(-2px);transform:translateX(-2px)}100%{-webkit-transform:translateX(0);transform:translateX(0)}}@keyframes error-shake{0%{-webkit-transform:translateX(0);transform:translateX(0)}10%{-webkit-transform:translateX(-10px);transform:translateX(-10px)}20%{-webkit-transform:translateX(10px);transform:translateX(10px)}30%{-webkit-transform:translateX(-8px);transform:translateX(-8px)}40%{-webkit-transform:translateX(8px);transform:translateX(8px)}50%{-webkit-transform:translateX(-6px);transform:translateX(-6px)}60%{-webkit-transform:translateX(6px);transform:translateX(6px)}70%{-webkit-transform:translateX(-4px);transform:translateX(-4px)}80%{-webkit-transform:translateX(4px);transform:translateX(4px)}90%{-webkit-transform:translateX(-2px);transform:translateX(-2px)}100%{-webkit-transform:translateX(0);transform:translateX(0)}}@-webkit-keyframes scale-in{0%{opacity:0;-webkit-transform:scale(0.25);transform:scale(0.25)}100%{opacity:1;-webkit-transform:scale(1);transform:scale(1)}}@keyframes scale-in{0%{opacity:0;-webkit-transform:scale(0.25);transform:scale(0.25)}100%{opacity:1;-webkit-transform:scale(1);transform:scale(1)}}@-webkit-keyframes scale-out{0%{opacity:1;-webkit-transform:scale(1);transform:scale(1)}100%{opacity:0;-webkit-transform:scale(0);transform:scale(0)}}@keyframes scale-out{0%{opacity:1;-webkit-transform:scale(1);transform:scale(1)}100%{opacity:0;-webkit-transform:scale(0);transform:scale(0)}}@-webkit-keyframes slide-up-in{0%{opacity:0;-webkit-transform:translateY(20px);transform:translateY(20px)}100%{opacity:1;-webkit-transform:translateX(0);transform:translateX(0)}}@keyframes slide-up-in{0%{opacity:0;-webkit-transform:translateY(20px);transform:translateY(20px)}100%{opacity:1;-webkit-transform:translateX(0);transform:translateX(0)}}@-webkit-keyframes bounce{0%{-webkit-transform:scale(1.05);transform:scale(1.05)}10%{-webkit-transform:scale(1.1);transform:scale(1.1)}20%{-webkit-transform:scale(1.15);transform:scale(1.15)}30%{-webkit-transform:scale(1.2);transform:scale(1.2)}40%{-webkit-transform:scale(1.25);transform:scale(1.25)}50%{-webkit-transform:scale(1.3);transform:scale(1.3)}60%{-webkit-transform:scale(1.375);transform:scale(1.375)}70%{-webkit-transform:scale(1.475);transform:scale(1.475)}80%{-webkit-transform:scale(1.4);transform:scale(1.4)}90%{-webkit-transform:scale(1.35);transform:scale(1.35)}100%{-webkit-transform:scale(1.3);transform:scale(1.3)}}@keyframes bounce{0%{-webkit-transform:scale(1.05);transform:scale(1.05)}10%{-webkit-transform:scale(1.1);transform:scale(1.1)}20%{-webkit-transform:scale(1.15);transform:scale(1.15)}30%{-webkit-transform:scale(1.2);transform:scale(1.2)}40%{-webkit-transform:scale(1.25);transform:scale(1.25)}50%{-webkit-transform:scale(1.3);transform:scale(1.3)}60%{-webkit-transform:scale(1.375);transform:scale(1.375)}70%{-webkit-transform:scale(1.475);transform:scale(1.475)}80%{-webkit-transform:scale(1.4);transform:scale(1.4)}90%{-webkit-transform:scale(1.35);transform:scale(1.35)}100%{-webkit-transform:scale(1.3);transform:scale(1.3)}}@-webkit-keyframes bounce-centered{0%{-webkit-transform:scale(1.05) translate(-50%, -50%);transform:scale(1.05) translate(-50%, -50%)}10%{-webkit-transform:scale(1.1) translate(-50%, -50%);transform:scale(1.1) translate(-50%, -50%)}20%{-webkit-transform:scale(1.15) translate(-50%, -50%);transform:scale(1.15) translate(-50%, -50%)}30%{-webkit-transform:scale(1.2) translate(-50%, -50%);transform:scale(1.2) translate(-50%, -50%)}40%{-webkit-transform:scale(1.25) translate(-50%, -50%);transform:scale(1.25) translate(-50%, -50%)}50%{-webkit-transform:scale(1.3) translate(-50%, -50%);transform:scale(1.3) translate(-50%, -50%)}60%{-webkit-transform:scale(1.375) translate(-50%, -50%);transform:scale(1.375) translate(-50%, -50%)}70%{-webkit-transform:scale(1.475) translate(-50%, -50%);transform:scale(1.475) translate(-50%, -50%)}80%{-webkit-transform:scale(1.4) translate(-50%, -50%);transform:scale(1.4) translate(-50%, -50%)}90%{-webkit-transform:scale(1.35) translate(-50%, -50%);transform:scale(1.35) translate(-50%, -50%)}100%{-webkit-transform:scale(1.3) translate(-50%, -50%);transform:scale(1.3) translate(-50%, -50%)}}@keyframes bounce-centered{0%{-webkit-transform:scale(1.05) translate(-50%, -50%);transform:scale(1.05) translate(-50%, -50%)}10%{-webkit-transform:scale(1.1) translate(-50%, -50%);transform:scale(1.1) translate(-50%, -50%)}20%{-webkit-transform:scale(1.15) translate(-50%, -50%);transform:scale(1.15) translate(-50%, -50%)}30%{-webkit-transform:scale(1.2) translate(-50%, -50%);transform:scale(1.2) translate(-50%, -50%)}40%{-webkit-transform:scale(1.25) translate(-50%, -50%);transform:scale(1.25) translate(-50%, -50%)}50%{-webkit-transform:scale(1.3) translate(-50%, -50%);transform:scale(1.3) translate(-50%, -50%)}60%{-webkit-transform:scale(1.375) translate(-50%, -50%);transform:scale(1.375) translate(-50%, -50%)}70%{-webkit-transform:scale(1.475) translate(-50%, -50%);transform:scale(1.475) translate(-50%, -50%)}80%{-webkit-transform:scale(1.4) translate(-50%, -50%);transform:scale(1.4) translate(-50%, -50%)}90%{-webkit-transform:scale(1.35) translate(-50%, -50%);transform:scale(1.35) translate(-50%, -50%)}100%{-webkit-transform:scale(1.3) translate(-50%, -50%);transform:scale(1.3) translate(-50%, -50%)}}@-webkit-keyframes bounce-center-lr{0%{-webkit-transform:scale(1.05) translateY(-50%);transform:scale(1.05) translateY(-50%)}10%{-webkit-transform:scale(1.1) translateY(-50%);transform:scale(1.1) translateY(-50%)}20%{-webkit-transform:scale(1.15) translateY(-50%);transform:scale(1.15) translateY(-50%)}30%{-webkit-transform:scale(1.2) translateY(-50%);transform:scale(1.2) translateY(-50%)}40%{-webkit-transform:scale(1.25) translateY(-50%);transform:scale(1.25) translateY(-50%)}50%{-webkit-transform:scale(1.3) translateY(-50%);transform:scale(1.3) translateY(-50%)}60%{-webkit-transform:scale(1.375) translateY(-50%);transform:scale(1.375) translateY(-50%)}70%{-webkit-transform:scale(1.475) translateY(-50%);transform:scale(1.475) translateY(-50%)}80%{-webkit-transform:scale(1.4) translateY(-50%);transform:scale(1.4) translateY(-50%)}90%{-webkit-transform:scale(1.35) translateY(-50%);transform:scale(1.35) translateY(-50%)}100%{-webkit-transform:scale(1.3) translateY(-50%);transform:scale(1.3) translateY(-50%)}}@keyframes bounce-center-lr{0%{-webkit-transform:scale(1.05) translateY(-50%);transform:scale(1.05) translateY(-50%)}10%{-webkit-transform:scale(1.1) translateY(-50%);transform:scale(1.1) translateY(-50%)}20%{-webkit-transform:scale(1.15) translateY(-50%);transform:scale(1.15) translateY(-50%)}30%{-webkit-transform:scale(1.2) translateY(-50%);transform:scale(1.2) translateY(-50%)}40%{-webkit-transform:scale(1.25) translateY(-50%);transform:scale(1.25) translateY(-50%)}50%{-webkit-transform:scale(1.3) translateY(-50%);transform:scale(1.3) translateY(-50%)}60%{-webkit-transform:scale(1.375) translateY(-50%);transform:scale(1.375) translateY(-50%)}70%{-webkit-transform:scale(1.475) translateY(-50%);transform:scale(1.475) translateY(-50%)}80%{-webkit-transform:scale(1.4) translateY(-50%);transform:scale(1.4) translateY(-50%)}90%{-webkit-transform:scale(1.35) translateY(-50%);transform:scale(1.35) translateY(-50%)}100%{-webkit-transform:scale(1.3) translateY(-50%);transform:scale(1.3) translateY(-50%)}}@-webkit-keyframes bounce-center-tb{0%{-webkit-transform:scale(1.05) translateX(-50%);transform:scale(1.05) translateX(-50%)}10%{-webkit-transform:scale(1.1) translateX(-50%);transform:scale(1.1) translateX(-50%)}20%{-webkit-transform:scale(1.15) translateX(-50%);transform:scale(1.15) translateX(-50%)}30%{-webkit-transform:scale(1.2) translateX(-50%);transform:scale(1.2) translateX(-50%)}40%{-webkit-transform:scale(1.25) translateX(-50%);transform:scale(1.25) translateX(-50%)}50%{-webkit-transform:scale(1.3) translateX(-50%);transform:scale(1.3) translateX(-50%)}60%{-webkit-transform:scale(1.375) translateX(-50%);transform:scale(1.375) translateX(-50%)}70%{-webkit-transform:scale(1.475) translateX(-50%);transform:scale(1.475) translateX(-50%)}80%{-webkit-transform:scale(1.4) translateX(-50%);transform:scale(1.4) translateX(-50%)}90%{-webkit-transform:scale(1.35) translateX(-50%);transform:scale(1.35) translateX(-50%)}100%{-webkit-transform:scale(1.3) translateX(-50%);transform:scale(1.3) translateX(-50%)}}@keyframes bounce-center-tb{0%{-webkit-transform:scale(1.05) translateX(-50%);transform:scale(1.05) translateX(-50%)}10%{-webkit-transform:scale(1.1) translateX(-50%);transform:scale(1.1) translateX(-50%)}20%{-webkit-transform:scale(1.15) translateX(-50%);transform:scale(1.15) translateX(-50%)}30%{-webkit-transform:scale(1.2) translateX(-50%);transform:scale(1.2) translateX(-50%)}40%{-webkit-transform:scale(1.25) translateX(-50%);transform:scale(1.25) translateX(-50%)}50%{-webkit-transform:scale(1.3) translateX(-50%);transform:scale(1.3) translateX(-50%)}60%{-webkit-transform:scale(1.375) translateX(-50%);transform:scale(1.375) translateX(-50%)}70%{-webkit-transform:scale(1.475) translateX(-50%);transform:scale(1.475) translateX(-50%)}80%{-webkit-transform:scale(1.4) translateX(-50%);transform:scale(1.4) translateX(-50%)}90%{-webkit-transform:scale(1.35) translateX(-50%);transform:scale(1.35) translateX(-50%)}100%{-webkit-transform:scale(1.3) translateX(-50%);transform:scale(1.3) translateX(-50%)}}[role=main]{width:100%;position:relative}[role=main]:after{content:"";display:table;clear:both}body{max-width:none;min-width:767px;width:100%;margin-left:auto;margin-right:auto;position:relative}body.locked{overflow:hidden}.main{position:relative}body[data-locale=en_GB] .main{max-width:600px;margin:0 auto}.blogs{border-right:1px solid #dbdbdb;padding-right:12px}.wide .below-entry__content{width:100%;max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto;margin:0 auto}.b-page .right-rail{display:none}@media (min-width: 1024px){.b-page .right-rail{display:block;-webkit-box-flex:0;-ms-flex:0 0 300px;flex:0 0 300px;margin-top:5px;padding-left:0}}.b-page .right-rail .below_right_rail_flex{margin-top:30px;margin-bottom:30px}.static .right-rail{margin-top:5px}.right-rail{max-width:303px}:target:before{content:"";display:block;height:78px;margin:-78px 0 0;pointer-events:none}.header{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;background-color:#000000;width:100%;margin-bottom:40px;position:static;top:0;height:78px;-webkit-transition:height 0.3s;transition:height 0.3s;z-index:5000005}.header__content{max-width:1440px;min-width:768px;width:100%;margin-left:auto;margin-right:auto;width:100%;height:78px;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-ms-flex-direction:row;flex-direction:row;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;-webkit-box-align:stretch;-ms-flex-align:stretch;align-items:stretch}.header__content-left,.header__content-right{-webkit-box-flex:0;-ms-flex:0 0 343px;flex:0 0 343px;display:-webkit-box;display:-ms-flexbox;display:flex}.header__content-left{padding-left:20px;-webkit-box-pack:start;-ms-flex-pack:start;justify-content:flex-start;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-ordinal-group:1;-ms-flex-order:0;order:0}.header__content-right{padding-right:20px;-webkit-box-pack:end;-ms-flex-pack:end;justify-content:flex-end;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2}.header__hamburger-link{width:30px;height:20px;cursor:pointer;position:relative}.header__hamburger-link:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:20px;color:#666;position:absolute;display:block;width:30px}.header__hamburger-link.open:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:24px;color:#666}.header__hamburger-link.open:hover:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:24px;color:#0DBE98}.header__hamburger-link:hover:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:20px;color:#0DBE98;position:absolute;display:block;width:30px}.header__editions{color:#ffffff;-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1;position:relative;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;-ms-flex-item-align:center;align-self:center;font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:0.6875rem;cursor:pointer;padding:15px;color:#888;-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none}.header__editions::-moz-selection{background-color:transparent}.header__editions::selection{background-color:transparent}.header__editions:hover,.header__editions.open{color:#0DBE98}.header__editions__label{-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1;font-size:0.625rem;letter-spacing:1.3px}.header__editions__edition{-webkit-box-ordinal-group:1;-ms-flex-order:0;order:0;font-size:1.25rem;font-style:italic;font-weight:normal;text-align:center;letter-spacing:1px}.header__logo{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-flex:1;-ms-flex:1 1 100%;flex:1 1 100%;-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1;-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none}.header__logo::-moz-selection{background-color:transparent}.header__logo::selection{background-color:transparent}.header__social-list{-webkit-box-ordinal-group:1;-ms-flex-order:0;order:0;display:-webkit-box;display:-ms-flexbox;display:flex;height:100%;-webkit-box-align:center;-ms-flex-align:center;align-items:center}.header__social-list__item{font-size:1.25rem;color:#ffffff;background-color:#242424;text-decoration:none;width:38px;height:38px;border-radius:50%;margin-right:12px;-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none}.header__social-list__item::-moz-selection{background-color:transparent}.header__social-list__item::selection{background-color:transparent}.header__social-list__item--newsletters:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1.125rem;color:#ffffff;position:relative;top:9px;left:10px}.header__social-list__item--newsletters:hover{background-color:#0dbe98}.header__social-list__item--facebook:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1.125rem;color:#ffffff;position:relative;top:8px;left:10px}.header__social-list__item--facebook:hover{background-color:#3b5999}.header__social-list__item--twitter:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1.125rem;color:#ffffff;position:relative;top:8px;left:11px}.header__social-list__item--twitter:hover{background-color:#55acef}.header__social-list__item--snapchat{background-image:url(//s.m.huffpost.com/assets/icon-snapchat-d9b47ee21b08668c392280c07aad41ba94f55ec741cb085253e419c2377d77f8.png);background-repeat:no-repeat;background-position:center center;background-size:26px}.header__social-list__item--snapchat:hover{background-color:#fffb00}.header__social-list__item--instagram:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1.125rem;color:#ffffff;position:relative;top:9px;left:10px}.header__social-list__item--instagram:hover{background:radial-gradient(circle farthest-corner at 32% 106%, #ffe17d 0%, #ffcd69 10%, #fa9137 28%, #eb4141 42%, transparent 82%),linear-gradient(135deg, #234bd7 12%, #c33cbe 58%)}@media (max-width: 981px){.header__social-list--sticky{display:none}}@media (max-width: 980px){.header .header__social-list{display:none}.header__content-left,.header__content-right{-webkit-box-flex:0;-ms-flex:0 0 150px;flex:0 0 150px}}.header--sticky{height:48px;background-color:rgba(0,0,0,0.9)}.header--sticky .logo__img{height:22px}.header--sticky .header__hamburger-link{width:24px;height:16px}.header--sticky .header__hamburger-link:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:16px;color:#666}.header--sticky .header__hamburger-link.open:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:16px;color:#666}.header--sticky .header__hamburger-link.open:hover:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:16px;color:#0DBE98}.header--sticky .header__hamburger-link:hover:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:16px;color:#0DBE98}.header--sticky .header__social-list__item{height:30px;width:30px;margin-right:9px}.header--sticky .header__social-list__item--newsletters:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1rem;color:#ffffff;position:relative;top:5px;left:7px}.header--sticky .header__social-list__item--facebook:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1rem;color:#ffffff;position:relative;top:5px;left:7px}.header--sticky .header__social-list__item--twitter:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1rem;color:#ffffff;position:relative;top:5px;left:7px}.header--sticky .header__social-list__item--snapchat{background-size:19px}.header--sticky .header__social-list__item--instagram:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:1rem;color:#ffffff;position:relative;top:5px;left:7px}.header--sticky .header__editions{display:none}.header--sticky .header__editions .editions__current-edition{padding:9px 15px}.header--transition{-webkit-transition-timing-function:ease;transition-timing-function:ease;-webkit-transition:height 0.2s, width 0.2s;transition:height 0.2s, width 0.2s}.logo{text-decoration:none;width:300px;display:block}.logo__img{height:28px;width:100%;display:block}.a-page:not(.section-video) .header{position:fixed;top:0;margin-bottom:0}.a-page:not(.section-video) .header__placeholder{background-color:#000000;height:78px;margin-bottom:40px}.a-page.front-page .header__placeholder{margin-bottom:0}.dropdown--closed{max-height:0;-webkit-transition:none;transition:none}.dropdown--closed .dropdown__container{display:none}.a-page .dropdown{top:78px}.a-page .dropdown.sticky{top:48px}.b-page .dropdown{top:78px;position:absolute}.b-page .dropdown.sticky{position:fixed;top:65px}.hamburger.dropdown{position:fixed;width:100%;top:78px}.hamburger__container{background-color:#000000;border-style:solid;border-color:#0DBE98;border-width:0 0 3px 0;overflow-y:scroll;background-image:url(//s.m.huffpost.com/assets/nav-bg-28c233db98f59d940f56847243216ef7150f6bb94f4f629dbc22eb551e9c5954.svg);background-repeat:no-repeat;background-position:center top;background-size:1452px 550px;max-height:inherit}.hamburger__wrapper{min-width:1024px;max-width:1024px;padding:20px;margin:0 auto;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-ms-flex-direction:row;flex-direction:row;-ms-flex-wrap:wrap;flex-wrap:wrap;-webkit-box-pack:justify;-ms-flex-pack:justify;justify-content:space-between}.hamburger__block{width:280px;margin-bottom:20px}.hamburger__block.partial-block .hamburger__section{float:left;width:50%}.hamburger__block.partial-block .hamburger__section-title{margin-bottom:20px}.hamburger__block.partial-block .hamburger__list-item{float:none;width:auto}@media (max-width: 1024px){.hamburger__inner-container{width:100%}.hamburger__block{width:270px;margin-left:10px;margin-right:10px}.hamburger__wrapper{-ms-flex-pack:distribute;justify-content:space-around;padding:20px 80px;min-width:768px}}.hamburger__section-title{display:inline-block;font-family:"ProximaNovaCond-Bold";font-size:1.125rem;letter-spacing:0.09375em;line-height:1.25rem;text-decoration:none;color:#ffffff;font-style:italic;border-style:solid;border-color:#0DBE98;border-width:0 0 3px 0;margin-bottom:15px;padding-top:4px;padding-left:2px;padding-right:2px}.hamburger__section-content{margin-bottom:10px}.hamburger__list{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;list-style:none;-ms-flex-flow:wrap;flex-flow:wrap}.hamburger__list-item{width:50%;margin-bottom:15px}.hamburger__link{font-size:1rem;text-decoration:none;line-height:1.0625rem;letter-spacing:0.01875em;color:#ffffff}.hamburger__link:hover{text-decoration:underline;color:#0DBE98}a.hamburger__section-title:hover{background-color:#0dbe98;padding-bottom:3px;border-bottom:none;-webkit-transition:background-color 350ms ease-out;transition:background-color 350ms ease-out}.footer{background-color:#000000;padding:40px 20px 0 20px;display:none}.text{line-height:1.5em}.text p{white-space:pre-wrap}.text p iframe{max-width:100%}.text{width:100%;margin:0}@media (min-width: 1024px){.text{max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}}.text p{white-space:normal}.text .link-out-link{font-style:italic}.text h3{color:#444444;font-size:1.75rem;line-height:1em;margin-top:20px;margin-bottom:20px}.text h4{color:#444444;font-size:1.375rem;line-height:1.1em;margin-top:20px;margin-bottom:20px}.wide .text{max-width:730px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.embed-asset{max-width:630px;min-width:initial;width:100%;margin-left:auto;margin-right:auto;margin-top:40px;margin-bottom:40px}.embed-asset embed,.embed-asset iframe,.embed-asset video,.embed-asset object{max-width:100%}.embed-asset iframe{display:block;margin:0 auto}.embed-asset:first-child{margin-top:0}.embed-asset iframe.twitter-tweet,.embed-asset iframe.instagram-media{margin:0 auto !important}.embed-asset blockquote{border:none;background-color:initial}.embed-asset .fb-post,.embed-asset .fb-video{display:block;margin:0 auto}.embed-asset .fb-post span,.embed-asset .fb-video span{display:block;margin:0 auto}.embed-asset .five-min-container .fmvps-wrapper div{margin:0 auto}.embed-asset .five-min-container a.back-to-original{display:block;padding-top:5px;text-decoration:none;color:#000000;-webkit-box-shadow:inset 0 -2px 0 #0DBE98;box-shadow:inset 0 -2px 0 #0DBE98}.embed-asset .five-min-container a.back-to-original:hover{background-color:rgba(13,190,152,0.7);padding-top:2px;padding-bottom:0;border-bottom:none;-webkit-transition:background-color 350ms ease-out;transition:background-color 350ms ease-out;-webkit-box-shadow:none;box-shadow:none}.embed-code>div:not([itemprop="video"])>.flex-video,.embed-code .flex-video>.flex-video{position:static}.unsupported-content{margin-top:5px;margin-bottom:5px;height:250px;width:100%;background:url(//s.m.huffpost.com/assets/notsupported-6c5dc5d5a2a21057b23669d70a1351a401d2c813866c8ec67ce8ab0b31d0e6a3.png) no-repeat center center}.embed-asset{max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.wide .embed-asset{max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.wide .embed-asset.full-width-embed{max-width:970px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.share-bar{display:none}.share-bar__item{float:left;display:block;font-size:1.875rem;height:3.125rem;width:3.125rem;line-height:3.125rem;text-decoration:none;text-align:center;color:#ffffff;overflow:hidden}.share-bar__item:before{display:inline-block}.share-bar__item-link{display:block;font-size:1.875rem;height:3.125rem;width:3.125rem;line-height:3.125rem;text-decoration:none;text-align:center;color:#ffffff;overflow:hidden}.share-bar__item-link:before{display:inline-block}.share-bar__social-counts{float:left;width:58px;height:45px;font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:1.5em;text-align:center;margin-right:1px}.share-bar__social-counts__number{line-height:2.25em;vertical-align:middle;line-height:45px}

.entry__header{margin-top:15px;margin-bottom:15px}.entry__body{color:#000000;word-wrap:break-word}@media (max-width: 580px){.entry__body div[style],.entry__body iframe[style],.entry__body img[style]{max-width:100%}}.entry__body .content-list-component{padding-left:0px;padding-right:0px}.entry__body p{font-size:1.125rem;line-height:1.75rem;margin-bottom:20px}.entry__body ul,.entry__body ol{padding-left:2em;padding-top:0;padding-bottom:0;margin-bottom:20px}.entry__body li{margin-bottom:5px}.entry__body ul li{font-size:16px;line-height:24px}.entry__body blockquote{margin-top:5px;margin-bottom:25px;background-color:transparent;border:none;color:#757575;padding-left:30px;font-size:15px;font-style:italic}.entry__body blockquote p:last-child{margin-bottom:0}.entry__text .text a,.entry__text p a{text-decoration:none;color:#000000;-webkit-box-shadow:inset 0 -2px 0 #0DBE98;box-shadow:inset 0 -2px 0 #0DBE98}.entry__text .text a:hover,.entry__text p a:hover{background-color:rgba(13,190,152,0.7);padding-top:2px;padding-bottom:0;border-bottom:none;-webkit-transition:background-color 350ms ease-out;transition:background-color 350ms ease-out;-webkit-box-shadow:none;box-shadow:none}.entry-static{width:66%}.entry{max-width:1080px;min-width:767px;width:100%;margin-left:auto;margin-right:auto}@media (max-width: 1300px){.entry{padding-left:20px;padding-right:20px}}.entry__content{display:-webkit-box;display:-ms-flexbox;display:flex}.entry__header{margin-top:0;margin-bottom:0}.entry__body{width:100%;font-size:1.125rem;line-height:1.75rem;margin-top:3px}@media (min-width: 1024px){.entry__body{max-width:705px;margin-right:36px}}@media (min-width: 1340px){.entry__body{margin-right:65px}}.entry__body ::-moz-selection{background-color:#dbdbdb}.entry__body ::selection{background-color:#dbdbdb}.entry__comments{max-width:1170px;min-width:initial;width:100%;margin-left:auto;margin-right:auto;margin-bottom:50px}@media (max-width: 1300px){.entry__comments{padding-left:10px;padding-right:10px}}.entry__comments .comments{max-width:700px}.entry__comments-pane{position:fixed;width:500px;height:100%;top:0;right:0;background-color:#ffffff;padding:5px 10px 10px 10px;-webkit-box-shadow:-1px 0px 10px 1px rgba(0,0,0,0.35);box-shadow:-1px 0px 10px 1px rgba(0,0,0,0.35);z-index:5000006;-webkit-transition:-webkit-transform .5s;transition:-webkit-transform .5s;transition:transform .5s;transition:transform .5s, -webkit-transform .5s;-webkit-transform:translateX(0);transform:translateX(0);-webkit-overflow-scrolling:touch}.entry__comments-pane__close{position:absolute;cursor:pointer;background-color:#ffffff;z-index:5000007;width:35px;height:65px;top:0;right:20px;text-align:center;line-height:65px;border-right:1px solid #ffffff}.entry__comments-pane__close:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:17px;color:#000000}.entry__comments-pane .comments{overflow:scroll;height:100%}.entry__comments-pane.hide{-webkit-transform:translateX(108%);transform:translateX(108%)}@media (min-width: 1024px) and (max-width: 1240px){.entry--standard{padding-left:80px;padding-right:20px;margin:0;max-width:1160px}}.entry--contributor .entry__body{width:100%}@media (max-width: 980px){.entry--contributor .entry__body{padding-right:0}}.entry--contributor .right-rail{width:300px}.wide .entry,.anthology .entry{max-width:970px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.wide .entry__header,.anthology .entry__header{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;text-align:center;max-width:767px;min-width:930px;width:100%;margin-left:auto;margin-right:auto;margin-bottom:20px}.wide .entry__header .entry-eyebrow,.anthology .entry__header .entry-eyebrow{-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1}.wide .entry__header .headline,.anthology .entry__header .headline{-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2;max-width:none;margin-bottom:20px}.wide .entry__header .headline__title,.anthology .entry__header .headline__title{max-width:none}.wide .entry__header .headline__subtitle,.anthology .entry__header .headline__subtitle{width:auto;max-width:730px;margin:0 auto}.wide .entry__header .wide_stroke,.anthology .entry__header .wide_stroke{width:140px;height:5px;margin:0 auto 20px auto;-webkit-box-ordinal-group:4;-ms-flex-order:3;order:3;background-color:#0DBE98}.wide .entry__header .byline,.anthology .entry__header .byline{-webkit-box-ordinal-group:5;-ms-flex-order:4;order:4;height:auto;margin-bottom:4px}.wide .entry__header .byline__authors,.anthology .entry__header .byline__authors{max-width:none;width:auto}.wide .entry__header .byline .author-card,.anthology .entry__header .byline .author-card{padding:0}.wide .entry__header .byline .author-card__details,.anthology .entry__header .byline .author-card__details{margin:0 auto;padding:0;line-height:normal}.wide .entry__header .byline .author-card__details__link,.anthology .entry__header .byline .author-card__details__link{line-height:normal}.wide .entry__header .byline .author-card__details__name,.anthology .entry__header .byline .author-card__details__name{font-size:0.875rem}.wide .entry__header .timestamp,.anthology .entry__header .timestamp{font-size:0.8125rem;-webkit-box-ordinal-group:6;-ms-flex-order:5;order:5;margin-top:0;margin-left:0}.wide .entry__body,.anthology .entry__body{max-width:none;margin-right:0}.wide .entry__body .extra-content__title,.anthology .entry__body .extra-content__title{text-align:left;max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.wide .entry__content,.anthology .entry__content{max-width:767px;min-width:930px;width:100%;margin-left:auto;margin-right:auto}.anthology .entry .headline__subtitle{max-width:none}.anthology .entry .wide_stroke{background-color:#B10000}.entry--anthology .entry__header{text-align:left}.entry--anthology .entry__content{max-width:930px}.entry--anthology .text{max-width:initial}
.card__link{color:inherit;text-decoration:none}.card{display:block;margin-bottom:20px;width:100%}.card:first-child{margin-top:0}.card__content{display:-webkit-box;display:-ms-flexbox;display:flex;overflow:hidden}.card__label{color:#2e7061}.card__headline{font-family:"ProximaNovaCond-Bold";color:#000000;margin-bottom:0;padding-right:5px;font-size:1.375rem;line-height:1.1em}.card__video{margin-bottom:15px;padding-bottom:15px;border-bottom:1px solid #eee}.card__video .video-image{float:right;width:100px}.card__video .video-title{font-family:"ProximaNovaCond-Bold";color:#000000;text-decoration:none;font-size:1.5rem;margin-right:10px}.card__image__cover-bg{width:100%;height:0;position:relative;padding-bottom:56.26%;display:block}.card__image__cover-bg .card__image__src{background-size:cover;background-position:50% 50%;width:100%;height:100%;position:absolute}.card--invert{background:#191919}.card--invert .card__label__text{background:#191919}.card--invert .card__headline{color:#ffffff}.card--no-label .card__label{display:none}.a-page .a-page__content__rail .card__label{color:#0DBE98}.a-page .a-page__content__rail .card__label::after{border-color:#0DBE98}.a-page .a-page__content__rail .card--section .card__headline{color:#0DBE98}.a-page .a-page__content__rail .card--section .card__headline .card__link{color:inherit}.middle-rail .card{border-bottom:1px solid #dbdbdb;margin-bottom:30px;padding-bottom:30px}.middle-rail .card:last-child{border-bottom:none;padding-bottom:0}.card__content{-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column}.card__label{font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:0.875rem;margin-bottom:8px;-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1}.card__label__text{letter-spacing:1.4px;font-family:inherit;font-size:inherit;font-weight:inherit;line-height:inherit}.card__headlines{-webkit-box-ordinal-group:4;-ms-flex-order:3;order:3}.card__pill-text{font-size:0.75rem;line-height:1.125rem;letter-spacing:-0.01em;color:#757575}.card__date{min-width:100%}.card__headline{font-size:1.625rem;line-height:1.75rem;margin-top:12px;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-ms-flex-flow:row wrap;flex-flow:row wrap}.card__headline.hover{text-decoration:underline}.card__link{color:inherit;text-decoration:none;-webkit-box-ordinal-group:4;-ms-flex-order:3;order:3}.card__video{margin-bottom:15px;padding-bottom:15px;border-bottom:1px solid #eee}.card__video .video-image{float:right;width:100px}.card__video .video-title{font-family:"ProximaNovaCond-Bold";color:#000000;text-decoration:none;font-size:1.5rem;margin-right:10px}.card__image{-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2;display:block;position:relative}.card__image__wrapper{line-height:0;position:relative}.card__image__credit{position:absolute;bottom:0;right:0;height:13px;opacity:0.5;background-color:#000000;font-family:Arial, sans-serif;font-size:8px;line-height:14px;vertical-align:middle;padding:0 5px;letter-spacing:0.025rem;text-align:right;color:#ffffff;text-transform:uppercase}.card__image__credit.hide{display:none}.card .author-card{padding-top:12px}.card .author-card__details__name{font-size:12px}.card--embed.card--media-right .card__image:before{left:40px}.middle-rail .card--media-right .card__details{width:50%}.middle-rail .card--media-right .card__headline{font-size:1.375rem;line-height:1.5rem}.card--media-right .card__content{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-ms-flex-direction:row;flex-direction:row;-ms-flex-wrap:wrap;flex-wrap:wrap}.card--media-right .card__label{width:100%}.card--media-right .card__headlines{-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1;-webkit-box-flex:1;-ms-flex:1 0 50%;flex:1 0 50%;max-width:100%}.card--media-right .card__headline:first-of-type{margin-top:0}.card--media-right .card__image{-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2;-webkit-box-flex:1;-ms-flex:1 1 50%;flex:1 1 50%;max-width:50%;padding-left:10px}.card--media-right .card__image__src{width:100%}.card--light .card__headline{font-size:1.5rem}.card--media-right.card--light .card__details{width:65%}.card--media-right.card--light .card__headlines{-webkit-box-flex:1;-ms-flex:1 0 65%;flex:1 0 65%}.card--media-right.card--light .card__date{-webkit-box-flex:1;-ms-flex:1 0 65%;flex:1 0 65%}.card--media-right.card--light .card__image{-webkit-box-flex:1;-ms-flex:1 1 35%;flex:1 1 35%;max-width:35%}.card--about .card__description{-webkit-box-ordinal-group:4;-ms-flex-order:3;order:3;font-size:0.875rem;line-height:1.125rem;color:#757575;font-family:"Helvetica Neue";padding-top:10px}.card--embed .card__embed{-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2}.card--multi-headline .card__headline{font-size:1.375rem;line-height:1.375rem;padding-bottom:12px;border-bottom:1px dotted #dbdbdb}.card--multi-headline .card__headline:last-of-type{border:none;padding-bottom:0;margin-bottom:0}.card--multi-headline .card__image__cover-bg{padding-bottom:100%}.card--multi-headline .card__byline{display:none}.card--multi-headline.card--media-right .card__headline:first-of-type{margin-top:0}.card--multi-headline.card--media-right .card__image__cover-bg{padding-bottom:100%}.card--section .card__label{olor:#2e7061;position:relative}.card--section .card__label__text{background:#ffffff;padding-right:10px;position:relative;z-index:2}.card--section .card__label:after{content:'';border-bottom:3px solid #2e7061;position:absolute;top:50%;left:0;margin-top:-2px;width:100%;z-index:1}.card--section .card__headline{display:block;border-bottom:1px solid #dbdbdb;padding-top:7px;padding-bottom:7px;font-size:1.375rem;color:#0DBE98}.card--section .card__headline:last-child{border:none}.recirc--bottom-trending .card__label,.recirc--most-shared .card__label{olor:#2e7061;position:relative}.recirc--bottom-trending .card__label__text,.recirc--most-shared .card__label__text{background:#ffffff;padding-right:10px;position:relative;z-index:2}.recirc--bottom-trending .card__label:after,.recirc--most-shared .card__label:after{content:'';border-bottom:3px solid #2e7061;position:absolute;top:50%;left:0;margin-top:-2px;width:100%;z-index:1}.recirc--horizontal .card--big_title .card__headline{font-size:1.625rem;line-height:1.625rem}.card--video-hub{padding-bottom:20px;margin-bottom:40px}.a-page__content__rail .card--video-hub .card__content{max-width:initial;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.card--video-hub .card__content{max-width:1280px;min-width:600px;width:100%;margin-left:auto;margin-right:auto;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;-webkit-box-pack:justify;-ms-flex-pack:justify;justify-content:space-between;width:100%;-ms-flex-flow:row;flex-flow:row;padding:0 20px}.card--video-hub .card__embed{max-width:800px;min-width:570px;width:100%;margin-left:auto;margin-right:auto;width:100%;padding:0}.card--video-hub .card__details{-webkit-box-flex:0;-ms-flex:0 0 300px;flex:0 0 300px;margin-right:30px;position:relative;min-height:450px}.card--video-hub .card__label{font-size:1.125rem;line-height:1.25rem;margin-top:0;margin-bottom:5px;-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2}.card--video-hub .card__label:after{display:none}.card--video-hub .card__label__text{padding-left:0;color:#0DBE98}.card--video-hub .card__headlines{-webkit-box-ordinal-group:4;-ms-flex-order:3;order:3}.card--video-hub .card__headline{border:0;color:#ffffff;font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:1.375rem;font-weight:bold;letter-spacing:0.04em;line-height:1.5rem;margin-top:0;margin-bottom:5px;max-width:720px;padding:0}.card--video-hub .card__content__ad{position:relative;bottom:0}.card--video-hub .card__image{-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;-webkit-box-flex:1;-ms-flex:1 0 600px;flex:1 0 600px;-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1}.card--video-hub .card__image:before{bottom:50%;left:50%;width:100px;height:100px;border-radius:50%;margin:0;font-size:44px;line-height:100px;padding-left:22px}.card--video-hub .card__headline:hover{opacity:1}.card--video-show-splash{padding-bottom:30px}.card--video-show-splash .card__content{max-width:600px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.card--video-show-card .card__headline{margin-bottom:5px}.card--video-show-card .card__image{padding-right:20px;max-width:250px;max-height:140px}.card--originals{-webkit-box-flex:1;-ms-flex:1 0 220px;flex:1 0 220px;margin-left:10px;margin-right:10px;margin-bottom:60px;max-width:280px}.card--originals .card__headline{font-family:"ProximaNovaCond-Bold";font-size:1.25rem;line-height:1.5rem}.card--autopilot-video-show .card__row{display:-webkit-box;display:-ms-flexbox;display:flex;margin-top:20px}.card--autopilot-video-show .card__label__text__link{color:#0DBE98;text-decoration:none}@media (max-width: 980px){.card--originals{-webkit-box-flex:1;-ms-flex:1 0 180px;flex:1 0 180px;max-width:220px}}@media (min-width: 1210px){.card--originals{-webkit-box-flex:1;-ms-flex:1 0 280px;flex:1 0 280px;max-width:280px}}.originals__content .card .card__content{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;-webkit-box-pack:start;-ms-flex-pack:start;justify-content:flex-start}.originals__content .card .card__content .card__image{-webkit-box-ordinal-group:1;-ms-flex-order:0;order:0;padding-left:0;padding-right:0}.originals__content .card .card__content .card__details{-webkit-box-ordinal-group:2;-ms-flex-order:1;order:1}.originals__content .card .card__content .card__headlines{-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2;font-size:1.375rem;line-height:1.5rem}.originals__content .card .card__content .card__headline{border-bottom:none;margin-top:10px;margin-bottom:8px;padding:0}
.recirc__item__image{width:100%}.recirc__item__image--external{height:0;position:relative;padding-bottom:56.26%}.recirc__item__image--external .recirc__item__image__src{background-size:cover;background-position:50% 50%;width:100%;height:100%;position:absolute}.recirc__item__author__name{color:#2e7061;font-weight:bold;font-size:0.75rem;line-height:0.9375rem;letter-spacing:-0.01em}.recirc__item__author__name:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;margin-right:3px}.recirc__item__author__teaser{font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;margin-bottom:10px;color:#757575;font-size:0.75rem;line-height:0.9375rem;letter-spacing:-0.01em;display:inline-block}.a-page .a-page__content__rail .recirc--vertical .recirc__title{color:#0DBE98}.a-page .a-page__content__rail .recirc--vertical .recirc__title::after{border-bottom:3px solid #0DBE98}.recirc{display:none}.recirc .author-card{line-height:0.9375rem;letter-spacing:-0.01em;color:#757575}.recirc--vertical{border-bottom:1px solid #dbdbdb;margin-bottom:20px}.recirc--vertical:last-child{border:0}.recirc--vertical .recirc__title{text-align:center}.recirc--vertical .card{border-bottom:1px solid #dbdbdb;padding-bottom:20px}.recirc--vertical .card__headline{font-size:1.125rem;line-height:1.25rem;margin:0}.recirc--vertical .card:last-child{border:none;margin:0}.recirc--vertical .card--multi-headline{border-bottom:none}.recirc--vertical .author-card{margin:0;padding:8px 0}.recirc--right-rail .card{padding:0}.recirc--right-rail .zone__title{padding:0}.recirc--left-rail{display:inherit}.recirc--left-rail .card{padding-bottom:20px;position:relative}.recirc--left-rail .card__label{display:none}.recirc--left-rail .card__headline{margin-top:10px;font-size:1.125rem}.recirc--left-rail .card .author-card{margin-bottom:8px;padding:0}.recirc--left-rail .card .author-card:first-of-type{padding-top:8px;padding-bottom:0;margin-bottom:0}.recirc--left-rail .card--headshot{padding-bottom:12px;margin-bottom:12px}.recirc--left-rail .card:first-of-type.card--autopilot .card__label,.recirc--left-rail .card:first-of-type.card--normal .card__label{display:block}.recirc--left-rail .card:first-of-type.card--autopilot .card__label__text,.recirc--left-rail .card:first-of-type.card--normal .card__label__text{visibility:hidden}.recirc--left-rail .card--postit{padding:0;margin-bottom:12px;border:none}.recirc--left-rail .card--postit .card__label{display:block}.recirc--left-rail-b-page{display:inherit}.recirc--left-rail-b-page .recirc__title{position:relative;font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:0.875rem;margin-bottom:10px}.recirc--left-rail-b-page .recirc__title__text{position:relative;padding-right:10px;letter-spacing:1.4px}.recirc--left-rail-b-page .recirc__title:after{content:'';border-bottom:3px solid #000000;position:absolute;top:50%;left:78px;margin-top:-2px;width:59%;z-index:1}.recirc--left-rail-b-page .card{padding-bottom:12px;padding-top:2px;margin-bottom:0;border-bottom:1px solid #dbdbdb}.recirc--left-rail-b-page .card__headline{margin-top:10px;margin-bottom:0;font-size:1.125rem;line-height:20px}.recirc--left-rail-b-page .card .author-card{padding-top:6px;padding-bottom:0}.recirc--suggested-story .recirc__title{font-family:Arial, Helvetica, sans-serif;font-size:0.875rem;line-height:22px;color:#999;margin-bottom:0px}.recirc--suggested-story .card__headline{font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:1.75rem;line-height:30px;color:#2e7062;text-decoration:none}.wide .recirc--suggested-story{max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}
.entry-eyebrow{color:#000000;font-size:1.125rem;line-height:1.1875rem;letter-spacing:0.11875em;font-style:italic;font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;margin-bottom:3px}.entry-eyebrow__link{color:#000000;text-decoration:none}.entry-eyebrow--sponsored .entry-eyebrow__link{color:#3d3d3d}.entry-eyebrow--sponsored .entry-eyebrow__link:hover{color:#3d3d3d}.entry-eyebrow--world{font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:0.8125rem;color:#757575;border-bottom:1px solid #cccccc;font-style:initial;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-pack:justify;-ms-flex-pack:justify;justify-content:space-between;margin-bottom:15px}.entry-eyebrow--world .entry-eyebrow__sponsor__text{display:inline-block;vertical-align:middle;line-height:13px;padding-bottom:9px}
.timestamp{padding-top:10px;padding-bottom:10px;color:#757575;font-size:0.75rem;line-height:2em}.timestamp{margin-top:5px;font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-size:0.8125rem;line-height:1.375rem;padding-top:0;padding-bottom:0;display:inline-block;margin-left:6px;height:20px}.timestamp--contributor{margin-left:0;margin-bottom:7px}
.headline__title{margin-right:10px;color:#000000;font-size:3.25rem;line-height:1.1em;letter-spacing:0.5px;max-width:900px;word-wrap:break-word;margin:15px 0}.headline__subtitle{margin-top:5px;color:#313131;font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-weight:normal;font-size:1.25rem;line-height:1.5rem;max-width:630px}.headline__subtitle .emoji{font-weight:normal;font-family:helvetica}.headline{margin-top:8px}.headline__title{margin-top:0;margin-bottom:0;line-height:3.25rem;max-width:1024px;letter-spacing:0.5px}@media (max-width: 1023px){.headline__title{font-size:2.625rem}}.headline__subtitle{margin-top:7px;line-height:1.5rem;width:720px;max-width:720px}.wide .headline{max-width:900px}
.byline{padding-top:15px;margin-bottom:15px;padding-left:0;padding-bottom:0;list-style:none;padding-top:10px;margin-bottom:10px}.byline{padding-top:0;padding-bottom:0;margin-bottom:0;display:-webkit-box;display:-ms-flexbox;display:flex}.byline__authors{max-width:630px;width:50%;-webkit-box-flex:1;-ms-flex:1 0 50%;flex:1 0 50%}.byline__share-bar{width:50%;-webkit-box-flex:1;-ms-flex:1 1 50%;flex:1 1 50%}.byline--contributor{margin-bottom:9px;height:auto}.byline--multiple{height:auto}.byline--left-rail{border-top:none;padding-right:20px}.byline--left-rail .author-card{-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;-webkit-box-flex:0;-ms-flex:0 1 100%;flex:0 1 100%}.byline--left-rail .author-card__headshot{width:50px;height:50px;border-radius:7%;margin-bottom:4px}.byline--left-rail .author-card__details-container{margin:4px 0}.byline--left-rail .author-card__details{padding:0;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column}.byline--left-rail .author-card__microbio{margin:4px 0;color:#666;-webkit-box-ordinal-group:3;-ms-flex-order:2;order:2}.byline--left-rail .author-card__social-container{margin:4px 0;-webkit-box-ordinal-group:4;-ms-flex-order:3;order:3}.byline--left-rail .author-card__social--facebook{background-color:transparent;color:#3b5999}.byline--left-rail .author-card__social--facebook:before{font-size:1rem}.byline--left-rail .author-card__social--twitter{background-color:transparent;color:#55acef}.byline--left-rail .author-card__social--twitter:before{font-size:1rem}.byline--left-rail .author-card__social--googleplus{background-color:transparent;color:#dd4b34}.byline--left-rail .author-card__social--googleplus:before{font-size:1rem}
.author-card{display:-webkit-inline-box;display:-ms-inline-flexbox;display:inline-flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-ms-flex-direction:row;flex-direction:row;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;-ms-flex-line-pack:stretch;align-content:stretch;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:start;-ms-flex-pack:start;justify-content:flex-start;font-size:0.75rem;line-height:1.125rem;width:100%;text-align:left}.author-card:last-child{margin-bottom:0}.author-card__headshot{background-repeat:no-repeat;background-size:cover;background-position:center center;margin-right:15px;-webkit-box-flex:0;-ms-flex:0 0 auto;flex:0 0 auto;border-radius:50%;border:solid 1px #cccccc;background-clip:content-box;height:50px;width:50px}.author-card__details{padding-right:15px;-webkit-box-flex:1;-ms-flex:1 1 auto;flex:1 1 auto}.author-card__details__link{text-decoration:none;color:inherit}.author-card__details__name{font-size:1rem;color:#444444;font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;font-weight:bold;white-space:pre-wrap}.author-card--full{display:-webkit-inline-box;display:-ms-inline-flexbox;display:inline-flex;-webkit-box-orient:horizontal;-webkit-box-direction:normal;-ms-flex-direction:row;flex-direction:row;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;-ms-flex-line-pack:stretch;align-content:stretch;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:start;-ms-flex-pack:start;justify-content:flex-start;color:#444444;font-size:0.75rem;line-height:1.125rem;width:100%;text-align:left;margin-bottom:20px}.author-card--full .author-card:last-child{margin-bottom:0}.author-card--full .author-card__headshot{background-repeat:no-repeat;background-size:cover;background-position:center center;margin-right:15px;-webkit-box-flex:0;-ms-flex:0 0 auto;flex:0 0 auto;border-radius:50%;background-clip:content-box;height:50px;width:50px}.author-card--full .author-card__details{padding-right:15px;-webkit-box-flex:1;-ms-flex:1 1 auto;flex:1 1 auto}.author-card--full .author-card__details__link{text-decoration:none;color:inherit}.author-card--full .author-card__details__name{font-weight:bold;font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif}.author-card--full .author-card__social-container{padding-left:7px;top:0}.author-card--full .author-card__social{-ms-flex-negative:0;flex-shrink:0;font-size:1.0625rem;text-decoration:none}.author-card--full .author-card__social--twitter{color:#55acef;position:relative;display:inline-block;background-color:initial;width:14px}.author-card--full .author-card__social--twitter:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:14px}.author-card--full .author-card__microbio{font-size:0.875rem;display:block}.author-card--byline .author-card__details__link{display:inline-block}.author-card--byline .author-card__details__name{padding-right:0}.author-card--contributor .author-card__headshot{width:40px;height:40px;border-radius:15%;margin-right:10px}.author-card--contributor .author-card__details__name{font-size:1rem}.author-card--contributor .author-card__microbio{font-size:0.875rem;color:#757575}.author-card--multiple{margin-bottom:10px}.author-card--multiple .author-card__headshot{width:30px;height:30px;border-radius:7%;margin-bottom:0}.author-card--multiple .author-card__microbio{display:inline}.author-card--contributor.author-card--full:last-child{margin-bottom:0}.author-card--contributor.author-card--byline .author-card__details{margin-top:0}.author-card--mega-phone .author-card__details__name{color:#2e7061}.author-card--mega-phone .author-card__details__name:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;margin-right:3px}.author-page .author-card{-webkit-box-align:center;-ms-flex-align:center;align-items:center}.author-page .author-card__headshot{width:100px;height:100px}.author-page .author-card__microbio,.author-page .author-card__minibio{font-family:Arial, sans-serif;font-size:1.125rem;color:#757575;margin-top:10px;display:block}.author-page .author-card__details{font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;margin-bottom:30px;padding:0}.author-page .author-card__details__name{color:#000000;display:inline-block;font-size:2.375rem}.author-page .author-card__social-container{display:inline-block;top:-8px;padding-left:5px}.author-page .author-card__social{-ms-flex-negative:0;flex-shrink:0;font-size:1.0625rem;text-decoration:none;margin-left:5px;margin-right:5px}.author-page .author-card__social--twitter{position:relative;top:1px;margin-left:0;margin-right:5px}.author-page .author-card__social--facebook{margin-left:0;margin-right:5px}.author-page .author-card__social--googleplus{color:#dd4b34;font-size:2rem}.author-page .author-card__social--googleplus:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:22px}.author-card__social--twitter,.author-card__social--facebook{border-radius:50%;width:22px;height:22px;line-height:19px;text-align:center;vertical-align:middle;margin-left:0;margin-right:3px}.author-list{width:630px}.author-list:after{content:"";display:table;clear:both}.author-card{padding:8px 0;-webkit-box-align:center;-ms-flex-align:center;align-items:center}.author-card__headshot{height:40px;width:40px;border:1px solid #cccccc;margin-right:10px}.author-card__details{-webkit-box-flex:0;-ms-flex:0 1 auto;flex:0 1 auto;margin-top:9px}.author-card__details-container{display:block}.author-card__details-container:after{content:"";display:table;clear:both}.author-card__details__name{font-size:0.875rem;padding-right:2px}.author-card__social-container{position:relative;top:-3px}.author-card__social{-ms-flex-negative:0;flex-shrink:0;font-size:1.0625rem;text-decoration:none}.author-card__social--twitter{background-color:#55acef;color:#ffffff;position:relative;display:inline-block}.author-card__social--twitter:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:11px}.author-card__social--facebook{background-color:#3b5999;color:#ffffff;display:inline-block;line-height:20px}.author-card__social--facebook:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:11px}.author-card__social--googleplus{color:#dd4b34;font-size:2rem}.author-card__social--googleplus:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:22px}.author-card__microbio{display:block}.author-card--byline{padding-bottom:12px}.author-card--byline .author-card__details{margin-top:4px}.author-card--no-teaser{-webkit-box-align:center;-ms-flex-align:center;align-items:center}.wide .author-list{width:980px}.wide .author-list-container .author-card{width:33%}
.top-media .slideshow-thumb__credits{padding-left:15px;padding-right:15px}.top-media iframe{max-width:100%;display:block;margin:0 auto}.top-media--image{margin-top:0;margin-bottom:30px}.top-media--image .image{margin-bottom:30px}.top-media--image .image__credit,.top-media--image .image__caption{padding-left:0px;padding-right:0px}.top-media--image .image__caption{border-bottom:1px dotted #cccccc}.top-media--video{margin:0 auto 30px}.top-media--video .back-to-original{text-decoration:none;color:#000000;-webkit-box-shadow:inset 0 -2px 0 #0DBE98;box-shadow:inset 0 -2px 0 #0DBE98}.top-media--video .back-to-original:hover{background-color:rgba(13,190,152,0.7);padding-top:2px;padding-bottom:0;border-bottom:none;-webkit-transition:background-color 350ms ease-out;transition:background-color 350ms ease-out;-webkit-box-shadow:none;box-shadow:none}.top-media--video .vdb_player{background-color:#f5f5f5}.wide .top-media{max-width:970px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.wide .top-media--image,.wide .top-media .image{max-width:970px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.top-media{margin-bottom:15px}.top-media--image .image{margin-top:0}.top-media .instagram-media{margin:0 auto !important}
.image{max-width:630px;min-width:initial;width:100%;margin-left:auto;margin-right:auto;margin-top:40px;margin-bottom:40px;line-height:0}.image:first-child{padding-top:0}.image__credit,.image__caption{line-height:1.4em;display:block;text-align:left}.image__credit{font-size:0.5625rem;color:#999999;margin-top:5px;text-transform:uppercase}.image__caption{font-size:0.75rem;border-bottom:1px dotted #cccccc;margin-top:7px;padding-bottom:8px;line-height:16px;color:#666}.image__caption p{padding-top:0;padding-bottom:0}.image{max-width:100%;display:block;text-align:center}.image__caption,.image__credit{max-width:100%;margin-left:auto;margin-right:auto}.image__caption{border-bottom:1px dotted #cccccc}
.text{line-height:1.5em}.text p{white-space:pre-wrap}.text p iframe{max-width:100%}.extra-content{margin-top:30px;margin-bottom:30px}.extra-content .slideshow-thumb,.extra-content .slideshow{margin-top:0px}.extra-content .slideshow-thumb-title{max-width:630px;min-width:initial;width:100%;margin-left:auto;margin-right:auto;line-height:22px}.extra-content .slideshow-thumb-title--amp{padding-left:15px}.extra-content__title{font-size:1rem;letter-spacing:0.0625em;font-weight:bold;line-height:1rem;color:#838383}.extra-content__media{margin-top:8px}.extra-content__media>div>div.slideshow-default__icon{margin-left:10px;margin-right:5px}.mobileweb .extra-content__title{padding:0 15px;max-width:630px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}
.share-text{color:#888}.slideshow.slideshow--a-page{width:300px;height:auto;background-color:initial;margin-bottom:50px;margin-left:15px;margin-right:15px;position:relative}.slideshow.slideshow--a-page .slideshow__container{position:relative;height:168px;width:300px;background-size:cover;background-repeat:no-repeat;background-position:center}.slideshow.slideshow--a-page .slideshow__label{color:#444444;font-family:"ProximaNovaCond-Bold";letter-spacing:0.03125em;font-size:13px;margin-top:10px}.slideshow.slideshow--a-page .slideshow__title{font-size:1.5rem;color:#000000;font-family:"ProximaNovaCond-Bold";margin-top:10px}.slideshow{max-width:630px;background-color:#000000;color:#ffffff;width:560px;height:315px;background-size:cover;background-repeat:no-repeat;background-position:center;margin:0;cursor:pointer}.slideshow.vertical{background-size:contain}.slideshow.dynamic-preview{-webkit-transition:background-image 1s;transition:background-image 1s}.slideshow__preview{position:relative;top:210px;left:20px;border-radius:40px;vertical-align:middle;line-height:70px;height:70px;width:390px;border:1px solid #ffffff;background:rgba(13,13,13,0.7);font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif}.slideshow__preview__icon{float:left;width:10%;margin-left:30px}.slideshow__preview__icon:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:25px;color:#ffffff}.slideshow__preview__text{height:70px}.slideshow__preview__text__title{font-size:1.25rem;text-overflow:ellipsis;white-space:nowrap;overflow:hidden;width:82%;height:50px}.slideshow__preview__text__count{height:20px;font-family:Arial;font-size:0.75rem}.wide .slideshow{max-width:720px;min-width:initial;width:100%;margin-left:auto;margin-right:auto}.wide .listicle__slide-caption{width:720px;margin:0 auto;text-align:left}.slideshow-default{cursor:pointer;max-width:630px;background:none;width:auto;height:auto}.slideshow-default__title{font-size:1.5rem;font-family:"ProximaNovaCond-Extrabld";line-height:24px;color:#333;float:left}.slideshow-default__icon{margin-left:5px}.slideshow-default__icon:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:23px;color:#333}.slideshow-default__slide-count{font-size:1.25rem;font-family:"ProximaNovaCond-Extrabld";color:#333;padding-top:2px;padding-left:3px}
.tag-cloud__header{font-weight:bold;font-family:"ProximaNovaCond-Extrabld", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;letter-spacing:0.0625em;font-size:1rem;line-height:1rem;color:#838383;margin:8px 5px 8px 0}.tag-cloud .tag{position:relative;font-family:"ProximaNova", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;padding:6px 12px;margin:5px 5px 5px 0;text-decoration:none;font-size:0.9375rem;border-radius:50px;display:inline-block;color:#838383;border:1px solid #838383}.tag-cloud{display:none}
.advertisement{display:none}
.inline-newsletter{display:none}
.nav-sticky{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;width:100%;height:65px;-webkit-box-shadow:0 0 10px 0 rgba(0,0,0,0.28);box-shadow:0 0 10px 0 rgba(0,0,0,0.28);z-index:5000005;position:fixed;top:0;left:0;background-color:#000000;-webkit-transition:-webkit-transform .33s ease-in-out;transition:-webkit-transform .33s ease-in-out;transition:transform .33s ease-in-out;transition:transform .33s ease-in-out, -webkit-transform .33s ease-in-out}.nav-sticky.hide{-webkit-transform:translateY(-100%);transform:translateY(-100%);-webkit-box-shadow:none;box-shadow:none}.nav-sticky__bar{width:100%;height:60px;display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center}.nav-sticky__item{display:-webkit-box;display:-ms-flexbox;display:flex;-webkit-box-align:center;-ms-flex-align:center;align-items:center;height:100%}.nav-sticky__hamburger{margin:0 20px}.nav-sticky__hamburger-link{width:27px;height:24px;text-decoration:none;cursor:pointer;overflow:hidden}.nav-sticky__hamburger-link:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:18px;color:#666;position:relative;top:2px}.nav-sticky__hamburger-link:hover:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:18px;color:#0DBE98;position:relative;top:2px}.nav-sticky__hamburger-link.open:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:18px;color:#666;position:relative;top:2px;left:2px}.nav-sticky__hamburger-link.open:hover:before{content:"";font-family:"huffington-icons";font-style:normal;text-rendering:optimizeLegibility;text-transform:none;-moz-osx-font-smoothing:grayscale;-webkit-font-smoothing:antialiased;font-smoothing:antialiased;font-size:18px;color:#0DBE98;position:relative;top:2px}.nav-sticky__logo{width:30px;height:30px;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center;text-decoration:none;-ms-flex-negative:0;flex-shrink:0}.nav-sticky__logo:hover{text-decoration:none}.nav-sticky__logo-img{border:0;-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none}.nav-sticky__logo-img::-moz-selection{background-color:transparent}.nav-sticky__logo-img::selection{background-color:transparent}.nav-sticky__title{padding-top:2px;display:inline-block;text-overflow:ellipsis;overflow:hidden;line-height:60px;white-space:nowrap;font-size:1.375rem;color:#ffffff;font-family:"ProximaNovaCond-Medium";letter-spacing:0.04375em;padding-left:20px;-webkit-box-flex:1;-ms-flex-positive:1;flex-grow:1}@media (min-width: 1024px){.nav-sticky__title{font-size:1.5rem}}.nav-sticky__sharebar{background-color:#000000;padding-left:20px;padding-right:10px;-ms-flex-negative:0;flex-shrink:0}.nav-sticky__progress-bar{width:100%;background:#000000;height:5px}.nav-sticky__progress-bar__fill{width:100%;background:-webkit-gradient(linear, right top, left top, from(#05f), color-stop(50%, #7201b2), to(#f83371));background:linear-gradient(to left, #05f, #7201b2 50%, #f83371);height:100%;max-width:100%}.nav-sticky--b-page{overflow:hidden;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column}
.below-entry-recirc__conversations-header{display:none}

</style>
<script>
  (function(){
    window.HP || (window.HP = {});
    HP.params = {"device":"desktop","is_front":null,"page_type":"b-page","entryId":"6805902","entryIds":["6805902"],"noRecirc":null,"recircTag":null,"topic":"Derek Cruice","dontFilterViewed":false,"social":{"newsletter":{"id":"0eb60568a730719cfead15d5782df96b","slug":"breaking-alerts","name":"Breaking Alerts","display_name":"Breaking Alerts","custom_toaster_text":null,"provider":"cm"}},"aPage":{"className":null,"slug":null},"section":{"slug":"crime","newsletter":{"id":"0eb60568a730719cfead15d5782df96b","slug":"breaking-alerts","name":"Breaking Alerts","display_name":"Breaking Alerts","custom_toaster_text":null,"provider":"cm"},"department":"politics-news"},"edition":{"id":"us","newsletter":{"id":"c1fb2d6bc59dbabb03c6369b73d9ece3","slug":"the-morning-email","name":"The Morning Email","display_name":"The Morning Email","custom_toaster_text":null,"provider":"cm"},"baseURL":"/"},"gravityJSON":null,"ab_config":[{"name":"outstreamStyle","id":1002,"variant_names":"control,enabled","variant_ids":"11,12","variant_distributions":"0.75,0.25"},{"name":"outstreamParents","id":1001,"variant_names":"control,enabled","variant_ids":"9,10","variant_distributions":"0.75,0.25"},{"name":"outstreamPolitics","id":1000,"variant_names":"control,enabled","variant_ids":"7,8","variant_distributions":"0.75,0.25"},{"name":"videoBelowSplash100Percent","id":92,"variant_names":"video","variant_ids":"138","variant_distributions":"1.0"},{"name":"vidibleOnHomepage2","id":91,"variant_names":"noVideo,autoPlayWithoutSound","variant_ids":"136,137","variant_distributions":"0.8,0.2"},{"name":"homepageSplash","id":1008,"variant_names":"control,splash","variant_ids":"26,27","variant_distributions":"0.5,0.5"},{"name":"mfp1829","id":1010,"variant_names":"embed,embedFlip,autoPlayWithoutSound","variant_ids":"31,32,33","variant_distributions":"0.4,0.4,0.2"}],"beacons":[{"type":"mlid","name":"masthead","nodeSelector":".bn-masthead"},{"type":"mnid","name":"editions","nodeSelector":".bn-editions"},{"type":"lnid","name":"open","nodeSelector":".bn-editions-link","clickToggle":"close"},{"type":"lnid","nameFromAttribute":"data-edition","nodeSelector":".bn-edition-item","trackPosition":true},{"type":"mnid","name":"hamburger_menu","nodeSelector":".bn-hamburger-menu"},{"type":"lnid","name":"open","nodeSelector":".bn-hamburger-button","clickToggle":"close"},{"type":"lnid","nameFromTextNode":true,"nodeSelector":".bn-hamburger-link","trackLink":true},{"type":"mnid","name":"logo","nodeSelector":".bn-logo-link"},{"type":"lnid","name":"homepage","nodeSelector":".bn-logo-image"},{"type":"mlid","nameFromAttribute":"data-zone","nodeSelector":".bn-zone"},{"type":"mlid","name":"right","nodeSelector":".bn-zone-right"},{"type":"mlid","name":"department_feed","nodeSelector":".bn-zone-department-feed"},{"type":"lnid","name":"zone_title","nodeSelector":".bn-zone-title","postLoaded":true},{"type":"lnid","name":"more_link","nodeSelector":".bn-zone-more-link","postLoaded":true},{"type":"mnid","nameFromAttribute":"data-card-type","nodeSelector":".bn-splash-card-main","trackLink":true,"linkFromAttribute":"data-card-link"},{"type":"mnid","nameFromAttribute":"data-card-type","nodeSelector":".bn-splash-card","parentNodeSelector":".bn-zone","trackPosition":true},{"type":"lnid","name":"image","nodeSelector":".bn-splash-image","trackLink":true,"linkFromAttribute":"data-card-link"},{"type":"lnid","name":"splash-headline","nodeSelector":".bn-splash-headline","trackLink":true,"linkFromAttribute":"data-card-link"},{"type":"mnid","nameFromAttribute":"data-card-type","nodeSelector":".bn-card","trackLink":true,"trackPosition":true,"linkFromAttribute":"data-link","parentNodeSelector":".bn-zone"},{"type":"mlid","name":"sticky-nav","nodeSelector":".bn-nav-sticky"},{"type":"mnid","name":"follow","nodeSelector":".bn-social-list"},{"type":"lnid","nameFromAttribute":"data-button-name","nodeSelector":".bn-follow-item","trackPosition":true},{"type":"mlid","name":"entry_header","nodeSelector":".bn-entry-header"},{"type":"mnid","name":"department_tag","nodeSelector":".bn-department-tag"},{"type":"lnid","nameFromAttribute":"data-linkname","nodeSelector":".bn-department-link","trackLink":true},{"type":"mnid","name":"byline","nodeSelector":".bn-byline"},{"type":"lnid","name":"author_image","nodeSelector":".bn-author-image"},{"type":"lnid","name":"author_name","nodeSelector":".bn-author-name"},{"type":"lnid","name":"twitter_follow","nodeSelector":".bn-author-twitter"},{"type":"mlid","name":"top_media","nodeSelector":".bn-top-media"},{"type":"mlid","name":"entry_text","nodeSelector":".bn-entry-text"},{"type":"mnid","name":"citation","nodeSelector":".bn-content-list-text"},{"type":"mnid","name":"entry_image","nodeSelector":".bn-content-list-image"},{"type":"lnid","nameFromTextNode":true,"nodeSelector":"a","excludeSelector":".bn-instream-share-link","parentNodeSelector":".bn-entry-text","trackPosition":true,"trackLink":true},{"type":"mnid","name":"share","nodeSelector":".bn-share"},{"type":"lnid","nameFromAttribute":"data-share-name","nodeSelector":".bn-instream-share-link","disableClickHandler":true},{"type":"lnid","name":"open","nodeSelector":".bn-read-more-link"},{"type":"mlid","name":"entry_footer","nodeSelector":".bn-entry-footer"},{"type":"mnid","name":"correction","nodeSelector":".bn-corrections"},{"type":"lnid","name":"contact","nodeSelector":".bn-correction-link"},{"type":"mnid","name":"topic_tag","nodeSelector":".bn-tag-cloud"},{"type":"lnid","nameFromAttribute":"data-slug","nodeSelector":".bn-tag","trackPosition":true,"trackLink":true},{"type":"mlid","name":"left_rail","nodeSelector":".bn-left-rail"},{"type":"lnid","nameFromAttribute":"data-share-name","nodeSelector":".bn-share-item","disableClickHandler":true},{"type":"mlid","name":"right_rail","nodeSelector":".bn-right-rail"},{"type":"mnid","nodeSelector":".bn-recirc","nameFromAttribute":"data-recirc-name","postLoaded":true},{"type":null,"nodeSelector":".bn-recirc-item","trackPosition":true,"parentNodeSelector":".bn-recirc","trackLink":true,"linkFromAttribute":"data-recirc-link","postLoaded":true},{"type":"mnid","name":"trending","nodeSelector":".bn-trending"},{"type":"mnid","name":"suggested-trending","nodeSelector":".bn-suggested-trending"},{"type":"mnid","name":"bottom-trending","nodeSelector":".bn-bottom-trending"},{"type":"lnid","name":"headline","nodeSelector":".bn-card-headline","trackLink":true,"linkFromAttribute":"data-card-link","postLoaded":true},{"type":"lnid","name":"image","nodeSelector":".bn-card-image","trackLink":true,"linkFromAttribute":"data-card-link","postLoaded":true},{"type":"lnid","name":"description","nodeSelector":".bn-card-description","trackLink":true,"linkFromAttribute":"data-card-link","postLoaded":true},{"type":"lnid","name":"author_name","nodeSelector":".bn-card-author"},{"type":"mnid","name":"follow","nodeSelector":".bn-follow-us"},{"type":"lnid","name":"subscribe","nodeSelector":".bn-signup-submit"},{"type":"mlid","name":"below_entry","nodeSelector":".bn-below-entry"},{"type":"mnid","name":"taboola","nodeSelector":".bn-taboola","trackPosition":true},{"type":"mlid","name":"footer","nodeSelector":".bn-footer"},{"type":"mnid","name":"corp-links","nodeSelector":".bn-footer-links"},{"type":"lnid","nameFromTextNode":true,"nodeSelector":".bn-footer-link-item","trackLink":true},{"type":"mnid","name":"slideshow-preview","nodeSelector":".bn-slideshow-preview"},{"type":"mlid","name":"extra-content","nodeSelector":".bn-slideshow-preview"},{"type":"mlid","name":"slideshow-header","nodeSelector":".bn-slideshow-header"},{"type":"mlid","name":"slideshow-body","nodeSelector":".bn-slideshow-body"},{"type":"lnid","name":"open","nodeSelector":".bn-slideshow-open","extra":{"dL_sDpt":"gallery"}},{"type":"lnid","name":"image","nodeSelector":".bn-slideshow-open-a-page","extra":{"mnid":"gallery"}},{"type":"lnid","name":"headline","nodeSelector":".bn-slideshow-open-a-page-headline","extra":{"mnid":"gallery"}},{"type":"lnid","name":"close","nodeSelector":".bn-close"},{"type":"mnid","name":"nav","nodeSelector":".bn-slideshow-nav"},{"type":"lnid","name":"prev","nodeSelector":".bn-slideshow-prev","extra":{"dL_sDpt":"gallery"}},{"type":"lnid","name":"next","nodeSelector":".bn-slideshow-next","extra":{"dL_sDpt":"gallery"}},{"type":"mlid","name":"comments-pane","nodeSelector":".bn-comments-pane"},{"type":"mnid","name":"comments","nodeSelector":".bn-comments-container"},{"type":"mlid","name":"modal","nodeSelector":".bn-modal","postLoaded":true},{"type":"mnid","name":"body","nodeSelector":".bn-modal-body","postLoaded":true},{"type":"lnid","name":"submit","nodeSelector":".bn-submit"},{"type":"lnid","name":"cancel","nodeSelector":".bn-cancel"},{"type":"mlid","name":"highlight","nodeSelector":".bn-highlight"},{"type":"mlid","name":"overlay","nodeSelector":".bn-overlay"},{"type":"mnid","name":"snapchat","nodeSelector":".bn-snapchat"},{"type":"lnid","name":"qr","nodeSelector":".bn-snapchat-qr"},{"type":"lnid","nameFromTextNode":true,"nodeSelector":".bn-card-link-text","trackLink":true}]};
  }());
</script>

<script>
  // critical desktop
  window.microAjax=function(e,t){if(this.getRequest=function(){return!!window.XMLHttpRequest&&new XMLHttpRequest},this.postBody=arguments[2]||"",this.callbackFunction=t,this.url=e,this.request=this.getRequest(),this.request){var n=this.request;n.onreadystatechange=function(){4==n.readyState&&t(n.responseText)},""!==this.postBody?(n.open("POST",e,!0),n.setRequestHeader("X-Requested-With","XMLHttpRequest"),n.setRequestHeader("Content-type","application/x-www-form-urlencoded"),n.setRequestHeader("Connection","close")):n.open("GET",e,!0),n.send(this.postBody)}},window.HP=window.HP||{},HP.modules=HP.modules||{},HP.functions=HP.functions||{},HP.elements=HP.elements||{},HP.params=HP.params||{},HP.modules.ads={banner:"",square:"",promotion:""},window.HP.functions.getQueryStringParameter=function(e,t){var n=new RegExp("([?|&])"+t+"=(.*?)(&|$)","i"),i=e.match(n)||["","",void 0];return i[2]},window.HP.functions.isEmpty=function(e){for(var t in e)if(e.hasOwnProperty(t))return!1;return!0},window.MW=HP,window.Mobileweb=HP.modules,function(){window.HP.functions.getCookie=function(e){var t,n,i=e+"=",r=document.cookie.split(";"),o=0;for(n=r.length;o<n;o++)if(t=r[o],t=t.trim(),0===t.indexOf(i))return t.substring(i.length,t.length);return""},window.HP.functions.setCookie=function(e,t,n){if(n=n||{},void 0!==e){var i=void 0===t||null===t||""===t;i&&(t=0);var r=e+"="+t;void 0!==n.domain&&(r+="; domain="+n.domain),void 0!==n.expires?(n.expires===parseInt(n.expires,10)&&(n.expires=new Date(n.expires).toString()),r+="; expires="+n.expires):i&&(r+="; expires=Thu, 01 Jan 1970 00:00:01 GMT"),void 0!==n.path&&(r+="; path="+n.path),document.cookie=r}},window.HP.functions.cookiesEnabled=function(){return window.document.cookie="huffposttestcookie",window.document.cookie.indexOf("huffposttestcookie")!==-1}}(),HP.modules.storage=function(){var e,t,n,i,r,o,s,a,c,l;return o=function(){try{var e="test";return a=window.localStorage,a.setItem(e,"1"),a.removeItem(e),!0}catch(e){return!1}}(),l=function(e){if("string"==typeof e)try{return JSON.parse(e)}catch(t){return e||void 0}},c=function(e){return JSON.stringify(e)},s=function(){},o?(e=function(e){return l(a.getItem(e))},t=function(e,t){try{a.setItem(e,c(t))}catch(e){}},n=function(e){a.removeItem(e)},i=function(){a.clear()},r=function(e){return void 0!==a.getItem(e)},e("lastPurge")<=1430758101108&&(i(),t("lastPurge",Date.now()))):(e=t=n=i=s,r=function(){return!1}),{enabled:o,get:e,set:t,remove:n,clear:i,has:r}}(),HP.functions.loadCSS=function(e,t,n){var i=window.document.createElement("link"),r=window.document.getElementsByTagName("script");t=t||r[r.length-1],i.rel="stylesheet",i.href=e,i.media="only x",t.parentNode.insertBefore(i,t),setTimeout(function(){i.media=n||"all"})},HP.functions.loadFont=function(){function e(e){var t=document.getElementsByTagName("head")[0],n=document.createElement("style");n.appendChild(document.createTextNode(e)),t.appendChild(n)}var t,n;return n=HP.modules.storage,t=function(t,i,r,o){var s=!1,a=setTimeout(function(){void 0!==r&&e(r.after),s=!0},o||500);void 0!==r&&e(r.before),n.get("file-"+t)===i&&n.get("font-"+t)?(e(n.get("font-"+t)),void 0!==r&&e(r.after),clearTimeout(a)):(n.remove("file-"+t),n.remove("font-"+t),microAjax(i,function(o){n.set("file-"+t,i),n.set("font-"+t,o),s||(e(o),void 0!==r&&e(r.after),clearTimeout(a))}))}}(),window.waitForGlobal=function(e,t,n){var i,r="function"==typeof e?e:function(){return!!window[e]};r()?t():i=setInterval(function(){r()&&(t(),clearInterval(i))},n||50)},window.modulousQueue=function(){function e(e){setTimeout(function(){try{e()}catch(e){}},0)}var t=[],n=!1;return{add:function(i){!n&&t.push(i),n&&e(i)},runAll:function(){var i=0;for(n=!0;i<t.length;i++)e(t[i]);t=[]}}}(),setTimeout(function(){waitForGlobal("jQuery",modulousQueue.runAll)},5e3),function(){function e(e){var t="",n=window.location.href,i=window.HP.functions.getQueryStringParameter(window.location.href,"ab"),r=window.HP.functions.getQueryStringParameter(window.location.href,"g");return void 0===i||void 0===r||i!==e||n.indexOf("staging")===-1&&n.indexOf("local")===-1&&n.indexOf("design")===-1||c[i]&&c[i].variants[r]&&(t=r),t}function t(e,t){if(c[e]){var n=c[e].id,i=c[e].variants[t];void 0===i&&console.error("Could not find AB test variant for value",t),a(e,n,t,i)}else console.error("Could not find AB test",e)}function n(e){var t=Object.keys(c[e].variants),n=c[e].distribution;return n?i(t,n):t[Math.floor(Math.random()*t.length)]}function i(e,t){for(var n=0,i=Math.random(),r=0;r<e.length;r++)if(n+=t[r],n=+n.toFixed(2),i<=n)return e[r]}function r(e){if(localStorage.getItem("abTest-"+e))return JSON.parse(localStorage.getItem("abTest-"+e)).value}function o(e,t){p[e]=t;var n=localStorage["abTest-"+e];("undefined"==typeof n||n.date+f<Date.now())&&d.set("abTest-"+e,{value:t,date:Date.now()})}function s(e){var t=r(e);return void 0===t&&(t=n(e),o(e,t)),t}function a(e,t,n,i){l.push([t,i].join(":")),u.push([e,n].join(":")),o(e,n),window.HP.devMode&&console.log("AB test",e,"set to",n)}var c={},l=[],u=[],d=window.HP.modules.storage,p={};window.HP.modules.AB={set:t,testMap:function(){return l},userGroupTestMap:function(){var e=Object.keys(c).map(function(e){var t=c[e].id,n=c[e].variants[s(e)];if(void 0!==t&&void 0!==n)return t+":"+n});return e},namedTestMap:function(){return u},registerTest:function(e,t,n,i){if(i=i||{},!(i.platforms&&i.platforms.indexOf(HP.params.device)===-1||i.editions&&i.editions.indexOf(HP.params.edition.id)===-1))return c[e]={id:t,variants:n},i.distribution&&(c[e].distribution=i.distribution),c[e]},sample:function(i,r){if(c[i]){var o=r;if(p[i])return p[i];var s=e(i);if(""!==s)o=s;else if(d.enabled||void 0===r){var a="abTest-"+i,l=d.get(a);o=l&&Date.now()<l.date+f?l.value:n(i)}return t(i,o),o}},sampleAnonymous:function(e,t){var n="anonymous-"+e;if(p[n])return p[n];var i=t[Math.floor(Math.random()*t.length)];if(d.enabled){var r="abTest-"+n,o=d.get(r);o&&Date.now()<o.date+f&&(i=parseInt(o.value,10))}return a(n,e,i,i),i}};var f=6048e5,h=HP.modules.AB;HP.params&&HP.params.ab_config&&HP.params.ab_config.forEach(function(e){var t={},n=e.variant_ids.split(","),i=e.variant_distributions.split(",").map(function(e){return parseFloat(e)});e.variant_names.split(",").forEach(function(e,i){t[e]=parseInt(n[i])}),h.registerTest(e.name,parseInt(e.id),t,{distribution:i})})}(),function(){var e=window.HP.modules.AB;e.registerTest("valenciaGroup",1003,{experiment:13,control:14}),e.registerTest("bingKeywordsNew",1005,{no_module:21,with_search:22,reserve:23},{distribution:[.2,.2,.6]}),e.registerTest("dynamicPreview",1022,{enabled:76,control:77}),e.registerTest("standardSplashCards2",1027,{splash:86,standard:87}),e.registerTest("irisPlaylist",1026,{iris_enabled:84,iris_disabled:85})}(),HP.modules.smarten=function(){function e(e){return e.replace(/(^|[-\u2014\s(\["])'/g,"$1\u2018").replace(/'/g,"\u2019").replace(/(^|[-\u2014/\[(\u2018\s])"/g,"$1\u201c").replace(/"/g,"\u201d").replace(/--/g,"\u2014")}function t(e){n(document.querySelectorAll(e))}function n(t){var n=[].reduce.call(t,function(e,t){return e.concat([].slice.call(t.querySelectorAll(":not(script):not(style)")))},[]),i=n.filter(function(e){return e.hasChildNodes()}).reduce(function(e,t){return e.concat([].slice.call(t.childNodes))},[]).filter(function(e){return e.nodeType===document.TEXT_NODE});i.forEach(function(t){t.textContent=e(t.textContent)})}return{selector:t,nodeList:n}}(),function(){window.getVidiblePlayer=function(e){var t=window.fbq,n=window.vidible;e.addEventListener(n.VIDEO_PAUSE,function(){t&&t("trackCustom","Vidible Pause")}),e.addEventListener(n.VIDEO_PLAY,function(){t&&t("trackCustom","Vidible Play")})}}(),HP.modules.cohorts=function(){function e(e){t(e,1)}function t(e,t){var i=n(e);return i[g]=(i[g]||0)+t,s(e,i),i[g]}function n(e){return h.get("cohort-"+e)||{}}function i(e){var t=n(e),i=0;return Object.keys(t).forEach(function(e){i+=t[e]}),i}function r(){return h.get("cohort_lists")||{}}function o(){return Object.keys(r())}function s(e,t){var n=r();n[e]=!0,h.set("cohort_lists",n),h.set("cohort-"+e,t)}function a(e){return parseInt(e,10)}function c(e){return g-30>=e}function l(){return o().reduce(function(e,t){return e[t]=n(t),e},{})}function u(e){var t=n(e),i=Object.keys(t).map(a);i=i.filter(c).forEach(function(e){delete t[e]}),s(e,t)}function d(){return h.get("cohort_start")||Math.min.apply(Math,Object.keys(n("pv")))||g}function p(){return g-d()}function f(){o().forEach(function(e){u(e)})}var h=HP.modules.storage,m=864e5,g=Math.round((new Date).valueOf()/m);return h.get("cohort_start")||h.set("cohort_start",d()),f(),{increment:e,add:t,total:i,cohorts:o,raw:n,allRaw:l,age:p}}();
</script>
<script>
  window.HP.functions.loadFont('ProximaNovaCond', 'http://s.m.huffpost.com/assets/fonts/proxima-nova-condensed-00413bf3c406df7a1c9b6218d92f311ced4bc12564bede72e457e7963bf16f44.css', {
    before: "h1,h2,h3,h4,h5{visibility:hidden;}",
    after: "h1,h2,h3,h4,h5{visibility:visible;}"
  }, 1000);
</script><style>h1,h2,h3,h4,h5{visibility:hidden;}</style>
<script>
  window.HP.functions.loadFont('ProximaNova', 'http://s.m.huffpost.com/assets/fonts/proxima-nova-ecb8d1091c7381b106b2e5ef8cb8b1f39236ba05d1d26e5479c511a5d10abcd2.css', {
    before: "h1,h2,h3,h4,h5{visibility:hidden;}",
    after: "h1,h2,h3,h4,h5{visibility:visible;}"
  }, 500);
</script><style>h1,h2,h3,h4,h5{visibility:hidden;}</style>

<script type="application/ld+json">
{"@type":"NewsArticle","headline":"Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant","articleSection":"Crime","image":{"@type":"ImageObject","url":"http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg","width":2000,"height":1000},"mainEntityOfPage":{"@type":"URL","@id":"http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html"},"speakable":{"@type":"SpeakableSpecification","cssSelector":[".headline__title",".headline__subtitle",".js-entry-text"]},"@context":"http://schema.org","dateCreated":"2015-03-05T07:57:45Z","datePublished":"2015-03-05T07:57:45Z","dateModified":"2015-03-05T07:59:01Z","author":{"@context":"http://schema.org","@type":"Person","name":"Andy Campbell","image":{"@type":"ImageObject","url":"http://s.huffpost.com/contributors/andy-campbell/headshot.jpg"},"url":"http://www.huffingtonpost.com/author/andy-campbell"},"creator":{"@context":"http://schema.org","@type":"Person","name":"Andy Campbell","image":{"@type":"ImageObject","url":"http://s.huffpost.com/contributors/andy-campbell/headshot.jpg"},"url":"http://www.huffingtonpost.com/author/andy-campbell"},"publisher":{"@type":"Organization","name":"HuffPost","url":"http://www.huffingtonpost.com","logo":{"@type":"ImageObject","url":"https://s.m.huffpost.com/hp-amp.gif","width":600,"height":60}},"description":"A Volusia County Sheriff's deputy shot an unarmed man in the face while serving a drug warrant on Wednesday.\r\n\r\nDeputies say Derek Cruice, 26, resisted a...","keywords":["derek-cruice","derek-cruice-dead","todd-raible","volusia-county-sheriff","todd-raible-volusia-county-sheriff","cop-shoots-pot-dealer-in-face","deputy-shoots-man-in-face-serving-warrant","video","police","@health_adhd","@health_depression","@health_gad","@health_hiv","@health_ht","@health_ibs","@health_ost","@health_pain","@health_erectile","@health_models","type: news"],"inLanguage":"en_US","thumbnailUrl":"http://i.huffpost.com/gen/2684130/images/n-DEREK-CRUICE-200x150.jpg","url":"http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html"}

</script>

<!-- Ad Header -->
<script type="text/javascript">
window.adTechCallback = function() {
  
  
  window.adSetAdURL('http://' + (document.domain || 'www.huffingtonpost.com') + (location.port?':'+location.port:'') + '/_uac/2014adpage.html');
  window.tacProp={tags:"kvpagetype=atf:bpage:news:news:slideshow;kvtopic=taser;kventryid=6805902;kvcmsid=hpo:6805902;kveditags=@health_ht:@health_gad:police:@health_pain:todd-raible-volusia-county-sheriff:deputy-shoots-man-in-face-serving-warrant:@health_depression:@health_adhd:@health_models:todd-raible:@health_hiv:@health_erectile:@health_ibs:volusia-county-sheriff:video:derek-cruice:cop-shoots-pot-dealer-in-face:@health_ost:derek-cruice-dead;kvvert=crime;kvblogger=andy-campbell"};
  
  window.tacProp.tags += ( (typeof window.HPAds!=="undefined") && (typeof window.HPAds.adtech_client_side_inf=="function") ) ? window.HPAds.adtech_client_side_inf() : "";
  window.tacProp.tags += ";kvrefd="+window.location.href;
  if (typeof window.adSetOthAT=="function") window.adSetOthAT(window.tacProp.tags);
  
  window.adSetMOAT('1');
  
};
</script>

<script src="http://o.aolcdn.com/ads/adsWrapper.js" async=""></script>
<script>
  
  window.waitForGlobal('adSetAdURL', function(){
  
    window.adTechCallback();
  
  });
  
</script>

<meta name="taboola.category" content="crime">

<script type="text/javascript">
    function doTaboola(taboolaParams) {
        if (typeof taboolaParams == 'undefined') return;
        
        window._taboola = window._taboola || [];
        if (!doTaboola.loaded) {
            _taboola.push({ article:'auto' });
            var _async_script = !function (e, f, u) {
                e.async = 1;
                e.src = u;
                f.parentNode.insertBefore(e, f);
            }(document.createElement('script'), document.getElementsByTagName('script')[0], taboolaParams.loader);
            doTaboola.loaded = true;
        }
        _taboola.push({
            "listenTo": 'render',
            "handler": function(e) {
                var detail = e.detail;
                var i = 0;
                while ( window.jQuery && window.JSON && detail.items[i] ) {
                    var dataItemId = detail.items[i].id;
                    var itemUrl = detail.items[i].url;
                    var taboolaDomNode = window.jQuery("[data-item-id='" + dataItemId + "']");
                    var dataBeacon = {
                        "p": {
                            "mpid": (i+1).toString(),
                            "plid": itemUrl,
                            "mlid": "taboola"
                        }
                    };
                    taboolaDomNode.attr('data-beacon', window.JSON.stringify(dataBeacon));
                    i++;
                }
            }
        });
        var cb = function(o){}; // function(o){ o.container.removeAttribute('id'); }
        _taboola.push({onrender: cb, mode:taboolaParams.mode, container:taboolaParams.container, placement:taboolaParams.placement, target_type:taboolaParams.target_type});
        
    }
</script>
<style>.trc-auto-size { height: auto !important; }</style>
<script>
var _HPJS = (function (_mod) {
  "use strict";

  _mod.loadJS = function (src, options) {
    var options = options || {};
    var ref = window.document.getElementsByTagName("script")[0];
    var script = window.document.createElement("script");
    script.src = src;
    script.async = options.async;
    script.defer = options.defer;
    ref.parentNode.insertBefore(script, ref);
    return script;
  }

  _mod.loadJSLater = function (url, timeout) {
    var ran = false;
    var loader = function () {
      if (ran) {
        return;
      }
      ran = true;
      _mod.loadJS(url, {async:true});
    }
    window.addEventListener ?
      window.addEventListener("load",loader,false) :
      window.attachEvent && window.attachEvent("onload",loader);
    // in case the load event never fires
    setTimeout(loader, timeout || 15000);
  }

  _mod.waitForGlobal = function(identifier, cb, poll) {
    var test = typeof identifier === "function" ? identifier : function() { return !!window[identifier]; };
    var interval;
    if(test()) {
      cb();
    } else {
      interval = setInterval(function(){
        if(test()) {
          cb();
          clearInterval(interval);
        }
      }, poll || 50);
    }
  };

  return _mod;

}(_HPJS || {}));
</script>
<style>h1,h2,h3,h4,h5{visibility:visible;}</style><style>h1,h2,h3,h4,h5{visibility:visible;}</style><script src="//o.aolcdn.com/os/moat/prod/moatuac.js"></script><script src="//o.aolcdn.com/os/moat/prod/moatuac.js"></script><script async="" src="http://cdn-gl.imrworldwide.com/novms/js/2/nlsDcrLite510.min.js#nol_sfcode=dcr&nol_ci=us-500136&nol_apid=P6987AC99-4DBA-45C3-BFE4-9CD649BA8C46&nol_assetname=crime&nol_apn=huffingtonpostCOM&nol_vc=c29&nol_segA=HPMG News&nol_segB=&nol_segC="></script><script src="/cr3wc89sw3-m.js"></script><script src="/cr3wc89sw3-c.js"></script><script type="text/javascript" charset="utf-8" async="" src="https://platform.twitter.com/js/tweet.6352a9eebf6f1aa3bcc0a57d66d2a72d.js"></script><style type="text/css"> .PIN_1498785571564_embed_grid { 
  width: 100%;
  max-width: 257px;
  min-width: 140px;
  display: inline-block;
  box-shadow: inset 0 0 1px #000;
  border-radius: 5px;
  overflow: hidden;
  font: 12px "Helvetica Neue", Helvetica, arial, sans-serif;
  color: rgb(54, 54, 54);
  box-sizing: border-box;
  background: #fff;
  cursor: pointer;
  -webkit-font-smoothing: antialiased;
}
 .PIN_1498785571564_embed_grid * { 
  display: block;
  position: relative;
  font: inherit;
  cursor: inherit;
  box-sizing: inherit;
  margin: 0;
  padding: 0;
  text-align: left;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_hd { 
  height: 55px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_hd .PIN_1498785571564_img { 
  position: absolute;
  top: 10px;
  left: 10px;
  height: 36px;
  width: 36px;
  border-radius: 18px;
  background: transparent url () 0 0 no-repeat;
  background-size: cover;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_hd .PIN_1498785571564_pinner { 
  white-space: nowrap;
  overflow: hidden;
  text-overflow: ellipsis;
  width: 75%;
  position: absolute;
  top: 20px;
  left: 56px;
  font-size: 14px;
  font-weight: bold;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_bd { 
  padding: 0 10px;
  -moz-scrollbars: none;
  -ms-overflow-style: none;
  overflow-x: hidden;
  overflow-y: auto;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_bd .PIN_1498785571564_ct { 
  width: 100%;
  height: auto;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  display: inline-block;
  width: 100%;
  padding: 1px;
  vertical-align: top;
  min-width: 60px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col .PIN_1498785571564_img { 
  margin: 0;
  display: inline-block;
  width: 100%;
  background: transparent url() 0 0 no-repeat;
  background-size: cover;
  box-shadow: inset 0 0 1px #000;
  border-radius: 2px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft { 
  padding: 11px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button { 
  border-radius: 3px;
  text-align: center;
  box-shadow: inset 0 0 1px #888;
  background-color: #efefef;
  position: relative;
  display: block;
  overflow: hidden;
  height: 33px;
  width: 100%;
  min-width: 70px;
  padding: 0 3px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button .PIN_1498785571564_label { 
  position: absolute;
  left: 0;
  width: 100%;
  text-align: center;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button .PIN_1498785571564_label.PIN_1498785571564_top { 
  top: 0;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button .PIN_1498785571564_label.PIN_1498785571564_bottom { 
  bottom: 0;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button .PIN_1498785571564_label .PIN_1498785571564_string { 
  white-space: pre;
  color: #746d6a;
  font-size: 13px;
  font-weight: bold;
  vertical-align: top;
  display: inline-block;
  height: 33px;
  line-height: 33px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button .PIN_1498785571564_label .PIN_1498785571564_logo { 
  display: inline-block;
  vertical-align: top;
  height: 33px;
  width: 65px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTZweCIgd2lkdGg9IjY0cHgiIHZpZXdCb3g9IjAgMCA2NCAxNiI+PGc+PHBhdGggZD0iTTMzLjIxNSw3LjY2NyBDMzMuODg4LDcuNjY3IDM0LjEyMiw4LjA3NSAzNC4xMjIsOC41NjIgQzM0LjEyMiw5LjM4NiAzMy40ODIsOS44ODkgMzIuNDgxLDkuODg5IEMzMi4xMjQsOS44ODkgMzEuNjI1LDkuODA0IDMxLjMyOSw5LjcwNCBDMzEuNDUyLDguOTQ4IDMxLjk0LDcuNjY3IDMzLjIxNSw3LjY2NyBMMzMuMjE1LDcuNjY3IFogTTQ3LjY2OSw3LjY2NyBDNDguMjg0LDcuNjY3IDQ4LjU3MSw4LjA3NSA0OC41NzEsOC41NjIgQzQ4LjU3MSw5LjM4NiA0Ny45MzYsOS44ODkgNDYuOTMxLDkuODg5IEM0Ni41NzIsOS44ODkgNDYuMDcsOS44MDQgNDUuNzcyLDkuNzA0IEM0NS44OTUsOC45NDggNDYuNDM5LDcuNjY3IDQ3LjY1OSw3LjY2NyBMNDcuNjY5LDcuNjY3IFogTTYyLjkwNiw3LjczNiBDNjMuMTExLDcuNzM2IDYzLjI0NSw3LjY1IDYzLjI4Niw3LjQ1NCBDNjMuMzQ3LDcuMTY2IDYzLjQ3LDYuNjY0IDYzLjUxMSw2LjUzMSBDNjMuNTUyLDYuMzU2IDYzLjQ1LDYuMjIzIDYzLjI1NSw2LjIyMyBMNjEuNzM3LDYuMjIzIEM2MS44NCw1Ljc4MiA2Mi4yNzEsNC4zNjcgNjIuMjcxLDMuNDQ0IEM2Mi4yNzEsMS41OTkgNjAuNzAyLDAuMDMgNTguNDg3LDAuMDMgQzU1Ljk3NSwwLjAzIDU0LjM0NCwxLjcxMSA1NC4zNDQsNC4wOCBDNTQuMzQ0LDQuNzY3IDU0LjYwMSw1LjU3NyA1NC45MTksNi4wNjkgQzU0Ljc4NSw2LjA1OSA1NC42NDIsNi4wNTQgNTQuNTA4LDYuMDU0IEM1Mi42MDEsNi4wNTQgNTEuNDAyLDcuMDkgNTEuNDAyLDguNTkzIEM1MS40MDIsMTAuMDA4IDUyLjUwOSwxMC41NTkgNTMuNTY1LDEwLjkyIEM1NC40ODgsMTEuMjQ5IDU1LjM0OSwxMS40IDU1LjM0OSwxMi4xMiBDNTUuMzQ5LDEyLjYxMiA1NC44NzgsMTIuODk5IDUzLjk1NSwxMi44OTkgQzUyLjg1OCwxMi44OTkgNTIuMjUzLDEyLjM4NyA1Mi4wOTksMTIuMTkyIEM1Mi4yMDEsMTIuMTEgNTIuMzU1LDExLjkxNSA1Mi4zNTUsMTEuNTk1IEM1Mi4zNTUsMTEuMTQ0IDUyLjAzOSwxMC43NzUgNTEuNDQ5LDEwLjc3NSBDNTAuOTQ4LDEwLjc3NSA1MC41MjYsMTEuMTg1IDUwLjQxOSwxMS43MTggQzQ5LjYxOSwxMi4zNzQgNDguNzQzLDEyLjgwMiA0Ny41MDMsMTIuODAyIEM0Ni4yODMsMTIuODAyIDQ1LjY0NywxMi4xNjYgNDUuNjQ3LDEwLjgxMyBDNDUuOTM0LDEwLjkwMiA0Ni42ODMsMTEuMDA4IDQ3LjE3NSwxMS4wMDggQzQ5LjI0NiwxMS4wMDggNTAuNjUxLDEwLjA2NCA1MC42NTEsOC4xODQgQzUwLjY1MSw3LjE3MiA0OS43NjksNi4xMDggNDguMDA1LDYuMTA4IEM0NS42OTIsNi4xMDggNDQuMzk5LDcuNTc1IDQzLjgzMiw5LjE4NCBMNDMuMzU1LDkuMDc2IEM0My40ODksOC43MTcgNDMuNTc3LDguMzAxIDQzLjU3Nyw3Ljg0NSBDNDMuNTc3LDcuMDQ1IDQzLjA4NSw2LjEwNyA0MS43NjIsNi4xMDcgQzQwLjg0NSw2LjEwNyAzOS45NzQsNi43MDggMzkuMzcsNy42MSBMMzkuNjI2LDYuNjA2IEMzOS42NjQsNi40NTIgMzkuNTk5LDYuMjgzIDM5LjM1OSw2LjI4MyBMMzguMDA2LDYuMjgzIEMzNy44MTQsNi4yODMgMzcuNjc4LDYuMzU0IDM3LjYyLDYuNTcgTDM2LjY2NywxMC4zMTIgQzM2LjM5LDExLjQzIDM0LjkxMywxMi44MTQgMzMuMDU3LDEyLjgxNCBDMzEuODM1LDEyLjgxNCAzMS4yMDEsMTIuMTM4IDMxLjIwMSwxMC43ODQgQzMxLjUyMiwxMC45MTIgMzIuMjM1LDExLjAyIDMyLjcyNiwxMS4wMiBDMzQuODc5LDExLjAyIDM2LjIwNSwxMC4wODEgMzYuMjA1LDguMTk3IEMzNi4yMDUsNy4xODcgMzUuMzEzLDYuMTI0IDMzLjU2LDYuMTI0IEMzMC45NDUsNi4xMjQgMjkuMzQ1LDguMTQ4IDI5LjA3OSwxMC40NDEgQzI4Ljk2NiwxMS4zOTQgMjguMTU2LDEyLjcwMyAyNy4yODQsMTIuNzAzIEMyNi44NjQsMTIuNzAzIDI2LjYyOCwxMi40MzYgMjYuNjI4LDExLjk4NSBDMjYuNjI4LDExLjU5NSAyNi45MDEsMTAuNjMxIDI3LjIwMiw5LjQzMiBDMjcuMzA1LDkuMDMyIDI3LjQ1Niw4LjQxNiAyNy42MjYsNy43NCBMMjkuMDUxLDcuNzQgQzI5LjI1OSw3Ljc0IDI5LjM4Miw3LjY0NyAyOS40MzEsNy40NTMgQzI5LjQ5OCw3LjE2NSAyOS42MjIsNi42NzMgMjkuNjU2LDYuNTQgQzI5LjcwNCw2LjM2NiAyOS42MDEsNi4yMjcgMjkuNDA0LDYuMjI3IEwyOC4wMDksNi4yMjcgTDI4LjY2NiwzLjYwNiBDMjguNzI3LDMuMzQ2IDI4LjUwOCwzLjE5NiAyOC4yODEsMy4yNDQgQzI4LjI4MSwzLjI0NCAyNy4yMDQsMy40NTUgMjcuMDA2LDMuNDk4IEMyNi44MDQsMy41MzkgMjYuNjQ3LDMuNjQ2IDI2LjU3NSwzLjkzOSBMMjYuMDExLDYuMjI1IEwyNC45MDQsNi4yMjUgQzI0LjY5MSw2LjIyNSAyNC41NjgsNi4zMTIgMjQuNTIyLDYuNTA5IEMyNC40NTMsNi43OTIgMjQuMzI3LDcuMjg4IDI0LjI5Myw3LjQyMiBDMjQuMjQ1LDcuNiAyNC4zNSw3LjczNSAyNC41NSw3LjczNSBMMjUuNjQ1LDcuNzM1IEMyNS42MzgsNy43NjkgMjUuMjUsOS4yNDIgMjQuOTM3LDEwLjU2NSBDMjQuNzg5LDExLjIwNCAyNC4yOTMsMTIuNjkxIDIzLjQ4MSwxMi42OTEgQzIyLjk5OSwxMi42OTEgMjIuNzk0LDEyLjQ0NSAyMi43OTQsMTEuOTIyIEMyMi43OTQsMTEuMDk2IDIzLjYwNiw5LjAzMSAyMy42MDYsOC4wOTMgQzIzLjYwNiw2Ljg0MiAyMi45MzUsNi4xMTQgMjEuNTU5LDYuMTE0IEMyMC42OTMsNi4xMTQgMTkuNzk1LDYuNjc4IDE5LjQxNCw3LjE3MSBDMTkuNDE0LDcuMTcxIDE5LjUyOSw2Ljc3MSAxOS41Nyw2LjYxNyBDMTkuNjEyLDYuNDUzIDE5LjUyOSw2LjI5MiAxOS4zMTQsNi4yOTIgTDE3Ljk3MSw2LjI5MiBDMTcuNjg0LDYuMjkyIDE3LjYwOSw2LjQ0NiAxNy41NjcsNi42MTIgTDE2LjU5OCwxMC40MDYgQzE2LjI3NSwxMS42NTcgMTUuNTAzLDEyLjcwMyAxNC42OCwxMi43MDMgQzE0LjI1LDEyLjcwMyAxNC4wNjUsMTIuNDM2IDE0LjA2NSwxMS45NzQgQzE0LjA2NSwxMS41ODUgMTQuMzEzLDEwLjYyMSAxNC42MTksOS40MjEgQzE0Ljk4Myw3Ljk2NSAxNS4zMDYsNi43NjYgMTUuMzQyLDYuNjIyIEMxNS4zODgsNi40NDQgMTUuMzA3LDYuMjg4IDE1LjA3NSw2LjI4OCBMMTMuNzIyLDYuMjg4IEMxMy40NzYsNi4yODggMTMuMzg4LDYuNDE0IDEzLjMzOCw2LjU5NSBDMTMuMzM4LDYuNTk1IDEyLjk1OSw4LjAzNCAxMi41NTcsOS42NDEgQzEyLjI3LDEwLjgwNiAxMS45NDIsMTEuOTkzIDExLjk0MiwxMi41NDkgQzExLjk0MiwxMy41NDMgMTIuMzg2LDE0LjI5MiAxMy42MTYsMTQuMjkyIEMxNC41NTksMTQuMjkyIDE1LjMwOCwxMy44MSAxNS44ODIsMTMuMjAyIEwxNS43MzksMTMuNzY2IEMxNS42ODcsMTMuOTU3IDE1Ljc0OSwxNC4xMjUgMTUuOTY3LDE0LjEyNSBMMTcuMzUyLDE0LjEyNSBDMTcuNTkzLDE0LjEyNSAxNy42ODUsMTQuMDI3IDE3LjczNiwxMy44MTcgTDE4LjgzMyw5LjU1MSBDMTkuMTAzLDguNDUxIDE5Ljc3Nyw3LjcyNiAyMC43Myw3LjcyNiBDMjEuMTgxLDcuNzI2IDIxLjU3MSw4LjAyNCAyMS41MjYsOC42MDUgQzIxLjQ3NSw5LjI0NCAyMC43MDYsMTEuNTM4IDIwLjcwNiwxMi41NDUgQzIwLjcwNiwxMy4zMDMgMjAuOTg5LDE0LjI4OCAyMi40MjIsMTQuMjg4IEMyMy4zOTYsMTQuMjg4IDI0LjExNCwxMy44MTYgMjQuNjM3LDEzLjIxMSBDMjQuODMyLDEzLjg0OSAyNS4zMzQsMTQuMjggMjYuMjY3LDE0LjI4IEMyNy44MTYsMTQuMjggMjguNzQ5LDEzLjM2IDI5LjMwMiwxMi40MzggQzI5Ljc1MSwxMy41MTUgMzAuODE1LDE0LjI4NCAzMi4zNzksMTQuMjg0IEMzMy45NjgsMTQuMjg0IDM1LjIwNCwxMy42MSAzNi4wNzYsMTIuNjk3IEwzNS44MDYsMTMuNzgxIEMzNS43NTUsMTMuOTczIDM1Ljg0MSwxNC4xMjUgMzYuMDY3LDE0LjEyNSBMMzcuNDM3LDE0LjEyNSBDMzcuNjIxLDE0LjEyNSAzNy43NTcsMTQuMDI4IDM3LjgwNiwxMy44MyBDMzcuODI4LDEzLjczNCAzOC4wMjcsMTIuOTY5IDM4LjMwOCwxMS44ODIgQzM4Ljg0Miw5LjgwMSAzOS43MDMsNy42NDIgNDAuOTkzLDcuNjQyIEM0MS40NDksNy42NDIgNDEuNjM3LDcuODk3IDQxLjYzNyw4LjI5OCBDNDEuNjM3LDguNDg3IDQxLjU4MSw4LjY0OSA0MS41MzQsOC43MzkgQzQwLjkwOSw4LjYxNCA0MC40MDQsOC45MjIgNDAuNDA0LDkuNTkgQzQwLjQwNCwxMC4wMjcgNDAuODYzLDEwLjQyMyA0MS40ODEsMTAuNDIzIEM0MS45NTMsMTAuNDIzIDQyLjM0NSwxMC4zMDggNDIuNjU3LDEwLjA5NyBDNDIuOTU0LDEwLjE1OSA0My4yNjIsMTAuMjI1IDQzLjU4LDEwLjI5NyBDNDMuNTM5LDEwLjYxIDQzLjUwOCwxMC45MTggNDMuNTA4LDExLjIwNyBDNDMuNTA4LDEyLjg3OSA0NC43NjksMTQuMjk3IDQ2LjkwMiwxNC4yOTcgQzQ4LjU4NCwxNC4yOTcgNDkuNjUsMTMuNzMzIDUwLjY2NSwxMi44OTggQzUxLjE0NywxMy42NzggNTIuMzQ3LDE0LjMwMyA1My45MjYsMTQuMzAzIEM1Ni4wOSwxNC4zMDMgNTcuMzIsMTMuMTc1IDU3LjMyLDExLjY4OCBDNTcuMzIsMTAuMzM1IDU2LjIxMyw5Ljg0MyA1NS4wNzQsOS40MzMgQzU0LjE1Miw5LjA5NCA1My4zNzIsOC45MiA1My4zNzIsOC4yNjQgQzUzLjM3Miw3LjcyIDUzLjgwMyw3LjQ3NCA1NC41MjEsNy40NzQgQzU0Ljk5Miw3LjQ3NCA1NS4zNDEsNy41NzcgNTUuNTM2LDcuNjI4IEM1NS43Miw4LjA4OSA1Ni4xMSw4LjYxMiA1Ni43NzcsOC42MTIgQzU3LjMzNSw4LjYxMiA1Ny41ODcsOC4xNzEgNTcuNTg3LDcuNzYxIEM1Ny41ODcsNi42MDIgNTUuNjU3LDYuNDc5IDU1LjY1NywzLjk5OCBDNTUuNjU3LDIuNDUgNTYuNTMzLDEuMTc4IDU4LjM0MywxLjE3OCBDNTkuNjAzLDEuMTc4IDYwLjQyNSwxLjk3OCA2MC40MjUsMy4yNiBDNjAuNDI1LDQuMzE2IDU5Ljc3OSw2LjIxMyA1OS43NzksNi4yMTMgTDU4LjY4Miw2LjIxMyBDNTguNDcyLDYuMjEzIDU4LjM0Nyw2LjI5NSA1OC4yOTksNi40OSBMNTguMDcxLDcuNDAyIEM1OC4wMjcsNy41NzcgNTguMTI4LDcuNzEgNTguMzI3LDcuNzEgTDU5LjM5Myw3LjcxIEM1OS42MTcsNy43NDkgNTguMjg2LDEyLjQ1IDU4LjI4NiwxMi41MjkgQzU4LjI4NiwxMy41MjQgNTguODQ4LDE0LjI3MiA2MC4wNzUsMTQuMjcyIEM2MS44MDgsMTQuMjcyIDYyLjg5NSwxMy4xNDQgNjMuMzY3LDExLjUwNCBDNjMuNDA0LDExLjM3MyA2My4zMzMsMTEuMjcgNjMuMjAzLDExLjI3IEw2Mi41MzYsMTEuMjcgQzYyLjQwMywxMS4yNyA2Mi4zMjcsMTEuMzQ0IDYyLjI5NCwxMS40NzUgQzYyLjE0OCwxMi4wNiA2MS43NzgsMTIuNjc1IDYxLjA0MywxMi42NzUgQzYwLjYyMywxMi42NzUgNjAuMzg1LDEyLjQwOCA2MC4zODUsMTEuOTUxIEM2MC4zODUsMTEuNTYxIDYxLjM5LDcuNzA5IDYxLjM5LDcuNzA5IEw2Mi45MDYsNy43MzYgTDYyLjkwNiw3LjczNiBaIE0xNi4wOTksNC4xMyBDMTYuMDUyLDQuODQ1IDE1LjQ1Myw1LjQyNiAxNC43NjMsNS40MjYgQzE0LjA3Niw1LjQyNiAxMy41NTQsNC44NDUgMTMuNjA1LDQuMTI5IEMxMy42NTYsMy40MTQgMTQuMjUxLDIuODMzIDE0Ljk0LDIuODMzIEMxNS42MjksMi44MzMgMTYuMTUsMy40MDkgMTYuMDk5LDQuMTI1IEwxNi4wOTksNC4xMyBaIE02LjQ1LDAgQzIuMjQ2LDAgMC4wMSwyLjgyNSAwLjAxLDUuOTAyIEMwLDcuMzQyIDAuNzY0LDkuMTIyIDEuOTc5LDkuNjkgQzIuMTY0LDkuNzc1IDIuMjYyLDkuNzM3IDIuMzA0LDkuNTU5IEMyLjMzNSw5LjQyMyAyLjUwMSw4Ljc2MiAyLjU3NSw4LjQ1MyBDMi42MDYsOC4zNTUgMi41OTUsOC4yNjkgMi41MTMsOC4xNzIgQzIuMTEzLDcuNjg2IDEuNzk1LDYuNzg4IDEuNzk1LDUuOTQ5IEMxLjc5NSwzLjggMy40MjYsMS43MiA2LjE5NCwxLjcyIEM4LjU5NCwxLjcyIDEwLjI2NSwzLjM1IDEwLjI2NSw1LjY4NSBDMTAuMjY1LDguMzIyIDguOTQyLDEwLjE0OCA3LjIwOSwxMC4xNDggQzYuMjU2LDEwLjE0OCA1LjUzOCw5LjM1OSA1Ljc2NCw4LjM4NyBDNi4wNDEsNy4yMjggNi41NzQsNS45NzcgNi41NzQsNS4xMzYgQzYuNTc0LDQuMzg4IDYuMTc0LDMuNzYyIDUuMzQzLDMuNzYyIEM0LjM2OSwzLjc2MiAzLjU4LDQuNzY3IDMuNTgsNi4xMzEgQzMuNTgsNi45OTIgMy44NzcsNy41NzcgMy44NzcsNy41NzcgTDIuNzA3LDEyLjQyOCBDMi40MDcsMTMuNzIgMi43NDgsMTUuODExIDIuNzc3LDE1Ljk5MyBDMi43OTIsMTYuMDkgMi45LDE2LjEyNCAyLjk2MSwxNi4wNDEgQzMuMDU4LDE1LjkxNSA0LjIyMywxNC4xNjkgNC41NTEsMTIuOTA2IEw1LjE1NiwxMC41ODggQzUuNDc3LDExLjE5MyA2LjQwNCwxMS43MDYgNy4zOTEsMTEuNzA2IEMxMC4zMzEsMTEuNzA2IDEyLjQ1LDkuMDAzIDEyLjQ1LDUuNjQ0IEMxMi40NSwyLjQyNCA5LjgyMiwwLjAxNCA2LjQ0MSwwLjAxNCBMNi40NSwwIFoiIGZpbGw9IiNiZDA4MWMiPjwvcGF0aD48L2c+PC9zdmc+) 50% 50% no-repeat;
  background-size: 60px 15px;
}
 .PIN_1498785571564_embed_grid .PIN_1498785571564_ft .PIN_1498785571564_button:hover { 
  box-shadow: inset 0 0 1px #000;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_noscroll .PIN_1498785571564_bd { 
  overflow: hidden;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_board .PIN_1498785571564_hd .PIN_1498785571564_pinner { 
  top: 10px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_board .PIN_1498785571564_hd .PIN_1498785571564_board { 
  white-space: nowrap;
  overflow: hidden;
  text-overflow: ellipsis;
  width: 75%;
  position: absolute;
  bottom: 10px;
  left: 56px;
  color: #363636;
  font-size: 12px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c2 { 
  max-width: 494px;
  min-width: 140px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c2 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 50%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c3 { 
  max-width: 731px;
  min-width: 200px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c3 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 33.33%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c4 { 
  max-width: 968px;
  min-width: 260px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c4 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 25%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c5 { 
  max-width: 1205px;
  min-width: 320px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c5 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 20%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c6 { 
  max-width: 1442px;
  min-width: 380px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c6 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 16.66%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c7 { 
  max-width: 1679px;
  min-width: 440px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c7 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 14.28%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c8 { 
  max-width: 1916px;
  min-width: 500px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c8 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 12.5%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c9 { 
  max-width: 2153px;
  min-width: 560px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c9 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 11.11%;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c10 { 
  max-width: 2390px;
  min-width: 620px;
}
 .PIN_1498785571564_embed_grid.PIN_1498785571564_c10 .PIN_1498785571564_bd .PIN_1498785571564_ct .PIN_1498785571564_col { 
  width: 10%;
}
 .PIN_1498785571564_embed_pin { 
  width: 100%;
  min-width: 160px;
  max-width: 236px;
  display: inline-block;
  box-sizing: border-box;
  box-shadow: inset 0 0 1px #000;
  border-radius: 5px;
  overflow: hidden;
  font: 12px "Helvetica Neue", Helvetica, arial, sans-serif;
  color: #363636;
  background: #fff;
  cursor: pointer;
  -webkit-font-smoothing: antialiased;
}
 .PIN_1498785571564_embed_pin * { 
  display: block;
  position: relative;
  font: inherit;
  cursor: inherit;
  box-sizing: inherit;
  margin: 0;
  padding: 0;
  text-align: left;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd { 
  border-bottom: 1px solid rgba(0, 0, 0, 0.09);
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd { 
  border-bottom: 1px solid rgba(0, 0, 0, 0.09);
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container { 
  width: 100%;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_img,  .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_iframe,  .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_video { 
  width: 100%;
  height: 100%;
  position: absolute;
  left: 0;
  background: transparent url() 0 0 no-repeat;
  background-size: cover;
  border: none;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_native { 
  height: 64px;
  width: 64px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iNjNweCIgd2lkdGg9IjYzcHgiIHZpZXdCb3g9IjAgMCA2NCA2NCI+PGc+PHBhdGggZD0iTSAzMiAzMiBtIC0zMiwgMCBhIDMxLCAzMSAwIDEsIDAgNjMsIDAgYSAzMSwgMzEgMCAxLDAgLTYzLCAwIHoiIGZpbGw9IiMwMDAiPjwvcGF0aD48cGF0aCBkPSJNIDI1IDE5IEwgNDggMzIgIEwgMjUgNDUgWiIgZmlsbD0iI2ZmZiI+PC9wYXRoPjwvZz48L3N2Zz4=) 0 0 no-repeat;
  position: absolute;
  top: 50%;
  left: 50%;
  margin-top: -32px;
  margin-left: -32px;
  opacity: .75;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_control { 
  width: 50px;
  height: 24px;
  position: absolute;
  bottom: 12px;
  left: 12px;
  background: rgba(0, 0, 0, .4);
  color: #fff;
  box-shadow: 0 0 2px rgba(0, 0, 0, .2);
  border: 2px solid rgba(255, 255, 255, .68);
  border-radius: 13px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_control.PIN_1498785571564_paused::after,  .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_control.PIN_1498785571564_playing::after { 
  position: absolute;
  top: 0;
  left: 0;
  width: 100%;
  height: 100%;
  font-size: 10px;
  line-height: 19px;
  white-space: pre;
  font-weight: bold;
  font-style: normal;
  text-align: center;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_control.PIN_1498785571564_paused::after { 
  content: "▶ GIF";
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_control.PIN_1498785571564_playing::after { 
  content: "❙ ❙ GIF";
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_repin { 
  position: absolute;
  top: 10px;
  left: 10px;
  height: 28px;
  width: auto;
  color: #fff;
  background: #bd081c url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMzBweCIgd2lkdGg9IjMwcHgiIHZpZXdCb3g9Ii0xIC0xIDMxIDMxIj48Zz48cGF0aCBkPSJNMjkuNDQ5LDE0LjY2MiBDMjkuNDQ5LDIyLjcyMiAyMi44NjgsMjkuMjU2IDE0Ljc1LDI5LjI1NiBDNi42MzIsMjkuMjU2IDAuMDUxLDIyLjcyMiAwLjA1MSwxNC42NjIgQzAuMDUxLDYuNjAxIDYuNjMyLDAuMDY3IDE0Ljc1LDAuMDY3IEMyMi44NjgsMC4wNjcgMjkuNDQ5LDYuNjAxIDI5LjQ0OSwxNC42NjIiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2ZmZiIgc3Ryb2tlLXdpZHRoPSIxIj48L3BhdGg+PHBhdGggZD0iTTE0LjczMywxLjY4NiBDNy41MTYsMS42ODYgMS42NjUsNy40OTUgMS42NjUsMTQuNjYyIEMxLjY2NSwyMC4xNTkgNS4xMDksMjQuODU0IDkuOTcsMjYuNzQ0IEM5Ljg1NiwyNS43MTggOS43NTMsMjQuMTQzIDEwLjAxNiwyMy4wMjIgQzEwLjI1MywyMi4wMSAxMS41NDgsMTYuNTcyIDExLjU0OCwxNi41NzIgQzExLjU0OCwxNi41NzIgMTEuMTU3LDE1Ljc5NSAxMS4xNTcsMTQuNjQ2IEMxMS4xNTcsMTIuODQyIDEyLjIxMSwxMS40OTUgMTMuNTIyLDExLjQ5NSBDMTQuNjM3LDExLjQ5NSAxNS4xNzUsMTIuMzI2IDE1LjE3NSwxMy4zMjMgQzE1LjE3NSwxNC40MzYgMTQuNDYyLDE2LjEgMTQuMDkzLDE3LjY0MyBDMTMuNzg1LDE4LjkzNSAxNC43NDUsMTkuOTg4IDE2LjAyOCwxOS45ODggQzE4LjM1MSwxOS45ODggMjAuMTM2LDE3LjU1NiAyMC4xMzYsMTQuMDQ2IEMyMC4xMzYsMTAuOTM5IDE3Ljg4OCw4Ljc2NyAxNC42NzgsOC43NjcgQzEwLjk1OSw4Ljc2NyA4Ljc3NywxMS41MzYgOC43NzcsMTQuMzk4IEM4Ljc3NywxNS41MTMgOS4yMSwxNi43MDkgOS43NDksMTcuMzU5IEM5Ljg1NiwxNy40ODggOS44NzIsMTcuNiA5Ljg0LDE3LjczMSBDOS43NDEsMTguMTQxIDkuNTIsMTkuMDIzIDkuNDc3LDE5LjIwMyBDOS40MiwxOS40NCA5LjI4OCwxOS40OTEgOS4wNCwxOS4zNzYgQzcuNDA4LDE4LjYyMiA2LjM4NywxNi4yNTIgNi4zODcsMTQuMzQ5IEM2LjM4NywxMC4yNTYgOS4zODMsNi40OTcgMTUuMDIyLDYuNDk3IEMxOS41NTUsNi40OTcgMjMuMDc4LDkuNzA1IDIzLjA3OCwxMy45OTEgQzIzLjA3OCwxOC40NjMgMjAuMjM5LDIyLjA2MiAxNi4yOTcsMjIuMDYyIEMxNC45NzMsMjIuMDYyIDEzLjcyOCwyMS4zNzkgMTMuMzAyLDIwLjU3MiBDMTMuMzAyLDIwLjU3MiAxMi42NDcsMjMuMDUgMTIuNDg4LDIzLjY1NyBDMTIuMTkzLDI0Ljc4NCAxMS4zOTYsMjYuMTk2IDEwLjg2MywyNy4wNTggQzEyLjA4NiwyNy40MzQgMTMuMzg2LDI3LjYzNyAxNC43MzMsMjcuNjM3IEMyMS45NSwyNy42MzcgMjcuODAxLDIxLjgyOCAyNy44MDEsMTQuNjYyIEMyNy44MDEsNy40OTUgMjEuOTUsMS42ODYgMTQuNzMzLDEuNjg2IiBmaWxsPSIjYmQwODFjIj48L3BhdGg+PC9nPjwvc3ZnPg==) 5px 50% no-repeat;
  background-size: 18px 18px;
  text-indent: 28px;
  font: 14px/28px "Helvetica Neue", Helvetica, Arial, "sans-serif";
  font-weight: bold;
  border-radius: 4px;
  padding: 0 6px 0 0;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_repin:hover { 
  background-color: #aa0719;
  box-shadow: none;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source { 
  height: 38px;
  border-bottom: 1px solid rgba(0, 0, 0, 0.09);
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_img { 
  position: absolute;
  top: 50%;
  margin-top: -8px;
  left: 10px;
  height: 16px;
  width: 16px;
  background: transparent url() 0 0 no-repeat;
  background-size: 16px 16px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_domain { 
  line-height: 38px;
  max-width: 75%;
  white-space: nowrap;
  overflow: hidden;
  text-overflow: ellipsis;
  font-weight: bold;
  color: #b9b9b9;
  position: absolute;
  top: 0px;
  left: 35px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source.PIN_1498785571564_nofav .PIN_1498785571564_img { 
  display: none;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source.PIN_1498785571564_nofav .PIN_1498785571564_domain { 
  max-width: 85%;
  left: 10px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_menu { 
  height: inherit;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_menu .PIN_1498785571564_toggle { 
  position: absolute;
  top: 0;
  right: 5px;
  height: inherit;
  width: 30px;
  border-bottom: 1px solid rgba(0, 0, 0, 0.09);
  background: #fff url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iNXB4IiB3aWR0aD0iMjBweCIgdmlld0JveD0iMCAwIDIwIDUiPjxnPjxwYXRoIGQ9Ik0xNy41LDUgQzE4Ljg4MSw1IDIwLDMuODgxIDIwLDIuNSBDMjAsMS4xMTkgMTguODgxLDAgMTcuNSwwIEMxNi4xMTksMCAxNSwxLjExOSAxNSwyLjUgQzE1LDMuODgxIDE2LjExOSw1IDE3LjUsNSBaIE0xMCw1IEMxMS4zOCw1IDEyLjUsMy44ODEgMTIuNSwyLjUgQzEyLjUsMS4xMTkgMTEuMzgsMCAxMCwwIEM4LjYyLDAgNy41LDEuMTE5IDcuNSwyLjUgQzcuNSwzLjg4MSA4LjYyLDUgMTAsNSBNMi41LDUgQzMuODgxLDUgNSwzLjg4MSA1LDIuNSBDNSwxLjExOSAzLjg4MSwwIDIuNSwwIEMxLjEyLDAgMCwxLjExOSAwLDIuNSBDMCwzLjg4MSAxLjEyIDUsMi41LDUgWiIgZmlsbD0iI2I1YjViNSI+PC9wYXRoPjwvZz48L3N2Zz4=) 50% 50% no-repeat;
  background-size: fill;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_menu .PIN_1498785571564_dropdown { 
  display: none;
  border-radius: 5px;
  box-shadow: 0 1px 3px rgba(0, 0, 0, .33);
  font-weight: bold;
  font-size: 12px;
  background: #fff;
  text-align: left;
  padding: 10px;
  position: absolute;
  max-width: 195px;
  right: 10px;
  top: 39px;
  z-index: 1;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_menu .PIN_1498785571564_dropdown::after { 
  content: "";
  position: absolute;
  top: -6px;
  right: 1px;
  border-bottom: 8px solid #fff;
  border-right: 8px solid transparent;
  border-left: 8px solid transparent;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_menu .PIN_1498785571564_dropdown::before { 
  content: "";
  position: absolute;
  top: -7px;
  right: 2px;
  border-bottom: 7px solid #e2e2e2;
  border-right: 7px solid transparent;
  border-left: 7px solid transparent;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_description { 
  font-size: 14px;
  line-height: 17px;
  margin: 8px 12px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_attribution { 
  height: 22px;
  margin-top: 5px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_attribution .PIN_1498785571564_img { 
  position: absolute;
  top: 50%;
  margin-top: -8px;
  left: 10px;
  height: 16px;
  width: 16px;
  background: transparent url() 0 0 no-repeat;
  background-size: 16px 16px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_attribution .PIN_1498785571564_by { 
  line-height: 22px;
  max-width: 75%;
  white-space: nowrap;
  overflow: hidden;
  text-overflow: ellipsis;
  font-size: 11px;
  font-weight: bold;
  color: #b9b9b9;
  position: absolute;
  top: 0px;
  left: 35px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_stats { 
  height: 24px;
  line-height: 24px;
  margin: 0 0 0 10px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_bd .PIN_1498785571564_stats .PIN_1498785571564_repins { 
  padding-left: 17px;
  padding-right: 10px;
  color: #a8a8a8;
  font-size: 11px;
  font-weight: bold;
  display: inline-block;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTRweCIgd2lkdGg9IjE0cHgiIHZpZXdCb3g9IjAgMCAxNCAxNCI+PGc+PHBhdGggZD0iTTExLjk3OSw2Ljg1OSBMMTMuOTksNS4wMTEgTDExLjI0MSw1LjAxMSBMMTAuNDg2LDAgTDkuNzM5LDUuMDExIEw3LDUuMDExIEw4Ljk4Niw2Ljg1OCBMOC45ODYsMTEuMDE3IEM4LjUwNSwxMC45ODUgOC4xNDMsMTEuMDEyIDguMTQzLDExLjIxMyBDOC4xNDMsMTEuNjg3IDguNTAzLDEyLjAwMSA4Ljk4NiwxMi4wMDEgTDExLjk4MiwxMiBDMTIuNDY1LDEyIDEyLjg4OCwxMS42ODYgMTIuODg4LDExLjIxMyBDMTIuODg4LDExLjAxMSAxMi40NjUsMTAuOTg1IDExLjk4MiwxMS4wMTcgTDExLjk3OSw2Ljg1OSBaIE00Ljk3OSw3LjE0MiBMNi45OSw4Ljk5IEw0LjI0MSw4Ljk5IEwzLjQ4NiwxNC4wMDEgTDIuNzM5LDguOTkgTDAsOC45OSBMMS45ODYsNy4xNDMgTDEuOTg2LDIuOTg0IEMxLjUwNSwzLjAxNiAxLjE0MywyLjk4OSAxLjE0MywyLjc4OCBDMS4xNDMsMi4zMTUgMS41MDMsMiAxLjk4NiwyIEw0Ljk4MiwyLjAwMSBDNS40NjUsMi4wMDEgNS44ODgsMi4zMTUgNS44ODgsMi43ODkgQzUuODg4LDIuOTkgNS40NjUsMy4wMTcgNC45ODIsMi45ODQgTDQuOTc5LDcuMTQyIFoiIGZpbGw9IiNiNWI1YjUiPjwvcGF0aD48L2c+PC9zdmc+) 0 50% no-repeat;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_ft { 
  height: 50px;
  margin-right: 10px;
  overflow: hidden;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_ft:after { 
  content: '-------------------------------------------------------------------------------------';
  display: block;
  height: 1px;
  line-height: 1px;
  color: #fff;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_ft .PIN_1498785571564_img { 
  position: absolute;
  left: 10px;
  top: 10px;
  height: 30px;
  width: 30px;
  border-radius: 30px;
  background: transparent url() 0 0 no-repeat;
  background-size: 30px 30px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_ft .PIN_1498785571564_pinner,  .PIN_1498785571564_embed_pin .PIN_1498785571564_ft  .PIN_1498785571564_board { 
  position: absolute;
  left: 50px;
  white-space: nowrap;
  overflow: hidden;
  text-overflow: ellipsis;
  width: 75%;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_ft .PIN_1498785571564_pinner { 
  font-weight: bold;
  top: 10px;
}
 .PIN_1498785571564_embed_pin .PIN_1498785571564_ft .PIN_1498785571564_board { 
  bottom: 10px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_ja .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_repin { 
  background-image: url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMThweCIgd2lkdGg9IjQxcHgiIHZpZXdCb3g9IjAgMCA0MSAxOCI+PGc+PHBhdGggZD0iTTE5LjgyMiw3LjE3MyBDMTkuODIyLDYuNTEgMTkuODM1LDYuMjc2IDE5Ljg4Nyw1Ljk2NCBMMTguMTQ1LDUuOTY0IEMxOC4xOTcsNi4yODkgMTguMTk3LDYuNDk3IDE4LjE5Nyw3LjE2IEwxOC4yMSwxMy4xOTIgQzE4LjIxLDEzLjk0NiAxOC4yMjMsMTQuMTY3IDE4LjI0OSwxNC4zODggQzE4LjMyNywxNS4wMjUgMTguNTIyLDE1LjQ0MSAxOC44ODYsMTUuNzE0IEMxOS4zOTMsMTYuMTA0IDIwLjI5LDE2LjI3MyAyMS45MjgsMTYuMjczIEMyMi43MjEsMTYuMjczIDI0LjM1OSwxNi4xOTUgMjUuMTI2LDE2LjExNyBDMjYuNTA0LDE1Ljk4NyAyNi41NjksMTUuOTc0IDI2Ljg0MiwxNS45NzQgTDI2Ljc2NCwxNC4yNDUgQzI2LjE5MiwxNC40MTQgMjUuOTA2LDE0LjQ3OSAyNS4yODIsMTQuNTU3IEMyNC4zMzMsMTQuNjg3IDIzLjEzNywxNC43NjUgMjIuMjY2LDE0Ljc2NSBDMjEuMDA1LDE0Ljc2NSAyMC4yNjQsMTQuNjQ4IDIwLjA0MywxNC40MjcgQzE5Ljg2MSwxNC4yNDUgMTkuODA5LDEzLjk1OSAxOS44MDksMTMuMjMxIEMxOS44MDksMTMuMTc5IDE5LjgwOSwxMy4xMDEgMTkuODIyLDEzLjAyMyBMMTkuODIyLDExLjMwNyBDMjEuOTkzLDEwLjkwNCAyNC4wMDgsMTAuMjI4IDI1LjkzMiw5LjI0IEwyNi4yNyw5LjA3MSBDMjYuMzc0LDkuMDE5IDI2LjQsOS4wMDYgMjYuNTQzLDguOTU0IEwyNS41MDMsNy40ODUgQzI0LjY1OCw4LjI3OCAyMS43ODUsOS40MzUgMTkuODIyLDkuNzk5IEwxOS44MjIsNy4xNzMgWiBNMjcuMzEsNC44NzIgQzI2LjQ5MSw0Ljg3MiAyNS44MTUsNS41NDggMjUuODE1LDYuMzY3IEMyNS44MTUsNy4xOTkgMjYuNDkxLDcuODc1IDI3LjMxLDcuODc1IEMyOC4xNDIsNy44NzUgMjguODE4LDcuMTk5IDI4LjgxOCw2LjM2NyBDMjguODE4LDUuNTQ4IDI4LjE0Miw0Ljg3MiAyNy4zMSw0Ljg3MiBMMjcuMzEsNC44NzIgWiBNMjcuMzEsNS41MjIgQzI3Ljc5MSw1LjUyMiAyOC4xNjgsNS44OTkgMjguMTY4LDYuMzY3IEMyOC4xNjgsNi44MzUgMjcuNzkxLDcuMjI1IDI3LjMxLDcuMjI1IEMyNi44NDIsNy4yMjUgMjYuNDY1LDYuODM1IDI2LjQ2NSw2LjM2NyBDMjYuNDY1LDUuODk5IDI2Ljg0Miw1LjUyMiAyNy4zMSw1LjUyMiBMMjcuMzEsNS41MjIgWiBNMzAuNTg2LDcuNjU0IEMzMS43OTUsOC4zMyAzMi44NjEsOS4xODggMzMuOTAxLDEwLjI5MyBMMzUuMDE5LDguODc2IEMzNC4wMTgsNy45MjcgMzMuMjEyLDcuMzI5IDMxLjY2NSw2LjM2NyBMMzAuNTg2LDcuNjU0IFogTTMxLjA0MSwxNi4yMzQgQzMxLjM0LDE2LjEzIDMxLjM3OSwxNi4xMTcgMzEuODk5LDE2LjAxMyBDMzMuOTE0LDE1LjU4NCAzNS41MjYsMTQuOTQ3IDM2Ljg1MiwxNC4wNjMgQzM4LjYzMywxMi44OCAzOS44NjgsMTEuMzQ2IDQwLjk3Myw4Ljk2NyBDNDAuMzEsOC40OTkgNDAuMTAyLDguMzA0IDM5LjU5NSw3LjY5MyBDMzkuMjA1LDguNzQ2IDM4Ljg0MSw5LjQ2MSAzOC4yNjksMTAuMjkzIEMzNy4yNDIsMTEuNzc1IDM2LjAzMywxMi43NzYgMzQuNDA4LDEzLjQ3OCBDMzMuMjI1LDEzLjk5OCAzMS42NzgsMTQuMzc1IDMwLjU2LDE0LjQ0IEwzMS4wNDEsMTYuMjM0IFogTTcuNDU4LDAgQzIuNSwwIDAsMy41MjIgMCw2LjQ1OSBDMCw4LjIzNyAwLjY4LDkuODE5IDIuMTM3LDEwLjQwOSBDMi4zNzYsMTAuNTA1IDIuNTksMTAuNDEyIDIuNjYsMTAuMTUgQzIuNzA4LDkuOTY5IDIuODIyLDkuNTExIDIuODczLDkuMzIgQzIuOTQzLDkuMDYxIDIuOTE2LDguOTcgMi43MjMsOC43NDQgQzIuMzAyLDguMjUzIDIuMDM0LDcuNjE3IDIuMDM0LDYuNzE2IEMyLjAzNCw0LjEwNCA0LjAwNywxLjc2NSA3LjE3MiwxLjc2NSBDOS45NzUsMS43NjUgMTEuNTE0LDMuNDYxIDExLjUxNCw1LjcyNiBDMTEuNTE0LDguNzA4IDEwLjE4MywxMS4xOCA4LjIwNiwxMS4xOCBDNy4xMTQsMTEuMTggNi4yOTcsMTAuMzI5IDYuNTU5LDkuMjMzIEM2Ljg3Miw3LjkyMiA3LjQ4LDYuNTA5IDcuNDgsNS41NjQgQzcuNDgsNC43MTcgNy4wMjIsNC4wMTEgNi4wNzIsNC4wMTEgQzQuOTU2LDQuMDExIDQuMDYsNS4xNTUgNC4wNiw2LjY4NyBDNC4wNiw3LjY2MyA0LjM5Myw4LjMyMyA0LjM5Myw4LjMyMyBDNC4zOTMsOC4zMjMgMy4yNTEsMTMuMTE3IDMuMDUxLDEzLjk1NyBDMi42NTIsMTUuNjI5IDIuOTkxLDE3LjY3OSAzLjAxOSwxNy44ODYgQzMuMDM2LDE4LjAwOSAzLjE5NSwxOC4wMzggMy4yNjcsMTcuOTQ2IEMzLjM3LDE3LjgxMiA0LjcsMTYuMTg3IDUuMTUxLDE0LjU2MiBDNS4yNzksMTQuMTAyIDUuODg1LDExLjcyIDUuODg1LDExLjcyIEM2LjI0OCwxMi40MDYgNy4zMDgsMTMuMDA5IDguNDM1LDEzLjAwOSBDMTEuNzksMTMuMDA5IDE0LjA2Niw5Ljk3OSAxNC4wNjYsNS45MjMgQzE0LjA2NiwyLjg1NyAxMS40NDQsMCA3LjQ1OCwwIFoiIGZpbGw9IiNmZmYiPjwvcGF0aD48L2c+PC9zdmc+);
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_ja .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_repin.PIN_1498785571564_save { 
  background: #bd081c url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMzBweCIgd2lkdGg9IjMwcHgiIHZpZXdCb3g9Ii0xIC0xIDMxIDMxIj48Zz48cGF0aCBkPSJNMjkuNDQ5LDE0LjY2MiBDMjkuNDQ5LDIyLjcyMiAyMi44NjgsMjkuMjU2IDE0Ljc1LDI5LjI1NiBDNi42MzIsMjkuMjU2IDAuMDUxLDIyLjcyMiAwLjA1MSwxNC42NjIgQzAuMDUxLDYuNjAxIDYuNjMyLDAuMDY3IDE0Ljc1LDAuMDY3IEMyMi44NjgsMC4wNjcgMjkuNDQ5LDYuNjAxIDI5LjQ0OSwxNC42NjIiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2ZmZiIgc3Ryb2tlLXdpZHRoPSIxIj48L3BhdGg+PHBhdGggZD0iTTE0LjczMywxLjY4NiBDNy41MTYsMS42ODYgMS42NjUsNy40OTUgMS42NjUsMTQuNjYyIEMxLjY2NSwyMC4xNTkgNS4xMDksMjQuODU0IDkuOTcsMjYuNzQ0IEM5Ljg1NiwyNS43MTggOS43NTMsMjQuMTQzIDEwLjAxNiwyMy4wMjIgQzEwLjI1MywyMi4wMSAxMS41NDgsMTYuNTcyIDExLjU0OCwxNi41NzIgQzExLjU0OCwxNi41NzIgMTEuMTU3LDE1Ljc5NSAxMS4xNTcsMTQuNjQ2IEMxMS4xNTcsMTIuODQyIDEyLjIxMSwxMS40OTUgMTMuNTIyLDExLjQ5NSBDMTQuNjM3LDExLjQ5NSAxNS4xNzUsMTIuMzI2IDE1LjE3NSwxMy4zMjMgQzE1LjE3NSwxNC40MzYgMTQuNDYyLDE2LjEgMTQuMDkzLDE3LjY0MyBDMTMuNzg1LDE4LjkzNSAxNC43NDUsMTkuOTg4IDE2LjAyOCwxOS45ODggQzE4LjM1MSwxOS45ODggMjAuMTM2LDE3LjU1NiAyMC4xMzYsMTQuMDQ2IEMyMC4xMzYsMTAuOTM5IDE3Ljg4OCw4Ljc2NyAxNC42NzgsOC43NjcgQzEwLjk1OSw4Ljc2NyA4Ljc3NywxMS41MzYgOC43NzcsMTQuMzk4IEM4Ljc3NywxNS41MTMgOS4yMSwxNi43MDkgOS43NDksMTcuMzU5IEM5Ljg1NiwxNy40ODggOS44NzIsMTcuNiA5Ljg0LDE3LjczMSBDOS43NDEsMTguMTQxIDkuNTIsMTkuMDIzIDkuNDc3LDE5LjIwMyBDOS40MiwxOS40NCA5LjI4OCwxOS40OTEgOS4wNCwxOS4zNzYgQzcuNDA4LDE4LjYyMiA2LjM4NywxNi4yNTIgNi4zODcsMTQuMzQ5IEM2LjM4NywxMC4yNTYgOS4zODMsNi40OTcgMTUuMDIyLDYuNDk3IEMxOS41NTUsNi40OTcgMjMuMDc4LDkuNzA1IDIzLjA3OCwxMy45OTEgQzIzLjA3OCwxOC40NjMgMjAuMjM5LDIyLjA2MiAxNi4yOTcsMjIuMDYyIEMxNC45NzMsMjIuMDYyIDEzLjcyOCwyMS4zNzkgMTMuMzAyLDIwLjU3MiBDMTMuMzAyLDIwLjU3MiAxMi42NDcsMjMuMDUgMTIuNDg4LDIzLjY1NyBDMTIuMTkzLDI0Ljc4NCAxMS4zOTYsMjYuMTk2IDEwLjg2MywyNy4wNTggQzEyLjA4NiwyNy40MzQgMTMuMzg2LDI3LjYzNyAxNC43MzMsMjcuNjM3IEMyMS45NSwyNy42MzcgMjcuODAxLDIxLjgyOCAyNy44MDEsMTQuNjYyIEMyNy44MDEsNy40OTUgMjEuOTUsMS42ODYgMTQuNzMzLDEuNjg2IiBmaWxsPSIjYmQwODFjIj48L3BhdGg+PC9nPjwvc3ZnPg==) 5px 50% no-repeat;
  background-size: 16px 16px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_medium { 
  min-width: 237px;
  max-width: 345px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large { 
  min-width: 345px;
  max-width: 600px;
  padding: 30px 34px 10px;
  font-size: 14px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd { 
  border-bottom: none;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_hd { 
  padding-top: 50px;
  border-bottom: none;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_img,  .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_container .PIN_1498785571564_video { 
  border-radius: 8px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_hd .PIN_1498785571564_repin { 
  top: 0px;
  left: 0px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_source { 
  height: 50px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_source .PIN_1498785571564_domain { 
  line-height: 50px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_stats { 
  position: absolute;
  top: 0;
  right: 0;
  text-align: right;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_description { 
  font-size: 16px;
  line-height: 20px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_menu { 
  height: inherit;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_bd .PIN_1498785571564_menu .PIN_1498785571564_dropdown { 
  top: 50px;
  font-size: 14px;
  padding: 10px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_ft { 
  height: 60px;
  font-size: 16px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_ft .PIN_1498785571564_board,  .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_ft .PIN_1498785571564_pinner { 
  left: 60px;
}
 .PIN_1498785571564_embed_pin.PIN_1498785571564_large .PIN_1498785571564_ft .PIN_1498785571564_img { 
  height: 40px;
  width: 40px;
  background-size: 40px 40px;
  border-radius: 20px;
}
 .PIN_1498785571564_button_follow { 
  display: inline-block;
  color: #363636;
  box-sizing: border-box;
  box-shadow: inset 0 0 1px #888;
  border-radius: 3px;
  font: 11px "Helvetica Neue", Helvetica, arial, sans-serif;
  font-weight: bold;
  cursor: pointer;
  -webkit-font-smoothing: antialiased;
  height: 20px;
  line-height: 20px;
  padding: 0 4px 0 20px;
  background-color: #efefef;
  position: relative;
  white-space: nowrap;
  vertical-align: baseline;
}
 .PIN_1498785571564_button_follow:hover { 
  box-shadow: inset 0 0 1px #000;
}
 .PIN_1498785571564_button_follow::after { 
  content: "";
  position: absolute;
  height: 14px;
  width: 14px;
  top: 3px;
  left: 3px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMzBweCIgd2lkdGg9IjMwcHgiIHZpZXdCb3g9Ii0xIC0xIDMxIDMxIj48Zz48cGF0aCBkPSJNMjkuNDQ5LDE0LjY2MiBDMjkuNDQ5LDIyLjcyMiAyMi44NjgsMjkuMjU2IDE0Ljc1LDI5LjI1NiBDNi42MzIsMjkuMjU2IDAuMDUxLDIyLjcyMiAwLjA1MSwxNC42NjIgQzAuMDUxLDYuNjAxIDYuNjMyLDAuMDY3IDE0Ljc1LDAuMDY3IEMyMi44NjgsMC4wNjcgMjkuNDQ5LDYuNjAxIDI5LjQ0OSwxNC42NjIiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2ZmZiIgc3Ryb2tlLXdpZHRoPSIxIj48L3BhdGg+PHBhdGggZD0iTTE0LjczMywxLjY4NiBDNy41MTYsMS42ODYgMS42NjUsNy40OTUgMS42NjUsMTQuNjYyIEMxLjY2NSwyMC4xNTkgNS4xMDksMjQuODU0IDkuOTcsMjYuNzQ0IEM5Ljg1NiwyNS43MTggOS43NTMsMjQuMTQzIDEwLjAxNiwyMy4wMjIgQzEwLjI1MywyMi4wMSAxMS41NDgsMTYuNTcyIDExLjU0OCwxNi41NzIgQzExLjU0OCwxNi41NzIgMTEuMTU3LDE1Ljc5NSAxMS4xNTcsMTQuNjQ2IEMxMS4xNTcsMTIuODQyIDEyLjIxMSwxMS40OTUgMTMuNTIyLDExLjQ5NSBDMTQuNjM3LDExLjQ5NSAxNS4xNzUsMTIuMzI2IDE1LjE3NSwxMy4zMjMgQzE1LjE3NSwxNC40MzYgMTQuNDYyLDE2LjEgMTQuMDkzLDE3LjY0MyBDMTMuNzg1LDE4LjkzNSAxNC43NDUsMTkuOTg4IDE2LjAyOCwxOS45ODggQzE4LjM1MSwxOS45ODggMjAuMTM2LDE3LjU1NiAyMC4xMzYsMTQuMDQ2IEMyMC4xMzYsMTAuOTM5IDE3Ljg4OCw4Ljc2NyAxNC42NzgsOC43NjcgQzEwLjk1OSw4Ljc2NyA4Ljc3NywxMS41MzYgOC43NzcsMTQuMzk4IEM4Ljc3NywxNS41MTMgOS4yMSwxNi43MDkgOS43NDksMTcuMzU5IEM5Ljg1NiwxNy40ODggOS44NzIsMTcuNiA5Ljg0LDE3LjczMSBDOS43NDEsMTguMTQxIDkuNTIsMTkuMDIzIDkuNDc3LDE5LjIwMyBDOS40MiwxOS40NCA5LjI4OCwxOS40OTEgOS4wNCwxOS4zNzYgQzcuNDA4LDE4LjYyMiA2LjM4NywxNi4yNTIgNi4zODcsMTQuMzQ5IEM2LjM4NywxMC4yNTYgOS4zODMsNi40OTcgMTUuMDIyLDYuNDk3IEMxOS41NTUsNi40OTcgMjMuMDc4LDkuNzA1IDIzLjA3OCwxMy45OTEgQzIzLjA3OCwxOC40NjMgMjAuMjM5LDIyLjA2MiAxNi4yOTcsMjIuMDYyIEMxNC45NzMsMjIuMDYyIDEzLjcyOCwyMS4zNzkgMTMuMzAyLDIwLjU3MiBDMTMuMzAyLDIwLjU3MiAxMi42NDcsMjMuMDUgMTIuNDg4LDIzLjY1NyBDMTIuMTkzLDI0Ljc4NCAxMS4zOTYsMjYuMTk2IDEwLjg2MywyNy4wNTggQzEyLjA4NiwyNy40MzQgMTMuMzg2LDI3LjYzNyAxNC43MzMsMjcuNjM3IEMyMS45NSwyNy42MzcgMjcuODAxLDIxLjgyOCAyNy44MDEsMTQuNjYyIEMyNy44MDEsNy40OTUgMjEuOTUsMS42ODYgMTQuNzMzLDEuNjg2IiBmaWxsPSIjYmQwODFjIj48L3BhdGg+PC9nPjwvc3ZnPg==) 0 0 no-repeat;
  background-size: 14px 14px;
}
 .PIN_1498785571564_button_follow.PIN_1498785571564_tall { 
  height: 26px;
  line-height: 26px;
  font-size: 13px;
  padding: 0 6px 0 25px;
  border-radius: 3px;
}
 .PIN_1498785571564_button_follow.PIN_1498785571564_tall::after { 
  height: 18px;
  width: 18px;
  top: 4px;
  left: 4px;
  background-size: 18px 18px;
}
 .PIN_1498785571564_button_pin { 
  cursor: pointer;
  display: inline-block;
  box-sizing: border-box;
  box-shadow: inset 0 0 1px #888;
  border-radius: 3px;
  height: 20px;
  width: 40px;
  -webkit-font-smoothing: antialiased;
  background: #efefef url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMThweCIgd2lkdGg9IjQycHgiIHZpZXdCb3g9IjAgMCA0MiAxOCI+PGc+PHBhdGggZD0iTTE2Ljg1Myw2LjM0NSBDMTcuNjMyLDYuMzQ1IDE4LjM4LDUuNzAyIDE4LjUxLDQuOTA5IEMxOC42NjQsNC4xMzggMTguMTM1LDMuNDk0IDE3LjM1NywzLjQ5NCBDMTYuNTc4LDMuNDk0IDE1LjgzLDQuMTM4IDE1LjY5OCw0LjkwOSBDMTUuNTQ2LDUuNzAyIDE2LjA1Myw2LjM0NSAxNi44NTMsNi4zNDUgWiBNNy40NTgsMCBDMi41LDAgMCwzLjUyMiAwLDYuNDU5IEMwLDguMjM3IDAuNjgsOS44MTkgMi4xMzcsMTAuNDA5IEMyLjM3NiwxMC41MDUgMi41OSwxMC40MTIgMi42NiwxMC4xNSBDMi43MDgsOS45NjkgMi44MjIsOS41MTEgMi44NzMsOS4zMiBDMi45NDMsOS4wNjEgMi45MTYsOC45NyAyLjcyMyw4Ljc0NCBDMi4zMDIsOC4yNTMgMi4wMzQsNy42MTcgMi4wMzQsNi43MTYgQzIuMDM0LDQuMTA0IDQuMDA3LDEuNzY1IDcuMTcyLDEuNzY1IEM5Ljk3NSwxLjc2NSAxMS41MTQsMy40NjEgMTEuNTE0LDUuNzI2IEMxMS41MTQsOC43MDggMTAuMTgzLDExLjE4IDguMjA2LDExLjE4IEM3LjExNCwxMS4xOCA2LjI5NywxMC4zMjkgNi41NTksOS4yMzMgQzYuODcyLDcuOTIyIDcuNDgsNi41MDkgNy40OCw1LjU2NCBDNy40OCw0LjcxNyA3LjAyMiw0LjAxMSA2LjA3Miw0LjAxMSBDNC45NTYsNC4wMTEgNC4wNiw1LjE1NSA0LjA2LDYuNjg3IEM0LjA2LDcuNjYzIDQuMzkzLDguMzIzIDQuMzkzLDguMzIzIEM0LjM5Myw4LjMyMyAzLjI1MSwxMy4xMTcgMy4wNTEsMTMuOTU3IEMyLjY1MiwxNS42MjkgMi45OTEsMTcuNjc5IDMuMDE5LDE3Ljg4NiBDMy4wMzYsMTguMDA5IDMuMTk1LDE4LjAzOCAzLjI2NywxNy45NDYgQzMuMzcsMTcuODEyIDQuNywxNi4xODcgNS4xNTEsMTQuNTYyIEM1LjI3OSwxNC4xMDIgNS44ODUsMTEuNzIgNS44ODUsMTEuNzIgQzYuMjQ4LDEyLjQwNiA3LjMwOCwxMy4wMDkgOC40MzUsMTMuMDA5IEMxMS43OSwxMy4wMDkgMTQuMDY2LDkuOTc5IDE0LjA2Niw1LjkyMyBDMTQuMDY2LDIuODU3IDExLjQ0NCwwIDcuNDU4LDAgWiBNMjYuODk2LDE0LjE4OSBDMjYuMzQ4LDE0LjE4OSAyNi4xMTcsMTMuOTE1IDI2LjExNywxMy4zMjggQzI2LjExNywxMi40MDQgMjcuMDM1LDEwLjA5MSAyNy4wMzUsOS4wNDEgQzI3LjAzNSw3LjYzOCAyNi4yNzYsNi44MjYgMjQuNzIsNi44MjYgQzIzLjczOSw2LjgyNiAyMi43MjIsNy40NTMgMjIuMjkxLDguMDAzIEMyMi4yOTEsOC4wMDMgMjIuNDIyLDcuNTUzIDIyLjQ2Nyw3LjM4IEMyMi41MTUsNy4xOTYgMjIuNDE1LDYuODg0IDIyLjE3Myw2Ljg4NCBMMjAuNjUxLDYuODg0IEMyMC4zMjgsNi44ODQgMjAuMjM4LDcuMDU1IDIwLjE5MSw3LjI0NCBDMjAuMTcyLDcuMzIgMTkuNjI0LDkuNTg0IDE5LjA5OCwxMS42MzIgQzE4LjczOCwxMy4wMzQgMTcuODYzLDE0LjIwNSAxNi45MjgsMTQuMjA1IEMxNi40NDcsMTQuMjA1IDE2LjIzMywxMy45MDYgMTYuMjMzLDEzLjM5OSBDMTYuMjMzLDEyLjk1OSAxNi41MTksMTEuODc3IDE2Ljg2LDEwLjUzNCBDMTcuMjc2LDguODk4IDE3LjY0Miw3LjU1MSAxNy42ODEsNy4zOTQgQzE3LjczMiw3LjE5MiAxNy42NDIsNy4wMTcgMTcuMzc5LDcuMDE3IEwxNS44NDksNy4wMTcgQzE1LjU3Miw3LjAxNyAxNS40NzMsNy4xNjEgMTUuNDE0LDcuMzYxIEMxNS40MTQsNy4zNjEgMTQuOTgzLDguOTc3IDE0LjUyNywxMC43NzUgQzE0LjE5NiwxMi4wNzkgMTMuODMsMTMuNDA5IDEzLjgzLDE0LjAzNCBDMTMuODMsMTUuMTQ4IDE0LjMzNiwxNS45NDQgMTUuNzI0LDE1Ljk0NCBDMTYuNzk2LDE1Ljk0NCAxNy42NDQsMTUuNDUgMTguMjkyLDE0Ljc2NCBDMTguMTk3LDE1LjEzNSAxOC4xMzYsMTUuNDE0IDE4LjEzLDE1LjQzOSBDMTguMDc0LDE1LjY1IDE4LjE0MiwxNS44MzggMTguMzk0LDE1LjgzOCBMMTkuOTYxLDE1LjgzOCBDMjAuMjMzLDE1LjgzOCAyMC4zMzcsMTUuNzMgMjAuMzk0LDE1LjQ5NCBDMjAuNDQ5LDE1LjI2OSAyMS42MTksMTAuNjY3IDIxLjYxOSwxMC42NjcgQzIxLjkyOCw5LjQ0MyAyMi42OTIsOC42MzIgMjMuNzY4LDguNjMyIEMyNC4yNzksOC42MzIgMjQuNzIsOC45NjcgMjQuNjY5LDkuNjE4IEMyNC42MTIsMTAuMzMzIDIzLjc0MSwxMi45MDMgMjMuNzQxLDE0LjAzMSBDMjMuNzQxLDE0Ljg4NCAyNC4wNiwxNS45NDUgMjUuNjgzLDE1Ljk0NSBDMjYuNzg5LDE1Ljk0NSAyNy42MDMsMTUuNDY0IDI4LjE5NSwxNC43ODYgTDI3LjQ4OSwxMy45NDEgQzI3LjMxMSwxNC4wOTQgMjcuMTE0LDE0LjE4OSAyNi44OTYsMTQuMTg5IFogTTQxLjcwMSw2Ljg3MyBMNDAuMTM0LDYuODczIEM0MC4xMzQsNi44NzMgNDAuODU2LDQuMTA5IDQwLjg3Myw0LjAzNSBDNDAuOTQyLDMuNzQ1IDQwLjY5OCwzLjU3OCA0MC40NDEsMy42MzEgQzQwLjQ0MSwzLjYzMSAzOS4yMywzLjg2NiAzOS4wMDUsMy45MTMgQzM4Ljc3OSwzLjk1OCAzOC42MDQsNC4wODEgMzguNTIyLDQuNDAzIEMzOC41MTIsNC40NDUgMzcuODgsNi44NzMgMzcuODgsNi44NzMgTDM2LjYyMiw2Ljg3MyBDMzYuMzg1LDYuODczIDM2LjI0NSw2Ljk2OCAzNi4xOTIsNy4xODggQzM2LjExNSw3LjUwNCAzNS45NzUsOC4xNDUgMzUuOTM2LDguMjk3IEMzNS44ODUsOC40OTQgMzYsOC42NDQgMzYuMjIyLDguNjQ0IEwzNy40NTcsOC42NDQgQzM3LjQ0OCw4LjY3NyAzNy4wNjQsMTAuMTI1IDM2LjcyNSwxMS41MjEgTDM2LjcyNCwxMS41MTYgQzM2LjcyLDExLjUzMiAzNi43MTYsMTEuNTQ2IDM2LjcxMiwxMS41NjIgTDM2LjcxMiwxMS41NTYgQzM2LjcxMiwxMS41NTYgMzYuNzA4LDExLjU3MSAzNi43MDIsMTEuNTk4IEMzNi4zMjQsMTIuOTY4IDM1LjExOCwxNC4yMDkgMzQuMjAxLDE0LjIwOSBDMzMuNzIxLDE0LjIwOSAzMy41MDYsMTMuOTA5IDMzLjUwNiwxMy40MDIgQzMzLjUwNiwxMi45NjMgMzMuNzkyLDExLjg4IDM0LjEzNCwxMC41MzcgQzM0LjU0OSw4LjkwMSAzNC45MTUsNy41NTUgMzQuOTU1LDcuMzk3IEMzNS4wMDYsNy4xOTYgMzQuOTE1LDcuMDIgMzQuNjUyLDcuMDIgTDMzLjEyMiw3LjAyIEMzMi44NDUsNy4wMiAzMi43NDYsNy4xNjQgMzIuNjg3LDcuMzY0IEMzMi42ODcsNy4zNjQgMzIuMjU3LDguOTggMzEuOCwxMC43NzggQzMxLjQ2OSwxMi4wODMgMzEuMTAzLDEzLjQxMiAzMS4xMDMsMTQuMDM3IEMzMS4xMDMsMTUuMTUxIDMxLjYwOSwxNS45NDggMzIuOTk3LDE1Ljk0OCBDMzQuMDcsMTUuOTQ4IDM1LjEzNiwxNS40NTMgMzUuNzgzLDE0Ljc2NyBDMzUuNzgzLDE0Ljc2NyAzNi4wMTEsMTQuNTIxIDM2LjIzLDE0LjIyOSBDMzYuMjQxLDE0LjU4MSAzNi4zMjQsMTQuODM3IDM2LjQxMSwxNS4wMTggQzM2LjQ1OCwxNS4xMTkgMzYuNTE1LDE1LjIxNSAzNi41ODEsMTUuMzAzIEMzNi41ODIsMTUuMzA0IDM2LjU4MywxNS4zMDYgMzYuNTg1LDE1LjMwOCBMMzYuNTg1LDE1LjMwOCBDMzYuODkxLDE1LjcxMyAzNy4zOTgsMTUuOTYyIDM4LjE1MSwxNS45NjIgQzM5Ljg5NCwxNS45NjIgNDAuOTQ0LDE0LjkzOCA0MS41NjIsMTMuOTA5IEw0MC43MDQsMTMuMjM5IEM0MC4zMzMsMTMuNzc0IDM5LjgzOSwxNC4xNzUgMzkuMzI0LDE0LjE3NSBDMzguODQ2LDE0LjE3NSAzOC41NzksMTMuODc4IDM4LjU3OSwxMy4zNzIgQzM4LjU3OSwxMi45MzUgMzguODg5LDExLjg2OCAzOS4yMjksMTAuNTMgQzM5LjM0NCwxMC4wODMgMzkuNTE2LDkuNDAxIDM5LjcwOCw4LjY0NCBMNDEuMzAyLDguNjQ0IEM0MS41MzksOC42NDQgNDEuNjc4LDguNTQ5IDQxLjczMiw4LjMyOSBDNDEuODA4LDguMDEyIDQxLjk0OCw3LjM3MiA0MS45ODgsNy4yMjEgQzQyLjAzOSw3LjAyMyA0MS45MjMsNi44NzMgNDEuNzAxLDYuODczIFogTTM0LjEyNiw2LjM0OCBDMzQuOTA1LDYuMzQ4IDM1LjY1Myw1LjcwNiAzNS43ODMsNC45MTIgQzM1LjkzNyw0LjE0MSAzNS40MDksMy40OTggMzQuNjMsMy40OTggQzMzLjg1MSwzLjQ5OCAzMy4xMDMsNC4xNDEgMzIuOTcxLDQuOTEyIEMzMi44MTksNS43MDYgMzMuMzI2LDYuMzQ4IDM0LjEyNiw2LjM0OCBaIiBmaWxsPSIjYmQwODFjIj48L3BhdGg+PC9nPjwvc3ZnPg==) 50% 50% no-repeat;
  background-size: 75%;
  position: relative;
  font: 12px "Helvetica Neue", Helvetica, arial, sans-serif;
  color: #555;
  text-align: center;
  vertical-align: baseline;
}
 .PIN_1498785571564_button_pin:hover { 
  box-shadow: inset 0 0 1px #000;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_above .PIN_1498785571564_count { 
  position: absolute;
  top: -28px;
  left: 0;
  height: 28px;
  width: inherit;
  line-height: 24px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iNzZweCIgd2lkdGg9IjExNHB4IiB2aWV3Qm94PSIwIDAgMTE0IDc2Ij48Zz48cGF0aCBkPSJNOSAxQzQuNiAxIDEgNC42IDEgOXY0M2MwIDQuMyAzLjYgOCA4IDhoMjZsMTggMTVoNy41bDE2LTE1SDEwNWM0LjQgMCA4LTMuNyA4LThWOWMwLTQuNC0zLjYtOC04LThIOXoiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2I1YjViNSIgc3Ryb2tlLXdpZHRoPSIyIj48L3BhdGg+PC9nPjwvc3ZnPg==) 0 0 no-repeat;
  background-size: 40px 28px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_above.PIN_1498785571564_padded { 
  margin-top: 28px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_beside .PIN_1498785571564_count { 
  position: absolute;
  right: -45px;
  text-align: center;
  text-indent: 5px;
  height: inherit;
  width: 45px;
  font-size: 11px;
  line-height: 20px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iNTZweCIgd2lkdGg9IjEyNnB4IiB2aWV3Qm94PSIyIDAgMTMwIDYwIj48Zz48cGF0aCBkPSJNMTE5LjYgMmM0LjUgMCA4IDMuNiA4IDh2NDBjMCA0LjQtMy41IDgtOCA4SDIzLjNMMS42IDMyLjR2LTQuNkwyMy4zIDJoOTYuM3oiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2I1YjViNSIgc3Ryb2tlLXdpZHRoPSIyIj48L3BhdGg+PC9nPjwvc3ZnPg==);
  background-size: cover;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_beside.PIN_1498785571564_padded { 
  margin-right: 45px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_ja { 
  background-image: url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMThweCIgd2lkdGg9IjQxcHgiIHZpZXdCb3g9IjAgMCA0MSAxOCI+PGc+PHBhdGggZD0iTTE5LjgyMiw3LjE3MyBDMTkuODIyLDYuNTEgMTkuODM1LDYuMjc2IDE5Ljg4Nyw1Ljk2NCBMMTguMTQ1LDUuOTY0IEMxOC4xOTcsNi4yODkgMTguMTk3LDYuNDk3IDE4LjE5Nyw3LjE2IEwxOC4yMSwxMy4xOTIgQzE4LjIxLDEzLjk0NiAxOC4yMjMsMTQuMTY3IDE4LjI0OSwxNC4zODggQzE4LjMyNywxNS4wMjUgMTguNTIyLDE1LjQ0MSAxOC44ODYsMTUuNzE0IEMxOS4zOTMsMTYuMTA0IDIwLjI5LDE2LjI3MyAyMS45MjgsMTYuMjczIEMyMi43MjEsMTYuMjczIDI0LjM1OSwxNi4xOTUgMjUuMTI2LDE2LjExNyBDMjYuNTA0LDE1Ljk4NyAyNi41NjksMTUuOTc0IDI2Ljg0MiwxNS45NzQgTDI2Ljc2NCwxNC4yNDUgQzI2LjE5MiwxNC40MTQgMjUuOTA2LDE0LjQ3OSAyNS4yODIsMTQuNTU3IEMyNC4zMzMsMTQuNjg3IDIzLjEzNywxNC43NjUgMjIuMjY2LDE0Ljc2NSBDMjEuMDA1LDE0Ljc2NSAyMC4yNjQsMTQuNjQ4IDIwLjA0MywxNC40MjcgQzE5Ljg2MSwxNC4yNDUgMTkuODA5LDEzLjk1OSAxOS44MDksMTMuMjMxIEMxOS44MDksMTMuMTc5IDE5LjgwOSwxMy4xMDEgMTkuODIyLDEzLjAyMyBMMTkuODIyLDExLjMwNyBDMjEuOTkzLDEwLjkwNCAyNC4wMDgsMTAuMjI4IDI1LjkzMiw5LjI0IEwyNi4yNyw5LjA3MSBDMjYuMzc0LDkuMDE5IDI2LjQsOS4wMDYgMjYuNTQzLDguOTU0IEwyNS41MDMsNy40ODUgQzI0LjY1OCw4LjI3OCAyMS43ODUsOS40MzUgMTkuODIyLDkuNzk5IEwxOS44MjIsNy4xNzMgWiBNMjcuMzEsNC44NzIgQzI2LjQ5MSw0Ljg3MiAyNS44MTUsNS41NDggMjUuODE1LDYuMzY3IEMyNS44MTUsNy4xOTkgMjYuNDkxLDcuODc1IDI3LjMxLDcuODc1IEMyOC4xNDIsNy44NzUgMjguODE4LDcuMTk5IDI4LjgxOCw2LjM2NyBDMjguODE4LDUuNTQ4IDI4LjE0Miw0Ljg3MiAyNy4zMSw0Ljg3MiBMMjcuMzEsNC44NzIgWiBNMjcuMzEsNS41MjIgQzI3Ljc5MSw1LjUyMiAyOC4xNjgsNS44OTkgMjguMTY4LDYuMzY3IEMyOC4xNjgsNi44MzUgMjcuNzkxLDcuMjI1IDI3LjMxLDcuMjI1IEMyNi44NDIsNy4yMjUgMjYuNDY1LDYuODM1IDI2LjQ2NSw2LjM2NyBDMjYuNDY1LDUuODk5IDI2Ljg0Miw1LjUyMiAyNy4zMSw1LjUyMiBMMjcuMzEsNS41MjIgWiBNMzAuNTg2LDcuNjU0IEMzMS43OTUsOC4zMyAzMi44NjEsOS4xODggMzMuOTAxLDEwLjI5MyBMMzUuMDE5LDguODc2IEMzNC4wMTgsNy45MjcgMzMuMjEyLDcuMzI5IDMxLjY2NSw2LjM2NyBMMzAuNTg2LDcuNjU0IFogTTMxLjA0MSwxNi4yMzQgQzMxLjM0LDE2LjEzIDMxLjM3OSwxNi4xMTcgMzEuODk5LDE2LjAxMyBDMzMuOTE0LDE1LjU4NCAzNS41MjYsMTQuOTQ3IDM2Ljg1MiwxNC4wNjMgQzM4LjYzMywxMi44OCAzOS44NjgsMTEuMzQ2IDQwLjk3Myw4Ljk2NyBDNDAuMzEsOC40OTkgNDAuMTAyLDguMzA0IDM5LjU5NSw3LjY5MyBDMzkuMjA1LDguNzQ2IDM4Ljg0MSw5LjQ2MSAzOC4yNjksMTAuMjkzIEMzNy4yNDIsMTEuNzc1IDM2LjAzMywxMi43NzYgMzQuNDA4LDEzLjQ3OCBDMzMuMjI1LDEzLjk5OCAzMS42NzgsMTQuMzc1IDMwLjU2LDE0LjQ0IEwzMS4wNDEsMTYuMjM0IFogTTcuNDU4LDAgQzIuNSwwIDAsMy41MjIgMCw2LjQ1OSBDMCw4LjIzNyAwLjY4LDkuODE5IDIuMTM3LDEwLjQwOSBDMi4zNzYsMTAuNTA1IDIuNTksMTAuNDEyIDIuNjYsMTAuMTUgQzIuNzA4LDkuOTY5IDIuODIyLDkuNTExIDIuODczLDkuMzIgQzIuOTQzLDkuMDYxIDIuOTE2LDguOTcgMi43MjMsOC43NDQgQzIuMzAyLDguMjUzIDIuMDM0LDcuNjE3IDIuMDM0LDYuNzE2IEMyLjAzNCw0LjEwNCA0LjAwNywxLjc2NSA3LjE3MiwxLjc2NSBDOS45NzUsMS43NjUgMTEuNTE0LDMuNDYxIDExLjUxNCw1LjcyNiBDMTEuNTE0LDguNzA4IDEwLjE4MywxMS4xOCA4LjIwNiwxMS4xOCBDNy4xMTQsMTEuMTggNi4yOTcsMTAuMzI5IDYuNTU5LDkuMjMzIEM2Ljg3Miw3LjkyMiA3LjQ4LDYuNTA5IDcuNDgsNS41NjQgQzcuNDgsNC43MTcgNy4wMjIsNC4wMTEgNi4wNzIsNC4wMTEgQzQuOTU2LDQuMDExIDQuMDYsNS4xNTUgNC4wNiw2LjY4NyBDNC4wNiw3LjY2MyA0LjM5Myw4LjMyMyA0LjM5Myw4LjMyMyBDNC4zOTMsOC4zMjMgMy4yNTEsMTMuMTE3IDMuMDUxLDEzLjk1NyBDMi42NTIsMTUuNjI5IDIuOTkxLDE3LjY3OSAzLjAxOSwxNy44ODYgQzMuMDM2LDE4LjAwOSAzLjE5NSwxOC4wMzggMy4yNjcsMTcuOTQ2IEMzLjM3LDE3LjgxMiA0LjcsMTYuMTg3IDUuMTUxLDE0LjU2MiBDNS4yNzksMTQuMTAyIDUuODg1LDExLjcyIDUuODg1LDExLjcyIEM2LjI0OCwxMi40MDYgNy4zMDgsMTMuMDA5IDguNDM1LDEzLjAwOSBDMTEuNzksMTMuMDA5IDE0LjA2Niw5Ljk3OSAxNC4wNjYsNS45MjMgQzE0LjA2NiwyLjg1NyAxMS40NDQsMCA3LjQ1OCwwIFoiIGZpbGw9IiNiZDA4MWMiPjwvcGF0aD48L2c+PC9zdmc+);
  background-size: 72%;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_red { 
  background-color: #bd081c;
  background-image: url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMThweCIgd2lkdGg9IjQycHgiIHZpZXdCb3g9IjAgMCA0MiAxOCI+PGc+PHBhdGggZD0iTTE2Ljg1Myw2LjM0NSBDMTcuNjMyLDYuMzQ1IDE4LjM4LDUuNzAyIDE4LjUxLDQuOTA5IEMxOC42NjQsNC4xMzggMTguMTM1LDMuNDk0IDE3LjM1NywzLjQ5NCBDMTYuNTc4LDMuNDk0IDE1LjgzLDQuMTM4IDE1LjY5OCw0LjkwOSBDMTUuNTQ2LDUuNzAyIDE2LjA1Myw2LjM0NSAxNi44NTMsNi4zNDUgWiBNNy40NTgsMCBDMi41LDAgMCwzLjUyMiAwLDYuNDU5IEMwLDguMjM3IDAuNjgsOS44MTkgMi4xMzcsMTAuNDA5IEMyLjM3NiwxMC41MDUgMi41OSwxMC40MTIgMi42NiwxMC4xNSBDMi43MDgsOS45NjkgMi44MjIsOS41MTEgMi44NzMsOS4zMiBDMi45NDMsOS4wNjEgMi45MTYsOC45NyAyLjcyMyw4Ljc0NCBDMi4zMDIsOC4yNTMgMi4wMzQsNy42MTcgMi4wMzQsNi43MTYgQzIuMDM0LDQuMTA0IDQuMDA3LDEuNzY1IDcuMTcyLDEuNzY1IEM5Ljk3NSwxLjc2NSAxMS41MTQsMy40NjEgMTEuNTE0LDUuNzI2IEMxMS41MTQsOC43MDggMTAuMTgzLDExLjE4IDguMjA2LDExLjE4IEM3LjExNCwxMS4xOCA2LjI5NywxMC4zMjkgNi41NTksOS4yMzMgQzYuODcyLDcuOTIyIDcuNDgsNi41MDkgNy40OCw1LjU2NCBDNy40OCw0LjcxNyA3LjAyMiw0LjAxMSA2LjA3Miw0LjAxMSBDNC45NTYsNC4wMTEgNC4wNiw1LjE1NSA0LjA2LDYuNjg3IEM0LjA2LDcuNjYzIDQuMzkzLDguMzIzIDQuMzkzLDguMzIzIEM0LjM5Myw4LjMyMyAzLjI1MSwxMy4xMTcgMy4wNTEsMTMuOTU3IEMyLjY1MiwxNS42MjkgMi45OTEsMTcuNjc5IDMuMDE5LDE3Ljg4NiBDMy4wMzYsMTguMDA5IDMuMTk1LDE4LjAzOCAzLjI2NywxNy45NDYgQzMuMzcsMTcuODEyIDQuNywxNi4xODcgNS4xNTEsMTQuNTYyIEM1LjI3OSwxNC4xMDIgNS44ODUsMTEuNzIgNS44ODUsMTEuNzIgQzYuMjQ4LDEyLjQwNiA3LjMwOCwxMy4wMDkgOC40MzUsMTMuMDA5IEMxMS43OSwxMy4wMDkgMTQuMDY2LDkuOTc5IDE0LjA2Niw1LjkyMyBDMTQuMDY2LDIuODU3IDExLjQ0NCwwIDcuNDU4LDAgWiBNMjYuODk2LDE0LjE4OSBDMjYuMzQ4LDE0LjE4OSAyNi4xMTcsMTMuOTE1IDI2LjExNywxMy4zMjggQzI2LjExNywxMi40MDQgMjcuMDM1LDEwLjA5MSAyNy4wMzUsOS4wNDEgQzI3LjAzNSw3LjYzOCAyNi4yNzYsNi44MjYgMjQuNzIsNi44MjYgQzIzLjczOSw2LjgyNiAyMi43MjIsNy40NTMgMjIuMjkxLDguMDAzIEMyMi4yOTEsOC4wMDMgMjIuNDIyLDcuNTUzIDIyLjQ2Nyw3LjM4IEMyMi41MTUsNy4xOTYgMjIuNDE1LDYuODg0IDIyLjE3Myw2Ljg4NCBMMjAuNjUxLDYuODg0IEMyMC4zMjgsNi44ODQgMjAuMjM4LDcuMDU1IDIwLjE5MSw3LjI0NCBDMjAuMTcyLDcuMzIgMTkuNjI0LDkuNTg0IDE5LjA5OCwxMS42MzIgQzE4LjczOCwxMy4wMzQgMTcuODYzLDE0LjIwNSAxNi45MjgsMTQuMjA1IEMxNi40NDcsMTQuMjA1IDE2LjIzMywxMy45MDYgMTYuMjMzLDEzLjM5OSBDMTYuMjMzLDEyLjk1OSAxNi41MTksMTEuODc3IDE2Ljg2LDEwLjUzNCBDMTcuMjc2LDguODk4IDE3LjY0Miw3LjU1MSAxNy42ODEsNy4zOTQgQzE3LjczMiw3LjE5MiAxNy42NDIsNy4wMTcgMTcuMzc5LDcuMDE3IEwxNS44NDksNy4wMTcgQzE1LjU3Miw3LjAxNyAxNS40NzMsNy4xNjEgMTUuNDE0LDcuMzYxIEMxNS40MTQsNy4zNjEgMTQuOTgzLDguOTc3IDE0LjUyNywxMC43NzUgQzE0LjE5NiwxMi4wNzkgMTMuODMsMTMuNDA5IDEzLjgzLDE0LjAzNCBDMTMuODMsMTUuMTQ4IDE0LjMzNiwxNS45NDQgMTUuNzI0LDE1Ljk0NCBDMTYuNzk2LDE1Ljk0NCAxNy42NDQsMTUuNDUgMTguMjkyLDE0Ljc2NCBDMTguMTk3LDE1LjEzNSAxOC4xMzYsMTUuNDE0IDE4LjEzLDE1LjQzOSBDMTguMDc0LDE1LjY1IDE4LjE0MiwxNS44MzggMTguMzk0LDE1LjgzOCBMMTkuOTYxLDE1LjgzOCBDMjAuMjMzLDE1LjgzOCAyMC4zMzcsMTUuNzMgMjAuMzk0LDE1LjQ5NCBDMjAuNDQ5LDE1LjI2OSAyMS42MTksMTAuNjY3IDIxLjYxOSwxMC42NjcgQzIxLjkyOCw5LjQ0MyAyMi42OTIsOC42MzIgMjMuNzY4LDguNjMyIEMyNC4yNzksOC42MzIgMjQuNzIsOC45NjcgMjQuNjY5LDkuNjE4IEMyNC42MTIsMTAuMzMzIDIzLjc0MSwxMi45MDMgMjMuNzQxLDE0LjAzMSBDMjMuNzQxLDE0Ljg4NCAyNC4wNiwxNS45NDUgMjUuNjgzLDE1Ljk0NSBDMjYuNzg5LDE1Ljk0NSAyNy42MDMsMTUuNDY0IDI4LjE5NSwxNC43ODYgTDI3LjQ4OSwxMy45NDEgQzI3LjMxMSwxNC4wOTQgMjcuMTE0LDE0LjE4OSAyNi44OTYsMTQuMTg5IFogTTQxLjcwMSw2Ljg3MyBMNDAuMTM0LDYuODczIEM0MC4xMzQsNi44NzMgNDAuODU2LDQuMTA5IDQwLjg3Myw0LjAzNSBDNDAuOTQyLDMuNzQ1IDQwLjY5OCwzLjU3OCA0MC40NDEsMy42MzEgQzQwLjQ0MSwzLjYzMSAzOS4yMywzLjg2NiAzOS4wMDUsMy45MTMgQzM4Ljc3OSwzLjk1OCAzOC42MDQsNC4wODEgMzguNTIyLDQuNDAzIEMzOC41MTIsNC40NDUgMzcuODgsNi44NzMgMzcuODgsNi44NzMgTDM2LjYyMiw2Ljg3MyBDMzYuMzg1LDYuODczIDM2LjI0NSw2Ljk2OCAzNi4xOTIsNy4xODggQzM2LjExNSw3LjUwNCAzNS45NzUsOC4xNDUgMzUuOTM2LDguMjk3IEMzNS44ODUsOC40OTQgMzYsOC42NDQgMzYuMjIyLDguNjQ0IEwzNy40NTcsOC42NDQgQzM3LjQ0OCw4LjY3NyAzNy4wNjQsMTAuMTI1IDM2LjcyNSwxMS41MjEgTDM2LjcyNCwxMS41MTYgQzM2LjcyLDExLjUzMiAzNi43MTYsMTEuNTQ2IDM2LjcxMiwxMS41NjIgTDM2LjcxMiwxMS41NTYgQzM2LjcxMiwxMS41NTYgMzYuNzA4LDExLjU3MSAzNi43MDIsMTEuNTk4IEMzNi4zMjQsMTIuOTY4IDM1LjExOCwxNC4yMDkgMzQuMjAxLDE0LjIwOSBDMzMuNzIxLDE0LjIwOSAzMy41MDYsMTMuOTA5IDMzLjUwNiwxMy40MDIgQzMzLjUwNiwxMi45NjMgMzMuNzkyLDExLjg4IDM0LjEzNCwxMC41MzcgQzM0LjU0OSw4LjkwMSAzNC45MTUsNy41NTUgMzQuOTU1LDcuMzk3IEMzNS4wMDYsNy4xOTYgMzQuOTE1LDcuMDIgMzQuNjUyLDcuMDIgTDMzLjEyMiw3LjAyIEMzMi44NDUsNy4wMiAzMi43NDYsNy4xNjQgMzIuNjg3LDcuMzY0IEMzMi42ODcsNy4zNjQgMzIuMjU3LDguOTggMzEuOCwxMC43NzggQzMxLjQ2OSwxMi4wODMgMzEuMTAzLDEzLjQxMiAzMS4xMDMsMTQuMDM3IEMzMS4xMDMsMTUuMTUxIDMxLjYwOSwxNS45NDggMzIuOTk3LDE1Ljk0OCBDMzQuMDcsMTUuOTQ4IDM1LjEzNiwxNS40NTMgMzUuNzgzLDE0Ljc2NyBDMzUuNzgzLDE0Ljc2NyAzNi4wMTEsMTQuNTIxIDM2LjIzLDE0LjIyOSBDMzYuMjQxLDE0LjU4MSAzNi4zMjQsMTQuODM3IDM2LjQxMSwxNS4wMTggQzM2LjQ1OCwxNS4xMTkgMzYuNTE1LDE1LjIxNSAzNi41ODEsMTUuMzAzIEMzNi41ODIsMTUuMzA0IDM2LjU4MywxNS4zMDYgMzYuNTg1LDE1LjMwOCBMMzYuNTg1LDE1LjMwOCBDMzYuODkxLDE1LjcxMyAzNy4zOTgsMTUuOTYyIDM4LjE1MSwxNS45NjIgQzM5Ljg5NCwxNS45NjIgNDAuOTQ0LDE0LjkzOCA0MS41NjIsMTMuOTA5IEw0MC43MDQsMTMuMjM5IEM0MC4zMzMsMTMuNzc0IDM5LjgzOSwxNC4xNzUgMzkuMzI0LDE0LjE3NSBDMzguODQ2LDE0LjE3NSAzOC41NzksMTMuODc4IDM4LjU3OSwxMy4zNzIgQzM4LjU3OSwxMi45MzUgMzguODg5LDExLjg2OCAzOS4yMjksMTAuNTMgQzM5LjM0NCwxMC4wODMgMzkuNTE2LDkuNDAxIDM5LjcwOCw4LjY0NCBMNDEuMzAyLDguNjQ0IEM0MS41MzksOC42NDQgNDEuNjc4LDguNTQ5IDQxLjczMiw4LjMyOSBDNDEuODA4LDguMDEyIDQxLjk0OCw3LjM3MiA0MS45ODgsNy4yMjEgQzQyLjAzOSw3LjAyMyA0MS45MjMsNi44NzMgNDEuNzAxLDYuODczIFogTTM0LjEyNiw2LjM0OCBDMzQuOTA1LDYuMzQ4IDM1LjY1Myw1LjcwNiAzNS43ODMsNC45MTIgQzM1LjkzNyw0LjE0MSAzNS40MDksMy40OTggMzQuNjMsMy40OTggQzMzLjg1MSwzLjQ5OCAzMy4xMDMsNC4xNDEgMzIuOTcxLDQuOTEyIEMzMi44MTksNS43MDYgMzMuMzI2LDYuMzQ4IDM0LjEyNiw2LjM0OCBaIiBmaWxsPSIjZmZmIj48L3BhdGg+PC9nPjwvc3ZnPg==);
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_red.PIN_1498785571564_ja { 
  background-image: url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMThweCIgd2lkdGg9IjQxcHgiIHZpZXdCb3g9IjAgMCA0MSAxOCI+PGc+PHBhdGggZD0iTTE5LjgyMiw3LjE3MyBDMTkuODIyLDYuNTEgMTkuODM1LDYuMjc2IDE5Ljg4Nyw1Ljk2NCBMMTguMTQ1LDUuOTY0IEMxOC4xOTcsNi4yODkgMTguMTk3LDYuNDk3IDE4LjE5Nyw3LjE2IEwxOC4yMSwxMy4xOTIgQzE4LjIxLDEzLjk0NiAxOC4yMjMsMTQuMTY3IDE4LjI0OSwxNC4zODggQzE4LjMyNywxNS4wMjUgMTguNTIyLDE1LjQ0MSAxOC44ODYsMTUuNzE0IEMxOS4zOTMsMTYuMTA0IDIwLjI5LDE2LjI3MyAyMS45MjgsMTYuMjczIEMyMi43MjEsMTYuMjczIDI0LjM1OSwxNi4xOTUgMjUuMTI2LDE2LjExNyBDMjYuNTA0LDE1Ljk4NyAyNi41NjksMTUuOTc0IDI2Ljg0MiwxNS45NzQgTDI2Ljc2NCwxNC4yNDUgQzI2LjE5MiwxNC40MTQgMjUuOTA2LDE0LjQ3OSAyNS4yODIsMTQuNTU3IEMyNC4zMzMsMTQuNjg3IDIzLjEzNywxNC43NjUgMjIuMjY2LDE0Ljc2NSBDMjEuMDA1LDE0Ljc2NSAyMC4yNjQsMTQuNjQ4IDIwLjA0MywxNC40MjcgQzE5Ljg2MSwxNC4yNDUgMTkuODA5LDEzLjk1OSAxOS44MDksMTMuMjMxIEMxOS44MDksMTMuMTc5IDE5LjgwOSwxMy4xMDEgMTkuODIyLDEzLjAyMyBMMTkuODIyLDExLjMwNyBDMjEuOTkzLDEwLjkwNCAyNC4wMDgsMTAuMjI4IDI1LjkzMiw5LjI0IEwyNi4yNyw5LjA3MSBDMjYuMzc0LDkuMDE5IDI2LjQsOS4wMDYgMjYuNTQzLDguOTU0IEwyNS41MDMsNy40ODUgQzI0LjY1OCw4LjI3OCAyMS43ODUsOS40MzUgMTkuODIyLDkuNzk5IEwxOS44MjIsNy4xNzMgWiBNMjcuMzEsNC44NzIgQzI2LjQ5MSw0Ljg3MiAyNS44MTUsNS41NDggMjUuODE1LDYuMzY3IEMyNS44MTUsNy4xOTkgMjYuNDkxLDcuODc1IDI3LjMxLDcuODc1IEMyOC4xNDIsNy44NzUgMjguODE4LDcuMTk5IDI4LjgxOCw2LjM2NyBDMjguODE4LDUuNTQ4IDI4LjE0Miw0Ljg3MiAyNy4zMSw0Ljg3MiBMMjcuMzEsNC44NzIgWiBNMjcuMzEsNS41MjIgQzI3Ljc5MSw1LjUyMiAyOC4xNjgsNS44OTkgMjguMTY4LDYuMzY3IEMyOC4xNjgsNi44MzUgMjcuNzkxLDcuMjI1IDI3LjMxLDcuMjI1IEMyNi44NDIsNy4yMjUgMjYuNDY1LDYuODM1IDI2LjQ2NSw2LjM2NyBDMjYuNDY1LDUuODk5IDI2Ljg0Miw1LjUyMiAyNy4zMSw1LjUyMiBMMjcuMzEsNS41MjIgWiBNMzAuNTg2LDcuNjU0IEMzMS43OTUsOC4zMyAzMi44NjEsOS4xODggMzMuOTAxLDEwLjI5MyBMMzUuMDE5LDguODc2IEMzNC4wMTgsNy45MjcgMzMuMjEyLDcuMzI5IDMxLjY2NSw2LjM2NyBMMzAuNTg2LDcuNjU0IFogTTMxLjA0MSwxNi4yMzQgQzMxLjM0LDE2LjEzIDMxLjM3OSwxNi4xMTcgMzEuODk5LDE2LjAxMyBDMzMuOTE0LDE1LjU4NCAzNS41MjYsMTQuOTQ3IDM2Ljg1MiwxNC4wNjMgQzM4LjYzMywxMi44OCAzOS44NjgsMTEuMzQ2IDQwLjk3Myw4Ljk2NyBDNDAuMzEsOC40OTkgNDAuMTAyLDguMzA0IDM5LjU5NSw3LjY5MyBDMzkuMjA1LDguNzQ2IDM4Ljg0MSw5LjQ2MSAzOC4yNjksMTAuMjkzIEMzNy4yNDIsMTEuNzc1IDM2LjAzMywxMi43NzYgMzQuNDA4LDEzLjQ3OCBDMzMuMjI1LDEzLjk5OCAzMS42NzgsMTQuMzc1IDMwLjU2LDE0LjQ0IEwzMS4wNDEsMTYuMjM0IFogTTcuNDU4LDAgQzIuNSwwIDAsMy41MjIgMCw2LjQ1OSBDMCw4LjIzNyAwLjY4LDkuODE5IDIuMTM3LDEwLjQwOSBDMi4zNzYsMTAuNTA1IDIuNTksMTAuNDEyIDIuNjYsMTAuMTUgQzIuNzA4LDkuOTY5IDIuODIyLDkuNTExIDIuODczLDkuMzIgQzIuOTQzLDkuMDYxIDIuOTE2LDguOTcgMi43MjMsOC43NDQgQzIuMzAyLDguMjUzIDIuMDM0LDcuNjE3IDIuMDM0LDYuNzE2IEMyLjAzNCw0LjEwNCA0LjAwNywxLjc2NSA3LjE3MiwxLjc2NSBDOS45NzUsMS43NjUgMTEuNTE0LDMuNDYxIDExLjUxNCw1LjcyNiBDMTEuNTE0LDguNzA4IDEwLjE4MywxMS4xOCA4LjIwNiwxMS4xOCBDNy4xMTQsMTEuMTggNi4yOTcsMTAuMzI5IDYuNTU5LDkuMjMzIEM2Ljg3Miw3LjkyMiA3LjQ4LDYuNTA5IDcuNDgsNS41NjQgQzcuNDgsNC43MTcgNy4wMjIsNC4wMTEgNi4wNzIsNC4wMTEgQzQuOTU2LDQuMDExIDQuMDYsNS4xNTUgNC4wNiw2LjY4NyBDNC4wNiw3LjY2MyA0LjM5Myw4LjMyMyA0LjM5Myw4LjMyMyBDNC4zOTMsOC4zMjMgMy4yNTEsMTMuMTE3IDMuMDUxLDEzLjk1NyBDMi42NTIsMTUuNjI5IDIuOTkxLDE3LjY3OSAzLjAxOSwxNy44ODYgQzMuMDM2LDE4LjAwOSAzLjE5NSwxOC4wMzggMy4yNjcsMTcuOTQ2IEMzLjM3LDE3LjgxMiA0LjcsMTYuMTg3IDUuMTUxLDE0LjU2MiBDNS4yNzksMTQuMTAyIDUuODg1LDExLjcyIDUuODg1LDExLjcyIEM2LjI0OCwxMi40MDYgNy4zMDgsMTMuMDA5IDguNDM1LDEzLjAwOSBDMTEuNzksMTMuMDA5IDE0LjA2Niw5Ljk3OSAxNC4wNjYsNS45MjMgQzE0LjA2NiwyLjg1NyAxMS40NDQsMCA3LjQ1OCwwIFoiIGZpbGw9IiNmZmYiPjwvcGF0aD48L2c+PC9zdmc+);
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_white { 
  background-color: #fff;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save { 
  border-radius: 2px;
  text-indent: 20px;
  width: auto;
  padding: 0 4px 0 0;
  text-align: center;
  font: 11px/20px "Helvetica Neue", Helvetica, sans-serif;
  font-weight: bold;
  color: #fff;
  background: #bd081c url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMzBweCIgd2lkdGg9IjMwcHgiIHZpZXdCb3g9Ii0xIC0xIDMxIDMxIj48Zz48cGF0aCBkPSJNMjkuNDQ5LDE0LjY2MiBDMjkuNDQ5LDIyLjcyMiAyMi44NjgsMjkuMjU2IDE0Ljc1LDI5LjI1NiBDNi42MzIsMjkuMjU2IDAuMDUxLDIyLjcyMiAwLjA1MSwxNC42NjIgQzAuMDUxLDYuNjAxIDYuNjMyLDAuMDY3IDE0Ljc1LDAuMDY3IEMyMi44NjgsMC4wNjcgMjkuNDQ5LDYuNjAxIDI5LjQ0OSwxNC42NjIiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2ZmZiIgc3Ryb2tlLXdpZHRoPSIxIj48L3BhdGg+PHBhdGggZD0iTTE0LjczMywxLjY4NiBDNy41MTYsMS42ODYgMS42NjUsNy40OTUgMS42NjUsMTQuNjYyIEMxLjY2NSwyMC4xNTkgNS4xMDksMjQuODU0IDkuOTcsMjYuNzQ0IEM5Ljg1NiwyNS43MTggOS43NTMsMjQuMTQzIDEwLjAxNiwyMy4wMjIgQzEwLjI1MywyMi4wMSAxMS41NDgsMTYuNTcyIDExLjU0OCwxNi41NzIgQzExLjU0OCwxNi41NzIgMTEuMTU3LDE1Ljc5NSAxMS4xNTcsMTQuNjQ2IEMxMS4xNTcsMTIuODQyIDEyLjIxMSwxMS40OTUgMTMuNTIyLDExLjQ5NSBDMTQuNjM3LDExLjQ5NSAxNS4xNzUsMTIuMzI2IDE1LjE3NSwxMy4zMjMgQzE1LjE3NSwxNC40MzYgMTQuNDYyLDE2LjEgMTQuMDkzLDE3LjY0MyBDMTMuNzg1LDE4LjkzNSAxNC43NDUsMTkuOTg4IDE2LjAyOCwxOS45ODggQzE4LjM1MSwxOS45ODggMjAuMTM2LDE3LjU1NiAyMC4xMzYsMTQuMDQ2IEMyMC4xMzYsMTAuOTM5IDE3Ljg4OCw4Ljc2NyAxNC42NzgsOC43NjcgQzEwLjk1OSw4Ljc2NyA4Ljc3NywxMS41MzYgOC43NzcsMTQuMzk4IEM4Ljc3NywxNS41MTMgOS4yMSwxNi43MDkgOS43NDksMTcuMzU5IEM5Ljg1NiwxNy40ODggOS44NzIsMTcuNiA5Ljg0LDE3LjczMSBDOS43NDEsMTguMTQxIDkuNTIsMTkuMDIzIDkuNDc3LDE5LjIwMyBDOS40MiwxOS40NCA5LjI4OCwxOS40OTEgOS4wNCwxOS4zNzYgQzcuNDA4LDE4LjYyMiA2LjM4NywxNi4yNTIgNi4zODcsMTQuMzQ5IEM2LjM4NywxMC4yNTYgOS4zODMsNi40OTcgMTUuMDIyLDYuNDk3IEMxOS41NTUsNi40OTcgMjMuMDc4LDkuNzA1IDIzLjA3OCwxMy45OTEgQzIzLjA3OCwxOC40NjMgMjAuMjM5LDIyLjA2MiAxNi4yOTcsMjIuMDYyIEMxNC45NzMsMjIuMDYyIDEzLjcyOCwyMS4zNzkgMTMuMzAyLDIwLjU3MiBDMTMuMzAyLDIwLjU3MiAxMi42NDcsMjMuMDUgMTIuNDg4LDIzLjY1NyBDMTIuMTkzLDI0Ljc4NCAxMS4zOTYsMjYuMTk2IDEwLjg2MywyNy4wNTggQzEyLjA4NiwyNy40MzQgMTMuMzg2LDI3LjYzNyAxNC43MzMsMjcuNjM3IEMyMS45NSwyNy42MzcgMjcuODAxLDIxLjgyOCAyNy44MDEsMTQuNjYyIEMyNy44MDEsNy40OTUgMjEuOTUsMS42ODYgMTQuNzMzLDEuNjg2IiBmaWxsPSIjYmQwODFjIj48L3BhdGg+PC9nPjwvc3ZnPg==) 3px 50% no-repeat;
  background-size: 14px 14px;
  -webkit-font-smoothing: antialiased;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save:hover { 
  background-color: #aa0719;
  box-shadow: none;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save .PIN_1498785571564_count { 
  text-indent: 0;
  position: absolute;
  color: #555;
  background: #efefef;
  border-radius: 2px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save .PIN_1498785571564_count::before { 
  content: "";
  position: absolute;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save.PIN_1498785571564_beside .PIN_1498785571564_count { 
  right: -46px;
  height: 20px;
  width: 40px;
  font-size: 10px;
  line-height: 20px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save.PIN_1498785571564_beside .PIN_1498785571564_count::before { 
  top: 3px;
  left: -4px;
  border-right: 7px solid #efefef;
  border-top: 7px solid transparent;
  border-bottom: 7px solid transparent;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save.PIN_1498785571564_above .PIN_1498785571564_count { 
  top: -36px;
  width: 100%;
  height: 30px;
  font-size: 10px;
  line-height: 30px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save.PIN_1498785571564_above .PIN_1498785571564_count::before { 
  bottom: -4px;
  left: 4px;
  border-top: 7px solid #efefef;
  border-right: 7px solid transparent;
  border-left: 7px solid transparent;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_save.PIN_1498785571564_above.PIN_1498785571564_padded { 
  margin-top: 28px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall { 
  height: 28px;
  width: 56px;
  border-radius: 3px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_above .PIN_1498785571564_count { 
  position: absolute;
  height: 37px;
  width: inherit;
  top: -37px;
  left: 0;
  line-height: 30px;
  font-size: 14px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iNzZweCIgd2lkdGg9IjExNHB4IiB2aWV3Qm94PSIwIDAgMTE0IDc2Ij48Zz48cGF0aCBkPSJNOSAxQzQuNiAxIDEgNC42IDEgOXY0M2MwIDQuMyAzLjYgOCA4IDhoMjZsMTggMTVoNy41bDE2LTE1SDEwNWM0LjQgMCA4LTMuNyA4LThWOWMwLTQuNC0zLjYtOC04LThIOXoiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2I1YjViNSIgc3Ryb2tlLXdpZHRoPSIyIj48L3BhdGg+PC9nPjwvc3ZnPg==);
  background-size: cover;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_above.PIN_1498785571564_padded { 
  margin-top: 37px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_beside .PIN_1498785571564_count { 
  text-indent: 5px;
  position: absolute;
  right: -63px;
  height: inherit;
  width: 63px;
  font-size: 14px;
  line-height: 28px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iNTZweCIgd2lkdGg9IjEyNnB4IiB2aWV3Qm94PSIyIDAgMTMwIDYwIj48Zz48cGF0aCBkPSJNMTE5LjYgMmM0LjUgMCA4IDMuNiA4IDh2NDBjMCA0LjQtMy41IDgtOCA4SDIzLjNMMS42IDMyLjR2LTQuNkwyMy4zIDJoOTYuM3oiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2I1YjViNSIgc3Ryb2tlLXdpZHRoPSIyIj48L3BhdGg+PC9nPjwvc3ZnPg==);
  background-size: cover;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_beside.PIN_1498785571564_padded { 
  margin-right: 63px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save { 
  border-radius: 4px;
  width: auto;
  background-position-x: 6px;
  background-size: 18px 18px;
  text-indent: 29px;
  font: 14px/28px "Helvetica Neue", Helvetica, Arial, "sans-serif";
  font-weight: bold;
  padding: 0 6px 0 0;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save .PIN_1498785571564_count { 
  position: absolute;
  color: #555;
  font-size: 12px;
  text-indent: 0;
  background: #efefef;
  border-radius: 4px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save .PIN_1498785571564_count::before { 
  content: "";
  position: absolute;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save.PIN_1498785571564_above .PIN_1498785571564_count { 
  font-size: 14px;
  top: -50px;
  width: 100%;
  height: 44px;
  line-height: 44px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save.PIN_1498785571564_above .PIN_1498785571564_count::before { 
  bottom: -4px;
  left: 7px;
  border-top: 7px solid #efefef;
  border-right: 7px solid transparent;
  border-left: 7px solid transparent;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save.PIN_1498785571564_beside .PIN_1498785571564_count { 
  font-size: 14px;
  right: -63px;
  width: 56px;
  height: 28px;
  line-height: 28px;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_tall.PIN_1498785571564_save.PIN_1498785571564_beside .PIN_1498785571564_count::before { 
  top: 7px;
  left: -4px;
  border-right: 7px solid #efefef;
  border-top: 7px solid transparent;
  border-bottom: 7px solid transparent;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_round { 
  height: 16px;
  width: 16px;
  background: transparent url(data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMzBweCIgd2lkdGg9IjMwcHgiIHZpZXdCb3g9Ii0xIC0xIDMxIDMxIj48Zz48cGF0aCBkPSJNMjkuNDQ5LDE0LjY2MiBDMjkuNDQ5LDIyLjcyMiAyMi44NjgsMjkuMjU2IDE0Ljc1LDI5LjI1NiBDNi42MzIsMjkuMjU2IDAuMDUxLDIyLjcyMiAwLjA1MSwxNC42NjIgQzAuMDUxLDYuNjAxIDYuNjMyLDAuMDY3IDE0Ljc1LDAuMDY3IEMyMi44NjgsMC4wNjcgMjkuNDQ5LDYuNjAxIDI5LjQ0OSwxNC42NjIiIGZpbGw9IiNmZmYiIHN0cm9rZT0iI2ZmZiIgc3Ryb2tlLXdpZHRoPSIxIj48L3BhdGg+PHBhdGggZD0iTTE0LjczMywxLjY4NiBDNy41MTYsMS42ODYgMS42NjUsNy40OTUgMS42NjUsMTQuNjYyIEMxLjY2NSwyMC4xNTkgNS4xMDksMjQuODU0IDkuOTcsMjYuNzQ0IEM5Ljg1NiwyNS43MTggOS43NTMsMjQuMTQzIDEwLjAxNiwyMy4wMjIgQzEwLjI1MywyMi4wMSAxMS41NDgsMTYuNTcyIDExLjU0OCwxNi41NzIgQzExLjU0OCwxNi41NzIgMTEuMTU3LDE1Ljc5NSAxMS4xNTcsMTQuNjQ2IEMxMS4xNTcsMTIuODQyIDEyLjIxMSwxMS40OTUgMTMuNTIyLDExLjQ5NSBDMTQuNjM3LDExLjQ5NSAxNS4xNzUsMTIuMzI2IDE1LjE3NSwxMy4zMjMgQzE1LjE3NSwxNC40MzYgMTQuNDYyLDE2LjEgMTQuMDkzLDE3LjY0MyBDMTMuNzg1LDE4LjkzNSAxNC43NDUsMTkuOTg4IDE2LjAyOCwxOS45ODggQzE4LjM1MSwxOS45ODggMjAuMTM2LDE3LjU1NiAyMC4xMzYsMTQuMDQ2IEMyMC4xMzYsMTAuOTM5IDE3Ljg4OCw4Ljc2NyAxNC42NzgsOC43NjcgQzEwLjk1OSw4Ljc2NyA4Ljc3NywxMS41MzYgOC43NzcsMTQuMzk4IEM4Ljc3NywxNS41MTMgOS4yMSwxNi43MDkgOS43NDksMTcuMzU5IEM5Ljg1NiwxNy40ODggOS44NzIsMTcuNiA5Ljg0LDE3LjczMSBDOS43NDEsMTguMTQxIDkuNTIsMTkuMDIzIDkuNDc3LDE5LjIwMyBDOS40MiwxOS40NCA5LjI4OCwxOS40OTEgOS4wNCwxOS4zNzYgQzcuNDA4LDE4LjYyMiA2LjM4NywxNi4yNTIgNi4zODcsMTQuMzQ5IEM2LjM4NywxMC4yNTYgOS4zODMsNi40OTcgMTUuMDIyLDYuNDk3IEMxOS41NTUsNi40OTcgMjMuMDc4LDkuNzA1IDIzLjA3OCwxMy45OTEgQzIzLjA3OCwxOC40NjMgMjAuMjM5LDIyLjA2MiAxNi4yOTcsMjIuMDYyIEMxNC45NzMsMjIuMDYyIDEzLjcyOCwyMS4zNzkgMTMuMzAyLDIwLjU3MiBDMTMuMzAyLDIwLjU3MiAxMi42NDcsMjMuMDUgMTIuNDg4LDIzLjY1NyBDMTIuMTkzLDI0Ljc4NCAxMS4zOTYsMjYuMTk2IDEwLjg2MywyNy4wNTggQzEyLjA4NiwyNy40MzQgMTMuMzg2LDI3LjYzNyAxNC43MzMsMjcuNjM3IEMyMS45NSwyNy42MzcgMjcuODAxLDIxLjgyOCAyNy44MDEsMTQuNjYyIEMyNy44MDEsNy40OTUgMjEuOTUsMS42ODYgMTQuNzMzLDEuNjg2IiBmaWxsPSIjYmQwODFjIj48L3BhdGg+PC9nPjwvc3ZnPg==) 0 0 no-repeat;
  background-size: 16px 16px;
  box-shadow: none;
}
 .PIN_1498785571564_button_pin.PIN_1498785571564_round.PIN_1498785571564_tall { 
  height: 32px;
  width: 32px;
  background-size: 32px 32px;
}
</style><style type="text/css">.trc_rbox_container{direction:ltr;text-align:left}.trc_rbox_header{border-style:solid;border-width:0}.trc_rbox_header_icon_div{display:table-cell;vertical-align:baseline}.trc_rbox_header_icon_img{vertical-align:middle}.trc_rbox_header_icon_span{display:inline-table}.in_trc_header{position:relative!important;float:right;margin:0}#trc_rbox_css_loaded{overflow:hidden;width:0;height:0}.trc_rbox{margin-top:0}.trc_rbox_div{margin:0 0 3px;direction:ltr;padding:0;box-sizing:border-box;-moz-box-sizing:border-box;-ms-box-sizing:border-box;-webkit-box-sizing:border-box;overflow:auto;position:relative;width:auto;*width:auto;border:solid #CCC 1px}.loading-animation span{display:block}.videoCube{zoom:1;cursor:pointer;float:none;overflow:hidden;box-sizing:border-box;-moz-box-sizing:border-box;-ms-box-sizing:border-box;-webkit-box-sizing:border-box}div.videoCube:hover,.videoCube_hover{cursor:pointer}.videoCube span.video-title:hover,.videoCube_hover span.video-title{text-decoration:underline}.videoCube a{text-decoration:none;border:0;color:#000;cursor:pointer}.videoCube a:hover,.videoCube_hover a,.videoCube a:link,.videoCube a{text-decoration:none!important;outline:0}.videoCube a .thumbBlock{float:left;display:block;overflow:hidden!important}.videoCube a img,.videoCube img{border:0;_border:0;display:block;margin:0;height:auto;width:auto}.videoCube .video-label{display:block;overflow:hidden}.videoCube .video-label{width:auto!important;white-space:pre-wrap;white-space:-moz-pre-wrap;white-space:-o-pre-wrap;word-wrap:break-word}.video-icon-container{float:left;height:100%}.videoCube .video-duration a{}.videoCube .video-duration{height:0;float:left;position:relative;color:#fff;font-size:11px}.videoCube .video-duration dt{border-radius:4px;-moz-border-radius:4px;-webkit-border-radius:4px;background-color:#000;opacity:.6;filter:alpha(opacity=60)}.videoCube span.video-label.trc_ellipsis{position:relative;overflow:hidden;display:-webkit-box;-webkit-box-orient:vertical}.videoCube span.video-label.trc-smart-ellipsis{position:relative;overflow:hidden}.videoCube span.video-label.trc-smart-ellipsis ins{display:inline-block;text-decoration:inherit}.videoCube span.video-label.trc-smart-ellipsis ins.lastLineEllipsis{display:block;overflow:hidden;text-overflow:ellipsis;white-space:nowrap;word-wrap:normal}.sponsored-default .video-description{font-weight:400;color:#000;font-size:9px;max-height:22px;*height:22px;overflow:hidden;line-height:9px}.video-duration.video-duration-detail div{color:#fff}.sponsored-default{background-color:#f7f6c6;padding-bottom:0!important}.trc_rbox .sponsored{position:relative;display:block;overflow:visible;height:auto;width:auto;padding-right:0;text-align:right;font-size:9px}.sponsored-url{white-space:nowrap;float:left;overflow:hidden;width:98%;height:auto;margin-top:0;margin-left:1px;padding-bottom:2px;color:green;font-size:9px;text-decoration:underline}.sponsored-default .video-title{height:26px;font-size:11px;line-height:13px;max-height:26px;*height:26px}.sponsored-default .thumbBlock img{}.sponsored-default .thumbBlock{text-align:center;background-color:transparent}.trc_rbox_div{height:410px}.videoCube{direction:ltr;font-size:11px;margin:0;color:#000;border-width:0}.videoCube.vertical:first-child{border-top:0;margin-top:0}.videoCube.horizontal:first-child{border-left:0;margin-left:0}.trc_rbox .trc_rbox_div .videoCube.vertical{_margin-top:expression(this.previousSibling==null?'0':'default')}.trc_rbox .trc_rbox_div .videoCube.horizontal{_margin-left:expression(this.previousSibling==null?'0':'default')}div.videoCube:hover,.videoCube_hover{background-color:#EBF0FF;color:#000}.videoCube .thumbBlock{margin-right:5px;margin-left:1px;border-style:solid}.videoCube a img,.videoCube img{border-color:#ececec}.videoCube .overlayImg{}.videoCube .video-label-box{margin-left:81px}.videoCube .video-label dt{font-weight:700}.videoCube .video-title{height:auto;margin-bottom:3px;white-space:normal}.videoCube .trc_inline_detail_spacer{display:inline-block;white-space:pre}.sponsored-default{padding-bottom:0}.loading-animation{font-family:sans;font-size:1.5em;text-align:center;color:gray;height:100%}.trc_rbox_header{font-family:Arial,Helvetica,sans-serif;font-size:12px;font-weight:700;text-decoration:none;color:#000}.trc_header_right_part{position:absolute;left:50%;top:0}.branding_div{overflow:visible;float:right}.branding_div img{height:20px}.trc_pager div{float:left;font-family:sans;cursor:pointer;margin-left:2px;margin-right:2px}.trc_pager_prev,.trc_pager_next{}.trc_pager{font-weight:400}.trc_pager_pages{display:block!important}.trc_related_container div.horizontal{float:left;box-sizing:border-box;-moz-box-sizing:border-box;-ms-box-sizing:border-box;-webkit-box-sizing:border-box}.trc_related_container div.vertical{}.trc_related_container DIV.videoCube.thumbnail_top .thumbBlock,.trc_related_container DIV.videoCube.thumbnail_bottom .thumbBlock{float:none}@media screen and (-webkit-min-device-pixel-ratio:0){.sponsored-url{margin-top:2px}}.vidiscovery-note{display:none}.pager_disabled{cursor:auto}.videoCube .thumbBlock .trc_sponsored_overlay_base{display:block;width:auto;margin-left:0;position:absolute;color:#fff!important}.videoCube .thumbBlock .trc_sponsored_overlay{filter:alpha(opacity=60);opacity:.6;display:block;position:absolute;*left:0}.videoCube .thumbBlock .trc_sponsored_overlay_base .sponsored{position:relative;display:block;overflow:visible;width:auto;text-align:center;padding:0 5px;margin-top:0}.videoCube .thumbBlock .trc_sponsored_overlay_base.round .trc_sponsored_overlay{border-radius:4px;-moz-border-radius:4px;-webkit-border-radius:4px}.videoCube .thumbBlock .trc_sponsored_overlay_base.round{margin-left:4px}.thumbnail-emblem{background-color:transparent;background-repeat:no-repeat;position:absolute;z-index:50}.playerCube .thumbnail-overlay,.videoCube .thumbnail-overlay{position:absolute;background-color:transparent;background-repeat:no-repeat;z-index:50}.videoCube:hover .thumbnail-overlay,.videoCube_hover .thumbnail-overlay{position:absolute;background-color:transparent;background-repeat:no-repeat;z-index:50}.thumbnail_bottom{padding-bottom:8px}.trc_pager_next img,trc_pager_next pager_disabled img,trc_pager_next:hover img{-moz-transform:scaleX(-1);-o-transform:scaleX(-1);-webkit-transform:scaleX(-1);transform:scaleX(-1);filter:FlipH;-ms-filter:"FlipH"}.trc_cover_iframe{position:absolute;z-index:100000;top:0;left:0;background:#000;opacity:.4;filter:alpha(opacity=40);border:0}.trc_lightbox_base{z-index:100001;position:fixed}.trc_lightbox_overlay{background-color:transparent;position:absolute;z-index:100002}.trc_lightbox_transparent{position:absolute;z-index:100003}#trc_lightbox_header{font-family:"Trebuchet MS",Helvetica,sans-serif}.trc_whitebox{background-color:#fff;position:relative;z-index:100004;padding:0 20px 20px;margin:0 auto;border:2px solid #000;-moz-box-shadow:3px 3px 20px #000;-webkit-box-shadow:3px 3px 20px #000;box-shadow:3px 3px 20px #000;-moz-box-sizing:content-box;-webkit-box-sizing:content-box;box-sizing:content-box;-moz-border-radius:5px;-webkit-border-radius:5px;border-radius:5px}.trc_whiteboxCloseButton{width:32px;height:32px;border:0;position:absolute;right:-16px;top:-14px;cursor:pointer;background:url(//cdn.taboola.com/taboola-generic/close.png) no-repeat top left transparent;color:#fff;color:transparent;font-size:0}.trc_related_container .logoDiv{font-family:Arial,Helvetica,sans-serif;white-space:nowrap;font-size:9px}.trc_related_container .logoDiv a{font-size:9px;text-decoration:none!important;color:#000;margin-right:1px;vertical-align:text-bottom}.logoDiv a span:hover{text-decoration:underline}.trc_rbox_header .logoDiv{font-size:1em}.trc_related_container .vertical-start-shade{background-image:url(//cdn.taboola.com/taboola-generic/shade-down.png);height:10px;width:auto;position:absolute;z-index:5}.trc_related_container .vertical-end-shade{background-image:url(//cdn.taboola.com/taboola-generic/shade-up.png);height:10px;width:auto;position:absolute;z-index:5}.trc_related_container .horizontal-start-shade{background-image:url(//cdn.taboola.com/taboola-generic/shade-right.png);height:auto;width:10px;position:absolute;z-index:5}.trc_related_container .horizontal-end-shade{background-image:url(//cdn.taboola.com/taboola-generic/shade-left.png);height:auto;width:10px;position:absolute;z-index:5}.trc_rbox_container.trc_expandable{overflow:hidden;max-height:0;transition-property:max-height;-webkit-transition-property:max-height;-moz-transition-property:max-height;-o-transition-property:max-height;-webkit-transform:translateZ(0);-moz-transform:translateZ(0);-ms-transform:translateZ(0);-o-transform:translateZ(0);transform:translateZ(0)}DIV.trc_autoplaylist-box{margin:6px auto 0;border:0;-moz-border-radius:5px;-webkit-border-radius:5px;border-radius:5px;padding:4px;font-family:"Trebuchet MS",Helvetica,sans-serif;width:320px;background:#191919;filter:progid:DXImageTransform.Microsoft.gradient(startColorstr='#191919', endColorstr='#393939');-ms-filter:"progid:DXImageTransform.Microsoft.gradient(startColorstr=#191919, endColorstr=#393939)";background:-webkit-gradient(linear,left top,left bottom,from(#191919),to(#393939));background:-moz-linear-gradient(top,#191919,#393939);background:linear-gradient(top,#191919,#393939)}DIV.trc_autoplaylist-box .trc-next-up-header{font-size:14px;color:#CCC;float:left;line-height:31px;margin:0 4px 0 6px;overflow:hidden;white-space:nowrap}DIV.trc_autoplaylist-box .trc-next-up-countdown{color:#fff;font-weight:700;font-size:15px;padding-left:4px}DIV.trc_autoplaylist-box .nextup{cursor:pointer}DIV.trc_autoplaylist-box .nextup .thumbblock{border:solid #FEFEFE 1px;margin-right:10px}DIV.trc_autoplaylist-box:hover .nextup .thumbblock SPAN.thumbnail-overlay{background-image:url(//cdn.taboola.com/taboola-generic/lightbox-overlay.png)}DIV.trc_autoplaylist-box .nextup .video-label-box{color:#fff;font-weight:700;font-size:12px;line-height:14px}.trc-syndication-link{text-decoration:none;color:inherit}.playerCube{width:100%;height:100%;position:relative;cursor:pointer}.playerCube .thumbnail-overlay{width:100%;height:100%;top:0;left:0;position:absolute}.trc_related_container .whatsThisSyndicated{position:absolute;cursor:pointer}.trc_related_container .whatsThisSyndicated.a-bottom-right,.trc_related_container .static-text.bottom-right{bottom:0;right:0}.trc_related_container .whatsThisSyndicated.a-top-right,.trc_related_container .static-text.top-right{top:0;right:0}.trc_related_container .whatsThisSyndicated.a-bottom-left,.trc_related_container .static-text.bottom-left{bottom:0;left:0}.trc_related_container .whatsThisSyndicated.a-top-left,.trc_related_container .static-text.top-left{top:0;left:0}.trc_related_container .videoCube .thumbBlock .branding{position:absolute;bottom:0;z-index:1;width:100%;margin:0;padding:5px 0;text-align:center}.syndicatedItem .branding{margin:0}.trc_related_container .videoCube .thumbBlock .static-text{position:absolute;z-index:1;margin:0;padding:5px;background-color:#000;color:#fff;display:block;font-family:Arial,Helvetica,sans-serif;font-size:10px;font-weight:400;text-align:left;text-decoration:none;opacity:.7}.trc_related_container .static-text.top{width:100%;top:0;padding:5px 0}.trc_related_container .static-text.bottom{width:100%;bottom:0;padding:2px 0}.trc-inplayer-rbox{background:#333;background:rgba(30,30,30,.9);*background:#333;bottom:0;position:absolute;height:300px;text-align:center}.trc-inplayer-rbox .trc_rbox_container{margin:50px auto 0;width:640px}.trc_rbox.trc-auto-size{width:100%;height:100%}.videoCube.thumbnail_under .thumbBlock{margin-left:0;margin-right:0}.videoCube.thumbnail_under .label-box-overlay{width:100%;height:100%;position:absolute;background:#000;opacity:.75;filter:alpha(opacity=75);top:0;_display:none}.videoCube.thumbnail_under .video-labels-anchor{width:100%;height:auto;position:absolute;z-index:1;left:0;bottom:0;min-height:2.58em;max-height:2.58em;padding-top:2px;padding-bottom:2px;_background-color:#CECECE;-webkit-transition:all .2s linear;-moz-transition:all .2s linear;-ms-transition:all .2s linear;-o-transition:all .2s linear;transition:all .2s linear;line-height:1.25em}.videoCube.thumbnail_under .video-labels-anchor span.video-title,.videoCube.thumbnail_under .video-labels-anchor span.branding{position:relative;z-index:1;padding:0 3px;margin:0}.videoCube.thumbnail_under .video-title{min-height:2.58em}.videoCube.thumbnail_under:hover .video-labels-anchor{opacity:1;max-height:6.45em}.trc-auto-size .trc_rbox_outer .trc_rbox_div{height:auto;width:auto}.trc-auto-size .trc_rbox_div .videoCube{height:auto}.trc-auto-size .trc_rbox_div .videoCube.trc-first-recommendation{margin-top:0}.trc_rbox .trc_rbox_outer .trc_rbox_div .videoCube.trc-first-in-row{margin-left:0}.trc_elastic .trc_rbox{width:auto}.trc_elastic .videoCube{overflow:hidden}.trc_elastic .videoCube .thumbBlock{display:block;background-color:transparent;background-position:center center;background-repeat:no-repeat;background-size:cover;position:absolute;display:inline-block;top:0;right:0;bottom:0;left:0;margin-left:0;margin-right:0}.trc_elastic .thumbBlock_holder{position:relative;width:100%}.trc_elastic .thumbnail_start .thumbBlock_holder{float:left;margin-right:10px}.trc_elastic .thumbnail_start.item-has-pre-label .thumbBlock_holder{margin-right:0}.trc_elastic .videoCube_aspect{width:1px}.trc_elastic .trc_rbox .trc_rbox_div{height:auto}.trc_elastic .thumbnail_start .trc-pre-label{float:left;padding-right:10px}.trc_elastic .thumbnail_start.trc-split-label .trc-main-label{float:left;padding-left:10px}.trc_elastic .thumbnail_start .video-label-box{box-sizing:border-box}.trc_elastic .video-label-box{display:block}.trc_user_adChoice_btn{background:url(//cdn.taboola.com/libtrc/static/thumbnails/3aadbb71016ecd7a194bbe2791777e9a.png) no-repeat scroll 0 0 rgba(255,255,255,.7);border-radius:0 0 0 5px;width:16px;height:16px;position:absolute;right:0;top:0;z-index:9000;cursor:pointer;visibility:hidden}.trc_user_adChoice_anchor{width:100%;height:100%;position:absolute}.videoCube:hover .trc_user_adChoice_btn,.videoCube_hover .trc_user_adChoice_btn{visibility:visible}.trc_user_exclude_btn{background:url(//cdn.taboola.com/libtrc/static/thumbnails/f539211219b796ffbb49949997c764f0.png) no-repeat scroll 0 0 transparent;width:12px;height:12px;position:absolute;right:2px;top:2px;z-index:9000;cursor:pointer;visibility:hidden}.trc_undo_btn{font-family:Arial,Helvetica,sans-serif;font-size:11px;line-height:14px;font-weight:400;color:#36C;text-decoration:underline;cursor:pointer;position:absolute;right:2px;top:2px;padding:0 1px;z-index:11000;visibility:hidden}.videoCube:hover .trc_user_exclude_btn,.videoCube_hover .trc_user_exclude_btn,.trc_user_excluded.videoCube:hover .trc_undo_btn,.trc_user_excluded.videoCube_hover .trc_undo_btn,.trc_undo_btn.trc_anchor{visibility:visible}.videoCube.trc_user_excluded .trc_user_exclude_btn{visibility:hidden}.trc_fade{opacity:0;filter:alpha(opacity=0);visibility:hidden;transition:opacity 500ms 0s,visibility 0s 500ms;-webkit-transition:opacity 500ms 0s,visibility 0s 500ms;-moz-transition:opacity 500ms 0s,visibility 0s 500ms;-o-transition:opacity 500ms 0s,visibility 0s 500ms;-ms-transition:opacity 500ms 0s,visibility 0s 500ms}.trc_fade.trc_in,.trc_user_excluded .trc_exclude_overlay{visibility:visible;opacity:1;filter:alpha(opacity=100);transition-delay:0s,0s;-moz-transition-delay:0s,0s;-webkit-transition-delay:0s,0s;-o-transition-delay:0s,0s;-ms-transition-delay:0s,0s}.trc_excludable .trc_exclude_overlay{position:absolute;z-index:10000;top:0;left:0;width:100%;height:100%;cursor:default;background-color:#fff}.videoCube.trc_excludable .trc_exclude_overlay.trc_fade{filter:alpha(opacity=80)\9}.videoCube.trc_user_excluded .trc_exclude_overlay{visibility:visible;opacity:.8;filter:alpha(opacity=80)}.videoCube.trc_user_excluded .thumbBlock{filter:url("data:image/svg+xml;utf8,<svg xmlns='http://www.w3.org/2000/svg'><filter id='grayscale'><feColorMatrix type='matrix' values='0.3333 0.3333 0.3333 0 0 0.3333 0.3333 0.3333 0 0 0.3333 0.3333 0.3333 0 0 0 0 0 1 0' /></filter></svg>#grayscale");filter:gray;-webkit-filter:grayscale(100%)}.videoCube.trc_user_excluded:hover a .video-label-box .video-title,.videoCube_hover.trc_user_excluded a .video-label-box .video-title{text-decoration:none}.videoCube.trc_user_excluded a .video-label-box *,.videoCube.trc_user_excluded:hover a .video-label-box *,.videoCube_hover.trc_user_excluded a .video-label-box *{color:#000;overflow:hidden;-webkit-transition:color 500ms 0s;-moz-transition:color 500ms 0s;-ms-transition:color 500ms 0s;-o-transition:color 500ms 0s;transition:color 500ms 0s}.videoCube.trc_user_excluded a .video-label-box .label-box-overlay{background-color:#BBB;-webkit-transition:background-color 500ms 0s;-moz-transition:background-color 500ms 0s;-ms-transition:background-color 500ms 0s;-o-transition:background-color 500ms 0s;transition:background-color 500ms 0s}.videoCube.thumbnail_under.trc_user_excluded .video-labels-anchor,.videoCube.thumbnail_under.trc_user_excluded:hover .video-labels-anchor{max-height:none;-webkit-transition:none;-moz-transition:none;-ms-transition:none;-o-transition:none;transition:none}.trc_related_container .trc_clearer{clear:both;height:0;overflow:hidden;font-size:0;line-height:0;visibility:hidden}.link-adc{float:right!important}.trc-widget-footer .link-adc a .trc_adc_wrapper,.trc_header_ext .link-adc a .trc_adc_wrapper{vertical-align:baseline;height:15px;width:19px;display:inline-block;position:relative}.trc-widget-footer .link-adc a .trc_adc_s_logo,.trc_header_ext .link-adc a .trc_adc_s_logo,.trc-widget-footer .link-adc a .trc_adc_b_logo,.trc_header_ext .link-adc a .trc_adc_b_logo{vertical-align:middle;height:15px;display:inline-block;margin-top:-1px}.trc-widget-footer .link-adc a .trc_adc_s_logo,.trc_header_ext .link-adc a .trc_adc_s_logo{width:19px;background:url(//cdn.taboola.com/libtrc/static/thumbnails/3aadbb71016ecd7a194bbe2791777e9a.png) no-repeat!important}.trc-widget-footer .link-adc a .trc_adc_b_logo,.trc_header_ext .link-adc a .trc_adc_b_logo{width:77px;background:#fff url(//cdn.taboola.com/libtrc/static/thumbnails/0781f9c5a8637d1e162874f157460048.png) no-repeat!important;right:-1px;display:none;position:absolute}.trc_mobile_disclosure_link,.trc_mobile_attribution_link,.trc_mobile_adc_link{display:none}.trc_desktop_disclosure_link,.trc_desktop_attribution_link,.trc_desktop_adc_link{display:inline}@media screen and (max-width:767px){.trc_mobile_disclosure_link{display:inline}.trc_mobile_attribution_link{display:inline}.trc_mobile_adc_link{display:inline}.trc_desktop_disclosure_link{display:none}.trc_desktop_attribution_link{display:none}.trc_desktop_adc_link{display:none}}.trc_in_iframe .trc_mobile_attribution_link,.trc_in_iframe .trc_mobile_disclosure_link{display:inline}.trc_in_iframe .trc_desktop_attribution_link,.trc_in_iframe .trc_desktop_disclosure_link{display:none}.trc_related_container .logoDiv,.trc_related_container .trc_header_ext .logoDiv{float:right}.trc_related_container .logoDiv+.logoDiv{margin-right:2px}.trc_related_container .attribution-disclosure-link-sponsored,.trc_related_container .attribution-disclosure-link-hybrid{display:none}.trc_related_container .trc-content-sponsored .attribution-disclosure-link-sponsored{display:block}.trc_related_container .trc-content-hybrid .attribution-disclosure-link-hybrid{display:block}.trc_related_container .trc-widget-footer:hover a span,.trc_related_container .trc_header_ext:hover a span{text-decoration:underline!important}.logoDiv a span.trc_logos_v_align{display:inline-block!important;font-size:15px!important;line-height:1em!important;width:0!important}.trc_related_container .trc_header_ext:hover a span.trc_logos_v_align,.trc_related_container .trc_header_ext:hover a span.trc_adc_wrapper,.trc_related_container .trc-widget-footer:hover a span.trc_logos_v_align,.trc_related_container .trc-widget-footer:hover a span.trc_adc_wrapper{text-decoration:none!important}.trc_related_container{clear:both}.tbl-loading-spinner{width:100%;height:40px;background:url(//cdn.taboola.com/static/91/91a25024-792d-4b52-84e6-ad1478c3f552.gif) center center no-repeat;background-size:40px}.tbl-hidden{display:none!important}.ab_thumbnails-a_abp-mode .video-title{font-family:Arial, Helvetica, sans-serif;font-size:17.0px;line-height:23.0px;font-weight:bold;max-height:69.0px;*height:69.0px;color:#444444;text-decoration:none;}.ab_thumbnails-a_abp-mode .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.ab_thumbnails-a_abp-mode .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.ab_thumbnails-a_abp-mode .videoCube .video-duration{left:36px;display:none;}.ab_thumbnails-a_abp-mode .videoCube .video-label-box{margin-left:0;margin-right:0px;}.ab_thumbnails-a_abp-mode .video-label,.ab_thumbnails-a_abp-mode .sponsored,.ab_thumbnails-a_abp-mode .sponsored-url{font-family:NotoNashkArabic;}.ab_thumbnails-a_abp-mode .trc_rbox_header{font-family:Proxima Nova Condensed;font-size:28.0px;font-weight:bold;text-decoration:none;color:#000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.ab_thumbnails-a_abp-mode .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.ab_thumbnails-a_abp-mode .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.ab_thumbnails-a_abp-mode .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.ab_thumbnails-a_abp-mode .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.ab_thumbnails-a_abp-mode .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.ab_thumbnails-a_abp-mode div.videoCube:hover,.ab_thumbnails-a_abp-mode  div.videoCube_hover{background-color:transparent;}.ab_thumbnails-a_abp-mode .sponsored-default{background-color:#F7F6C6;}.ab_thumbnails-a_abp-mode div.sponsored-default:hover,.ab_thumbnails-a_abp-mode  div.sponsored-default.videoCube_hover{background-color:inherit;}.ab_thumbnails-a_abp-mode .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.ab_thumbnails-a_abp-mode .videoCube:hover .thumbnail-overlay,.ab_thumbnails-a_abp-mode  .videoCube_hover .thumbnail-overlay{background-image:null;}.ab_thumbnails-a_abp-mode .trc_rbox_border_elm{border-color:darkgray;}.ab_thumbnails-a_abp-mode .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.ab_thumbnails-a_abp-mode div.videoCube:hover .thumbBlock{border-color:inherit;}.ab_thumbnails-a_abp-mode .pager_enabled{color:#0056b3;}.ab_thumbnails-a_abp-mode .trc_pager_counter{color:#000000;}.ab_thumbnails-a_abp-mode .pager_disabled{color:#7d898f;}.ab_thumbnails-a_abp-mode .trc_pager_prev:hover,.ab_thumbnails-a_abp-mode  .trc_pager_next:hover{color:#6497ED;}.ab_thumbnails-a_abp-mode .trc_pager_selected{color:#0056b3;}.ab_thumbnails-a_abp-mode .trc_pager_unselected{color:#7d898f;}.ab_thumbnails-a_abp-mode div.trc_pager_pages div:hover{color:#6497ED;}.ab_thumbnails-a_abp-mode .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.ab_thumbnails-a_abp-mode .video-label-box{text-align:left;}.ab_thumbnails-a_abp-mode .trc_sponsored_overlay{background-color:black;}.ab_thumbnails-a_abp-mode .thumbnail-emblem{background-position:5% 5%;}.ab_thumbnails-a_abp-mode .videoCube .sponsored{margin-top:-7px;}.ab_thumbnails-a_abp-mode {width:300px;_width:300px;border-width:2px 0px 2px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:5px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.ab_thumbnails-a_abp-mode .videoCube.vertical{border-style:solid none none none;}.ab_thumbnails-a_abp-mode .videoCube.horizontal{border-style:none;}.ab_thumbnails-a_abp-mode .trc_pager_prev,.ab_thumbnails-a_abp-mode .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .trc_pager div{font-family:serif;}.ab_thumbnails-a_abp-mode .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.ab_thumbnails-a_abp-mode .playerCube:hover .thumbnail-overlay,.ab_thumbnails-a_abp-mode  .playerCube_hover .thumbnail-overlay{background-image:null;}.ab_thumbnails-a_abp-mode .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.ab_thumbnails-a_abp-mode .playerCube .videoCube.horizontal{border-style:none none none none;}.ab_thumbnails-a_abp-mode .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.ab_thumbnails-a_abp-mode .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-label-box{text-align:left;}.ab_thumbnails-a_abp-mode .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.ab_thumbnails-a_abp-mode .playerCube .videoCube .video-duration{display:block;left:36px;}.ab_thumbnails-a_abp-mode .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.ab_thumbnails-a_abp-mode .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-a_abp-mode .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.ab_thumbnails-a_abp-mode .playerCube div.videoCube:hover,.ab_thumbnails-a_abp-mode  div.videoCube_hover{background-color:transparent;}.ab_thumbnails-a_abp-mode .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.ab_thumbnails-a_abp-mode div.syndicatedItem:hover,.ab_thumbnails-a_abp-mode  div.syndicatedItem.videoCube_hover{background-color:transparent;}.ab_thumbnails-a_abp-mode div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem.horizontal{border-style:none;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.ab_thumbnails-a_abp-mode .syndicatedItem{background-color:transparent;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-title{max-height:46.0px;*height:46.0px;color:#444444;font-family:NotoNashkArabic;font-size:17.0px;line-height:23.0px;font-weight:bold;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-a_abp-mode .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:NotoNashkArabic;background-image:null;text-align:left;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.ab_thumbnails-a_abp-mode .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.ab_thumbnails-a_abp-mode .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.ab_thumbnails-a_abp-mode .trc_rbox_header_icon_img{margin:0px;height:18px;}.ab_thumbnails-a_abp-mode .video-icon-img{margin:0px;height:18px;}.ab_thumbnails-a_abp-mode .video-label-box.trc-pre-label{height:auto;}.ab_thumbnails-a_abp-mode .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.ab_thumbnails-a_abp-mode .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.ab_thumbnails-a_abp-mode .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.ab_thumbnails-a_abp-mode .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.ab_thumbnails-a_abp-mode .videoCube .video-label-box.trc-pre-label{margin:0;}.ab_thumbnails-o_abp-mode .video-title{font-family:Arial, Helvetica, sans-serif;font-size:18.0px;line-height:24.0px;font-weight:bold;max-height:96.0px;*height:96.0px;color:#000000;text-decoration:none;}.ab_thumbnails-o_abp-mode .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.ab_thumbnails-o_abp-mode .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.ab_thumbnails-o_abp-mode .videoCube .video-duration{left:36px;display:none;}.ab_thumbnails-o_abp-mode .videoCube .video-label-box{margin-left:0;margin-right:0px;}.ab_thumbnails-o_abp-mode .video-label,.ab_thumbnails-o_abp-mode .sponsored,.ab_thumbnails-o_abp-mode .sponsored-url{font-family:ProximaNovaCond-Medium;}.ab_thumbnails-o_abp-mode .trc_rbox_header{font-family:ProximaNovaCond-Bold;font-size:20.0px;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.ab_thumbnails-o_abp-mode .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.ab_thumbnails-o_abp-mode .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.ab_thumbnails-o_abp-mode .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.ab_thumbnails-o_abp-mode .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.ab_thumbnails-o_abp-mode .videoCube{width:auto;_width:auto;background-color:transparent;border-width:0px 0px 0px 0px;border-color:#E4E4E4;padding:0px 0px 0px 0px;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:SOLID;}.ab_thumbnails-o_abp-mode div.videoCube:hover,.ab_thumbnails-o_abp-mode  div.videoCube_hover{background-color:transparent;}.ab_thumbnails-o_abp-mode .sponsored-default{background-color:#F7F6C6;}.ab_thumbnails-o_abp-mode div.sponsored-default:hover,.ab_thumbnails-o_abp-mode  div.sponsored-default.videoCube_hover{background-color:inherit;}.ab_thumbnails-o_abp-mode .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.ab_thumbnails-o_abp-mode .videoCube:hover .thumbnail-overlay,.ab_thumbnails-o_abp-mode  .videoCube_hover .thumbnail-overlay{background-image:null;}.ab_thumbnails-o_abp-mode .trc_rbox_border_elm{border-color:darkgray;}.ab_thumbnails-o_abp-mode .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.ab_thumbnails-o_abp-mode div.videoCube:hover .thumbBlock{border-color:inherit;}.ab_thumbnails-o_abp-mode .pager_enabled{color:#0056b3;}.ab_thumbnails-o_abp-mode .trc_pager_counter{color:#000000;}.ab_thumbnails-o_abp-mode .pager_disabled{color:#7d898f;}.ab_thumbnails-o_abp-mode .trc_pager_prev:hover,.ab_thumbnails-o_abp-mode  .trc_pager_next:hover{color:#6497ED;}.ab_thumbnails-o_abp-mode .trc_pager_selected{color:#0056b3;}.ab_thumbnails-o_abp-mode .trc_pager_unselected{color:#7d898f;}.ab_thumbnails-o_abp-mode div.trc_pager_pages div:hover{color:#6497ED;}.ab_thumbnails-o_abp-mode .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.ab_thumbnails-o_abp-mode .video-label-box{text-align:left;}.ab_thumbnails-o_abp-mode .trc_sponsored_overlay{background-color:black;}.ab_thumbnails-o_abp-mode .thumbnail-emblem{background-position:5% 5%;}.ab_thumbnails-o_abp-mode .videoCube .sponsored{margin-top:-7px;}.ab_thumbnails-o_abp-mode {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.ab_thumbnails-o_abp-mode .videoCube.vertical{border-style:solid none none none;}.ab_thumbnails-o_abp-mode .videoCube.horizontal{border-style:none;}.ab_thumbnails-o_abp-mode .trc_pager_prev,.ab_thumbnails-o_abp-mode .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .trc_pager div{font-family:serif;}.ab_thumbnails-o_abp-mode .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.ab_thumbnails-o_abp-mode .playerCube:hover .thumbnail-overlay,.ab_thumbnails-o_abp-mode  .playerCube_hover .thumbnail-overlay{background-image:null;}.ab_thumbnails-o_abp-mode .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.ab_thumbnails-o_abp-mode .playerCube .videoCube.horizontal{border-style:none none none none;}.ab_thumbnails-o_abp-mode .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.ab_thumbnails-o_abp-mode .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-label-box{text-align:left;}.ab_thumbnails-o_abp-mode .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.ab_thumbnails-o_abp-mode .playerCube .videoCube .video-duration{display:block;left:36px;}.ab_thumbnails-o_abp-mode .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.ab_thumbnails-o_abp-mode .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.ab_thumbnails-o_abp-mode .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.ab_thumbnails-o_abp-mode .playerCube div.videoCube:hover,.ab_thumbnails-o_abp-mode  div.videoCube_hover{background-color:transparent;}.ab_thumbnails-o_abp-mode .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.ab_thumbnails-o_abp-mode div.syndicatedItem:hover,.ab_thumbnails-o_abp-mode  div.syndicatedItem.videoCube_hover{background-color:transparent;}.ab_thumbnails-o_abp-mode div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem{background-color:transparent;border-color:#E4E4E4;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:0px 0px 0px 0px;border-style:SOLID;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem.horizontal{border-style:none;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.ab_thumbnails-o_abp-mode .syndicatedItem{background-color:transparent;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-title{max-height:72.0px;*height:72.0px;color:#000000;font-family:ProximaNovaCond-Medium;font-size:18.0px;line-height:24.0px;font-weight:bold;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.ab_thumbnails-o_abp-mode .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:ProximaNovaCond-Medium;background-image:null;text-align:left;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.ab_thumbnails-o_abp-mode .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.ab_thumbnails-o_abp-mode .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.ab_thumbnails-o_abp-mode .trc_rbox_header_icon_img{margin:0px;height:18px;}.ab_thumbnails-o_abp-mode .video-icon-img{margin:0px;height:18px;}.ab_thumbnails-o_abp-mode .video-label-box.trc-pre-label{height:auto;}.ab_thumbnails-o_abp-mode .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.ab_thumbnails-o_abp-mode .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.ab_thumbnails-o_abp-mode .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.ab_thumbnails-o_abp-mode .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.ab_thumbnails-o_abp-mode .videoCube .video-label-box.trc-pre-label{margin:0;}.rbox-blended .video-title{font-family:Arial, Helvetica, sans-serif;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;text-decoration:none;}.rbox-blended .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.rbox-blended .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:1px;padding:0;}.rbox-blended .videoCube .video-duration{left:36px;display:block;}.rbox-blended .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.rbox-blended .video-label,.rbox-blended .sponsored,.rbox-blended .sponsored-url{font-family:Arial, Helvetica, sans-serif;}.rbox-blended .trc_rbox_header{font-family:Arial, Helvetica, sans-serif;font-size:16px;font-weight:bold;text-decoration:none;color:black;border-width:0;background:transparent;border-style:none none solid none;border-color:#D6D5D3;padding:0;}.rbox-blended .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.rbox-blended .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.rbox-blended .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.rbox-blended .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.rbox-blended .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:3px;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.rbox-blended div.videoCube:hover,.rbox-blended  div.videoCube_hover{background-color:transparent;}.rbox-blended .sponsored-default{background-color:#F7F6C6;}.rbox-blended div.sponsored-default:hover,.rbox-blended  div.sponsored-default.videoCube_hover{background-color:inherit;}.rbox-blended .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.rbox-blended .videoCube:hover .thumbnail-overlay,.rbox-blended  .videoCube_hover .thumbnail-overlay{background-image:null;}.rbox-blended .trc_rbox_border_elm{border-color:darkgray;}.rbox-blended .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.rbox-blended div.videoCube:hover .thumbBlock{border-color:inherit;}.rbox-blended .pager_enabled{color:#0056b3;}.rbox-blended .trc_pager_counter{color:#000000;}.rbox-blended .pager_disabled{color:#7d898f;}.rbox-blended .trc_pager_prev:hover,.rbox-blended  .trc_pager_next:hover{color:#6497ED;}.rbox-blended .trc_pager_selected{color:#0056b3;}.rbox-blended .trc_pager_unselected{color:#7d898f;}.rbox-blended div.trc_pager_pages div:hover{color:#6497ED;}.rbox-blended .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.rbox-blended .video-label-box{text-align:left;}.rbox-blended .trc_sponsored_overlay{background-color:black;}.rbox-blended .thumbnail-emblem{background-position:5% 5%;}.rbox-blended .videoCube .sponsored{margin-top:-7px;}.rbox-blended {width:300px;_width:300px;border-width:0px;border-style:solid solid solid solid;border-color:#000000;padding:0;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.rbox-blended .videoCube.vertical{border-style:solid none none none;}.rbox-blended .videoCube.horizontal{border-style:none none none solid;}.rbox-blended .trc_pager_prev,.rbox-blended .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.rbox-blended .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.rbox-blended .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .trc_pager div{font-family:serif;}.rbox-blended .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.rbox-blended .playerCube:hover .thumbnail-overlay,.rbox-blended  .playerCube_hover .thumbnail-overlay{background-image:null;}.rbox-blended .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.rbox-blended .playerCube .videoCube.horizontal{border-style:none none none none;}.rbox-blended .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.rbox-blended .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-label-box{text-align:left;}.rbox-blended .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.rbox-blended .playerCube .videoCube .video-duration{display:block;left:36px;}.rbox-blended .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.rbox-blended .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.rbox-blended .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.rbox-blended .playerCube div.videoCube:hover,.rbox-blended  div.videoCube_hover{background-color:transparent;}.rbox-blended .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.rbox-blended div.syndicatedItem:hover,.rbox-blended  div.syndicatedItem.videoCube_hover{background-color:transparent;}.rbox-blended div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.rbox-blended .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.rbox-blended .videoCube.syndicatedItem.horizontal{border-style:none none none solid;}.rbox-blended .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.rbox-blended .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.rbox-blended .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.rbox-blended .videoCube.syndicatedItem .video-duration{display:block;left:36px;}.rbox-blended .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.rbox-blended .syndicatedItem{background-color:transparent;}.rbox-blended .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.rbox-blended .syndicatedItem .video-title{max-height:2.58em;*height:2.58em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:14px;line-height:17.5px;font-weight:bold;text-decoration:none;}.rbox-blended .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.rbox-blended .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.rbox-blended .syndicatedItem .branding{color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;text-align:left;}.rbox-blended .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:block;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.rbox-blended .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:block;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.rbox-blended .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.rbox-blended .trc_rbox_header_icon_img{margin:0px;height:18px;}.rbox-blended .video-icon-img{margin:0px;height:18px;}.rbox-blended .video-label-box.trc-pre-label{height:auto;}.rbox-blended .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.rbox-blended .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.rbox-blended .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.rbox-blended .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.rbox-blended .videoCube .video-label-box.trc-pre-label{margin:0;}.text-links-a .video-title{font-family:Arial, Helvetica, sans-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;max-height:38.0px;*height:38.0px;color:#000000;text-decoration:none;}.text-links-a .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.text-links-a .trc_rbox_div{width:auto;_width:99%;height:auto;border-width:0;padding:0;}.text-links-a .videoCube .video-duration{left:36px;display:none;}.text-links-a .videoCube .video-label-box{margin-left:0;margin-right:0px;}.text-links-a .video-label,.text-links-a .sponsored,.text-links-a .sponsored-url{font-family:Arial, Helvetica, san-serif;}.text-links-a .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif\3b ;font-size:17.0px;font-weight:bold;text-decoration:none;color:#000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.text-links-a .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.text-links-a .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.text-links-a .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.text-links-a .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.text-links-a .videoCube{width:48%;_width:48%;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:3px 3px 3px 0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.text-links-a div.videoCube:hover,.text-links-a  div.videoCube_hover{background-color:transparent;}.text-links-a .sponsored-default{background-color:#F7F6C6;}.text-links-a div.sponsored-default:hover,.text-links-a  div.sponsored-default.videoCube_hover{background-color:inherit;}.text-links-a .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.text-links-a .videoCube:hover .thumbnail-overlay,.text-links-a  .videoCube_hover .thumbnail-overlay{background-image:null;}.text-links-a .trc_rbox_border_elm{border-color:darkgray;}.text-links-a .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.text-links-a div.videoCube:hover .thumbBlock{border-color:inherit;}.text-links-a .pager_enabled{color:#0056b3;}.text-links-a .trc_pager_counter{color:#000000;}.text-links-a .pager_disabled{color:#7d898f;}.text-links-a .trc_pager_prev:hover,.text-links-a  .trc_pager_next:hover{color:#6497ED;}.text-links-a .trc_pager_selected{color:#0056b3;}.text-links-a .trc_pager_unselected{color:#7d898f;}.text-links-a div.trc_pager_pages div:hover{color:#6497ED;}.text-links-a .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.text-links-a .video-label-box{text-align:left;}.text-links-a .trc_sponsored_overlay{background-color:black;}.text-links-a .thumbnail-emblem{background-position:5% 5%;}.text-links-a .videoCube .sponsored{margin-top:-7px;}.text-links-a {width:auto;_width:auto;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.text-links-a .videoCube.vertical{border-style:solid none none none;}.text-links-a .videoCube.horizontal{border-style:none;}.text-links-a .trc_pager_prev,.text-links-a .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.text-links-a .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.text-links-a .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .trc_pager div{font-family:serif;}.text-links-a .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.text-links-a .playerCube:hover .thumbnail-overlay,.text-links-a  .playerCube_hover .thumbnail-overlay{background-image:null;}.text-links-a .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.text-links-a .playerCube .videoCube.horizontal{border-style:none none none none;}.text-links-a .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.text-links-a .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-label-box{text-align:left;}.text-links-a .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.text-links-a .playerCube .videoCube .video-duration{display:block;left:36px;}.text-links-a .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.text-links-a .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.text-links-a .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.text-links-a .playerCube div.videoCube:hover,.text-links-a  div.videoCube_hover{background-color:transparent;}.text-links-a .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.text-links-a div.syndicatedItem:hover,.text-links-a  div.syndicatedItem.videoCube_hover{background-color:transparent;}.text-links-a div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.text-links-a .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.text-links-a .videoCube.syndicatedItem.horizontal{border-style:none;}.text-links-a .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.text-links-a .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.text-links-a .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.text-links-a .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.text-links-a .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.text-links-a .syndicatedItem{background-color:transparent;}.text-links-a .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.text-links-a .syndicatedItem .video-title{max-height:38.0px;*height:38.0px;color:#000000;font-family:Arial, Helvetica, san-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;text-decoration:none;}.text-links-a .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.text-links-a .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.text-links-a .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, san-serif;background-image:null;text-align:left;}.text-links-a .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.text-links-a .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.text-links-a .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.text-links-a .trc_rbox_header_icon_img{margin:0px;height:18px;}.text-links-a .video-icon-img{margin:0px;height:18px;}.text-links-a .video-label-box.trc-pre-label{height:auto;}.text-links-a .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.text-links-a .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.text-links-a .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.text-links-a .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.text-links-a .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-a .video-title{font-family:Arial, Helvetica, sans-serif;font-size:18.0px;line-height:24.0px;font-weight:bold;max-height:96.0px;*height:96.0px;color:#000;text-decoration:none;}.thumbnails-a .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-a .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-a .videoCube .video-duration{left:36px;display:none;}.thumbnails-a .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-a .video-label,.thumbnails-a .sponsored,.thumbnails-a .sponsored-url{font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';}.thumbnails-a .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif\3b ;font-size:28.0px;font-weight:bold;text-decoration:none;color:#000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-a .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-a .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-a .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-a .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-a .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-a div.videoCube:hover,.thumbnails-a  div.videoCube_hover{background-color:transparent;}.thumbnails-a .sponsored-default{background-color:#F7F6C6;}.thumbnails-a div.sponsored-default:hover,.thumbnails-a  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-a .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-a .videoCube:hover .thumbnail-overlay,.thumbnails-a  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-a .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-a .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-a div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-a .pager_enabled{color:#0056b3;}.thumbnails-a .trc_pager_counter{color:#000000;}.thumbnails-a .pager_disabled{color:#7d898f;}.thumbnails-a .trc_pager_prev:hover,.thumbnails-a  .trc_pager_next:hover{color:#6497ED;}.thumbnails-a .trc_pager_selected{color:#0056b3;}.thumbnails-a .trc_pager_unselected{color:#7d898f;}.thumbnails-a div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-a .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-a .video-label-box{text-align:left;}.thumbnails-a .trc_sponsored_overlay{background-color:black;}.thumbnails-a .thumbnail-emblem{background-position:5% 5%;}.thumbnails-a .videoCube .sponsored{margin-top:-7px;}.thumbnails-a {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-a .videoCube.vertical{border-style:solid none none none;}.thumbnails-a .videoCube.horizontal{border-style:none;}.thumbnails-a .trc_pager_prev,.thumbnails-a .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-a .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-a .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .trc_pager div{font-family:serif;}.thumbnails-a .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-a .playerCube:hover .thumbnail-overlay,.thumbnails-a  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-a .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-a .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-a .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-a .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-label-box{text-align:left;}.thumbnails-a .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-a .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-a .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-a .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-a .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-a .playerCube div.videoCube:hover,.thumbnails-a  div.videoCube_hover{background-color:transparent;}.thumbnails-a .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-a div.syndicatedItem:hover,.thumbnails-a  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-a div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-a .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-a .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-a .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-a .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-a .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-a .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-a .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-a .syndicatedItem{background-color:transparent;}.thumbnails-a .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-a .syndicatedItem .video-title{max-height:72.0px;*height:72.0px;color:#000;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';font-size:18.0px;line-height:24.0px;font-weight:bold;text-decoration:none;}.thumbnails-a .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-a .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-a .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';background-image:null;text-align:left;}.thumbnails-a .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-a .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-a .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-a .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-a .video-icon-img{margin:0px;height:18px;}.thumbnails-a .video-label-box.trc-pre-label{height:auto;}.thumbnails-a .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-a .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-a .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-a .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-a .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-b .video-title{font-family:Arial, Helvetica, sans-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;max-height:76.0px;*height:76.0px;color:#ffffff;text-decoration:none;}.thumbnails-b .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-b .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-b .videoCube .video-duration{left:36px;display:none;}.thumbnails-b .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-b .video-label,.thumbnails-b .sponsored,.thumbnails-b .sponsored-url{font-family:'NotoNashkArabic', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;}.thumbnails-b .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;font-size:28.0px;font-weight:bold;text-decoration:none;color:#ffffff;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-b .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-b .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-b .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-b .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-b .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-b div.videoCube:hover,.thumbnails-b  div.videoCube_hover{background-color:transparent;}.thumbnails-b .sponsored-default{background-color:#F7F6C6;}.thumbnails-b div.sponsored-default:hover,.thumbnails-b  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-b .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-b .videoCube:hover .thumbnail-overlay,.thumbnails-b  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-b .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-b .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-b div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-b .pager_enabled{color:#0056b3;}.thumbnails-b .trc_pager_counter{color:#000000;}.thumbnails-b .pager_disabled{color:#7d898f;}.thumbnails-b .trc_pager_prev:hover,.thumbnails-b  .trc_pager_next:hover{color:#6497ED;}.thumbnails-b .trc_pager_selected{color:#0056b3;}.thumbnails-b .trc_pager_unselected{color:#7d898f;}.thumbnails-b div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-b .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-b .video-label-box{text-align:left;}.thumbnails-b .trc_sponsored_overlay{background-color:black;}.thumbnails-b .thumbnail-emblem{background-position:5% 5%;}.thumbnails-b .videoCube .sponsored{margin-top:-7px;}.thumbnails-b {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-b .videoCube.vertical{border-style:solid none none none;}.thumbnails-b .videoCube.horizontal{border-style:none;}.thumbnails-b .trc_pager_prev,.thumbnails-b .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-b .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-b .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .trc_pager div{font-family:serif;}.thumbnails-b .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-b .playerCube:hover .thumbnail-overlay,.thumbnails-b  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-b .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-b .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-b .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-b .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-label-box{text-align:left;}.thumbnails-b .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-b .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-b .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-b .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-b .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-b .playerCube div.videoCube:hover,.thumbnails-b  div.videoCube_hover{background-color:transparent;}.thumbnails-b .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-b div.syndicatedItem:hover,.thumbnails-b  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-b div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-b .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-b .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-b .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-b .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-b .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-b .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-b .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-b .syndicatedItem{background-color:transparent;}.thumbnails-b .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-b .syndicatedItem .video-title{max-height:57.0px;*height:57.0px;color:#ffffff;font-family:'NotoNashkArabic', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;text-decoration:none;}.thumbnails-b .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-b .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-b .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'NotoNashkArabic', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;background-image:null;text-align:left;}.thumbnails-b .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-b .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-b .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-b .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-b .video-icon-img{margin:0px;height:18px;}.thumbnails-b .video-label-box.trc-pre-label{height:auto;}.thumbnails-b .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-b .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-b .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-b .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-b .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-c .video-title{font-family:Arial, Helvetica, sans-serif;font-size:16.0px;line-height:18.0px;font-weight:bold;max-height:54.0px;*height:54.0px;color:#1a1a1a;text-decoration:none;}.thumbnails-c .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-c .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-c .videoCube .video-duration{left:36px;display:none;}.thumbnails-c .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-c .video-label,.thumbnails-c .sponsored,.thumbnails-c .sponsored-url{font-family:Arial, 'Helvetica Neue', Helvetica, sans-serif;}.thumbnails-c .trc_rbox_header{font-family:Arial, Helvetica, sans-serif;font-size:100%;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0;}.thumbnails-c .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-c .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-c .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-c .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-c .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-c div.videoCube:hover,.thumbnails-c  div.videoCube_hover{background-color:transparent;}.thumbnails-c .sponsored-default{background-color:#F7F6C6;}.thumbnails-c div.sponsored-default:hover,.thumbnails-c  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-c .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-c .videoCube:hover .thumbnail-overlay,.thumbnails-c  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-c .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-c .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-c div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-c .pager_enabled{color:#0056b3;}.thumbnails-c .trc_pager_counter{color:#000000;}.thumbnails-c .pager_disabled{color:#7d898f;}.thumbnails-c .trc_pager_prev:hover,.thumbnails-c  .trc_pager_next:hover{color:#6497ED;}.thumbnails-c .trc_pager_selected{color:#0056b3;}.thumbnails-c .trc_pager_unselected{color:#7d898f;}.thumbnails-c div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-c .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-c .video-label-box{text-align:left;}.thumbnails-c .trc_sponsored_overlay{background-color:black;}.thumbnails-c .thumbnail-emblem{background-position:5% 5%;}.thumbnails-c .videoCube .sponsored{margin-top:-7px;}.thumbnails-c {width:300px;_width:300px;border-width:0px 0px 1px 0px;border-style:solid solid solid solid;border-color:#c2c2c2;padding:13px 0px 12px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-c .videoCube.vertical{border-style:solid none none none;}.thumbnails-c .videoCube.horizontal{border-style:none;}.thumbnails-c .trc_pager_prev,.thumbnails-c .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-c .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-c .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .trc_pager div{font-family:serif;}.thumbnails-c .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-c .playerCube:hover .thumbnail-overlay,.thumbnails-c  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-c .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-c .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-c .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-c .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-label-box{text-align:left;}.thumbnails-c .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-c .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-c .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-c .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-c .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-c .playerCube div.videoCube:hover,.thumbnails-c  div.videoCube_hover{background-color:transparent;}.thumbnails-c .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-c div.syndicatedItem:hover,.thumbnails-c  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-c div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-c .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-c .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-c .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-c .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-c .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-c .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-c .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-c .syndicatedItem{background-color:transparent;}.thumbnails-c .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-c .syndicatedItem .video-title{max-height:36.0px;*height:36.0px;color:#1a1a1a;font-family:Arial, 'Helvetica Neue', Helvetica, sans-serif;font-size:16.0px;line-height:18.0px;font-weight:bold;text-decoration:none;}.thumbnails-c .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-c .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-c .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:Arial, 'Helvetica Neue', Helvetica, sans-serif;background-image:null;text-align:left;}.thumbnails-c .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-c .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-c .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-c .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-c .video-icon-img{margin:0px;height:18px;}.thumbnails-c .video-label-box.trc-pre-label{height:auto;}.thumbnails-c .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-c .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-c .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-c .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-c .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-d .video-title{font-family:Arial, Helvetica, sans-serif;font-size:16.0px;line-height:18.0px;font-weight:bold;max-height:54.0px;*height:54.0px;color:#1a1a1a;text-decoration:none;}.thumbnails-d .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-d .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-d .videoCube .video-duration{left:36px;display:none;}.thumbnails-d .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-d .video-label,.thumbnails-d .sponsored,.thumbnails-d .sponsored-url{font-family:Arial, Helvetica, sans-serif;}.thumbnails-d .trc_rbox_header{font-family:Arial, Helvetica, sans-serif;font-size:100%;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0;}.thumbnails-d .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-d .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-d .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-d .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-d .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-d div.videoCube:hover,.thumbnails-d  div.videoCube_hover{background-color:transparent;}.thumbnails-d .sponsored-default{background-color:#F7F6C6;}.thumbnails-d div.sponsored-default:hover,.thumbnails-d  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-d .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-d .videoCube:hover .thumbnail-overlay,.thumbnails-d  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-d .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-d .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-d div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-d .pager_enabled{color:#0056b3;}.thumbnails-d .trc_pager_counter{color:#000000;}.thumbnails-d .pager_disabled{color:#7d898f;}.thumbnails-d .trc_pager_prev:hover,.thumbnails-d  .trc_pager_next:hover{color:#6497ED;}.thumbnails-d .trc_pager_selected{color:#0056b3;}.thumbnails-d .trc_pager_unselected{color:#7d898f;}.thumbnails-d div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-d .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-d .video-label-box{text-align:left;}.thumbnails-d .trc_sponsored_overlay{background-color:black;}.thumbnails-d .thumbnail-emblem{background-position:5% 5%;}.thumbnails-d .videoCube .sponsored{margin-top:-7px;}.thumbnails-d {width:300px;_width:300px;border-width:0px 0px 1px 0px;border-style:dotted;border-color:#c2c2c2;padding:13px 0px 12px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-d .videoCube.vertical{border-style:solid none none none;}.thumbnails-d .videoCube.horizontal{border-style:none;}.thumbnails-d .trc_pager_prev,.thumbnails-d .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-d .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-d .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .trc_pager div{font-family:serif;}.thumbnails-d .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-d .playerCube:hover .thumbnail-overlay,.thumbnails-d  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-d .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-d .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-d .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-d .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-label-box{text-align:left;}.thumbnails-d .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-d .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-d .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-d .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-d .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-d .playerCube div.videoCube:hover,.thumbnails-d  div.videoCube_hover{background-color:transparent;}.thumbnails-d .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-d div.syndicatedItem:hover,.thumbnails-d  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-d div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-d .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-d .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-d .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-d .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-d .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-d .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-d .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-d .syndicatedItem{background-color:transparent;}.thumbnails-d .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-d .syndicatedItem .video-title{max-height:36.0px;*height:36.0px;color:#1a1a1a;font-family:Arial, Helvetica, sans-serif;font-size:16.0px;line-height:18.0px;font-weight:bold;text-decoration:none;}.thumbnails-d .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-d .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-d .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;text-align:left;}.thumbnails-d .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-d .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-d .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-d .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-d .video-icon-img{margin:0px;height:18px;}.thumbnails-d .video-label-box.trc-pre-label{height:auto;}.thumbnails-d .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-d .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-d .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-d .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-d .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-e .video-title{font-family:Arial, Helvetica, sans-serif;font-size:22.0px;line-height:24.0px;font-weight:bold;max-height:144.0px;*height:144.0px;color:#1b1b1b;text-decoration:none;}.thumbnails-e .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-e .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-e .videoCube .video-duration{left:36px;display:none;}.thumbnails-e .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-e .video-label,.thumbnails-e .sponsored,.thumbnails-e .sponsored-url{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;}.thumbnails-e .trc_rbox_header{font-family:Arial, Helvetica, sans-serif;font-size:100%;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0;}.thumbnails-e .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-e .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-e .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-e .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-e .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-e div.videoCube:hover,.thumbnails-e  div.videoCube_hover{background-color:transparent;}.thumbnails-e .sponsored-default{background-color:#F7F6C6;}.thumbnails-e div.sponsored-default:hover,.thumbnails-e  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-e .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-e .videoCube:hover .thumbnail-overlay,.thumbnails-e  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-e .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-e .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-e div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-e .pager_enabled{color:#0056b3;}.thumbnails-e .trc_pager_counter{color:#000000;}.thumbnails-e .pager_disabled{color:#7d898f;}.thumbnails-e .trc_pager_prev:hover,.thumbnails-e  .trc_pager_next:hover{color:#6497ED;}.thumbnails-e .trc_pager_selected{color:#0056b3;}.thumbnails-e .trc_pager_unselected{color:#7d898f;}.thumbnails-e div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-e .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-e .video-label-box{text-align:left;}.thumbnails-e .trc_sponsored_overlay{background-color:black;}.thumbnails-e .thumbnail-emblem{background-position:5% 5%;}.thumbnails-e .videoCube .sponsored{margin-top:-7px;}.thumbnails-e {width:300px;_width:300px;border-width:0px 0px 1px 0px;border-style:dotted;border-color:#DFDFDF;padding:22px 0px 22px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-e .videoCube.vertical{border-style:solid none none none;}.thumbnails-e .videoCube.horizontal{border-style:none;}.thumbnails-e .trc_pager_prev,.thumbnails-e .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-e .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-e .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .trc_pager div{font-family:serif;}.thumbnails-e .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-e .playerCube:hover .thumbnail-overlay,.thumbnails-e  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-e .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-e .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-e .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-e .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-label-box{text-align:left;}.thumbnails-e .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-e .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-e .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-e .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-e .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-e .playerCube div.videoCube:hover,.thumbnails-e  div.videoCube_hover{background-color:transparent;}.thumbnails-e .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-e div.syndicatedItem:hover,.thumbnails-e  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-e div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-e .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-e .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-e .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-e .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-e .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-e .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-e .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-e .syndicatedItem{background-color:transparent;}.thumbnails-e .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-e .syndicatedItem .video-title{max-height:120.0px;*height:120.0px;color:#1b1b1b;font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;font-size:22.0px;line-height:24.0px;font-weight:bold;text-decoration:none;}.thumbnails-e .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-e .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-e .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;background-image:null;text-align:left;}.thumbnails-e .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-e .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-e .videoCube.thumbnail_start .thumbBlock_holder{width:41%;_width:41%;}.thumbnails-e .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-e .video-icon-img{margin:0px;height:18px;}.thumbnails-e .video-label-box.trc-pre-label{height:auto;}.thumbnails-e .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-e .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-e .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-e .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-e .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-f .video-title{font-family:Arial, Helvetica, sans-serif;font-size:22.0px;line-height:30.0px;font-weight:bold;max-height:150.0px;*height:150.0px;color:#1b1b1b;text-decoration:none;}.thumbnails-f .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-f .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-f .videoCube .video-duration{left:36px;display:none;}.thumbnails-f .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-f .video-label,.thumbnails-f .sponsored,.thumbnails-f .sponsored-url{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;}.thumbnails-f .trc_rbox_header{font-family:Arial, Helvetica, sans-serif;font-size:100%;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0;}.thumbnails-f .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-f .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-f .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-f .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-f .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-f div.videoCube:hover,.thumbnails-f  div.videoCube_hover{background-color:transparent;}.thumbnails-f .sponsored-default{background-color:#F7F6C6;}.thumbnails-f div.sponsored-default:hover,.thumbnails-f  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-f .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-f .videoCube:hover .thumbnail-overlay,.thumbnails-f  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-f .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-f .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-f div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-f .pager_enabled{color:#0056b3;}.thumbnails-f .trc_pager_counter{color:#000000;}.thumbnails-f .pager_disabled{color:#7d898f;}.thumbnails-f .trc_pager_prev:hover,.thumbnails-f  .trc_pager_next:hover{color:#6497ED;}.thumbnails-f .trc_pager_selected{color:#0056b3;}.thumbnails-f .trc_pager_unselected{color:#7d898f;}.thumbnails-f div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-f .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-f .video-label-box{text-align:left;}.thumbnails-f .trc_sponsored_overlay{background-color:black;}.thumbnails-f .thumbnail-emblem{background-position:5% 5%;}.thumbnails-f .videoCube .sponsored{margin-top:-7px;}.thumbnails-f {width:300px;_width:300px;border-width:0px 0px 1px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 22px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-f .videoCube.vertical{border-style:solid none none none;}.thumbnails-f .videoCube.horizontal{border-style:none;}.thumbnails-f .trc_pager_prev,.thumbnails-f .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-f .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-f .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .trc_pager div{font-family:serif;}.thumbnails-f .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-f .playerCube:hover .thumbnail-overlay,.thumbnails-f  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-f .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-f .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-f .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-f .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-label-box{text-align:left;}.thumbnails-f .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-f .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-f .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-f .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-f .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-f .playerCube div.videoCube:hover,.thumbnails-f  div.videoCube_hover{background-color:transparent;}.thumbnails-f .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-f div.syndicatedItem:hover,.thumbnails-f  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-f div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-f .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-f .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-f .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-f .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-f .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-f .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-f .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-f .syndicatedItem{background-color:transparent;}.thumbnails-f .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-f .syndicatedItem .video-title{max-height:120.0px;*height:120.0px;color:#1b1b1b;font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;font-size:22.0px;line-height:30.0px;font-weight:bold;text-decoration:none;}.thumbnails-f .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-f .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-f .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;background-image:null;text-align:left;}.thumbnails-f .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-f .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-f .videoCube.thumbnail_start .thumbBlock_holder{width:41%;_width:41%;}.thumbnails-f .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-f .video-icon-img{margin:0px;height:18px;}.thumbnails-f .video-label-box.trc-pre-label{height:auto;}.thumbnails-f .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-f .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-f .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-f .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-f .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-g .video-title{font-family:Arial, Helvetica, sans-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;max-height:76.0px;*height:76.0px;color:#fff;text-decoration:none;}.thumbnails-g .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-g .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-g .videoCube .video-duration{left:36px;display:none;}.thumbnails-g .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-g .video-label,.thumbnails-g .sponsored,.thumbnails-g .sponsored-url{font-family:'Helvetica Neue';}.thumbnails-g .trc_rbox_header{font-family:ProximaNovaCond-Extrabld;font-size:20.0px;font-weight:bold;text-decoration:none;color:#fff;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-g .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-g .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-g .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-g .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-g .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-g div.videoCube:hover,.thumbnails-g  div.videoCube_hover{background-color:transparent;}.thumbnails-g .sponsored-default{background-color:#F7F6C6;}.thumbnails-g div.sponsored-default:hover,.thumbnails-g  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-g .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-g .videoCube:hover .thumbnail-overlay,.thumbnails-g  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-g .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-g .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-g div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-g .pager_enabled{color:#0056b3;}.thumbnails-g .trc_pager_counter{color:#000000;}.thumbnails-g .pager_disabled{color:#7d898f;}.thumbnails-g .trc_pager_prev:hover,.thumbnails-g  .trc_pager_next:hover{color:#6497ED;}.thumbnails-g .trc_pager_selected{color:#0056b3;}.thumbnails-g .trc_pager_unselected{color:#7d898f;}.thumbnails-g div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-g .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-g .video-label-box{text-align:left;}.thumbnails-g .trc_sponsored_overlay{background-color:black;}.thumbnails-g .thumbnail-emblem{background-position:5% 5%;}.thumbnails-g .videoCube .sponsored{margin-top:-7px;}.thumbnails-g {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-g .videoCube.vertical{border-style:solid none none none;}.thumbnails-g .videoCube.horizontal{border-style:none;}.thumbnails-g .trc_pager_prev,.thumbnails-g .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-g .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-g .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .trc_pager div{font-family:serif;}.thumbnails-g .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-g .playerCube:hover .thumbnail-overlay,.thumbnails-g  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-g .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-g .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-g .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-g .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-label-box{text-align:left;}.thumbnails-g .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-g .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-g .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-g .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-g .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-g .playerCube div.videoCube:hover,.thumbnails-g  div.videoCube_hover{background-color:transparent;}.thumbnails-g .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-g div.syndicatedItem:hover,.thumbnails-g  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-g div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-g .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-g .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-g .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-g .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-g .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-g .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-g .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-g .syndicatedItem{background-color:transparent;}.thumbnails-g .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-g .syndicatedItem .video-title{max-height:57.0px;*height:57.0px;color:#fff;font-family:'Helvetica Neue';font-size:14.0px;line-height:19.0px;font-weight:bold;text-decoration:none;}.thumbnails-g .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-g .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-g .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'Helvetica Neue';background-image:null;text-align:left;}.thumbnails-g .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-g .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-g .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-g .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-g .video-icon-img{margin:0px;height:18px;}.thumbnails-g .video-label-box.trc-pre-label{height:auto;}.thumbnails-g .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-g .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-g .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-g .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-g .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-h .video-title{font-family:Arial, Helvetica, sans-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;max-height:76.0px;*height:76.0px;color:#fff;text-decoration:none;}.thumbnails-h .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-h .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-h .videoCube .video-duration{left:36px;display:none;}.thumbnails-h .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-h .video-label,.thumbnails-h .sponsored,.thumbnails-h .sponsored-url{font-family:'Helvetica Neue';}.thumbnails-h .trc_rbox_header{font-family:ProximaNovaCond-Extrabld;font-size:20.0px;font-weight:bold;text-decoration:none;color:#fff;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-h .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-h .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-h .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-h .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-h .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-h div.videoCube:hover,.thumbnails-h  div.videoCube_hover{background-color:transparent;}.thumbnails-h .sponsored-default{background-color:#F7F6C6;}.thumbnails-h div.sponsored-default:hover,.thumbnails-h  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-h .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-h .videoCube:hover .thumbnail-overlay,.thumbnails-h  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-h .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-h .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-h div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-h .pager_enabled{color:#0056b3;}.thumbnails-h .trc_pager_counter{color:#000000;}.thumbnails-h .pager_disabled{color:#7d898f;}.thumbnails-h .trc_pager_prev:hover,.thumbnails-h  .trc_pager_next:hover{color:#6497ED;}.thumbnails-h .trc_pager_selected{color:#0056b3;}.thumbnails-h .trc_pager_unselected{color:#7d898f;}.thumbnails-h div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-h .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-h .video-label-box{text-align:left;}.thumbnails-h .trc_sponsored_overlay{background-color:black;}.thumbnails-h .thumbnail-emblem{background-position:5% 5%;}.thumbnails-h .videoCube .sponsored{margin-top:-7px;}.thumbnails-h {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-h .videoCube.vertical{border-style:solid none none none;}.thumbnails-h .videoCube.horizontal{border-style:none;}.thumbnails-h .trc_pager_prev,.thumbnails-h .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-h .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-h .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .trc_pager div{font-family:serif;}.thumbnails-h .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-h .playerCube:hover .thumbnail-overlay,.thumbnails-h  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-h .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-h .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-h .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-h .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-label-box{text-align:left;}.thumbnails-h .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-h .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-h .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-h .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-h .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-h .playerCube div.videoCube:hover,.thumbnails-h  div.videoCube_hover{background-color:transparent;}.thumbnails-h .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-h div.syndicatedItem:hover,.thumbnails-h  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-h div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-h .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-h .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-h .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-h .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-h .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-h .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-h .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-h .syndicatedItem{background-color:transparent;}.thumbnails-h .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-h .syndicatedItem .video-title{max-height:57.0px;*height:57.0px;color:#fff;font-family:'Helvetica Neue';font-size:14.0px;line-height:19.0px;font-weight:bold;text-decoration:none;}.thumbnails-h .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-h .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-h .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'Helvetica Neue';background-image:null;text-align:left;}.thumbnails-h .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-h .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-h .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-h .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-h .video-icon-img{margin:0px;height:18px;}.thumbnails-h .video-label-box.trc-pre-label{height:auto;}.thumbnails-h .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-h .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-h .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-h .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-h .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-i .video-title{font-family:Arial, Helvetica, sans-serif;font-size:17.0px;line-height:23.0px;font-weight:bold;max-height:92.0px;*height:92.0px;color:#444;text-decoration:none;}.thumbnails-i .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-i .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-i .videoCube .video-duration{left:36px;display:none;}.thumbnails-i .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-i .video-label,.thumbnails-i .sponsored,.thumbnails-i .sponsored-url{font-family:NotoNashkArabic;}.thumbnails-i .trc_rbox_header{font-family:'Proxima Nova Condensed';font-size:28.0px;font-weight:bold;text-decoration:none;color:#222;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-i .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-i .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-i .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-i .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-i .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-i div.videoCube:hover,.thumbnails-i  div.videoCube_hover{background-color:transparent;}.thumbnails-i .sponsored-default{background-color:#F7F6C6;}.thumbnails-i div.sponsored-default:hover,.thumbnails-i  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-i .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-i .videoCube:hover .thumbnail-overlay,.thumbnails-i  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-i .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-i .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-i div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-i .pager_enabled{color:#0056b3;}.thumbnails-i .trc_pager_counter{color:#000000;}.thumbnails-i .pager_disabled{color:#7d898f;}.thumbnails-i .trc_pager_prev:hover,.thumbnails-i  .trc_pager_next:hover{color:#6497ED;}.thumbnails-i .trc_pager_selected{color:#0056b3;}.thumbnails-i .trc_pager_unselected{color:#7d898f;}.thumbnails-i div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-i .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-i .video-label-box{text-align:left;}.thumbnails-i .trc_sponsored_overlay{background-color:black;}.thumbnails-i .thumbnail-emblem{background-position:5% 5%;}.thumbnails-i .videoCube .sponsored{margin-top:-7px;}.thumbnails-i {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-i .videoCube.vertical{border-style:solid none none none;}.thumbnails-i .videoCube.horizontal{border-style:none;}.thumbnails-i .trc_pager_prev,.thumbnails-i .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-i .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-i .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .trc_pager div{font-family:serif;}.thumbnails-i .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-i .playerCube:hover .thumbnail-overlay,.thumbnails-i  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-i .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-i .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-i .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-i .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-label-box{text-align:left;}.thumbnails-i .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-i .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-i .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-i .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-i .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-i .playerCube div.videoCube:hover,.thumbnails-i  div.videoCube_hover{background-color:transparent;}.thumbnails-i .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-i div.syndicatedItem:hover,.thumbnails-i  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-i div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-i .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-i .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-i .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-i .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-i .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-i .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-i .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-i .syndicatedItem{background-color:transparent;}.thumbnails-i .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-i .syndicatedItem .video-title{max-height:69.0px;*height:69.0px;color:#444;font-family:NotoNashkArabic;font-size:17.0px;line-height:23.0px;font-weight:bold;text-decoration:none;}.thumbnails-i .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-i .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-i .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:NotoNashkArabic;background-image:null;text-align:left;}.thumbnails-i .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-i .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-i .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-i .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-i .video-icon-img{margin:0px;height:18px;}.thumbnails-i .video-label-box.trc-pre-label{height:auto;}.thumbnails-i .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-i .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-i .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-i .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-i .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-j .video-title{font-family:Arial, Helvetica, sans-serif;font-size:17.0px;line-height:23.0px;font-weight:bold;max-height:92.0px;*height:92.0px;color:#444;text-decoration:none;}.thumbnails-j .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-j .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-j .videoCube .video-duration{left:36px;display:none;}.thumbnails-j .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-j .video-label,.thumbnails-j .sponsored,.thumbnails-j .sponsored-url{font-family:NotoNashkArabic;}.thumbnails-j .trc_rbox_header{font-family:'Proxima Nova Condensed';font-size:28.0px;font-weight:bold;text-decoration:none;color:#222;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-j .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-j .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-j .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-j .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-j .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-j div.videoCube:hover,.thumbnails-j  div.videoCube_hover{background-color:transparent;}.thumbnails-j .sponsored-default{background-color:#F7F6C6;}.thumbnails-j div.sponsored-default:hover,.thumbnails-j  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-j .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-j .videoCube:hover .thumbnail-overlay,.thumbnails-j  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-j .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-j .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-j div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-j .pager_enabled{color:#0056b3;}.thumbnails-j .trc_pager_counter{color:#000000;}.thumbnails-j .pager_disabled{color:#7d898f;}.thumbnails-j .trc_pager_prev:hover,.thumbnails-j  .trc_pager_next:hover{color:#6497ED;}.thumbnails-j .trc_pager_selected{color:#0056b3;}.thumbnails-j .trc_pager_unselected{color:#7d898f;}.thumbnails-j div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-j .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-j .video-label-box{text-align:left;}.thumbnails-j .trc_sponsored_overlay{background-color:black;}.thumbnails-j .thumbnail-emblem{background-position:5% 5%;}.thumbnails-j .videoCube .sponsored{margin-top:-7px;}.thumbnails-j {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-j .videoCube.vertical{border-style:solid none none none;}.thumbnails-j .videoCube.horizontal{border-style:none;}.thumbnails-j .trc_pager_prev,.thumbnails-j .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-j .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-j .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .trc_pager div{font-family:serif;}.thumbnails-j .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-j .playerCube:hover .thumbnail-overlay,.thumbnails-j  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-j .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-j .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-j .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-j .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-label-box{text-align:left;}.thumbnails-j .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-j .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-j .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-j .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-j .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-j .playerCube div.videoCube:hover,.thumbnails-j  div.videoCube_hover{background-color:transparent;}.thumbnails-j .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-j div.syndicatedItem:hover,.thumbnails-j  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-j div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-j .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-j .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-j .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-j .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-j .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-j .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-j .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-j .syndicatedItem{background-color:transparent;}.thumbnails-j .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-j .syndicatedItem .video-title{max-height:69.0px;*height:69.0px;color:#444;font-family:NotoNashkArabic;font-size:17.0px;line-height:23.0px;font-weight:bold;text-decoration:none;}.thumbnails-j .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-j .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-j .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:NotoNashkArabic;background-image:null;text-align:left;}.thumbnails-j .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-j .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-j .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-j .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-j .video-icon-img{margin:0px;height:18px;}.thumbnails-j .video-label-box.trc-pre-label{height:auto;}.thumbnails-j .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-j .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-j .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-j .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-j .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-k .video-title{font-family:Arial, Helvetica, sans-serif;font-size:18.0px;line-height:20.0px;font-weight:bold;max-height:96.0px;*height:96.0px;color:#000000;text-decoration:none;}.thumbnails-k .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-k .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-k .videoCube .video-duration{left:36px;display:none;}.thumbnails-k .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-k .video-label,.thumbnails-k .sponsored,.thumbnails-k .sponsored-url{font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;}.thumbnails-k .trc_rbox_header{font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;font-size:18.0px;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-k .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-k .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-k .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-k .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-k .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-k div.videoCube:hover,.thumbnails-k  div.videoCube_hover{background-color:transparent;}.thumbnails-k .sponsored-default{background-color:#F7F6C6;}.thumbnails-k div.sponsored-default:hover,.thumbnails-k  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-k .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-k .videoCube:hover .thumbnail-overlay,.thumbnails-k  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-k .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-k .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-k div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-k .pager_enabled{color:#0056b3;}.thumbnails-k .trc_pager_counter{color:#000000;}.thumbnails-k .pager_disabled{color:#7d898f;}.thumbnails-k .trc_pager_prev:hover,.thumbnails-k  .trc_pager_next:hover{color:#6497ED;}.thumbnails-k .trc_pager_selected{color:#0056b3;}.thumbnails-k .trc_pager_unselected{color:#7d898f;}.thumbnails-k div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-k .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-k .video-label-box{text-align:left;}.thumbnails-k .trc_sponsored_overlay{background-color:black;}.thumbnails-k .thumbnail-emblem{background-position:5% 5%;}.thumbnails-k .videoCube .sponsored{margin-top:-7px;}.thumbnails-k {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-k .videoCube.vertical{border-style:solid none none none;}.thumbnails-k .videoCube.horizontal{border-style:none;}.thumbnails-k .trc_pager_prev,.thumbnails-k .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-k .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-k .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .trc_pager div{font-family:serif;}.thumbnails-k .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-k .playerCube:hover .thumbnail-overlay,.thumbnails-k  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-k .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-k .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-k .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-k .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-label-box{text-align:left;}.thumbnails-k .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-k .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-k .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-k .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-k .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-k .playerCube div.videoCube:hover,.thumbnails-k  div.videoCube_hover{background-color:transparent;}.thumbnails-k .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-k div.syndicatedItem:hover,.thumbnails-k  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-k div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-k .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-k .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-k .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-k .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-k .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-k .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-k .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-k .syndicatedItem{background-color:transparent;}.thumbnails-k .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-k .syndicatedItem .video-title{max-height:72.0px;*height:72.0px;color:#000000;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;font-size:18.0px;line-height:20.0px;font-weight:bold;text-decoration:none;}.thumbnails-k .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-k .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-k .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif;background-image:null;text-align:left;}.thumbnails-k .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-k .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-k .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-k .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-k .video-icon-img{margin:0px;height:18px;}.thumbnails-k .video-label-box.trc-pre-label{height:auto;}.thumbnails-k .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-k .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-k .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-k .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-k .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-m .video-title{font-family:Arial, Helvetica, sans-serif;font-size:16.0px;line-height:22.0px;font-weight:bold;max-height:88.0px;*height:88.0px;color:#444444;text-decoration:none;}.thumbnails-m .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-m .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-m .videoCube .video-duration{left:36px;display:none;}.thumbnails-m .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-m .video-label,.thumbnails-m .sponsored,.thumbnails-m .sponsored-url{font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';}.thumbnails-m .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif\3b ;font-size:17.0px;font-weight:bold;text-decoration:none;color:#444444;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-m .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-m .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-m .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-m .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-m .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-m div.videoCube:hover,.thumbnails-m  div.videoCube_hover{background-color:transparent;}.thumbnails-m .sponsored-default{background-color:#F7F6C6;}.thumbnails-m div.sponsored-default:hover,.thumbnails-m  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-m .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-m .videoCube:hover .thumbnail-overlay,.thumbnails-m  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-m .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-m .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-m div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-m .pager_enabled{color:#0056b3;}.thumbnails-m .trc_pager_counter{color:#000000;}.thumbnails-m .pager_disabled{color:#7d898f;}.thumbnails-m .trc_pager_prev:hover,.thumbnails-m  .trc_pager_next:hover{color:#6497ED;}.thumbnails-m .trc_pager_selected{color:#0056b3;}.thumbnails-m .trc_pager_unselected{color:#7d898f;}.thumbnails-m div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-m .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-m .video-label-box{text-align:left;}.thumbnails-m .trc_sponsored_overlay{background-color:black;}.thumbnails-m .thumbnail-emblem{background-position:5% 5%;}.thumbnails-m .videoCube .sponsored{margin-top:-7px;}.thumbnails-m {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 15px 0px 15px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-m .videoCube.vertical{border-style:solid none none none;}.thumbnails-m .videoCube.horizontal{border-style:none;}.thumbnails-m .trc_pager_prev,.thumbnails-m .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-m .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-m .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .trc_pager div{font-family:serif;}.thumbnails-m .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-m .playerCube:hover .thumbnail-overlay,.thumbnails-m  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-m .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-m .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-m .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-m .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-label-box{text-align:left;}.thumbnails-m .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-m .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-m .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-m .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-m .playerCube div.videoCube:hover,.thumbnails-m  div.videoCube_hover{background-color:transparent;}.thumbnails-m .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-m div.syndicatedItem:hover,.thumbnails-m  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-m div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-m .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-m .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-m .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-m .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-m .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-m .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-m .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-m .syndicatedItem{background-color:transparent;}.thumbnails-m .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-m .syndicatedItem .video-title{max-height:66.0px;*height:66.0px;color:#444444;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';font-size:16.0px;line-height:22.0px;font-weight:bold;text-decoration:none;}.thumbnails-m .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-m .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';background-image:null;text-align:left;}.thumbnails-m .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-m .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-m .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-m .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-m .video-icon-img{margin:0px;height:18px;}.thumbnails-m .video-label-box.trc-pre-label{height:auto;}.thumbnails-m .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-m .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-m .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-m .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-m .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-m-AMP .video-title{font-family:Arial, Helvetica, sans-serif;font-size:14.0px;line-height:19.0px;font-weight:bold;max-height:76.0px;*height:76.0px;color:#030508;text-decoration:none;}.thumbnails-m-AMP .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-m-AMP .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-m-AMP .videoCube .video-duration{left:36px;display:none;}.thumbnails-m-AMP .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-m-AMP .video-label,.thumbnails-m-AMP .sponsored,.thumbnails-m-AMP .sponsored-url{font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';}.thumbnails-m-AMP .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif\3b ;font-size:17.0px;font-weight:bold;text-decoration:none;color:#fcfcfc;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-m-AMP .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-m-AMP .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-m-AMP .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-m-AMP .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-m-AMP .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-m-AMP div.videoCube:hover,.thumbnails-m-AMP  div.videoCube_hover{background-color:transparent;}.thumbnails-m-AMP .sponsored-default{background-color:#F7F6C6;}.thumbnails-m-AMP div.sponsored-default:hover,.thumbnails-m-AMP  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-m-AMP .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-m-AMP .videoCube:hover .thumbnail-overlay,.thumbnails-m-AMP  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-m-AMP .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-m-AMP .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-m-AMP div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-m-AMP .pager_enabled{color:#0056b3;}.thumbnails-m-AMP .trc_pager_counter{color:#000000;}.thumbnails-m-AMP .pager_disabled{color:#7d898f;}.thumbnails-m-AMP .trc_pager_prev:hover,.thumbnails-m-AMP  .trc_pager_next:hover{color:#6497ED;}.thumbnails-m-AMP .trc_pager_selected{color:#0056b3;}.thumbnails-m-AMP .trc_pager_unselected{color:#7d898f;}.thumbnails-m-AMP div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-m-AMP .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-m-AMP .video-label-box{text-align:left;}.thumbnails-m-AMP .trc_sponsored_overlay{background-color:black;}.thumbnails-m-AMP .thumbnail-emblem{background-position:5% 5%;}.thumbnails-m-AMP .videoCube .sponsored{margin-top:-7px;}.thumbnails-m-AMP {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 15px 0px 15px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-m-AMP .videoCube.vertical{border-style:solid none none none;}.thumbnails-m-AMP .videoCube.horizontal{border-style:none;}.thumbnails-m-AMP .trc_pager_prev,.thumbnails-m-AMP .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .trc_pager div{font-family:serif;}.thumbnails-m-AMP .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-m-AMP .playerCube:hover .thumbnail-overlay,.thumbnails-m-AMP  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-m-AMP .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-m-AMP .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-m-AMP .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-m-AMP .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-label-box{text-align:left;}.thumbnails-m-AMP .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-m-AMP .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-m-AMP .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-m-AMP .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-m-AMP .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-m-AMP .playerCube div.videoCube:hover,.thumbnails-m-AMP  div.videoCube_hover{background-color:transparent;}.thumbnails-m-AMP .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-m-AMP div.syndicatedItem:hover,.thumbnails-m-AMP  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-m-AMP div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-m-AMP .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-m-AMP .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-m-AMP .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-m-AMP .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-m-AMP .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-m-AMP .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-m-AMP .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-m-AMP .syndicatedItem{background-color:transparent;}.thumbnails-m-AMP .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-title{max-height:57.0px;*height:57.0px;color:#030508;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';font-size:14.0px;line-height:19.0px;font-weight:bold;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-m-AMP .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-m-AMP .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';background-image:null;text-align:left;}.thumbnails-m-AMP .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-m-AMP .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-m-AMP .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-m-AMP .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-m-AMP .video-icon-img{margin:0px;height:18px;}.thumbnails-m-AMP .video-label-box.trc-pre-label{height:auto;}.thumbnails-m-AMP .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-m-AMP .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-m-AMP .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-m-AMP .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-m-AMP .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-mg .video-title{font-family:Arial, Helvetica, sans-serif;font-size:16.0px;line-height:22.0px;font-weight:bold;max-height:88.0px;*height:88.0px;color:#444444;text-decoration:none;}.thumbnails-mg .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-mg .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-mg .videoCube .video-duration{left:36px;display:none;}.thumbnails-mg .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-mg .video-label,.thumbnails-mg .sponsored,.thumbnails-mg .sponsored-url{font-family:Arial, Helvetica, san-serif;}.thumbnails-mg .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif\3b ;font-size:17.0px;font-weight:bold;text-decoration:none;color:#444444;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-mg .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-mg .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-mg .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-mg .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-mg .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-mg div.videoCube:hover,.thumbnails-mg  div.videoCube_hover{background-color:transparent;}.thumbnails-mg .sponsored-default{background-color:#F7F6C6;}.thumbnails-mg div.sponsored-default:hover,.thumbnails-mg  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-mg .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-mg .videoCube:hover .thumbnail-overlay,.thumbnails-mg  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-mg .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-mg .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-mg div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-mg .pager_enabled{color:#0056b3;}.thumbnails-mg .trc_pager_counter{color:#000000;}.thumbnails-mg .pager_disabled{color:#7d898f;}.thumbnails-mg .trc_pager_prev:hover,.thumbnails-mg  .trc_pager_next:hover{color:#6497ED;}.thumbnails-mg .trc_pager_selected{color:#0056b3;}.thumbnails-mg .trc_pager_unselected{color:#7d898f;}.thumbnails-mg div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-mg .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-mg .video-label-box{text-align:left;}.thumbnails-mg .trc_sponsored_overlay{background-color:black;}.thumbnails-mg .thumbnail-emblem{background-position:5% 5%;}.thumbnails-mg .videoCube .sponsored{margin-top:-7px;}.thumbnails-mg {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-mg .videoCube.vertical{border-style:solid none none none;}.thumbnails-mg .videoCube.horizontal{border-style:none;}.thumbnails-mg .trc_pager_prev,.thumbnails-mg .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-mg .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-mg .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .trc_pager div{font-family:serif;}.thumbnails-mg .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-mg .playerCube:hover .thumbnail-overlay,.thumbnails-mg  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-mg .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-mg .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-mg .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-mg .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-label-box{text-align:left;}.thumbnails-mg .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-mg .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-mg .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-mg .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-mg .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-mg .playerCube div.videoCube:hover,.thumbnails-mg  div.videoCube_hover{background-color:transparent;}.thumbnails-mg .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-mg div.syndicatedItem:hover,.thumbnails-mg  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-mg div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-mg .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-mg .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-mg .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-mg .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-mg .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-mg .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-mg .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-mg .syndicatedItem{background-color:transparent;}.thumbnails-mg .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-title{max-height:66.0px;*height:66.0px;color:#444444;font-family:Arial, Helvetica, san-serif;font-size:16.0px;line-height:22.0px;font-weight:bold;text-decoration:none;}.thumbnails-mg .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-mg .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-mg .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:Arial, Helvetica, san-serif;background-image:null;text-align:left;}.thumbnails-mg .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-mg .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-mg .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-mg .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-mg .video-icon-img{margin:0px;height:18px;}.thumbnails-mg .video-label-box.trc-pre-label{height:auto;}.thumbnails-mg .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-mg .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-mg .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-mg .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-mg .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-n .video-title{font-family:Arial, Helvetica, sans-serif;font-size:18.0px;line-height:24.0px;font-weight:bold;max-height:96.0px;*height:96.0px;color:#000;text-decoration:none;}.thumbnails-n .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-n .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-n .videoCube .video-duration{left:36px;display:none;}.thumbnails-n .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-n .video-label,.thumbnails-n .sponsored,.thumbnails-n .sponsored-url{font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';}.thumbnails-n .trc_rbox_header{font-family:'ProximaNovaCond-Extrabld', 'NotoKufiArabic-Bold', 'Helvetica Neue', 'Helvetica', Roboto, Arial, sans-serif\3b ;font-size:28.0px;font-weight:bold;text-decoration:none;color:#222;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-n .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-n .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-n .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-n .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-n .videoCube{width:auto;_width:auto;background-color:transparent;border-width:1px;border-color:#D6D5D3;padding:0;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:none;}.thumbnails-n div.videoCube:hover,.thumbnails-n  div.videoCube_hover{background-color:transparent;}.thumbnails-n .sponsored-default{background-color:#F7F6C6;}.thumbnails-n div.sponsored-default:hover,.thumbnails-n  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-n .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-n .videoCube:hover .thumbnail-overlay,.thumbnails-n  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-n .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-n .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-n div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-n .pager_enabled{color:#0056b3;}.thumbnails-n .trc_pager_counter{color:#000000;}.thumbnails-n .pager_disabled{color:#7d898f;}.thumbnails-n .trc_pager_prev:hover,.thumbnails-n  .trc_pager_next:hover{color:#6497ED;}.thumbnails-n .trc_pager_selected{color:#0056b3;}.thumbnails-n .trc_pager_unselected{color:#7d898f;}.thumbnails-n div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-n .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-n .video-label-box{text-align:left;}.thumbnails-n .trc_sponsored_overlay{background-color:black;}.thumbnails-n .thumbnail-emblem{background-position:5% 5%;}.thumbnails-n .videoCube .sponsored{margin-top:-7px;}.thumbnails-n {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-n .videoCube.vertical{border-style:solid none none none;}.thumbnails-n .videoCube.horizontal{border-style:none;}.thumbnails-n .trc_pager_prev,.thumbnails-n .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-n .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-n .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .trc_pager div{font-family:serif;}.thumbnails-n .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-n .playerCube:hover .thumbnail-overlay,.thumbnails-n  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-n .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-n .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-n .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-n .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-label-box{text-align:left;}.thumbnails-n .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-n .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-n .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-n .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-n .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-n .playerCube div.videoCube:hover,.thumbnails-n  div.videoCube_hover{background-color:transparent;}.thumbnails-n .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-n div.syndicatedItem:hover,.thumbnails-n  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-n div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-n .videoCube.syndicatedItem{background-color:transparent;border-color:#D6D5D3;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:1px;border-style:none;}.thumbnails-n .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-n .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-n .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-n .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-n .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-n .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-n .syndicatedItem{background-color:transparent;}.thumbnails-n .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-n .syndicatedItem .video-title{max-height:72.0px;*height:72.0px;color:#000;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';font-size:18.0px;line-height:24.0px;font-weight:bold;text-decoration:none;}.thumbnails-n .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-n .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-n .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:'ProximaNovaCond-bold', 'ProximaNovaCond-Extrabld';background-image:null;text-align:left;}.thumbnails-n .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-n .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-n .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-n .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-n .video-icon-img{margin:0px;height:18px;}.thumbnails-n .video-label-box.trc-pre-label{height:auto;}.thumbnails-n .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-n .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-n .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-n .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-n .videoCube .video-label-box.trc-pre-label{margin:0;}.thumbnails-o .video-title{font-family:Arial, Helvetica, sans-serif;font-size:18.0px;line-height:24.0px;font-weight:bold;max-height:96.0px;*height:96.0px;color:#000000;text-decoration:none;}.thumbnails-o .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;max-height:2.2em;*height:2.2em;color:black;text-decoration:none;}.thumbnails-o .trc_rbox_div{width:auto;_width:99%;height:410px;border-width:0;padding:0;}.thumbnails-o .videoCube .video-duration{left:36px;display:none;}.thumbnails-o .videoCube .video-label-box{margin-left:0;margin-right:0px;}.thumbnails-o .video-label,.thumbnails-o .sponsored,.thumbnails-o .sponsored-url{font-family:ProximaNovaCond-Medium;}.thumbnails-o .trc_rbox_header{font-family:ProximaNovaCond-Bold;font-size:20.0px;font-weight:bold;text-decoration:none;color:#000000;border-width:0;background:transparent;border-style:none;border-color:#D6D5D3;padding:0 0 6px 0;}.thumbnails-o .sponsored-url{font-size:9px;font-weight:bold;text-decoration:underline;color:green;}.thumbnails-o .sponsored{font-size:9px;font-weight:normal;text-decoration:none;color:#9C9A9C;}.thumbnails-o .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .sponsored-default .video-title{max-height:2.58em;*height:2.58em;}.thumbnails-o .sponsored-default .video-description{max-height:2.2em;*height:2.2em;}.thumbnails-o .videoCube{width:auto;_width:auto;background-color:transparent;border-width:0px 0px 0px 0px;border-color:#E4E4E4;padding:0px 0px 0px 0px;height:auto;margin-left:0px;margin-top:0px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-style:SOLID;}.thumbnails-o div.videoCube:hover,.thumbnails-o  div.videoCube_hover{background-color:transparent;}.thumbnails-o .sponsored-default{background-color:#F7F6C6;}.thumbnails-o div.sponsored-default:hover,.thumbnails-o  div.sponsored-default.videoCube_hover{background-color:inherit;}.thumbnails-o .videoCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-o .videoCube:hover .thumbnail-overlay,.thumbnails-o  .videoCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-o .trc_rbox_border_elm{border-color:darkgray;}.thumbnails-o .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-o div.videoCube:hover .thumbBlock{border-color:inherit;}.thumbnails-o .pager_enabled{color:#0056b3;}.thumbnails-o .trc_pager_counter{color:#000000;}.thumbnails-o .pager_disabled{color:#7d898f;}.thumbnails-o .trc_pager_prev:hover,.thumbnails-o  .trc_pager_next:hover{color:#6497ED;}.thumbnails-o .trc_pager_selected{color:#0056b3;}.thumbnails-o .trc_pager_unselected{color:#7d898f;}.thumbnails-o div.trc_pager_pages div:hover{color:#6497ED;}.thumbnails-o .trc_lightbox_overlay{background-color:#000000;opacity:0.70;filter:alpha(opacity=70);}.thumbnails-o .video-label-box{text-align:left;}.thumbnails-o .trc_sponsored_overlay{background-color:black;}.thumbnails-o .thumbnail-emblem{background-position:5% 5%;}.thumbnails-o .videoCube .sponsored{margin-top:-7px;}.thumbnails-o {width:300px;_width:300px;border-width:0px 0px 0px 0px;border-style:solid solid solid solid;border-color:#DFDFDF;padding:0px 0px 0px 0px;border-radius:0;-moz-border-radius:0;-webkit-border-radius:0;box-shadow:none;}.thumbnails-o .videoCube.vertical{border-style:solid none none none;}.thumbnails-o .videoCube.horizontal{border-style:none;}.thumbnails-o .trc_pager_prev,.thumbnails-o .trc_pager_next{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-o .trc_pager_pages div{font-size:12px;font-weight:normal;text-decoration:none;}.thumbnails-o .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .trc_pager div{font-family:serif;}.thumbnails-o .playerCube .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-o .playerCube:hover .thumbnail-overlay,.thumbnails-o  .playerCube_hover .thumbnail-overlay{background-image:null;}.thumbnails-o .playerCube .videoCube{background-color:transparent;border-color:#D6D5D3;border-width:1px;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;margin-left:0px;margin-top:0px;padding:3px;}.thumbnails-o .playerCube .videoCube.horizontal{border-style:none none none none;}.thumbnails-o .playerCube .videoCube .video-label-box{margin-left:81px;margin-right:0px;}.thumbnails-o .playerCube .video-duration-detail{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-external-data{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-label-box{text-align:left;}.thumbnails-o .playerCube .video-published-date{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-category{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-description{font-family:Arial, Helvetica, sans-serif;font-size:10px;line-height:11px;font-weight:normal;text-decoration:none;max-height:2.2em;*height:2.2em;color:black;}.thumbnails-o .playerCube .videoCube .video-duration{display:block;left:36px;}.thumbnails-o .playerCube .videoCube .thumbBlock{border-width:0px;border-color:darkgray;}.thumbnails-o .playerCube .video-rating{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-uploader{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-views{font-size:10px;font-weight:normal;text-decoration:none;color:black;}.thumbnails-o .playerCube .video-title{font-family:Arial, Helvetica, sans-serif;text-decoration:none;font-size:14px;line-height:17.5px;font-weight:bold;max-height:2.58em;*height:2.58em;color:black;}.thumbnails-o .playerCube div.videoCube:hover,.thumbnails-o  div.videoCube_hover{background-color:transparent;}.thumbnails-o .whatsThisSyndicated{font-family:Arial, Verdana, sans-serif;font-size:9px;font-weight:normal;color:black;text-decoration:none;padding:0;}.thumbnails-o div.syndicatedItem:hover,.thumbnails-o  div.syndicatedItem.videoCube_hover{background-color:transparent;}.thumbnails-o div.syndicatedItem:hover .thumbBlock{border-color:inherit;}.thumbnails-o .videoCube.syndicatedItem{background-color:transparent;border-color:#E4E4E4;border-radius:0px;-moz-border-radius:0px;-webkit-border-radius:0px;border-width:0px 0px 0px 0px;border-style:SOLID;}.thumbnails-o .videoCube.syndicatedItem.horizontal{border-style:none;}.thumbnails-o .videoCube.syndicatedItem .thumbBlock{border-color:darkgray;border-width:0px;}.thumbnails-o .videoCube.syndicatedItem .thumbnail-overlay{background-image:null;background-position:5% 5%;}.thumbnails-o .videoCube.syndicatedItem.vertical{border-style:solid none none none;}.thumbnails-o .videoCube.syndicatedItem .video-duration{display:none;left:36px;}.thumbnails-o .videoCube.syndicatedItem .video-label-box{margin-left:0px;}.thumbnails-o .syndicatedItem{background-color:transparent;}.thumbnails-o .syndicatedItem .video-description{max-height:2.2em;*height:2.2em;color:black;font-family:Arial, Helvetica, sans-serif;font-size:10px;font-weight:normal;line-height:11px;text-decoration:none;}.thumbnails-o .syndicatedItem .video-title{max-height:72.0px;*height:72.0px;color:#000000;font-family:ProximaNovaCond-Medium;font-size:18.0px;line-height:24.0px;font-weight:bold;text-decoration:none;}.thumbnails-o .syndicatedItem .sponsored{color:#9C9A9C;font-size:9px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .sponsored-url{color:green;font-size:9px;font-weight:bold;text-decoration:underline;}.thumbnails-o .syndicatedItem .video-category{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .video-duration-detail{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .video-external-data{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .video-published-date{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .video-rating{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .video-uploader{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .video-views{color:black;font-size:10px;font-weight:normal;text-decoration:none;}.thumbnails-o .syndicatedItem .branding{color:#999999;font-size:11.0px;font-weight:bold;text-decoration:none;font-family:ProximaNovaCond-Medium;background-image:null;text-align:left;}.thumbnails-o .videoCube.syndicatedItem .thumbBlock .branding{text-align:left;background-color:transparent;display:none;left:0px;color:black;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;background-image:null;}.thumbnails-o .videoCube.syndicatedItem .thumbBlock .static-text{text-align:left;background-color:black;display:none;color:white;font-size:10px;font-weight:normal;text-decoration:none;font-family:Arial, Helvetica, sans-serif;}.thumbnails-o .videoCube.thumbnail_start .thumbBlock_holder{width:40%;_width:40%;}.thumbnails-o .trc_rbox_header_icon_img{margin:0px;height:18px;}.thumbnails-o .video-icon-img{margin:0px;height:18px;}.thumbnails-o .video-label-box.trc-pre-label{height:auto;}.thumbnails-o .syndicatedItem .video-label-box.trc-pre-label{height:auto;}.thumbnails-o .videoCube.thumbnail_start .trc-pre-label{width:60%;_width:60%;}.thumbnails-o .videoCube.thumbnail_start.trc-split-label .trc-main-label{width:30%;_width:30%;}.thumbnails-o .videoCube.thumbnail_start.trc-split-label .trc-pre-label{width:30%;_width:30%;}.thumbnails-o .videoCube .video-label-box.trc-pre-label{margin:0;}.ab_thumbnails-a_abp-mode img {
        max-width: none;
}

/* override bootstrap default span definitions */
.ab_thumbnails-a_abp-mode [class*=span] {
    float:none;
    margin-left:0;
}


.ab_thumbnails-a_abp-mode .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.ab_thumbnails-a_abp-mode .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.ab_thumbnails-a_abp-mode .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.ab_thumbnails-a_abp-mode .logoDiv a {
    font-size: 100%;
}

.ab_thumbnails-a_abp-mode .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.ab_thumbnails-a_abp-mode .videoCube a {
    padding: 0;
}

.ab_thumbnails-a_abp-mode .thumbBlock {
    margin: 0;
}

.trc_elastic .ab_thumbnails-a_abp-mode .video-label-box {
    height: 69.0px;
}

.ab_thumbnails-a_abp-mode .videoCube .video-label-box {
    margin-top: 5px;
}

.ab_thumbnails-a_abp-mode .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.ab_thumbnails-a_abp-mode .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.ab_thumbnails-a_abp-mode .video-label-box .branding {
    display: block;
}

.ab_thumbnails-a_abp-mode .syndicatedItem .branding {
    line-height: 23.0px;
}
.ab_thumbnails-a_abp-mode .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.ab_thumbnails-a_abp-mode .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.ab_thumbnails-a_abp-mode .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.ab_thumbnails-o_abp-mode img {
        max-width: none;
}

/* override bootstrap default span definitions */
.ab_thumbnails-o_abp-mode [class*=span] {
    float:none;
    margin-left:0;
}


.ab_thumbnails-o_abp-mode .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: block;
        width: auto;
	margin: 0px 0px 0px 0px;
	background: transparent;
	height: auto;
	background-color: transparent;
	padding: 0 0 6px 0;
	box-sizing: initial;
}

.ab_thumbnails-o_abp-mode .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.ab_thumbnails-o_abp-mode .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.ab_thumbnails-o_abp-mode .logoDiv a {
    font-size: 100%;
}

.ab_thumbnails-o_abp-mode .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.ab_thumbnails-o_abp-mode .videoCube a {
    padding: 0;
}

.ab_thumbnails-o_abp-mode .thumbBlock {
    margin: 0;
}

.trc_elastic .ab_thumbnails-o_abp-mode .video-label-box {
    height: 96.0px;
}

.ab_thumbnails-o_abp-mode .videoCube .video-label-box {
    margin-top: 5px;
}

.ab_thumbnails-o_abp-mode .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.ab_thumbnails-o_abp-mode .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.ab_thumbnails-o_abp-mode .video-label-box .branding {
    display: block;
}

.ab_thumbnails-o_abp-mode .syndicatedItem .branding {
    line-height: 24.0px;
}
.ab_thumbnails-o_abp-mode .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.ab_thumbnails-o_abp-mode .trc_header_right_part {
	margin-top: 0px;
}

.ab_thumbnails-o_abp-mode .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.ab_thumbnails-o_abp-mode .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
/* override bootstrap default span definitions */
.text-links-a [class*=span] {
    float:none;
    margin-left:0;
}

.text-links-a .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.text-links-a .trc_rbox_header_span .trc_header_right_column {
        position: absolute;
        width: 48%;
        left: 52%;
        top: 0;
	background: transparent;
	height: auto;
}

.text-links-a .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
        position: relative;
        z-index: 1;
}

.text-links-a .logoDiv a {
    font-size: 100%;
}


.text-links-a .logoDiv a span {
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.text-links-a .syndicatedItem .video-title .branding {
        line-height: 1.3em;
}

.text-links-a .trc_rbox_div .videoCube.horizontal {
        clear: left;
}

.text-links-a .trc_rbox_div .videoCube.trc_tl_right_col {
        float: none;
        clear: right;
        margin-left: auto;
}

.text-links-a .videoCube:hover span.branding,
.text-links-a .videoCube_hover span.branding {
        text-decoration: none;
}

.text-links-a .trc_rbox_div .videoCube.thumbnail_none a{
    display: list-item;
    list-style: none outside none;
    vertical-align: top;
    overflow: visible;
    margin-left: 0;
    padding: 0;
}

.text-links-a .videoCube .video-label-box {
        display: inline-block!important;
        vertical-align: top;
        width: 100%;
        *margin-top: -3px;
	min-height: 19.0px;
}

.text-links-a .trc_rbox_div .videoCube:hover .video-title {
        text-decoration: underline;
}
.text-links-a .trc_header_left_column {
	width: auto;
	display: inline-block;
	background: transparent;
	height: auto;
}

.text-links-a .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.text-links-a .syndicatedItem .branding {
	line-height: 19.0px;
}

.text-links-a .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-a img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-a [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-a .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-a .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-a .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-a .logoDiv a {
    font-size: 100%;
}

.thumbnails-a .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-a .videoCube a {
    padding: 0;
}

.thumbnails-a .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-a .video-label-box {
    height: 96.0px;
}

.thumbnails-a .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-a .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-a .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-a .video-label-box .branding {
    display: block;
}

.thumbnails-a .syndicatedItem .branding {
    line-height: 24.0px;
}
.thumbnails-a .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-a .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-a .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-b img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-b [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-b .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-b .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-b .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-b .logoDiv a {
    font-size: 100%;
}

.thumbnails-b .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-b .videoCube a {
    padding: 0;
}

.thumbnails-b .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-b .video-label-box {
    height: 76.0px;
}

.thumbnails-b .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-b .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-b .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-b .video-label-box .branding {
    display: block;
}

.thumbnails-b .syndicatedItem .branding {
    line-height: 19.0px;
}
.thumbnails-b .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-b .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-b .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-c img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-c [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-c .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: none;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-c .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-c .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-c .logoDiv a {
    font-size: 100%;
}

.thumbnails-c .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11px;
}

.thumbnails-c .videoCube a {
    padding: 0;
}

.thumbnails-c .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-c .video-label-box {
    height: 54.0px;
}

.thumbnails-c .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-c .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-c .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-c .video-label-box .branding {
    display: block;
}

.thumbnails-c .syndicatedItem .branding {
    line-height: 18.0px;
}
.thumbnails-c .trc_header_left_column {
	background: transparent;
	height: auto;
}

.thumbnails-c .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-c .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.trc_elastic .thumbnails-c  .trc_rbox_outer .videoCube{
	  margin-bottom: 0px;
}
.thumbnails-d img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-d [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-d .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: none;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-d .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-d .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-d .logoDiv a {
    font-size: 100%;
}

.thumbnails-d .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11px;
}

.thumbnails-d .videoCube a {
    padding: 0;
}

.thumbnails-d .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-d .video-label-box {
    height: 54.0px;
}

.thumbnails-d .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-d .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-d .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-d .video-label-box .branding {
    display: block;
}

.thumbnails-d .syndicatedItem .branding {
    line-height: 18.0px;
}
.thumbnails-d .trc_header_left_column {
	background: transparent;
	height: auto;
}

.thumbnails-d .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-d .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.trc_elastic .thumbnails-d  .trc_rbox_outer .videoCube{
	  margin-bottom: 0px;
}
.thumbnails-e img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-e [class*=span] {
    float:none;
    margin-left:0;
}

.thumbnails-e .trc_rbox_div {
	margin-bottom: 0;
}

.thumbnails-e .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: none;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-e .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-e .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-e .logoDiv a {
    font-size: 100%;
}

.thumbnails-e .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11px;
}

.thumbnails-e .videoCube a {
    padding: 0;
}

.thumbnails-e .trc_rbox_outer .videoCube {
	margin-bottom: 10px;
}

.thumbnails-e .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-e .video-label-box {
    height: auto;
}

.thumbnails-e .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin-bottom: 0;
}

.thumbnails-e .videoCube:hover .video-label-box .video-title {
    text-decoration: none;
}

.thumbnails-e .video-label-box .branding {
    display: block;
}

.thumbnails-e .syndicatedItem .branding {
    line-height: 24.0px;
}
.thumbnails-e .trc_header_left_column {
	background: transparent;
	height: auto;
}

.thumbnails-e .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-e .videoCube .video-label-box {
	margin-top: 0px;
}

.thumbnails-e .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.trc_elastic .thumbnails-e  .trc_rbox_outer .videoCube{
	  margin-bottom: 0px;
}
.trc_elastic .thumbnails-e .thumbnail_start .thumbBlock_holder {
    float: right;
    margin-left: 15px;
}
.thumbnails-e .videoCube.thumbnail_start .thumbBlock_holder {
    width: 120px;
    height: 120px;
}
.thumbnails-f img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-f [class*=span] {
    float:none;
    margin-left:0;
}

.thumbnails-f .trc_rbox_div {
	margin-bottom: 0;
}

.thumbnails-f .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: none;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-f .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-f .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-f .logoDiv a {
    font-size: 100%;
}

.thumbnails-f .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11px;
}

.thumbnails-f .videoCube a {
    padding: 0;
}

.thumbnails-f .trc_rbox_outer .videoCube {
	margin-bottom: 10px;
}

.thumbnails-f .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-f .video-label-box {
    height: 76.0px;
}

.thumbnails-f .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin-bottom: 0;
}

.thumbnails-f .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-f .video-label-box .branding {
    display: block;
}

.thumbnails-f .syndicatedItem .branding {
    line-height: 30.0px;
}
.thumbnails-f .trc_header_left_column {
	background: transparent;
	height: auto;
}

.thumbnails-f .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-f .videoCube .video-label-box {
	margin-top: 0px;
}

.thumbnails-f .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-g img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-g [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-g .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-g .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-g .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-g .logoDiv a {
    font-size: 100%;
}

.thumbnails-g .logoDiv a span {
    display: inline;
    color: #fff;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-g .videoCube a {
    padding: 0;
}

.thumbnails-g .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-g .video-label-box {
    height: 76.0px;
}

.thumbnails-g .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-g .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-g .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-g .video-label-box .branding {
    display: block;
}

.thumbnails-g .syndicatedItem .branding {
    line-height: 19.0px;
}
.thumbnails-g .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-g .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-g .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-h img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-h [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-h .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-h .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-h .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-h .logoDiv a {
    font-size: 100%;
}

.thumbnails-h .logoDiv a span {
    display: inline;
    color: #fff;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-h .videoCube a {
    padding: 0;
}

.thumbnails-h .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-h .video-label-box {
    height: 76.0px;
}

.thumbnails-h .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-h .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-h .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-h .video-label-box .branding {
    display: block;
}

.thumbnails-h .syndicatedItem .branding {
    line-height: 19.0px;
}
.thumbnails-h .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-h .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-h .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-i img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-i [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-i .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-i .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-i .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-i .logoDiv a {
    font-size: 100%;
}

.thumbnails-i .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-i .videoCube a {
    padding: 0;
}

.thumbnails-i .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-i .video-label-box {
    height: 92.0px;
}

.thumbnails-i .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-i .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-i .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-i .video-label-box .branding {
    display: block;
}

.thumbnails-i .syndicatedItem .branding {
    line-height: 23.0px;
}
.thumbnails-i .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-i .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-i .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-j img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-j [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-j .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-j .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-j .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-j .logoDiv a {
    font-size: 100%;
}

.thumbnails-j .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-j .videoCube a {
    padding: 0;
}

.thumbnails-j .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-j .video-label-box {
    height: 92.0px;
}

.thumbnails-j .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-j .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-j .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-j .video-label-box .branding {
    display: block;
}

.thumbnails-j .syndicatedItem .branding {
    line-height: 23.0px;
}
.thumbnails-j .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-j .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-j .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-k img {
        max-width: none;
}
/* override bootstrap default span definitions */
.thumbnails-k [class*=span] {
    float:none;
    margin-left:0;
}
.thumbnails-k {
	padding-bottom: 35px;
}
.thumbnails-k .trc_rbox_header {
    width: 100%;
	line-height: 0.1em;
	text-align: center;
	margin: 10px 0;
	padding: 0;
	border-bottom: 3px solid #000;
}
.thumbnails-k .trc_rbox_header span {
	padding: 0 10px;
	letter-spacing: 1.4px;
	background:#fff;
}
.thumbnails-k .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}
.thumbnails-k .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}
.thumbnails-k .logoDiv a {
    font-size: 100%;
}
.thumbnails-k .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}
.thumbnails-k .videoCube a {
    padding: 0;
}
.thumbnails-k .thumbBlock {
    margin: 0;
}
.trc_elastic .thumbnails-k .video-label-box {
    height: 96.0px;
}
.thumbnails-k .videoCube .video-label-box {
    margin-top: 5px;
}
.thumbnails-k .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}
.thumbnails-k .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}
.thumbnails-k .video-label-box .branding {
    display: block;
}
.thumbnails-k .syndicatedItem .branding {
    line-height: 24.0px;
}
.thumbnails-k .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}
.thumbnails-k .trc_rbox_header .logoDiv a {
	font-size: 100%;
}
.thumbnails-k .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-m img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-m [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-m .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-m .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-m .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-m .logoDiv a {
    font-size: 100%;
}

.thumbnails-m .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-m .videoCube a {
    padding: 0;
}

.thumbnails-m .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-m .video-label-box {
    height: 88.0px;
}

.thumbnails-m .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-m .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-m .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-m .video-label-box .branding {
    display: block;
}

.thumbnails-m .syndicatedItem .branding {
    line-height: 22.0px;
}
.thumbnails-m .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-m .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-m .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-m-AMP img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-m-AMP [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-m-AMP .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-m-AMP .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-m-AMP .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-m-AMP .logoDiv a {
    font-size: 100%;
}

.thumbnails-m-AMP .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-m-AMP .videoCube a {
    padding: 0;
}

.thumbnails-m-AMP .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-m-AMP .video-label-box {
    height: 76.0px;
}

.thumbnails-m-AMP .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-m-AMP .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-m-AMP .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-m-AMP .video-label-box .branding {
    display: block;
}

.thumbnails-m-AMP .syndicatedItem .branding {
    line-height: 19.0px;
}
.thumbnails-m-AMP .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-m-AMP .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-m-AMP .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-mg img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-mg [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-mg .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-mg .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-mg .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-mg .logoDiv a {
    font-size: 100%;
}

.thumbnails-mg .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-mg .videoCube a {
    padding: 0;
}

.thumbnails-mg .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-mg .video-label-box {
    height: 88.0px;
}

.thumbnails-mg .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-mg .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-mg .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-mg .video-label-box .branding {
    display: block;
}

.thumbnails-mg .syndicatedItem .branding {
    line-height: 22.0px;
}
.thumbnails-mg .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-mg .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-mg .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-n img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-n [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-n .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: inline-block;
        width: 100%;
	background: transparent;
	height: auto;
}

.thumbnails-n .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-n .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-n .logoDiv a {
    font-size: 100%;
}

.thumbnails-n .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-n .videoCube a {
    padding: 0;
}

.thumbnails-n .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-n .video-label-box {
    height: 96.0px;
}

.thumbnails-n .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-n .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-n .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-n .video-label-box .branding {
    display: block;
}

.thumbnails-n .syndicatedItem .branding {
    line-height: 24.0px;
}
.thumbnails-n .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-n .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-n .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
.thumbnails-o img {
        max-width: none;
}

/* override bootstrap default span definitions */
.thumbnails-o [class*=span] {
    float:none;
    margin-left:0;
}


.thumbnails-o .trc_rbox_header {
        line-height: 1.2em;
        position: relative;
        display: block;
        width: auto;
	margin: 0px 0px 0px 0px;
	background: transparent;
	height: auto;
	background-color: transparent;
	padding: 0 0 6px 0;
	box-sizing: initial;
}

.thumbnails-o .trc_rbox_header_span .trc_header_right_column {
        display: none;
	background: transparent;
	height: auto;
}

.thumbnails-o .trc_rbox_header .logoDiv {
        font-size: inherit;
        line-height: normal;
}

.thumbnails-o .logoDiv a {
    font-size: 100%;
}

.thumbnails-o .logoDiv a span {
    display: inline;
    color: #000000;
    font-weight: normal;
    font-size: 11.0px;
}

.thumbnails-o .videoCube a {
    padding: 0;
}

.thumbnails-o .thumbBlock {
    margin: 0;
}

.trc_elastic .thumbnails-o .video-label-box {
    height: 96.0px;
}

.thumbnails-o .videoCube .video-label-box {
    margin-top: 5px;
}

.thumbnails-o .videoCube .video-label-box .video-title {
    text-decoration: none;
    margin: 0;
}

.thumbnails-o .videoCube:hover .video-label-box .video-title {
    text-decoration: underline;
}

.thumbnails-o .video-label-box .branding {
    display: block;
}

.thumbnails-o .syndicatedItem .branding {
    line-height: 24.0px;
}
.thumbnails-o .trc_header_left_column {
	width: 48%;
	display: inline-block;
	background: transparent;
	height: auto;
}

.thumbnails-o .trc_header_right_part {
	margin-top: 0px;
}

.thumbnails-o .trc_rbox_header .logoDiv a {
	font-size: 100%;
}

.thumbnails-o .trc_rbox_header .trc_header_ext {
	position: relative;
	top: auto;
	right: auto;
}
</style><script type="text/javascript" src="//widget.perfectmarket.com/aol-huffingtonpost/pmk-201725001.2.js"></script><script src="//mms.huffingtonpost.com/"></script><style type="text/css"></style><style type="text/css">@media screen and (min-width: 0px) {.trc_elastic_trc_11200 .trc_rbox_outer{margin-left:-2%;}.trc_elastic_trc_11200 .videoCube_aspect{padding-bottom:55.172413793103445%; width: 100%;}.trc_elastic_trc_11200 .videoCube{width: 31.33%; position: relative; float: left; margin: 0 0 2% 0; margin-left: 2%;}.trc_elastic_trc_11200 div.videoCube:nth-of-type(-n+6){display:block;visibility:visible;}.trc_elastic_trc_11200 div.videoCube:nth-of-type(n+7){display:none;visibility:hidden;}} </style><style type="text/css">.trc_popover_aug_container { position: static;}#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover {position: absolute; font-family: Arial, Helvetica, sans-serif; font-size: 12px; line-height: 16px; color: #000000; cursor: default; position: absolute; top: 0; right: 0; z-index: 12000; width: 180px; padding: 1px; text-align: left; white-space: normal; background-color: #ffffff; border: 1px solid #ccc; border: 1px solid rgba(0, 0, 0, 0.2); -webkit-border-radius: 6px; -moz-border-radius: 6px; border-radius: 6px; -webkit-box-shadow: 0 5px 10px rgba(0, 0, 0, 0.2); -moz-box-shadow: 0 5px 10px rgba(0, 0, 0, 0.2); box-shadow: 0 5px 10px rgba(0, 0, 0, 0.2); -webkit-background-clip: padding-box; -moz-background-clip: padding; background-clip: padding-box;-moz-box-sizing: content-box; -webkit-box-sizing: content-box; box-sizing: content-box;}#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover iframe{width: 100%;}#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover .trc_popover_arrow, .trc_popover .trc_popover_arrow:after { position: absolute; display: block; width: 0; height: 0; border-color: transparent; border-style: solid; }#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover .trc_popover_arrow { border-width: 11px; }#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover .trc_popover_arrow:after { border-width: 10px; content: "";}#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover.trc_bottom { margin-top: 10px; }#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover.trc_bottom .trc_popover_arrow { top: -11px; right: 11px; margin-left: -11px; border-bottom-color: #999; border-bottom-color: rgba(0, 0, 0, 0.25); border-top-width: 0; }#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover.trc_bottom .trc_popover_arrow:after { top: 1px; margin-left: -10px; border-bottom-color: #ffffff; border-top-width: 0; }#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover_fade.trc_in { visibility: visible; opacity: 1; filter: alpha(opacity=100); transition-delay: 0s, 0s; -moz-transition-delay: 0s, 0s; -webkit-transition-delay: 0s, 0s; -o-transition-delay: 0s, 0s; -ms-transition-delay: 0s, 0s;}#tbl-aug-ipsash #tbl-aug-o1omwa #tbl-aug-63lmh2 .trc_popover_fade { visibility: hidden; opacity: 0; filter: alpha(opacity=0); transition: opacity 500ms 0s, visibility 0s 500ms; -webkit-transition: opacity 500ms 0s, visibility 0s 500ms;  -moz-transition: opacity 500ms 0s, visibility 0s 500ms; -o-transition: opacity 500ms 0s, visibility 0s 500ms; -ms-transition: opacity 500ms 0s, visibility 0s 500ms;}</style><style type="text/css">.fb_hidden{position:absolute;top:-10000px;z-index:10001}.fb_reposition{overflow:hidden;position:relative}.fb_invisible{display:none}.fb_reset{background:none;border:0;border-spacing:0;color:#000;cursor:auto;direction:ltr;font-family:"lucida grande", tahoma, verdana, arial, sans-serif;font-size:11px;font-style:normal;font-variant:normal;font-weight:normal;letter-spacing:normal;line-height:1;margin:0;overflow:visible;padding:0;text-align:left;text-decoration:none;text-indent:0;text-shadow:none;text-transform:none;visibility:visible;white-space:normal;word-spacing:normal}.fb_reset>div{overflow:hidden}.fb_link img{border:none}@keyframes fb_transform{from{opacity:0;transform:scale(.95)}to{opacity:1;transform:scale(1)}}.fb_animate{animation:fb_transform .3s forwards}
.fb_dialog{background:rgba(82, 82, 82, .7);position:absolute;top:-10000px;z-index:10001}.fb_reset .fb_dialog_legacy{overflow:visible}.fb_dialog_advanced{padding:10px;-moz-border-radius:8px;-webkit-border-radius:8px;border-radius:8px}.fb_dialog_content{background:#fff;color:#333}.fb_dialog_close_icon{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/yq/r/IE9JII6Z1Ys.png) no-repeat scroll 0 0 transparent;cursor:pointer;display:block;height:15px;position:absolute;right:18px;top:17px;width:15px}.fb_dialog_mobile .fb_dialog_close_icon{top:5px;left:5px;right:auto}.fb_dialog_padding{background-color:transparent;position:absolute;width:1px;z-index:-1}.fb_dialog_close_icon:hover{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/yq/r/IE9JII6Z1Ys.png) no-repeat scroll 0 -15px transparent}.fb_dialog_close_icon:active{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/yq/r/IE9JII6Z1Ys.png) no-repeat scroll 0 -30px transparent}.fb_dialog_loader{background-color:#f6f7f9;border:1px solid #606060;font-size:24px;padding:20px}.fb_dialog_top_left,.fb_dialog_top_right,.fb_dialog_bottom_left,.fb_dialog_bottom_right{height:10px;width:10px;overflow:hidden;position:absolute}.fb_dialog_top_left{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/ye/r/8YeTNIlTZjm.png) no-repeat 0 0;left:-10px;top:-10px}.fb_dialog_top_right{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/ye/r/8YeTNIlTZjm.png) no-repeat 0 -10px;right:-10px;top:-10px}.fb_dialog_bottom_left{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/ye/r/8YeTNIlTZjm.png) no-repeat 0 -20px;bottom:-10px;left:-10px}.fb_dialog_bottom_right{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/ye/r/8YeTNIlTZjm.png) no-repeat 0 -30px;right:-10px;bottom:-10px}.fb_dialog_vert_left,.fb_dialog_vert_right,.fb_dialog_horiz_top,.fb_dialog_horiz_bottom{position:absolute;background:#525252;filter:alpha(opacity=70);opacity:.7}.fb_dialog_vert_left,.fb_dialog_vert_right{width:10px;height:100%}.fb_dialog_vert_left{margin-left:-10px}.fb_dialog_vert_right{right:0;margin-right:-10px}.fb_dialog_horiz_top,.fb_dialog_horiz_bottom{width:100%;height:10px}.fb_dialog_horiz_top{margin-top:-10px}.fb_dialog_horiz_bottom{bottom:0;margin-bottom:-10px}.fb_dialog_iframe{line-height:0}.fb_dialog_content .dialog_title{background:#6d84b4;border:1px solid #365899;color:#fff;font-size:14px;font-weight:bold;margin:0}.fb_dialog_content .dialog_title>span{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/yd/r/Cou7n-nqK52.gif) no-repeat 5px 50%;float:left;padding:5px 0 7px 26px}body.fb_hidden{-webkit-transform:none;height:100%;margin:0;overflow:visible;position:absolute;top:-10000px;left:0;width:100%}.fb_dialog.fb_dialog_mobile.loading{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/ya/r/3rhSv5V8j3o.gif) white no-repeat 50% 50%;min-height:100%;min-width:100%;overflow:hidden;position:absolute;top:0;z-index:10001}.fb_dialog.fb_dialog_mobile.loading.centered{width:auto;height:auto;min-height:initial;min-width:initial;background:none}.fb_dialog.fb_dialog_mobile.loading.centered #fb_dialog_loader_spinner{width:100%}.fb_dialog.fb_dialog_mobile.loading.centered .fb_dialog_content{background:none}.loading.centered #fb_dialog_loader_close{color:#fff;display:block;padding-top:20px;clear:both;font-size:18px}#fb-root #fb_dialog_ipad_overlay{background:rgba(0, 0, 0, .45);position:absolute;bottom:0;left:0;right:0;top:0;width:100%;min-height:100%;z-index:10000}#fb-root #fb_dialog_ipad_overlay.hidden{display:none}.fb_dialog.fb_dialog_mobile.loading iframe{visibility:hidden}.fb_dialog_content .dialog_header{-webkit-box-shadow:white 0 1px 1px -1px inset;background:-webkit-gradient(linear, 0% 0%, 0% 100%, from(#738ABA), to(#2C4987));border-bottom:1px solid;border-color:#1d4088;color:#fff;font:14px Helvetica, sans-serif;font-weight:bold;text-overflow:ellipsis;text-shadow:rgba(0, 30, 84, .296875) 0 -1px 0;vertical-align:middle;white-space:nowrap}.fb_dialog_content .dialog_header table{-webkit-font-smoothing:subpixel-antialiased;height:43px;width:100%}.fb_dialog_content .dialog_header td.header_left{font-size:12px;padding-left:5px;vertical-align:middle;width:60px}.fb_dialog_content .dialog_header td.header_right{font-size:12px;padding-right:5px;vertical-align:middle;width:60px}.fb_dialog_content .touchable_button{background:-webkit-gradient(linear, 0% 0%, 0% 100%, from(#4966A6), color-stop(.5, #355492), to(#2A4887));border:1px solid #29487d;-webkit-background-clip:padding-box;-webkit-border-radius:3px;-webkit-box-shadow:rgba(0, 0, 0, .117188) 0 1px 1px inset, rgba(255, 255, 255, .167969) 0 1px 0;display:inline-block;margin-top:3px;max-width:85px;line-height:18px;padding:4px 12px;position:relative}.fb_dialog_content .dialog_header .touchable_button input{border:none;background:none;color:#fff;font:12px Helvetica, sans-serif;font-weight:bold;margin:2px -12px;padding:2px 6px 3px 6px;text-shadow:rgba(0, 30, 84, .296875) 0 -1px 0}.fb_dialog_content .dialog_header .header_center{color:#fff;font-size:16px;font-weight:bold;line-height:18px;text-align:center;vertical-align:middle}.fb_dialog_content .dialog_content{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/y9/r/jKEcVPZFk-2.gif) no-repeat 50% 50%;border:1px solid #555;border-bottom:0;border-top:0;height:150px}.fb_dialog_content .dialog_footer{background:#f6f7f9;border:1px solid #555;border-top-color:#ccc;height:40px}#fb_dialog_loader_close{float:left}.fb_dialog.fb_dialog_mobile .fb_dialog_close_button{text-shadow:rgba(0, 30, 84, .296875) 0 -1px 0}.fb_dialog.fb_dialog_mobile .fb_dialog_close_icon{visibility:hidden}#fb_dialog_loader_spinner{animation:rotateSpinner 1.2s linear infinite;background-color:transparent;background-image:url(http://static.xx.fbcdn.net/rsrc.php/v3/yD/r/t-wz8gw1xG1.png);background-repeat:no-repeat;background-position:50% 50%;height:24px;width:24px}@keyframes rotateSpinner{0%{transform:rotate(0deg)}100%{transform:rotate(360deg)}}
.fb_iframe_widget{display:inline-block;position:relative}.fb_iframe_widget span{display:inline-block;position:relative;text-align:justify}.fb_iframe_widget iframe{position:absolute}.fb_iframe_widget_fluid_desktop,.fb_iframe_widget_fluid_desktop span,.fb_iframe_widget_fluid_desktop iframe{max-width:100%}.fb_iframe_widget_fluid_desktop iframe{min-width:220px;position:relative}.fb_iframe_widget_lift{z-index:1}.fb_hide_iframes iframe{position:relative;left:-10000px}.fb_iframe_widget_loader{position:relative;display:inline-block}.fb_iframe_widget_fluid{display:inline}.fb_iframe_widget_fluid span{width:100%}.fb_iframe_widget_loader iframe{min-height:32px;z-index:2;zoom:1}.fb_iframe_widget_loader .FB_Loader{background:url(http://static.xx.fbcdn.net/rsrc.php/v3/y9/r/jKEcVPZFk-2.gif) no-repeat;height:32px;width:32px;margin-left:-16px;position:absolute;left:50%;z-index:4}</style><style type="text/css">
.plr-card-t{
    display: block;
    margin-bottom: 20px;
    width: 100%;
}

.plr-card__content-t{
    padding-bottom: 20px;
}

.plr-card__headlines-t{
    max-width: 100%;
    min-height: 64px;
    overflow: hidden;
}

.plr-card__headline-t{
    margin-top: 0;
    letter-spacing: 0.25px;
    font-family: "ProximaNovaCond-Medium", "Helvetica Neue", "Helvetica", Roboto, Arial, sans-serif;
    font-size: 1.125rem;
    line-height: 1.25rem;
}

.plr-card__link-t{
    color: inherit;
    text-decoration: none;
}

.plr-card__link-t:hover{
    text-decoration: underline;
}

.plr-card__image-t{
    width: 64px;
    max-width: 64px;
    float: left;
    margin-right: 15px;
}

.plr-card__image__wrapper-t{
    line-height: 0;
    position: relative;
    overflow: hidden;
    background-size: cover;
    background-position: center center;
    height: 64px;
}

.plr-card__pill-text-t{
    font-size: 0.75rem;
    line-height: 1.125rem;
    letter-spacing: -0.01em;
    color: #f83371;
}
</style><script type="text/javascript" async="" src="//r.skimresources.com/api/?callback=skimlinksApplyHandlers&data=%7B%22pubcode%22%3A%2238395X1559799%22%2C%22domains%22%3A%5B%22facebook.com%22%2C%22twitter.com%22%2C%22instagram.com%22%2C%22highline.huffingtonpost.com%22%2C%22testkitchen.huffingtonpost.com%22%2C%22elections.huffingtonpost.com%22%2C%22horoscopes.huffingtonpost.com%22%2C%22huffpostarabi.com%22%2C%22huffingtonpost.com.au%22%2C%22huffpostbrasil.com%22%2C%22huffingtonpost.ca%22%2C%22huffingtonpost.de%22%2C%22huffingtonpost.es%22%2C%22huffingtonpost.fr%22%2C%22huffingtonpost.gr%22%2C%22huffingtonpost.in%22%2C%22huffingtonpost.it%22%2C%22huffingtonpost.jp%22%2C%22huffingtonpost.kr%22%2C%22huffpostmaghreb.com%22%2C%22huffingtonpost.com.mx%22%2C%22quebec.huffingtonpost.ca%22%2C%22huffingtonpost.co.za%22%2C%22huffingtonpost.co.uk%22%2C%22adinfo.aol.com%22%2C%22wftv.com%22%2C%22mynews13.com%22%2C%22popup.taboola.com%22%2C%22stylebistro.com%22%2C%22sn.sptrkr.com%22%2C%22quiz.howstuffworks.com%22%2C%22go.babbel.com%22%2C%22plarium.com%22%2C%22livingly.com%22%2C%22advertising.aol.com%22%2C%22careers.smartrecruiters.com%22%2C%22privacy.aol.com%22%2C%22oath.com%22%5D%2C%22page%22%3A%22http%3A%2F%2Fwww.huffingtonpost.com%2F2015%2F03%2F05%2Fderek-cruice-shot-in-face_n_6805902.html%22%7D"></script></head>
<body class="desktop b-page department--politics-news section--crime legacy-a standard" data-locale="en_US">
<div id="recaptcha-container"></div>

<div class="advertisement asset_inserts">
<script type="text/javascript">
waitForGlobal("adSetMOAT", function() {
  adSetMOAT("1");
});
</script>
</div>



<div style="position: absolute; left: -9999px; top: -9999px;"><object type="application/x-shockwave-flash" classid="clsid:d27cdb6e-ae6d-11cf-96b8-444553540000" width="1" height="1" id="moatMessageSender"><param name="allowScriptAccess" value="always"><param name="allowFullScreen" value="false"><param name="movie" value="http://o.aolcdn.com/os/moat/prod/MessageSenderV2.swf"><param name="FlashVars" value="r=MoatSuperV5.yh.zb&s=MoatSuperV5.yh.zc&e=MoatSuperV5.yh.zd&td=afs.moatads.com"><param name="quality" value="high"><param name="bgcolor" value="#ffffff"><embed type="application/x-shockwave-flash" src="http://o.aolcdn.com/os/moat/prod/MessageSenderV2.swf" quality="high" flashvars="r=MoatSuperV5.yh.zb&s=MoatSuperV5.yh.zc&e=MoatSuperV5.yh.zd&td=afs.moatads.com" bgcolor="#ffffff" width="1" height="1" id="moatMessageSenderEmbed" align="middle" allowscriptaccess="always" allowfullscreen="false"></object></div><div class="main">

<!-- Header -->
<div class="header__placeholder"></div>
<header class="header js-header">
<div class="header__content bn-masthead" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;masthead&quot;}}" data-beacon-parsed="true">
<div class="header__content-left">
<div class="header__hamburger bn-hamburger-menu" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;hamburger_menu&quot;}}" data-beacon-parsed="true">
<div class="header__hamburger-link header--transition bn-hamburger-button js-hamburger-link bn-clickable" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;open&quot;}}" data-beacon-parsed="true" data-beacon-toggle-type="lnid" data-beacon-toggle-value="close"></div>
</div>
</div>
<div class="header__logo">
<a class="logo bn-logo-link" href="/" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;logo&quot;}}" data-beacon-parsed="true">
<img alt="HuffPost" class="logo__img header--transition bn-logo-image bn-clickable" src="http://s.m.huffpost.com/assets/HuffPostHero-d783f680fb974e80346bdfa145131d058e596e1d32f791981875aa481702c770.svg" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;homepage&quot;}}" data-beacon-parsed="true">
</a>
</div>
<div class="header__content-right">
<div class="header__social-list bn-social-list" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;follow&quot;}}" data-beacon-parsed="true">
<a class="bn-follow-item header--transition header__social-list__item header__social-list__item--newsletters bn-clickable" data-button-name="newsletters" href="/newsletters" target="" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;newsletters&quot;,&quot;mpid&quot;:1}}" data-beacon-parsed="true"></a>
<a class="bn-follow-item header--transition header__social-list__item header__social-list__item--facebook bn-clickable" data-button-name="facebook" href="http://facebook.com/HuffPost" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;facebook&quot;,&quot;mpid&quot;:2}}" data-beacon-parsed="true"></a>
<a class="bn-follow-item header--transition header__social-list__item header__social-list__item--twitter bn-clickable" data-button-name="twitter" href="https://twitter.com/HuffPost" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter&quot;,&quot;mpid&quot;:3}}" data-beacon-parsed="true"></a>
<a class="bn-follow-item header--transition header__social-list__item header__social-list__item--instagram bn-clickable" data-button-name="instagram" href="http://instagram.com/HuffPost" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;instagram&quot;,&quot;mpid&quot;:4}}" data-beacon-parsed="true"></a>
<a class="bn-follow-item header--transition header__social-list__item header__social-list__item--snapchat js-snapchat-follow bn-clickable" data-button-name="snapchat" href="" target="" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;snapchat&quot;,&quot;mpid&quot;:5}}" data-beacon-parsed="true"></a>
</div>
<div class="header__editions bn-editions js-editions-button bn-editions-link bn-multiple bn-clickable" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;editions&quot;,&quot;lnid&quot;:&quot;open&quot;}}" data-beacon-parsed="true" data-beacon-toggle-type="lnid" data-beacon-toggle-value="close">
<span class="header__editions__label bn-clickable">EDITION</span>
<div class="header__editions__edition us bn-clickable">US</div>
</div>
</div>
</div>
</header>
<div class="dropdowns">
<nav class="dropdown hamburger js-hamburger-menu bn-masthead dropdown--closed sticky" data-max-height="550" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;masthead&quot;}}" data-beacon-parsed="true">
<div class="hamburger__container js-hamburger-container dropdown__container bn-hamburger-menu" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;hamburger_menu&quot;}}" data-beacon-parsed="true">
<div class="hamburger__wrapper">
<div class="full-block hamburger__block">
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;NEWS&quot;,&quot;plid&quot;:&quot;/&quot;}}" data-beacon-parsed="true">
NEWS
</a>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://highline.huffingtonpost.com/" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Highline&quot;,&quot;plid&quot;:&quot;http://highline.huffingtonpost.com/&quot;}}" data-beacon-parsed="true">
Highline
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/science" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Science&quot;,&quot;plid&quot;:&quot;/section/science&quot;}}" data-beacon-parsed="true">
Science
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/education" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Education&quot;,&quot;plid&quot;:&quot;/section/education&quot;}}" data-beacon-parsed="true">
Education
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/weird-news" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Weird News&quot;,&quot;plid&quot;:&quot;/section/weird-news&quot;}}" data-beacon-parsed="true">
Weird News
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/business" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Business&quot;,&quot;plid&quot;:&quot;/section/business&quot;}}" data-beacon-parsed="true">
Business
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://testkitchen.huffingtonpost.com/" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;TestKitchen&quot;,&quot;plid&quot;:&quot;http://testkitchen.huffingtonpost.com/&quot;}}" data-beacon-parsed="true">
TestKitchen
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/technology" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Tech&quot;,&quot;plid&quot;:&quot;/section/technology&quot;}}" data-beacon-parsed="true">
Tech
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/college" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;College&quot;,&quot;plid&quot;:&quot;/section/college&quot;}}" data-beacon-parsed="true">
College
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/media" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Media&quot;,&quot;plid&quot;:&quot;/section/media&quot;}}" data-beacon-parsed="true">
Media
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="full-block hamburger__block">
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/section/politics" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;POLITICS&quot;,&quot;plid&quot;:&quot;/section/politics&quot;}}" data-beacon-parsed="true">
POLITICS
</a>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://elections.huffingtonpost.com/pollster" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Pollster&quot;,&quot;plid&quot;:&quot;http://elections.huffingtonpost.com/pollster&quot;}}" data-beacon-parsed="true">
Pollster
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/heroin-epidemic" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Heroin Epidemic&quot;,&quot;plid&quot;:&quot;/topic/heroin-epidemic&quot;}}" data-beacon-parsed="true">
Heroin Epidemic
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/donald-trump" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Donald Trump&quot;,&quot;plid&quot;:&quot;/topic/donald-trump&quot;}}" data-beacon-parsed="true">
Donald Trump
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/racial-inequality" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Racial Inequality&quot;,&quot;plid&quot;:&quot;/topic/racial-inequality&quot;}}" data-beacon-parsed="true">
Racial Inequality
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/us-senate" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;US Senate&quot;,&quot;plid&quot;:&quot;/topic/us-senate&quot;}}" data-beacon-parsed="true">
US Senate
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://elections.huffingtonpost.com/2016/results/president" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Election Results&quot;,&quot;plid&quot;:&quot;http://elections.huffingtonpost.com/2016/results/president&quot;}}" data-beacon-parsed="true">
Election Results
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/huffpost-hill" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;HuffPost Hill&quot;,&quot;plid&quot;:&quot;/topic/huffpost-hill&quot;}}" data-beacon-parsed="true">
HuffPost Hill
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/police-brutality" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Police Brutality&quot;,&quot;plid&quot;:&quot;/topic/police-brutality&quot;}}" data-beacon-parsed="true">
Police Brutality
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/hate-crimes" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Hate Crimes&quot;,&quot;plid&quot;:&quot;/topic/hate-crimes&quot;}}" data-beacon-parsed="true">
Hate Crimes
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/supreme-court" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Supreme Court&quot;,&quot;plid&quot;:&quot;/topic/supreme-court&quot;}}" data-beacon-parsed="true">
Supreme Court
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/congress" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Congress&quot;,&quot;plid&quot;:&quot;/topic/congress&quot;}}" data-beacon-parsed="true">
Congress
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/so-that-happened" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;So That Happened&quot;,&quot;plid&quot;:&quot;/topic/so-that-happened&quot;}}" data-beacon-parsed="true">
So That Happened
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="full-block hamburger__block">
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/dept/entertainment" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;ENTERTAINMENT&quot;,&quot;plid&quot;:&quot;/dept/entertainment&quot;}}" data-beacon-parsed="true">
ENTERTAINMENT
</a>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/entertainment" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Entertainment&quot;,&quot;plid&quot;:&quot;/section/entertainment&quot;}}" data-beacon-parsed="true">
Entertainment
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/comedy" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Comedy&quot;,&quot;plid&quot;:&quot;/section/comedy&quot;}}" data-beacon-parsed="true">
Comedy
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/celebrity" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Celebrity&quot;,&quot;plid&quot;:&quot;/section/celebrity&quot;}}" data-beacon-parsed="true">
Celebrity
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/tv" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;TV&quot;,&quot;plid&quot;:&quot;/section/tv&quot;}}" data-beacon-parsed="true">
TV
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/arts" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Arts + Culture&quot;,&quot;plid&quot;:&quot;/section/arts&quot;}}" data-beacon-parsed="true">
Arts + Culture
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/backspace" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Backspace&quot;,&quot;plid&quot;:&quot;/topic/backspace&quot;}}" data-beacon-parsed="true">
Backspace
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/movies" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Movies&quot;,&quot;plid&quot;:&quot;/topic/movies&quot;}}" data-beacon-parsed="true">
Movies
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="full-block hamburger__block">
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/dept/lifestyle" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;LIFESTYLE&quot;,&quot;plid&quot;:&quot;/dept/lifestyle&quot;}}" data-beacon-parsed="true">
LIFESTYLE
</a>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/healthy-living" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Healthy Living&quot;,&quot;plid&quot;:&quot;/section/healthy-living&quot;}}" data-beacon-parsed="true">
Healthy Living
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/travel" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Travel&quot;,&quot;plid&quot;:&quot;/section/travel&quot;}}" data-beacon-parsed="true">
Travel
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/style" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Style&quot;,&quot;plid&quot;:&quot;/section/style&quot;}}" data-beacon-parsed="true">
Style
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/taste" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Taste&quot;,&quot;plid&quot;:&quot;/section/taste&quot;}}" data-beacon-parsed="true">
Taste
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/huffpost-home" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Home&quot;,&quot;plid&quot;:&quot;/section/huffpost-home&quot;}}" data-beacon-parsed="true">
Home
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/relationships" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Relationships&quot;,&quot;plid&quot;:&quot;/topic/relationships&quot;}}" data-beacon-parsed="true">
Relationships
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://horoscopes.huffingtonpost.com/" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Horoscopes&quot;,&quot;plid&quot;:&quot;http://horoscopes.huffingtonpost.com/&quot;}}" data-beacon-parsed="true">
Horoscopes
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="full-block hamburger__block">
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/dept/voices" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;VOICES&quot;,&quot;plid&quot;:&quot;/dept/voices&quot;}}" data-beacon-parsed="true">
VOICES
</a>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/black-voices" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Black Voices&quot;,&quot;plid&quot;:&quot;/section/black-voices&quot;}}" data-beacon-parsed="true">
Black Voices
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/latino-voices" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Latino Voices&quot;,&quot;plid&quot;:&quot;/section/latino-voices&quot;}}" data-beacon-parsed="true">
Latino Voices
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/women" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Women&quot;,&quot;plid&quot;:&quot;/section/women&quot;}}" data-beacon-parsed="true">
Women
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/fifty" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Fifty&quot;,&quot;plid&quot;:&quot;/section/fifty&quot;}}" data-beacon-parsed="true">
Fifty
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/queer-voices" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Queer Voices&quot;,&quot;plid&quot;:&quot;/section/queer-voices&quot;}}" data-beacon-parsed="true">
Queer Voices
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/parents" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Parents&quot;,&quot;plid&quot;:&quot;/section/parents&quot;}}" data-beacon-parsed="true">
Parents
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="hamburger__block partial-block">
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/dept/impact" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;IMPACT&quot;,&quot;plid&quot;:&quot;/dept/impact&quot;}}" data-beacon-parsed="true">
IMPACT
</a>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/reclaim" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Reclaim&quot;,&quot;plid&quot;:&quot;/topic/reclaim&quot;}}" data-beacon-parsed="true">
Reclaim
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/project-zero" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Project Zero&quot;,&quot;plid&quot;:&quot;/topic/project-zero&quot;}}" data-beacon-parsed="true">
Project Zero
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/good-news" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Good News&quot;,&quot;plid&quot;:&quot;/topic/good-news&quot;}}" data-beacon-parsed="true">
Good News
</a>
</li>
</ul>
</div>
</div>
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="/section/video" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;VIDEO&quot;,&quot;plid&quot;:&quot;/section/video&quot;}}" data-beacon-parsed="true">
VIDEO
</a>
</div>
<div class="hamburger__section">
<a class="hamburger__section-title bn-hamburger-link bn-clickable" href="http://www.huffingtonpost.com/newsletters" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;NEWSLETTERS&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/newsletters&quot;}}" data-beacon-parsed="true">
NEWSLETTERS
</a>
</div>
</div>
<div class="full-block hamburger__block">
<div class="hamburger__section">
<h3 class="hamburger__section-title">
FEATURED
</h3>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/own" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;OWN&quot;,&quot;plid&quot;:&quot;/section/own&quot;}}" data-beacon-parsed="true">
OWN
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/paving-the-way" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Paving the Way&quot;,&quot;plid&quot;:&quot;/topic/paving-the-way&quot;}}" data-beacon-parsed="true">
Paving the Way
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/power-of-humanity" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;The Power Of Humanity&quot;,&quot;plid&quot;:&quot;/topic/power-of-humanity&quot;}}" data-beacon-parsed="true">
The Power Of Humanity
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/retire-well" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Retire Well&quot;,&quot;plid&quot;:&quot;/topic/retire-well&quot;}}" data-beacon-parsed="true">
Retire Well
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/sleep-wellness" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Sleep + Wellness&quot;,&quot;plid&quot;:&quot;/topic/sleep-wellness&quot;}}" data-beacon-parsed="true">
Sleep + Wellness
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/purpose-profit" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;What's Working: Purpose + Profit&quot;,&quot;plid&quot;:&quot;/topic/purpose-profit&quot;}}" data-beacon-parsed="true">
What's Working: Purpose + Profit
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/theworldpost" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;WorldPost&quot;,&quot;plid&quot;:&quot;/section/theworldpost&quot;}}" data-beacon-parsed="true">
WorldPost
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="full-block hamburger__block">
<div class="hamburger__section">
<h3 class="hamburger__section-title">
MORE
</h3>
<div class="hamburger__section-content">
<ul class="hamburger__list">
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/books" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Books&quot;,&quot;plid&quot;:&quot;/section/books&quot;}}" data-beacon-parsed="true">
Books
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://www.huffingtonpost.com/news/candidate-confessional-podcast/" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Candidate Confessional&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/news/candidate-confessional-podcast/&quot;}}" data-beacon-parsed="true">
Candidate Confessional
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/college" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;College&quot;,&quot;plid&quot;:&quot;/section/college&quot;}}" data-beacon-parsed="true">
College
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/crime" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Crime&quot;,&quot;plid&quot;:&quot;/section/crime&quot;}}" data-beacon-parsed="true">
Crime
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/divorce" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Divorce&quot;,&quot;plid&quot;:&quot;/section/divorce&quot;}}" data-beacon-parsed="true">
Divorce
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="http://www.huffingtonpost.com/news/eat-the-press/" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Eat the Press&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/news/eat-the-press/&quot;}}" data-beacon-parsed="true">
Eat the Press
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/education" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Education&quot;,&quot;plid&quot;:&quot;/section/education&quot;}}" data-beacon-parsed="true">
Education
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/green" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Green&quot;,&quot;plid&quot;:&quot;/section/green&quot;}}" data-beacon-parsed="true">
Green
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/outspeak" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Outspeak&quot;,&quot;plid&quot;:&quot;/topic/outspeak&quot;}}" data-beacon-parsed="true">
Outspeak
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/religion" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Religion&quot;,&quot;plid&quot;:&quot;/section/religion&quot;}}" data-beacon-parsed="true">
Religion
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/topic/small-business" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Small Business&quot;,&quot;plid&quot;:&quot;/topic/small-business&quot;}}" data-beacon-parsed="true">
Small Business
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/sports" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Sports&quot;,&quot;plid&quot;:&quot;/section/sports&quot;}}" data-beacon-parsed="true">
Sports
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/teen" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Teen&quot;,&quot;plid&quot;:&quot;/section/teen&quot;}}" data-beacon-parsed="true">
Teen
</a>
</li>
<li class="hamburger__list-item">
<a class="hamburger__link bn-hamburger-link bn-clickable" href="/section/weddings" target="_top" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Weddings&quot;,&quot;plid&quot;:&quot;/section/weddings&quot;}}" data-beacon-parsed="true">
Weddings
</a>
</li>
</ul>
</div>
</div>
</div>
<div class="blank hamburger__block">
</div>

</div>
</div>
</nav>

<div class="dropdown editions js-dropdown-editions bn-masthead dropdown--closed sticky" data-max-height="400" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;masthead&quot;}}" data-beacon-parsed="true">
<div class="dropdown__editions-container bn-editions dropdown__container" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;editions&quot;}}" data-beacon-parsed="true">
<div class="dropdown__editions-inner-container js-editions-container">
<ul class="dropdown__editions-column">
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="ar" href="http://www.huffpostarabi.com/?country=AR" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;ar&quot;,&quot;mpid&quot;:1}}" data-beacon-parsed="true">Arabi</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="au" href="http://www.huffingtonpost.com.au/?country=AU" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;au&quot;,&quot;mpid&quot;:2}}" data-beacon-parsed="true">Australia</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="br" href="http://www.huffpostbrasil.com/?country=BR" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;br&quot;,&quot;mpid&quot;:3}}" data-beacon-parsed="true">Brasil</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="ca" href="http://www.huffingtonpost.ca/?country=CA" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;ca&quot;,&quot;mpid&quot;:4}}" data-beacon-parsed="true">Canada</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="de" href="http://www.huffingtonpost.de/?country=DE" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;de&quot;,&quot;mpid&quot;:5}}" data-beacon-parsed="true">Deutschland</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="es" href="http://www.huffingtonpost.es/?country=ES" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;es&quot;,&quot;mpid&quot;:6}}" data-beacon-parsed="true">España</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="fr" href="http://www.huffingtonpost.fr/?country=FR" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;fr&quot;,&quot;mpid&quot;:7}}" data-beacon-parsed="true">France</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="gr" href="http://www.huffingtonpost.gr/?country=GR" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;gr&quot;,&quot;mpid&quot;:8}}" data-beacon-parsed="true">Greece</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="in" href="http://www.huffingtonpost.in/?country=IN" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;in&quot;,&quot;mpid&quot;:9}}" data-beacon-parsed="true">India</a>
</li>
</ul>
<ul class="dropdown__editions-column">
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="it" href="http://www.huffingtonpost.it/?country=IT" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;it&quot;,&quot;mpid&quot;:10}}" data-beacon-parsed="true">Italia</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="jp" href="http://www.huffingtonpost.jp/?country=JP" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;jp&quot;,&quot;mpid&quot;:11}}" data-beacon-parsed="true">Japan</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="kr" href="http://www.huffingtonpost.kr/?country=KR" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;kr&quot;,&quot;mpid&quot;:12}}" data-beacon-parsed="true">Korea</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="mg" href="http://www.huffpostmaghreb.com/?country=MG" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;mg&quot;,&quot;mpid&quot;:13}}" data-beacon-parsed="true">Maghreb</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="mx" href="http://www.huffingtonpost.com.mx/?country=MX" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;mx&quot;,&quot;mpid&quot;:14}}" data-beacon-parsed="true">México</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="qc" href="http://quebec.huffingtonpost.ca/?country=QC" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;qc&quot;,&quot;mpid&quot;:15}}" data-beacon-parsed="true">Québec</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="za" href="http://www.huffingtonpost.co.za/?country=ZA" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;za&quot;,&quot;mpid&quot;:16}}" data-beacon-parsed="true">South Africa</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="uk" href="http://www.huffingtonpost.co.uk/?country=UK" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;uk&quot;,&quot;mpid&quot;:17}}" data-beacon-parsed="true">United Kingdom</a>
</li>
<li class="dropdown__editions-list-item">
<a class="dropdown__editions-item js-edition-item bn-edition-item bn-clickable" data-edition="us" href="http://www.huffingtonpost.com/?country=US" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;us&quot;,&quot;mpid&quot;:18}}" data-beacon-parsed="true">United States</a>
</li>
</ul>

</div>
</div>
</div>

</div>
<div class="overlays">
<div class="snapchat-overlay bn-overlay hidden" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;overlay&quot;}}" data-beacon-parsed="true">
<div class="snapchat-overlay__container bn-snapchat" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;snapchat&quot;}}" data-beacon-parsed="true">
<div class="snapchat-overlay__container__close bn-close bn-clickable" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;close&quot;}}" data-beacon-parsed="true">
<div class="snapchat-overlay__container__close__icon bn-clickable"></div>
<span class="bn-clickable">Close</span>
</div>
<div class="snapchat-overlay__container__qr bn-snapchat-qr bn-clickable" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;qr&quot;}}" data-beacon-parsed="true">
<img src="http://s.m.huffpost.com/assets/snapchat-QR-a98b071227601c8bd787d668be38d7220002e41178fe4db1577c894b942534b3.png" class="bn-clickable">
</div>
</div>
</div>

</div>


<div class="advertisement leaderboard_flex treated sticky-nav-on" style="top: -163px;">
<style type="text/css"> #topnav_margin_btm { margin: 0 !important; } </style>
<div id="ad_leaderboard_flex" class="ad_wrapper_"><div class="adtech-adspot ad-leaderboard_flex" id="adsDiv0"><iframe itemscope="" itemtype="https://schema.org/WPAdBlock" width="728" height="90" title="Ad" marginwidth="0" marginheight="0" allowtransparency="true" frameborder="0" scrolling="no" id="atwAdFrame0" style="visibility: visible; display: inline;" banid="17977404|110882637" class="uac_728x90"></iframe><div id="atwAdFrame0EAN" style="width: 728px; height: 15px; top: 0px; left: 0px; margin: 0px auto; position: relative;"><a id="atwAdFrame0EANA" style="position:absolute;right:0px;z-index:4999;margin:0 auto;width:77px;height:15px" href="http://adinfo.aol.com/" target="_blank"><img src="//o.aolcdn.com/ads/adchoices.png" style="border:none;width:77px;height:15px;vertical-align:top" alt="AdChoices"></a></div></div>
<script>
(function() {
  var adsDiv = document.querySelector('#adtech_leaderboard_flex');
  adsDiv.removeAttribute('id');
  window.modulousQueue.add(function(){
    
      
        waitForGlobal('htmlAdWH', function() {
          adsDiv.reloadCount = 0;
          var recoveryId = '';
          var needsRecovery = recoveryId && recoveryId.length && kraken__adBlock && kraken__adBlock.active;
          var adId = needsRecovery ? recoveryId : '93318049';
          try { adId = io_c3sd.ads["leaderboard_flex"] || adId; } catch(e) {} 
          
          // Override adtech_call_type
          var delivery_type = "" || "";
          var adtech_call_type = "";
          switch (delivery_type) {
            case "by_request": adtech_call_type = "ajax"; break;
            case "text": adtech_call_type = "text"; break;
            case "iframe": adtech_call_type = "iframe"; break;
            case "iframe_proxy": adtech_call_type = "f"; break;
            case "collapsed": adtech_call_type = "fc"; break;
            default: adtech_call_type = "fc"; break;
          }

          
          
          
          var args = ['lb', 'lb'
          ];
          
          

          if(needsRecovery) {
            waitForGlobal('readyToRecover', function() {
              adsDiv = document.querySelector('.leaderboard_flex');
              //htmlAdWH(adId, args[0], args[1], "fileless", adsDiv);
            });
          } else {
            htmlAdWH(adId, args[0], args[1], adtech_call_type, adsDiv);
          }
          
          if (typeof debugadcode == 'function') { debugadcode('AdTech', {placement: 'leaderboard_flex', mnid: adId}, adsDiv); }
          
          jQuery(document).on('reloadAd', function(ev, elem) {
            if (elem && !elem.contains(adsDiv)) { return; }
            var refreshRate = 1;  
            adsDiv.reloadCount++;
            if (adsDiv.reloadCount % refreshRate === 0 && typeof adsReloadAd === 'function') { adsReloadAd(adsDiv); }
          });
          
          adSetInView('0');
        }); // wFG:htmlAdWH
      
    
  });
}());
</script></div>
<script type="text/javascript">if (typeof HPAds !== 'undefined') { 
HPAds.ad_onload('div[id^="ad_leaderboard_flex"] iframe','HPAds.ad_uac_onload_handler(\' iframe[class^="uac_"]\',this,\'bottom_ad_border_div\',10)');
}</script>
</div>


<div id="main" role="main">




<article class="entry js-entry component loaded entry--mt entry--standard" data-mobilepath="/us/entry/6805902" data-sharingimage="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" data-sharingtitle="Florida Deputy Shoots Unarmed Man While Serving Warrant" data-sharingtweetname="HuffPostCrime" data-sharingtweettext="Florida+Deputy+Shoots+Unarmed+Man+While+Serving+Warrant" data-sharingurl="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html" data-socialmessaging="{}" id="6805902" path="/us/entry/6805902" type="article">



<header class="entry__header bn-entry-header" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;entry_header&quot;}}" data-beacon-parsed="true">


<span class="entry-eyebrow">
<span class="bn-department-tag" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;department_tag&quot;}}" data-beacon-parsed="true">
<a class="entry-eyebrow__link bn-department-link bn-clickable" data-linkname="crime" href="/section/crime" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;crime&quot;,&quot;plid&quot;:&quot;/section/crime&quot;}}" data-beacon-parsed="true">CRIME
</a></span>
</span>

<div class="timestamp">
<span class="timestamp__date--published">03/05/2015 02:57 am ET</span>
</div>






<div class="headline js-headline">
<h1 class="headline__title">Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant</h1>
</div>




<div class="bn-byline byline" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;byline&quot;}}" data-beacon-parsed="true">
<div class="byline__authors">
<div class="author-card author-card--byline">
<a class="bn-author-image bn-clickable" href="/author/andy-campbell" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;author_image&quot;}}" data-beacon-parsed="true"><div class="author-card__headshot bn-clickable" style="background-image: url(http://s.huffpost.com/contributors/andy-campbell/headshot.jpg), url(http://s.huffpost.com/contributors/andy-campbell/headshot_color.jpg);;"></div>
</a><div class="author-card__details">
<span class="author-card__details-container"><a class="author-card__details__link bn-author-name bn-clickable" href="/author/andy-campbell" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;author_name&quot;}}" data-beacon-parsed="true"><span class="author-card__details__name bn-clickable">By Andy Campbell</span>
</a></span></div>
</div>

</div>
</div>

</header>
<div class="entry__content js-entry-content">
<div class="entry__body js-entry-body">


<div class="top-media js-top-media bn-top-media" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;top_media&quot;}}" data-beacon-parsed="true">
<div class="top-media--video">
<object id="flashObj" width="570" height="321" classid="clsid:D27CDB6E-AE6D-11cf-96B8-444553540000" codebase="http://download.macromedia.com/pub/shockwave/cabs/flash/swflash.cab#version=9,0,47,0"><param name="movie" value="http://c.brightcove.com/services/viewer/federated_f9?isVid=1"><param name="bgcolor" value="#FFFFFF"><param name="flashVars" value="videoId=4094131467001&linkBaseURL=http%3A%2F%2Fwww.wftv.com%2Fvideos%2Fnews%2Ffriend-of-man-shot-by-volusia-co-deputy-calls%2FvDJgcZ%2F&playerID=836827755001&playerKey=AQ~~,AAAAPmbRNRk~,eMJgSV_RKKcXKDRGL0_yBQnqcdQLaidg&domain=embed&dynamicStreaming=true"><param name="base" value="http://admin.brightcove.com"><param name="seamlesstabbing" value="false"><param name="allowFullScreen" value="true"><param name="swLiveConnect" value="true"><param name="allowScriptAccess" value="always"><embed src="http://c.brightcove.com/services/viewer/federated_f9?isVid=1" bgcolor="#FFFFFF" flashvars="videoId=4094131467001&linkBaseURL=http%3A%2F%2Fwww.wftv.com%2Fvideos%2Fnews%2Ffriend-of-man-shot-by-volusia-co-deputy-calls%2FvDJgcZ%2F&playerID=836827755001&playerKey=AQ~~,AAAAPmbRNRk~,eMJgSV_RKKcXKDRGL0_yBQnqcdQLaidg&domain=embed&dynamicStreaming=true" base="http://admin.brightcove.com" name="flashObj" width="570" height="321" seamlesstabbing="false" type="application/x-shockwave-flash" allowfullscreen="true" swliveconnect="true" allowscriptaccess="always" pluginspage="http://www.macromedia.com/shockwave/download/index.cgi?P1_Prod_Version=ShockwaveFlash"></object>
</div>
<script></script>
<iframe style="display: none;"></iframe>

</div>

<div class="bn-left-rail js-sharebar share-bar share-bar--sticky fixed" data-mobilepath="/us/entry/6805902" data-sharingimage="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" data-sharingtitle="Florida Deputy Shoots Unarmed Man While Serving Warrant" data-sharingtweetname="HuffPostCrime" data-sharingtweettext="Florida+Deputy+Shoots+Unarmed+Man+While+Serving+Warrant" data-sharingurl="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;left_rail&quot;}}" data-beacon-parsed="true" style="top: 120px; left: 20px;">
<ul class="share-bar__list bn-share" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;share&quot;}}" data-beacon-parsed="true">
<div class="share-bar__social-counts">
<span class="share-bar__social-counts__number">1.4k</span>
</div>
<li class="bn-share-item facebook js-sharebar-facebook share-bar__item bn-clickable" data-share-name="facebook" data-type="facebook" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;facebook&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="facebook share-bar__item-link bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item js-sharebar-twitter share-bar__item twitter bn-clickable" data-share-name="twitter" data-type="twitter" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="share-bar__item-link twitter bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item js-sharebar-pinterest pinterest share-bar__item bn-clickable" data-share-name="pinterest" data-type="pinterest" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;pinterest&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="pinterest share-bar__item-link bn-clickable" data-pin-custom="true" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item email js-sharebar-email share-bar__item bn-clickable" data-share-name="email" data-type="email" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;email&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="email share-bar__item-link bn-clickable" href="#" target="" data-beacon-click-disabled="true"></a>
</li>
<div class="bn-share-item js-sharebar-comments share-bar__comments with-count bn-clickable" data-share-name="comments_open" data-type="comments" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;comments_open&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<div class="share-bar__comments__icon with-count bn-clickable" data-beacon-click-disabled="true"></div>
<div class="share-bar__comments__count with-count bn-clickable" data-beacon-click-disabled="true">390</div>
</div>
</ul>
</div>


<div class="share-bar-highlight js-highlight hidden bn-highlight" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;highlight&quot;}}" data-beacon-parsed="true">
<div class="bn-share" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;share&quot;}}" data-beacon-parsed="true">
<a class="share-bar-highlight__twitter js-highlight-share-button bn-share-item bn-clickable" data-share-name="twitter" data-type="twitter" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true"></a>
<a class="share-bar-highlight__facebook js-highlight-share-button bn-share-item bn-clickable" data-share-name="facebook" data-type="facebook" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;facebook&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true"></a>
</div>
</div>

<div class="entry__text js-entry-text bn-entry-text" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;entry_text&quot;}}" data-beacon-parsed="true">
<p>A Volusia County Sheriff’s deputy shot an unarmed man in the face while serving a drug warrant on Wednesday.</p>
<p>Deputies say Derek Cruice, 26, resisted arrest during the incident at his Deltona, Florida home. Cruice’s friends, six of whom were at the house when a SWAT team entered, called the fatal shooting “murder.”</p>
<p>At 6:32 a.m., Deputy Todd Raible, 36, <a href="http://www.wftv.com/news/news/local/officials-man-shot-face-while-deputies-attempt-ser/nkNWP/" rel="nofollow" target="_hplink" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;fired one shot&quot;,&quot;mpid&quot;:1,&quot;plid&quot;:&quot;http://www.wftv.com/news/news/local/officials-man-shot-face-while-deputies-attempt-ser/nkNWP/&quot;}}" data-beacon-parsed="true" class="bn-clickable">fired one shot</a>, hitting Cruice in the face as he stood inside the doorway, police told WFTV. </p>
<p>Cruice was pronounced dead a short time later at an area hospital.</p>
<p>“They were met with resistance and a shooting incurred,” Volusia County Sheriff Ben Johnson told My News 13.</p>
<p>The sheriff’s department <a href="http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2015/3/4/man_shot_by_volusia_.html" rel="nofollow" target="_hplink" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;confirmed that Cruice was unarmed&quot;,&quot;mpid&quot;:2,&quot;plid&quot;:&quot;http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2015/3/4/man_shot_by_volusia_.html&quot;}}" data-beacon-parsed="true" class="bn-clickable">confirmed that Cruice was unarmed</a> at the time.</p>
<blockquote class="twitter-tweet twitter-tweet-error" lang="en" data-twitter-extracted-i1498785551377818427="true"></blockquote>
<p>Deputy involved in shooting while serving warrant in Deltona. Suspect shot once in face after advancing on deputy. No injury to deputies.</p>
<p>— Volusia Co. Sheriff (@VolusiaSheriff) <a href="https://twitter.com/VolusiaSheriff/status/573110522876452864" rel="nofollow" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;March 4, 2015&quot;,&quot;mpid&quot;:3,&quot;plid&quot;:&quot;https://twitter.com/VolusiaSheriff/status/573110522876452864&quot;}}" data-beacon-parsed="true" class="bn-clickable">March 4, 2015</a></p>

<script async="" src="//platform.twitter.com/widgets.js" charset="utf-8">try{}catch(e){}</script>
<p>One of the men inside the home, 24-year-old Matthew Grady, contradicted deputies’ claims that Cruice had resisted arrest.</p>
<p>“There’s a couple of seconds between opening the door, walking out, getting to my knee and halfway out there’s gunfire,” Grady told My News 13. “I look back as the guy’s grabbing me, and my friend is dead or dying.”</p>
<p>When asked if Cruice resisted, Grady said: “No.”</p>
<p>Roommate Steven Cochran said Cruice wasn’t wearing a shirt at the time of the shooting, and didn’t pose a threat of carrying a concealed weapon.</p>
<p>“It’s kind of hard to conceal anything or hide anything when this is all you have on,” said Cochran. “They entered the house and fired.”</p>
<p>Raible, a SWAT team member and narcotics investigator with the department for 10 years, reportedly has a clean record. He was placed on administrative leave, and the Florida Department of Law Enforcement has taken over the investigation. Raible wasn’t injured in the incident, <a href="https://twitter.com/VolusiaSheriff/status/573110522876452864" rel="nofollow" target="_hplink" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;according to the sheriff’s department&quot;,&quot;mpid&quot;:4,&quot;plid&quot;:&quot;https://twitter.com/VolusiaSheriff/status/573110522876452864&quot;}}" data-beacon-parsed="true" class="bn-clickable">according to the sheriff’s department</a>.</p>
<p>The sheriff’s office allegedly recovered 217 grams of marijuana at the home, along with a scale, pipes, plastic bags and about $3,000 in cash.</p>
<p>Cruice had previously been arrested on misdemeanor drug charges in the past, but was never convicted, WFTV reported.</p>

<script class="entry-paragraph-1-ad-script-template" type="text/template">

</script>
<script class="sharebox-ad-script-template" type="text/template">

</script>
<script class="in-line-ad-unit-script-template" type="text/template">

</script>
<script class="mobileweb_transporter_mid_script-template" type="text/template">

</script>
<script class="mobileweb_transporter_mid_2_script-template" type="text/template">

</script>
<script class="entry-paragraph-3-ad-script-template" type="text/template">

</script>

</div>
<script></script>
<iframe style="display: none;"></iframe>


<script class="article-bottom-ad-script-template" type="text/template">

</script>



<section class="extra-content">
<h4 class="extra-content__title">RELATED ON HUFFPOST:</h4>
<div class="extra-content__media">

<div class="dynamic-preview-container slideshow-container" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;slideshow-dynamic-preview&quot;,&quot;mlid&quot;:&quot;extra-content&quot;}}" data-beacon-parsed="true">
<div class="bn-slideshow-open dynamic-preview slideshow bn-clickable" data-data-url="/mapi/v2/us/entry/6805902/slideshow/239858.json?device=desktop" data-slides-srcs="http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_3030621_free.jpg http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_1888070_free.jpg http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_3030622_free.jpg" data-slideshow-id="239858" id="slideshow-239858" style="background-image: url(&quot;http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_1888070_free.jpg&quot;);" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;open&quot;,&quot;dL_sDpt&quot;:&quot;gallery&quot;}}" data-beacon-parsed="true">
<div class="slideshow__preview bn-clickable">
<div class="slideshow__preview__icon bn-clickable"></div>
<div class="slideshow__preview__text bn-clickable">
<div class="slideshow__preview__text__title bn-clickable">Shootings In America</div>
</div>
</div>
</div>
<div class="bn-slideshow-open hidden slideshow static-slideshow bn-clickable" data-data-url="/mapi/v2/us/entry/6805902/slideshow/239858.json?device=desktop" data-slideshow-id="239858" id="slideshow-239858" style="background-image:url(http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_3030621_free.jpg);" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;open&quot;,&quot;dL_sDpt&quot;:&quot;gallery&quot;}}" data-beacon-parsed="true">
<div class="slideshow__preview bn-clickable">
<div class="slideshow__preview__icon bn-clickable"></div>
<div class="slideshow__preview__text bn-clickable">
<div class="slideshow__preview__text__title bn-clickable">Shootings In America</div>
</div>
</div>
</div>
</div>
<div class="slideshow-overlay hidden" id="overlay-239858">
<div class="slideshow-overlay__header bn-slideshow-header" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;slideshow-header&quot;}}" data-beacon-parsed="true">
<div class="slideshow-overlay__header__hp">
<img alt="The Huffington Post" class="slideshow-overlay__header__hp-logo" src="http://s.m.huffpost.com/assets/H-icon-72fd5782b36a1a42ab82a4af65cbc6dd6ddc4d4a71190a91c5cfaab7c66fd836.svg">
</div>
<h2 class="slideshow-overlay__header__title">Shootings In America</h2>
<div class="slideshow-overlay__header__share bn-share" data-sharingimage="http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_3030621_free.jpg" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;share&quot;}}" data-beacon-parsed="true">
<div class="js-sharebar share-bar share-bar--slideshow-top">
<ul class="share-bar__list bn-share" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;share&quot;}}" data-beacon-parsed="true">
<li class="bn-share-item facebook js-sharebar-facebook share-bar__item bn-clickable" data-share-name="facebook" data-type="facebook" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;facebook&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="facebook share-bar__item-link bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item js-sharebar-twitter share-bar__item twitter bn-clickable" data-share-name="twitter" data-type="twitter" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="share-bar__item-link twitter bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item email js-sharebar-email share-bar__item bn-clickable" data-share-name="email" data-type="email" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;email&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="email share-bar__item-link bn-clickable" href="#" target="" data-beacon-click-disabled="true"></a>
</li>
</ul>
</div>

</div>
<div class="slideshow-overlay__header__close bn-close bn-clickable" data-slideshow-id="239858" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;close&quot;}}" data-beacon-parsed="true"></div>
</div>
<div class="slideshow-overlay__container__ad-con">
<div class="slideshow-overlay__container__ad-con__top-ad"></div>
</div>
<div class="slideshow-overlay__container bn-slideshow-body" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;slideshow-body&quot;}}" data-beacon-parsed="true">
<div class="slideshow-overlay__container__left">
<div class="slideshow-overlay__container__left__nav bn-slideshow-nav" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;nav&quot;}}" data-beacon-parsed="true">
<div class="slideshow-overlay__container__left__nav__prev bn-slideshow-prev bn-clickable" data-slideshow-id="239858" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;prev&quot;,&quot;dL_sDpt&quot;:&quot;gallery&quot;}}" data-beacon-parsed="true"></div>
<div class="slideshow-overlay__container__left__nav__current">1</div>
<div class="slideshow-overlay__container__left__nav__of">
<div class="slideshow-overlay__container__left__nav__of__line"></div>
<div class="slideshow-overlay__container__left__nav__of__word">of</div>
</div>
<div class="slideshow-overlay__container__left__nav__total">14</div>
<div class="slideshow-overlay__container__left__nav__next bn-slideshow-next bn-clickable" data-slideshow-id="239858" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;next&quot;,&quot;dL_sDpt&quot;:&quot;gallery&quot;}}" data-beacon-parsed="true"></div>
</div>
<div class="slideshow-overlay__container__left__info">
<div class="slideshow-overlay__container__left__info__slide-title"> </div>
<div class="slideshow-overlay__container__left__info__slide-caption">Swat team members secure the scene near Sparks Middle School in Sparks, Nev., after a shooting there on Monday, Oct. 21, 2013. Authorities are reporting that two people were killed and two wounded at the Nevada middle school. (AP Photo/Kevin Clifford)</div>
</div>
<div class="slideshow-overlay__container__left__side-share" data-sharingimage="http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_3030621_free.jpg">
<p class="share-text">Share this slide:</p>
<div class="js-sharebar share-bar share-bar--slideshow-top">
<ul class="share-bar__list bn-share" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;share&quot;}}" data-beacon-parsed="true">
<li class="bn-share-item facebook js-sharebar-facebook share-bar__item bn-clickable" data-share-name="facebook" data-type="facebook" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;facebook&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="facebook share-bar__item-link bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item js-sharebar-twitter share-bar__item twitter bn-clickable" data-share-name="twitter" data-type="twitter" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="share-bar__item-link twitter bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item email js-sharebar-email share-bar__item bn-clickable" data-share-name="email" data-type="email" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;email&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="email share-bar__item-link bn-clickable" href="#" target="" data-beacon-click-disabled="true"></a>
</li>
</ul>
</div>

</div>
<div class="slideshow-overlay__container__left__side-ad"></div>
</div>
<div class="slideshow-overlay__container__content js-slideshow-content">
<div class="slideshow-overlay__container__content__slide">
<img src="http://i.huffpost.com/gadgets/slideshows/239858/slide_239858_3030621_free.jpg" itemprop="image">

</div>
<div class="slideshow-overlay__container__content__credit">AP</div>
</div>
</div>
<div class="slideshow-overlay__data">
<div class="slideshow-overlay__data__preload"></div>
</div>
</div>


</div>
</section>
<script></script>
<iframe style="display: none;"></iframe>



<div class="entry_footer bn-entry-footer" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;entry_footer&quot;}}" data-beacon-parsed="true">

<div class="byline-footer">
<div class="entry-corrections js-entry-corrections bn-corrections" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;correction&quot;}}" data-beacon-parsed="true">
<a class="suggest-correction bn-correction-link bn-clickable" href="javascript:void(0)" id="suggest-correction-link" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;contact&quot;}}" data-beacon-parsed="true">Suggest a correction</a>
</div>

<div class="byline-footer__authors">
<div class="author-card--byline author-card--full bn-byline" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;byline&quot;}}" data-beacon-parsed="true">
<a href="/author/andy-campbell"><div class="author-card__headshot bn-author-image bn-clickable" style="background-image: url(http://s.huffpost.com/contributors/andy-campbell/headshot.jpg), url(http://s.huffpost.com/contributors/andy-campbell/headshot_color.jpg);;" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;author_image&quot;}}" data-beacon-parsed="true"></div>
</a><div class="author-card__details">
<span class="author-card__details-container"><a class="author-card__details__link bn-author-name bn-clickable" href="/author/andy-campbell" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;author_name&quot;}}" data-beacon-parsed="true"><span class="author-card__details__name bn-clickable">Andy Campbell</span>
</a><span class="author-card__social-container">
<a class="author-card__social author-card__social--twitter bn-author-twitter bn-clickable" href="http://twitter.com/AndyBCampbell" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter_follow&quot;}}" data-beacon-parsed="true"></a>
</span>
</span><span class="author-card__microbio">
Crime And Weird News Editor, The Huffington Post
</span>
</div>
</div>

</div>
</div>


<div class="tag-cloud js-tag-cloud bn-tag-cloud" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;topic_tag&quot;}}" data-beacon-parsed="true">
<h2 class="tag-cloud__header">MORE:</h2>
<a class="tag bn-tag bn-clickable" data-slug="Derek Cruice" href="/topic/Derek Cruice" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Derek Cruice&quot;,&quot;mpid&quot;:1,&quot;plid&quot;:&quot;/topic/Derek Cruice&quot;}}" data-beacon-parsed="true">
Derek Cruice
</a>
<a class="tag bn-tag bn-clickable" data-slug="Derek Cruice dead" href="/topic/Derek Cruice dead" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Derek Cruice dead&quot;,&quot;mpid&quot;:2,&quot;plid&quot;:&quot;/topic/Derek Cruice dead&quot;}}" data-beacon-parsed="true">
Derek Cruice Dead
</a>
<a class="tag bn-tag bn-clickable" data-slug="todd raible" href="/topic/todd raible" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;todd raible&quot;,&quot;mpid&quot;:3,&quot;plid&quot;:&quot;/topic/todd raible&quot;}}" data-beacon-parsed="true">
Todd Raible
</a>
<a class="tag bn-tag bn-clickable" data-slug="Volusia County Sheriff" href="/topic/Volusia County Sheriff" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Volusia County Sheriff&quot;,&quot;mpid&quot;:4,&quot;plid&quot;:&quot;/topic/Volusia County Sheriff&quot;}}" data-beacon-parsed="true">
Volusia County Sheriff
</a>
<a class="tag bn-tag bn-clickable" data-slug="todd raible volusia county sheriff" href="/topic/todd raible volusia county sheriff" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;todd raible volusia county sheriff&quot;,&quot;mpid&quot;:5,&quot;plid&quot;:&quot;/topic/todd raible volusia county sheriff&quot;}}" data-beacon-parsed="true">
Todd Raible Volusia County Sheriff
</a>
</div>

<div id="suggested-story"></div>


</div>
<div class="below-entry js-below-entry" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;below_entry&quot;}}">
<div class="below-entry__content">


<div class="advertisement below_entry_recirc treated">
<div id="taboola-below-article-thumbnails-5177491609862348" class="taboola hp-entry-you_may_like trc_related_container trc_spotlight_widget trc_elastic trc_elastic_trc_11200 " data-placement-name="Below Article Thumbnails"><div class="trc_rbox_container" style="display: block;"><div><div id="trc_wrapper_11200" class="trc_rbox thumbnails-o trc-content-sponsored " style="overflow: hidden; display: block;"><div id="trc_header_11200" class="trc_rbox_header trc_rbox_border_elm"><div class="trc_header_ext"><div class="logoDiv link-adc "><a class="trc_desktop_adc_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span class="trc_adc_wrapper"><span class="trc_adc_s_logo"></span>&nbsp;</span><span class="trc_logos_v_align">&nbsp;</span></a><a class="trc_mobile_adc_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span class="trc_adc_wrapper"><span class="trc_adc_s_logo"></span>&nbsp;</span><span class="trc_logos_v_align">&nbsp;</span></a></div><div class="logoDiv link-attribution "><a class="trc_desktop_attribution_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span>by Taboola</span><span class="trc_logos_v_align">&nbsp;</span></a><a class="trc_mobile_attribution_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span>by Taboola</span><span class="trc_logos_v_align">&nbsp;</span></a></div><div class="logoDiv link-disclosure  attribution-disclosure-link-sponsored"><a class="trc_desktop_disclosure_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span>Sponsored Links</span><span class="trc_logos_v_align">&nbsp;</span></a><a class="trc_mobile_disclosure_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span>Sponsored Links</span><span class="trc_logos_v_align">&nbsp;</span></a></div><div class="logoDiv link-disclosure  attribution-disclosure-link-hybrid"><a class="trc_desktop_disclosure_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span>Promoted Links</span><span class="trc_logos_v_align">&nbsp;</span></a><a class="trc_mobile_disclosure_link trc_attribution_position_top" rel="nofollow" href="http://popup.taboola.com/en/?template=colorbox&utm_source=aol-huffingtonpost&utm_medium=referral&utm_content=thumbnails-o:Below Article Thumbnails:" target="_blank"><span>Promoted Links</span><span class="trc_logos_v_align">&nbsp;</span></a></div></div><span class="trc_rbox_header_span">You May Like</span></div><div id="outer_11200" class="trc_rbox_outer"><div id="rbox-t2m" class="trc_rbox_div trc_rbox_border_elm"><div id="internal_trc_11200"><div data-item-id="~~V1~~-3855892779090245063~~gzb9aGhgIDA3l5GmWEus-8KkJbiEquim0vePrRQTYNWLWDEoSQwuhc4ZvLOUkuMCftZYEZdxVIARG00h6_pFjzHLaeWSs14Edab5ZbVOdayOGk-YHaGh0F8ZOTIP3n7goR9_aMALbfUvo7eNuAaR8g" data-item-title="25 Stars Who Are Literally Unrecognizable Without Makeup" data-item-thumb="https://console.brax-cdn.com/dff22586-d4d6-4d09-8593-579dfbcc5259.jpg" data-item-syndicated="true" class="videoCube trc_spotlight_item origin-default thumbnail_top syndicatedItem photoItem videoCube_1_child trc-first-recommendation trc-spotlight-first-recommendation  trc_excludable " data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:&quot;1&quot;,&quot;plid&quot;:&quot;http://www.stylebistro.com/Barefaced+Beauties+-+Celebrities+Without+Makeup?utm_source=tabo&utm_medium=cpc&utm_campaign=Tabo-SB-UK-Desktop-Specials-Barefaced-Beauties&utm_content=aol-huffingtonpost&quot;,&quot;mlid&quot;:&quot;taboola&quot;}}"><a title="25 Stars Who Are Literally Unrecognizable Without Makeup" href="http://www.stylebistro.com/Barefaced+Beauties+-+Celebrities+Without+Makeup?utm_source=tabo&utm_medium=cpc&utm_campaign=Tabo-SB-UK-Desktop-Specials-Barefaced-Beauties&utm_content=aol-huffingtonpost" rel="nofollow" target="_blank" class=" item-thumbnail-href "><div class="thumbBlock_holder"><span class="thumbBlock" style="background-image: url(&quot;http://images.taboola.com/taboola/image/fetch/f_jpg%2Cq_auto%2Ch_148%2Cw_280%2Cc_fill%2Cg_faces:auto%2Ce_sharpen/https%3A//console.brax-cdn.com/dff22586-d4d6-4d09-8593-579dfbcc5259.jpg&quot;);"><span class="thumbnail-overlay"></span><span class="branding">StyleBistro</span><span class="static-text top-right"></span></span><div class="videoCube_aspect"></div></div></a><a title="25 Stars Who Are Literally Unrecognizable Without Makeup" href="http://www.stylebistro.com/Barefaced+Beauties+-+Celebrities+Without+Makeup?utm_source=tabo&utm_medium=cpc&utm_campaign=Tabo-SB-UK-Desktop-Specials-Barefaced-Beauties&utm_content=aol-huffingtonpost" rel="nofollow" target="_blank" class=" item-label-href "><span class="video-label-box trc-main-label "><span class="video-label video-title trc_ellipsis " style="-webkit-line-clamp: 3;">25 Stars Who Are Literally Unrecognizable Without Makeup</span><span class="branding">StyleBistro</span></span></a><div class=" trc_user_exclude_btn " title="Remove this item"></div><div class=" trc_exclude_overlay trc_fade "></div><div class=" trc_undo_btn ">Undo</div></div><div data-item-id="~~V1~~4048768638394217214~~hkx9Ui5VaCAMGi1d0NRXE75JXqzlx-HayPNr0iUxi7MSO_fFkG-bpaW2-qyqBDD6hbvTJ9f5uoHZ9n0f0Rxsj9Z3uhcZH3NvLIMM0XLNNnpULF2ZqOy-MWoLYDSbniS78kPHpmKzIb6f9PDGye4U9Q" data-item-title="24 Rare Historical Photos That Will Leave You Speechless" data-item-thumb="http://b1.spassets.com/gallery/389/12735/c8cc859b01e6ac0096aee3eff8a9b26a.jpg" data-item-syndicated="true" class="videoCube trc_spotlight_item origin-default thumbnail_top syndicatedItem photoItem videoCube_2_child trc_excludable " data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:&quot;2&quot;,&quot;plid&quot;:&quot;http://sn.sptrkr.com/494b6106?utm_source=Taboola&utm_medium=aol-huffingtonpost&utm_campaign=SN_GB_D_Taboola_3058_615804&utm_content=44666069&quot;,&quot;mlid&quot;:&quot;taboola&quot;}}"><a title="24 Rare Historical Photos That Will Leave You Speechless" href="http://sn.sptrkr.com/494b6106?utm_source=Taboola&utm_medium=aol-huffingtonpost&utm_campaign=SN_GB_D_Taboola_3058_615804&utm_content=44666069" rel="nofollow" target="_blank" class=" item-thumbnail-href "><div class="thumbBlock_holder"><span class="thumbBlock" style="background-image: url(&quot;http://images.taboola.com/taboola/image/fetch/f_jpg%2Cq_auto%2Ch_148%2Cw_280%2Cc_fill%2Cg_faces:auto%2Ce_sharpen/http%3A//b1.spassets.com/gallery/389/12735/c8cc859b01e6ac0096aee3eff8a9b26a.jpg&quot;);"><span class="thumbnail-overlay"></span><span class="branding">StandardNews</span><span class="static-text top-right"></span></span><div class="videoCube_aspect"></div></div></a><a title="24 Rare Historical Photos That Will Leave You Speechless" href="http://sn.sptrkr.com/494b6106?utm_source=Taboola&utm_medium=aol-huffingtonpost&utm_campaign=SN_GB_D_Taboola_3058_615804&utm_content=44666069" rel="nofollow" target="_blank" class=" item-label-href "><span class="video-label-box trc-main-label "><span class="video-label video-title trc_ellipsis " style="-webkit-line-clamp: 3;">24 Rare Historical Photos That Will Leave You Speechless</span><span class="branding">StandardNews</span></span></a><div class=" trc_user_exclude_btn " title="Remove this item"></div><div class=" trc_exclude_overlay trc_fade "></div><div class=" trc_undo_btn ">Undo</div></div><div data-item-id="~~V1~~3272264891947756139~~xpg5r0G6o29pJAbh9aA-afO9gIl_ly0gxu0Ud1XMNYASO_fFkG-bpaW2-qyqBDD6boj4nsjC6tw9ul26EUANANZ3uhcZH3NvLIMM0XLNNnofk0yu0wfvbKEwkxpNYM1AYFTpzOxp7jLKs_zraWu8Hg" data-item-title="Most Can&#39;t Identify These 50 Movies from One Screenshot. Can You?" data-item-thumb="http://gleaned-images.stuff.com/signals-test/fe9d73fef7dfda1b0bc45199138dff15622c2c892c9b7eec9cabf598" data-item-syndicated="true" class="videoCube trc_spotlight_item origin-default thumbnail_top syndicatedItem textItem videoCube_3_child trc_excludable " data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:&quot;3&quot;,&quot;plid&quot;:&quot;http://quiz.howstuffworks.com/entertainment/89-of-people-cant-identify-all-50-80s-movies-from-just-one-screenshot-can-you.htm?mkcpgn=i600011302&utm_source=taboola&utm_medium=paid&utm_campaign=UK-80sMovies-Screenshots%28desktop%29&utm_term=aol-huffingtonpost&utm_content=Most+Can%27t+Identify+These+50+Movies+from+One+Screenshot.+Can+You%3F&quot;,&quot;mlid&quot;:&quot;taboola&quot;}}"><a title="Most Can't Identify These 50 Movies from One Screenshot. Can You?" href="http://quiz.howstuffworks.com/entertainment/89-of-people-cant-identify-all-50-80s-movies-from-just-one-screenshot-can-you.htm?mkcpgn=i600011302&utm_source=taboola&utm_medium=paid&utm_campaign=UK-80sMovies-Screenshots%28desktop%29&utm_term=aol-huffingtonpost&utm_content=Most+Can%27t+Identify+These+50+Movies+from+One+Screenshot.+Can+You%3F" rel="nofollow" target="_blank" class=" item-thumbnail-href "><div class="thumbBlock_holder"><span class="thumbBlock" style="background-image: url(&quot;http://images.taboola.com/taboola/image/fetch/f_jpg%2Cq_auto%2Ch_148%2Cw_280%2Cc_fill%2Cg_faces:auto%2Ce_sharpen/http%3A//gleaned-images.stuff.com/signals-test/fe9d73fef7dfda1b0bc45199138dff15622c2c892c9b7eec9cabf598&quot;);"><span class="thumbnail-overlay"></span><span class="branding">HowStuffWorks</span><span class="static-text top-right"></span></span><div class="videoCube_aspect"></div></div></a><a title="Most Can't Identify These 50 Movies from One Screenshot. Can You?" href="http://quiz.howstuffworks.com/entertainment/89-of-people-cant-identify-all-50-80s-movies-from-just-one-screenshot-can-you.htm?mkcpgn=i600011302&utm_source=taboola&utm_medium=paid&utm_campaign=UK-80sMovies-Screenshots%28desktop%29&utm_term=aol-huffingtonpost&utm_content=Most+Can%27t+Identify+These+50+Movies+from+One+Screenshot.+Can+You%3F" rel="nofollow" target="_blank" class=" item-label-href "><span class="video-label-box trc-main-label "><span class="video-label video-title trc_ellipsis " style="-webkit-line-clamp: 3;">Most Can't Identify These 50 Movies from One Screenshot. Can You?</span><span class="branding">HowStuffWorks</span></span></a><div class=" trc_user_exclude_btn " title="Remove this item"></div><div class=" trc_exclude_overlay trc_fade "></div><div class=" trc_undo_btn ">Undo</div></div><div data-item-id="~~V1~~-7527002819135068892~~BVr-GE0AjYrTn-s26BYuVbQ1Nv3LsHyVDTVYtlw2MeYSO_fFkG-bpaW2-qyqBDD6zU5Xh5Yvqkieis1PbDO3DtZ3uhcZH3NvLIMM0XLNNnof2GMNlgL-5SUL08r2w9648kPHpmKzIb6f9PDGye4U9Q" data-item-title="You&#39;ll be speaking a new language in 3 weeks thanks to this app made in Germany" data-item-thumb="http://cdn.taboolasyndication.com/libtrc/static/thumbnails/fe90c2c06d6ca733c8ebbf688df67c0e.png" data-item-syndicated="true" class="videoCube trc_spotlight_item origin-default thumbnail_top syndicatedItem videoCube_4_child trc_excludable " data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:&quot;4&quot;,&quot;plid&quot;:&quot;https://go.babbel.com/engmag-a189-appsanywhere-cd-gbr-tb/1_gbr_tab_cd?utm_source=Taboola&utm_medium=CON&utm_campaign=CD_GBRALL_gEN_cUK_SpanishChallenge&utm_term=aol-huffingtonpost&quot;,&quot;mlid&quot;:&quot;taboola&quot;}}"><a title="You'll be speaking a new language in 3 weeks thanks to this app made in Germany" href="https://go.babbel.com/engmag-a189-appsanywhere-cd-gbr-tb/1_gbr_tab_cd?utm_source=Taboola&utm_medium=CON&utm_campaign=CD_GBRALL_gEN_cUK_SpanishChallenge&utm_term=aol-huffingtonpost" rel="nofollow" target="_blank" class=" item-thumbnail-href "><div class="thumbBlock_holder"><span class="thumbBlock" style="background-image: url(&quot;http://images.taboola.com/taboola/image/fetch/f_jpg%2Cq_auto%2Ch_148%2Cw_280%2Cc_fill%2Cg_faces:auto%2Ce_sharpen/http%3A//cdn.taboolasyndication.com/libtrc/static/thumbnails/fe90c2c06d6ca733c8ebbf688df67c0e.png&quot;);"><span class="thumbnail-overlay"></span><span class="branding">Babbel</span><span class="static-text top-right"></span></span><div class="videoCube_aspect"></div></div></a><a title="You'll be speaking a new language in 3 weeks thanks to this app made in Germany" href="https://go.babbel.com/engmag-a189-appsanywhere-cd-gbr-tb/1_gbr_tab_cd?utm_source=Taboola&utm_medium=CON&utm_campaign=CD_GBRALL_gEN_cUK_SpanishChallenge&utm_term=aol-huffingtonpost" rel="nofollow" target="_blank" class=" item-label-href "><span class="video-label-box trc-main-label "><span class="video-label video-title trc_ellipsis " style="-webkit-line-clamp: 3;">You'll be speaking a new language in 3 weeks thanks to this app made in Germany</span><span class="branding">Babbel</span></span></a><div class=" trc_user_exclude_btn " title="Remove this item"></div><div class=" trc_exclude_overlay trc_fade "></div><div class=" trc_undo_btn ">Undo</div></div><div data-item-id="~~V1~~-5946300570125137099~~EeuF_EpbdbysgjL8a8oIRnGQ8r7wkoWe0iuV1BIEwtESO_fFkG-bpaW2-qyqBDD699Qc-gAVgcBVSl_mVDmD5tZ3uhcZH3NvLIMM0XLNNnrZKF83No0MlKdnOn7S8GqNYFTpzOxp7jLKs_zraWu8Hg" data-item-title="If you own a computer you must try this game" data-item-thumb="https://api-int.x-plarium.com/uploads/FVC.jpeg" data-item-syndicated="true" class="videoCube trc_spotlight_item origin-default thumbnail_top syndicatedItem textItem videoCube_5_child trc_excludable " data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:&quot;5&quot;,&quot;plid&quot;:&quot;https://plarium.com/play/en/vikings/005_village_anim_g?publisherID=aol-huffingtonpost&plid=101502&pxl=taboola_fr&quot;,&quot;mlid&quot;:&quot;taboola&quot;}}"><a title="If you own a computer you must try this game" href="https://plarium.com/play/en/vikings/005_village_anim_g?publisherID=aol-huffingtonpost&plid=101502&pxl=taboola_fr" rel="nofollow" target="_blank" class=" item-thumbnail-href "><div class="thumbBlock_holder"><span class="thumbBlock" style="background-image: url(&quot;http://images.taboola.com/taboola/image/fetch/f_jpg%2Cq_auto%2Ch_148%2Cw_280%2Cc_fill%2Cg_faces:auto%2Ce_sharpen/https%3A//api-int.x-plarium.com/uploads/FVC.jpeg&quot;);"><span class="thumbnail-overlay"></span><span class="branding">Vikings: Free Online Game</span><span class="static-text top-right"></span></span><div class="videoCube_aspect"></div></div></a><a title="If you own a computer you must try this game" href="https://plarium.com/play/en/vikings/005_village_anim_g?publisherID=aol-huffingtonpost&plid=101502&pxl=taboola_fr" rel="nofollow" target="_blank" class=" item-label-href "><span class="video-label-box trc-main-label "><span class="video-label video-title trc_ellipsis " style="-webkit-line-clamp: 2;">If you own a computer you must try this game</span><span class="branding">Vikings: Free Online Game</span></span></a><div class=" trc_user_exclude_btn " title="Remove this item"></div><div class=" trc_exclude_overlay trc_fade "></div><div class=" trc_undo_btn ">Undo</div></div><div data-item-id="~~V1~~-3361922757362746208~~9OUD6FhUvxlEkTgqaFzUqmAhdCjzvnL4PhEsKziBUTwSO_fFkG-bpaW2-qyqBDD6ElAJaEr7rU9vx-bTiwZGodZ3uhcZH3NvLIMM0XLNNnoiTgRPaTSavZq6snjrOR1A1mTO636rCKHciJUY36Chsg" data-item-title="The Most Daring Dresses on the Cannes Red Carpet" data-item-thumb="http://www4.pictures.zimbio.com/mp/Jo_SFBGqgsKx.jpg" data-item-syndicated="true" class="videoCube trc_spotlight_item origin-default thumbnail_top syndicatedItem photoItem videoCube_6_child trc_excludable " data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:&quot;6&quot;,&quot;plid&quot;:&quot;http://www.livingly.com/The+Most+Daring+Dresses+on+the+Cannes+Red+Carpet?utm_source=tabo&utm_medium=cpc&utm_campaign=Tabo-LV-Specials-X1-Desktop-Daring-Cannes-Dresses&utm_content=aol-huffingtonpost&utm_term=http%3A%2F%2Fwww4.pictures.zimbio.com%2Fmp%2FJo_SFBGqgsKx.jpg}&utm_term=http%3A%2F%2Fwww4.pictures.zimbio.com%2Fmp%2FJo_SFBGqgsKx.jpg_The+Most+Daring+Dresses+on+the+Cannes+Red+Carpet&quot;,&quot;mlid&quot;:&quot;taboola&quot;}}"><a title="The Most Daring Dresses on the Cannes Red Carpet" href="http://www.livingly.com/The+Most+Daring+Dresses+on+the+Cannes+Red+Carpet?utm_source=tabo&utm_medium=cpc&utm_campaign=Tabo-LV-Specials-X1-Desktop-Daring-Cannes-Dresses&utm_content=aol-huffingtonpost&utm_term=http%3A%2F%2Fwww4.pictures.zimbio.com%2Fmp%2FJo_SFBGqgsKx.jpg}&utm_term=http%3A%2F%2Fwww4.pictures.zimbio.com%2Fmp%2FJo_SFBGqgsKx.jpg_The+Most+Daring+Dresses+on+the+Cannes+Red+Carpet" rel="nofollow" target="_blank" class=" item-thumbnail-href "><div class="thumbBlock_holder"><span class="thumbBlock" style="background-image: url(&quot;http://images.taboola.com/taboola/image/fetch/f_jpg%2Cq_auto%2Ch_148%2Cw_280%2Cc_fill%2Cg_faces:auto%2Ce_sharpen/http%3A//www4.pictures.zimbio.com/mp/Jo_SFBGqgsKx.jpg&quot;);"><span class="thumbnail-overlay"></span><span class="branding">Livingly</span><span class="static-text top-right"></span></span><div class="videoCube_aspect"></div></div></a><a title="The Most Daring Dresses on the Cannes Red Carpet" href="http://www.livingly.com/The+Most+Daring+Dresses+on+the+Cannes+Red+Carpet?utm_source=tabo&utm_medium=cpc&utm_campaign=Tabo-LV-Specials-X1-Desktop-Daring-Cannes-Dresses&utm_content=aol-huffingtonpost&utm_term=http%3A%2F%2Fwww4.pictures.zimbio.com%2Fmp%2FJo_SFBGqgsKx.jpg}&utm_term=http%3A%2F%2Fwww4.pictures.zimbio.com%2Fmp%2FJo_SFBGqgsKx.jpg_The+Most+Daring+Dresses+on+the+Cannes+Red+Carpet" rel="nofollow" target="_blank" class=" item-label-href "><span class="video-label-box trc-main-label "><span class="video-label video-title trc_ellipsis " style="-webkit-line-clamp: 2;">The Most Daring Dresses on the Cannes Red Carpet</span><span class="branding">Livingly</span></span></a><div class=" trc_user_exclude_btn " title="Remove this item"></div><div class=" trc_exclude_overlay trc_fade "></div><div class=" trc_undo_btn ">Undo</div></div></div></div></div><div class="trc_clearer"></div></div></div></div></div>
<div id="hp-more-gravity"></div>
<script>
    (function(){
        var c = document.getElementById('taboola-below-article-thumbnails');
        c.id += '-' + Math.round(Math.random()*1e16);
        
        var taboolaParams = {
            loader: "http://cdn.taboola.com/libtrc/aol-huffingtonpost/loader.js",
            mode: "thumbnails-o",
            container: c.id,
            placement: "Below Article Thumbnails",
            target_type: "mix"
        };
        
        
        window.modulousQueue.add(function(){ doTaboola(taboolaParams); });
        
    }());
</script>
</div>



<div class="advertisement bottom_article_text treated">
<div id="zergnet-widget-49669" class="widget-loaded"><div class="zergheader"><span class="zerghead1">Around The Web</span><span class="zerghead2">Powered By ZergNet</span></div>
<div class="zerglayoutcl"><div class="zergrow">
			<div class="zergentity">
				<a href="http://www.zergnet.com/i/1822497/49669/0/0/0" target="_blank" rel="nofollow" class="zergimg" data-beacon="{&quot;p&quot;:{&quot;plid&quot;:&quot;http://www.zergnet.com/i/1822497/49669/0/0/0&quot;,&quot;mpid&quot;:1}}">
					<img src="http://img2.zergnet.com/1822497_300.jpg" nopin="true" data-pin-no-hover="true">
				</a>
				<div class="zergheadline">
					<a href="http://www.zergnet.com/i/1822497/49669/0/0/0" target="_blank" rel="nofollow" data-beacon="{&quot;p&quot;:{&quot;plid&quot;:&quot;http://www.zergnet.com/i/1822497/49669/0/0/0&quot;,&quot;mpid&quot;:1}}">10 Great Athletes You Forgot Were in Jail<span class="zergdestW">Grunge.com</span></a>
				</div>
			</div>
			<div class="zergentity">
				<a href="http://www.zergnet.com/i/1832838/49669/0/0/0" target="_blank" rel="nofollow" class="zergimg" data-beacon="{&quot;p&quot;:{&quot;plid&quot;:&quot;http://www.zergnet.com/i/1832838/49669/0/0/0&quot;,&quot;mpid&quot;:2}}">
					<img src="http://img5.zergnet.com/1832838_300.jpg" nopin="true" data-pin-no-hover="true">
				</a>
				<div class="zergheadline">
					<a href="http://www.zergnet.com/i/1832838/49669/0/0/0" target="_blank" rel="nofollow" data-beacon="{&quot;p&quot;:{&quot;plid&quot;:&quot;http://www.zergnet.com/i/1832838/49669/0/0/0&quot;,&quot;mpid&quot;:2}}">Famous Symbols That Used to Mean Something Completely Different<span class="zergdestW">Grunge.com</span></a>
				</div>
			</div>
			<div class="zergentity">
				<a href="http://www.zergnet.com/i/1753281/49669/0/0/0" target="_blank" rel="nofollow" class="zergimg" data-beacon="{&quot;p&quot;:{&quot;plid&quot;:&quot;http://www.zergnet.com/i/1753281/49669/0/0/0&quot;,&quot;mpid&quot;:3}}">
					<img src="http://img2.zergnet.com/1753281_300.jpg" nopin="true" data-pin-no-hover="true">
				</a>
				<div class="zergheadline">
					<a href="http://www.zergnet.com/i/1753281/49669/0/0/0" target="_blank" rel="nofollow" data-beacon="{&quot;p&quot;:{&quot;plid&quot;:&quot;http://www.zergnet.com/i/1753281/49669/0/0/0&quot;,&quot;mpid&quot;:3}}">What it Really Means When a Dog Tilts Their Head<span class="zergdestW">Grunge.com</span></a>
				</div>
			</div></div></div><div style="clear: both"></div><style type="text/css">#zergnet-widget-49669 .zerglayoutcl {
width: 100%;
clear: both;
}

#zergnet-widget-49669 .zergentity {
float: left;
width: 32%;
margin-left: 2%;
overflow: hidden;
margin-bottom: 10px;
}

#zergnet-widget-49669 .zergentity:nth-of-type(1) {
margin-left: 0;
}

#zergnet-widget-49669 .zergheader {
display: block;
margin-bottom: 15px;
line-height: 24px;
}

#zergnet-widget-49669 .zergheader::after {
display: block;
content: "";
clear: both;
}

#zergnet-widget-49669 .zerghead1 {
float: left;
color: #000;
font-size: 20px;
font-weight: bold;
font-family: ProximaNovaCond, sans-serif;
}

#zergnet-widget-49669 .zerghead2 {
float: right;
color: #000;
font-size: 13px;
font-weight: lighter;
font-family: ProximaNova, sans-serif;
}

#zergnet-widget-49669 .zergheadline {
width: 100%;
margin-top: 10px;
text-align: left;
line-height: 24px;
}
 
#zergnet-widget-49669 .zergheadline a {
font-size: 18px;
font-weight: 500;
text-decoration: none;
font-family: ProximaNovaCond-Medium, sans-serif;
color: #000;
}

#zergnet-widget-49669 .zergdestW {
display: block;
margin-top: 5px;
color: #999;
font-size: 11px;
font-weight: normal;
font-family: ProximaNovaCond-Medium, sans-serif;
line-height: 1.23;
letter-spacing: 0.5px;
}

#zergnet-widget-49669 .zergentity img {
width: 100%;
-ms-transform: translateY(20%);
-webkit-transform: translateY(20%);
transform: translateY(20%);
height: auto;
}

#zergnet-widget-49669 .zergimg {
display: block;
margin-top: -40%;
overflow: hidden;
}

@media (max-width: 600px) {
#zergnet-widget-49669 {display: none;}
#zergnet-widget-49669 .zergentity { width:48% !important; margin-left:2%}
#zergnet-widget-49669 .zergentity:nth-of-type(1) {margin-left:2%}
#zergnet-widget-49669 .zergentity:nth-of-type(2n+1) {clear:left; margin-left:0;} 
}</style></div>

<script language="javascript" type="text/javascript">
    (function() {
        var zergnet = document.createElement('script');
        zergnet.type = 'text/javascript'; zergnet.async = true;
        zergnet.src = (document.location.protocol == "https:" ? "https:" : "http:") + '//www.zergnet.com/zerg.js?id=49669';
        var znscr = document.getElementsByTagName('script')[0];
        znscr.parentNode.insertBefore(zergnet, znscr);
    })();
</script>
</div>

</div>
</div>

</div>
<script>
  window.HP.modules.smarten.selector('.js-entry');
</script>
<div class="js-right-rail right-rail bn-right-rail" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;right_rail&quot;}}" data-beacon-parsed="true">

<div class="advertisement right_rail_flex">
<script type="text/javascript">
  if ( "function" === typeof(adSetSyncDelay) ) {
    adSetSyncDelay('125');
  }
</script>
<div id="ad_right_rail_flex" class="ad_wrapper_top"><div class="adtech-adspot ad-right_rail_flex" id="adsDiv1"><iframe itemscope="" itemtype="https://schema.org/WPAdBlock" width="300" height="250" title="Ad" marginwidth="0" marginheight="0" allowtransparency="true" frameborder="0" scrolling="no" id="atwAdFrame1" banid="18017820|109970437" class="uac_300x250" data-integralas-id-2b3ddb6e-9532-1fe0-1416-47d97499d70b=""></iframe><div id="atwAdFrame1EAN" style="width: 300px; height: 15px; top: 0px; left: 0px; margin: 0px auto; position: relative;"><a id="atwAdFrame1EANA" style="position:absolute;right:0px;z-index:4999;margin:0 auto;width:77px;height:15px" href="http://adinfo.aol.com/" target="_blank"><img src="//o.aolcdn.com/ads/adchoices.png" style="border:none;width:77px;height:15px;vertical-align:top" alt="AdChoices"></a></div></div>
<script>
(function() {
  var adsDiv = document.querySelector('#adtech_right_rail_flex');
  adsDiv.removeAttribute('id');
  window.modulousQueue.add(function(){
    
      
        waitForGlobal('htmlAdWH', function() {
          adsDiv.reloadCount = 0;
          var recoveryId = '';
          var needsRecovery = recoveryId && recoveryId.length && kraken__adBlock && kraken__adBlock.active;
          var adId = needsRecovery ? recoveryId : '93317250';
          try { adId = io_c3sd.ads["right_rail_flex"] || adId; } catch(e) {} 
          
          // Override adtech_call_type
          var delivery_type = "" || "";
          var adtech_call_type = "";
          switch (delivery_type) {
            case "by_request": adtech_call_type = "ajax"; break;
            case "text": adtech_call_type = "text"; break;
            case "iframe": adtech_call_type = "iframe"; break;
            case "iframe_proxy": adtech_call_type = "f"; break;
            case "collapsed": adtech_call_type = "fc"; break;
            default: adtech_call_type = "f"; break;
          }

          
          
          
          var args = ['RR', 'RR'
          ];
          
          

          if(needsRecovery) {
            waitForGlobal('readyToRecover', function() {
              adsDiv = document.querySelector('.right_rail_flex');
              //htmlAdWH(adId, args[0], args[1], "fileless", adsDiv);
            });
          } else {
            htmlAdWH(adId, args[0], args[1], adtech_call_type, adsDiv);
          }
          
          if (typeof debugadcode == 'function') { debugadcode('AdTech', {placement: 'right_rail_flex', mnid: adId}, adsDiv); }
          
          jQuery(document).on('reloadAd', function(ev, elem) {
            if (elem && !elem.contains(adsDiv)) { return; }
            var refreshRate = 1;  
            adsDiv.reloadCount++;
            if (adsDiv.reloadCount % refreshRate === 0 && typeof adsReloadAd === 'function') { adsReloadAd(adsDiv); }
          });
          
          adSetInView('0');
        }); // wFG:htmlAdWH
      
    
  });
}());
</script></div>
</div>











<!-- Polar targeted placement -->
<div class="placement-sponsorship"></div>
<div class="bn-trending" id="right-rail-more-on" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;trending&quot;}}" data-beacon-parsed="true" data-mediaconductor-processed="true"><section class="bn-recirc component js-scroll loaded recirc recirc--right-rail recirc--zone_page" data-recirc-name="" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;&quot;}}" data-beacon-parsed="true">
  <div class="zone__title">
  <h1 class="zone__title__text">
        TRENDING
  </h1>
</div>
<div class="zone__content">
  <div class="card recirc__item bn-recirc-item card--standard card--trending card--media-left  " id="us_5954fcb8e4b05c37bb7c6d0a" data-recirc-link="us_5954fcb8e4b05c37bb7c6d0a" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:1,&quot;plid&quot;:&quot;us_5954fcb8e4b05c37bb7c6d0a&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_5954fcb8e4b05c37bb7c6d0a" href="http://www.huffingtonpost.com/entry/donald-trump-morning-joe_us_5954fcb8e4b05c37bb7c6d0a" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Trump Tweets Disgustingly Sexist Attack Against ‘Morning Joe’ Host</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/donald-trump-morning-joe_us_5954fcb8e4b05c37bb7c6d0a" target="" data-entry-id="us_5954fcb8e4b05c37bb7c6d0a" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/5954fd591500006303bfb5ea.jpeg?cache=4pocrk5stp);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item card--standard card--trending card--media-left  " id="us_59554203e4b02734df30ed11" data-recirc-link="us_59554203e4b02734df30ed11" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:2,&quot;plid&quot;:&quot;us_59554203e4b02734df30ed11&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59554203e4b02734df30ed11" href="http://www.huffingtonpost.com/entry/sarah-huckabee-sanders-donald-trump-tweets_us_59554203e4b02734df30ed11" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Sarah Huckabee Sanders Defends Donald Trump’s Tweets About Mika Brzezinski</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/sarah-huckabee-sanders-donald-trump-tweets_us_59554203e4b02734df30ed11" target="" data-entry-id="us_59554203e4b02734df30ed11" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(https://img.huffingtonpost.com/asset/5955427a1500002000bfb747.jpeg?ops=432_243,quality_75);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item card--standard card--trending card--media-left  " id="us_59555a21e4b05c37bb7d0a64" data-recirc-link="us_59555a21e4b05c37bb7d0a64" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:3,&quot;plid&quot;:&quot;us_59555a21e4b05c37bb7d0a64&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59555a21e4b05c37bb7d0a64" href="http://www.huffingtonpost.com/entry/greta-van-susteren-msnbc_us_59555a21e4b05c37bb7d0a64" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Greta Van Susteren Out At MSNBC</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/greta-van-susteren-msnbc_us_59555a21e4b05c37bb7d0a64" target="" data-entry-id="us_59555a21e4b05c37bb7d0a64" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(https://img.huffingtonpost.com/asset/59555b7c1500006303bfb7c3.jpeg?cache=pwqxnf5xkb&ops=432_243,quality_75);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item card--standard card--trending card--media-left  " id="us_59551b99e4b02734df30951a" data-recirc-link="us_59551b99e4b02734df30951a" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:4,&quot;plid&quot;:&quot;us_59551b99e4b02734df30951a&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59551b99e4b02734df30951a" href="http://www.huffingtonpost.com/entry/republicans-trump-sexism_us_59551b99e4b02734df30951a" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Republicans Express Concern About Trump’s Sexism, Still Won’t Do Anything About It</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/republicans-trump-sexism_us_59551b99e4b02734df30951a" target="" data-entry-id="us_59551b99e4b02734df30951a" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/595532551500002000bfb6b9.jpeg);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item card--standard card--trending card--media-left  " id="us_59552a16e4b02734df30abc5" data-recirc-link="us_59552a16e4b02734df30abc5" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:5,&quot;plid&quot;:&quot;us_59552a16e4b02734df30abc5&quot;}}" data-beacon-parsed="true" data-mediaconductor-processed="true">
  <div class="card__content">
        <div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59552a16e4b02734df30abc5" href="http://www.huffingtonpost.com/entry/fox-news-donald-trump-tweets_us_59552a16e4b02734df30abc5" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Even Fox News Hosts Are ‘Grossed Out’ By Donald Trump’s Sexist Tweets</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/fox-news-donald-trump-tweets_us_59552a16e4b02734df30abc5" target="" data-entry-id="us_59552a16e4b02734df30abc5" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/59552f851500006303bfb6ad.png?cache=iojqfobbpj);"></div>
        </div>
      </div>
    </a>
  </div>
</div><div class="mediaconductor-tag-begin polar-tag-begin" style="display: none;"></div> <div class="plr-card-t noskim">
    <div class="plr-card__content-t">
        <a class="plr-card__image-t plr-card__link-t" href="http://www.huffingtonpost.co.uk/entry/win-holiday-vouchers_uk_57592898e4b0069193651619" target="_blank" rel="nofollow">
            <div class="plr-card__image__wrapper-t" style="background-image: url(//meraxes-cdn.polarmobile.com/image/v1.0.0/bin/57bef7453e9221343df68dea?v=2efe8&w=124&h=124)">
         
            </div>
        </a>
        <div class="plr-card__headlines-t">
            <h2 class="plr-card__headline-t">
                <a class="plr-card__link-t" href="http://www.huffingtonpost.co.uk/entry/win-holiday-vouchers_uk_57592898e4b0069193651619" target="_blank" rel="nofollow">Win £1,000 Worth Of Holiday Vouchers</a>
            </h2>
            <span class="plr-card__pill-text-t">Promoted by Voltarol</span>
        </div>
    </div>
</div><div id="mediaconductor-nonce-10" style="display: none;"></div>
<script>
(function() {
  window.PolarAOL.inject("trending", {
    unit: {
      "server": "mvdirect",
      "id": "promofeed_8da0d7e4d30544559a6c6e6e0c13cffc"
    },
    nonce: "mediaconductor-nonce-10",
    siteName: "HuffPost - USA",
    placementName: "BPage - Desktop - Trending ",
    tagName: "UK- USEdition-  Desktop - BPage - PF - Trending ",
    originCountry: "UK"
  });
})();
</script> <script src="//z.moatads.com/polar984REuc19/moatad.js#moatClientLevel1=b2732eb1141c4c48b32b48842b54ba84&moatClientLevel2=d05ce97492f5478d864f1b629b02a278&moatClientLevel3=a8034e9e232b46bb85b07dc7eb41f4f4&moatClientLevel4=8dd7cbecb9804885ade7a8da8a11a837&moatClientSlicer1=GB&moatClientSlicer2=&zMoatDay=Fri&zMoatHour=01&zMoatTimeout=10000"></script><div class="mediaconductor-tag-end polar-tag-end" style="display: none;"></div>
</div>
</section></div><div class="mediaconductor-tag-begin polar-tag-begin" style="display: none;"></div> <div id="mediaconductor-nonce-9" style="display: block; margin-top: 30px;"><iframe style="border: none; height: 270px;"></iframe></div>
<script>

  
var PolarCreativeLabel = "HuffPost US Edition - Article Page Right Rail";
var PolarCreative = "promofeed_" + "8da0d7e4d30544559a6c6e6e0c13cffc";


    var iframe = document.createElement("iframe");

    var iframeContents = [
        "<!doctype html>",
        "<html>",
        "  <body>",
        "    <div id='mv-native-ad'></div>",
        "  </body>",
        "</html>",
    ].join("\n");

    document.getElementById("mediaconductor-nonce-9").appendChild(iframe);
    iframe.style.border = "none";


    var iframeDocument = iframe.contentWindow.document;
    iframeDocument.open("text/html", "replace");
    iframeDocument.write(iframeContents);
    iframeDocument.close();
    iframe.contentWindow.PolarCreative = PolarCreative;
    iframe.contentWindow.PolarCreativeLabel = PolarCreativeLabel;
    iframe.contentWindow.PolarCompiledTemplate = function (Handlebars,depth0,helpers,partials,data) {
  this.compilerInfo = [4,'>= 1.0.0'];
helpers = this.merge(helpers, Handlebars.helpers); data = data || {};
  var buffer = "", stack1, stack2, functionType="function", escapeExpression=this.escapeExpression;


  buffer += "<div class=\"plr-frame\">\n  <a href=\"";
  if (stack1 = helpers.link) { stack1 = stack1.call(depth0, {hash:{},data:data}); }
  else { stack1 = depth0.link; stack1 = typeof stack1 === functionType ? stack1.apply(depth0) : stack1; }
  buffer += escapeExpression(stack1)
    + "\" target=\"_blank\">\n        <div class=\"plr-ad\">\n          <div class=\"plr-line\" style=\"text-align:center;\">\n          <div class=\"plr-sponsor-name plr-line-text\">"
    + escapeExpression(((stack1 = ((stack1 = depth0.sponsor),stack1 == null || stack1 === false ? stack1 : stack1.name)),typeof stack1 === functionType ? stack1.apply(depth0) : stack1))
    + "</div>\n          </div>\n            <div class=\"plr-img-wrapper\">\n              <div style=\"background: url('";
  if (stack2 = helpers.getThumbHref) { stack2 = stack2.call(depth0, {hash:{},data:data}); }
  else { stack2 = depth0.getThumbHref; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }
  buffer += escapeExpression(stack2)
    + "') no-repeat center center;\"></div>\n            </div>\n          <div class=\"plr-title\">";
  if (stack2 = helpers.title) { stack2 = stack2.call(depth0, {hash:{},data:data}); }
  else { stack2 = depth0.title; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }
  buffer += escapeExpression(stack2)
    + "</div>\n        </div>\n </a>\n</div>\n<link rel=\"stylesheet\" href=\"http://s.m.huffpost.com/assets/fonts/proxima-nova-bold-6bdfc864744b144366b8f735598378d4.css\">\n\n<style>\n  body {\n  	margin: 0px;\n    padding: 0px;\n  }\n   .plr-frame {\n      box-sizing: content-box;\n      width: 300px;\n      height: 250px;\n      margin: auto;\n      padding-bottom: 20px;\n      background-color: white; }\n\n    .plr-img-wrapper {\n      position: relative;\n      overflow: hidden;\n      width: 300px;\n      height:168px}\n\n    .plr-img-wrapper div {\n      position: absolute;\n      top: 0;\n      right: 0;\n      bottom: 0;\n      left: 0;\n      background-size: cover !important; }\n\n    .plr-disclosure {\n      font-family: Arial;\n      font-size: 12px;\n      letter-spacing: 1px;\n      color: #2c7061; }\n\n    .plr-sponsor-name {\n     font-family: ProximaNovaCond-bold, NotoKufiArabic-Bold, Helvetica Neue, Helvetica, Roboto, Arial, sans-serif;\n      font-size: 14px;\n      font-weight: bold;\n      display: inline-block;\n      padding-right: 6px;\n      padding-left:10px;\n      text-transform: uppercase;\n      color: #000;\n      background-color: white}\n\n    .plr-ad::before {\n      position: relative;\n      z-index: -1;\n      top: 25px;\n      display: inherit;\n      content: '';\n      border-top: 3px solid #2c7061; }\n\n    .plr-ad {\n      padding: 10px 0x 0 0px; }\n\n    .plr-line {\n     position:relative;\n     font-family: ProximaNovaCond-bold, NotoKufiArabic-Bold, Helvetica Neue, Helvetica, Roboto, Arial, sans-serif;\n     font-size: 0.875rem;\n     margin-bottom: 10px;\n     -webkit-box-ordinal-group: 2;\n      color:#000; }\n\n    .plr-line:after{\n     content: '';\n     border-bottom: 3px solid #000;\n     position: absolute;\n     top: 50%;\n     left: 0;\n     margin-top: -2px;\n     width: 100%;\n     z-index: 1;\n}\n.plr-line-text{\n    position: relative;\n    padding-right: 10px;\n    letter-spacing: 1.4px;\n    background: #ffffff;\n    display: inline-block;\n    z-index: 2;\n}\n\n    .plr-title {\n     font-family: ProximaNovaCond-bold, ProximaNovaCond-Extrabld, NotoKufiArabic-Bold, Helvetica Neue, Helvetica, Roboto, Arial, sans-serif;\n      font-size: 18px;\n      line-height: 20px;\n      margin-top: 8px;\n    }\n\n.plr-frame a{\n text-decoration:none;\n color:inherit;\n}\n</style>";
  return buffer;
  };

    var js = document.createElement('script');
    js.src = "//plugin.mediavoice.com/plugin.js";
    js.id = "nativeads-plugin"; js.type = "text/javascript";
    iframe.contentWindow.document.getElementsByTagName('head')[0].appendChild(js);

    js = document.createElement('script');
    var code = '(function(){       window.NATIVEADS = window.NATIVEADS || {};       window.NATIVEADS_QUEUE = window.NATIVEADS_QUEUE || [];        var q = function() {           return window.NATIVEADS_QUEUE;       };        q().push(["setPropertyID", "NA-HUFFPOSTUSEDIT-11238859"]);       q().push(["setWillLinkRedirect", function(){return false;}]);        q().push(["insertPreview", {           label: PolarCreativeLabel,           unit: {"server":"mvdirect","id":PolarCreative},           location: "#mv-native-ad",           template: PolarCompiledTemplate,           onRender: function($element) {                 var height = $element.outerHeight();            		console.log("Setting iFrame height: " + height);             	window.frameElement.style.height = height + "px";             	window.frameElement.parentElement.style.display = "block";             	window.frameElement.parentElement.style.marginTop = "30px";           },           onFill: function(data) {             	if (data.sponsor.name.indexOf(" ") == 0) {                 	data.sponsor.name = undefined;                 }           },           onError: function(error) { }       }]);  })();';
    try {
      js.appendChild(document.createTextNode(code));
    } catch (e) {
      js.text = code;
    }
    iframe.contentWindow.document.getElementsByTagName('head')[0].appendChild(js);
             

  

</script> <script src="//z.moatads.com/polar984REuc19/moatad.js#moatClientLevel1=b2732eb1141c4c48b32b48842b54ba84&moatClientLevel2=d05ce97492f5478d864f1b629b02a278&moatClientLevel3=f6d9d1eb1de241acbcf436ee7db6a02e&moatClientLevel4=82df2dc2b3d041ac8489d5248a4bf671&moatClientSlicer1=GB&moatClientSlicer2=&zMoatDay=Fri&zMoatHour=01&zMoatTimeout=10000"></script><div class="mediaconductor-tag-end polar-tag-end" style="display: none;"></div>
<div class="inline-newsletter inline-newsletter--breaking-alerts inline-newsletter--square" style="background-image:url('http://s.m.huffpost.com/assets/newsletters/square/breaking-alerts-ebfc5fb12567e237812c01301eb063e7b851709308e6e030807bb53622ded24c.png');">
<div class="inline-newsletter__logo">
<!-- Generator: Adobe Illustrator 19.1.0, SVG Export Plug-In . SVG Version: 6.00 Build 0)  --><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="0px" y="0px" viewBox="0 0 720 90" style="enable-background:new 0 0 720 90;" xml:space="preserve" height="14px" width="99px">
<style type="text/css">
	.st0{display:none;}
	.st1{display:inline;}
	.st2{fill:#FFFFFF;}
	.st3{fill:#0DBE98;}
</style>
<g id="Layer_2" class="st0">
	<rect class="st1" width="720" height="90"></rect>
</g>
<g id="Layer_1">
	<g>
		<g>
			<path class="st2" d="M93.1,85.4h26.3l14.1-80.7h-26.3L102,33.6H79.2l5.2-28.9H58.1L44,85.4h26.3l5.3-30.2h22.8L93.1,85.4z"></path>
		</g>
		<g>
			<path class="st2" d="M148.4,4.6l-8,44.9c-0.7,3.5-1,6.9-1,10.1c0,23.8,20.8,27.4,33.3,27.4c26.2,0,37.5-8.7,41.6-32.2l8.8-50.3     h-26.3l-7.5,42.2c-2.4,13.2-4.1,20.1-13.9,20.1c-6.4,0-9.5-3.4-9.5-10.4c0-2.7,0.4-6,1.2-10.1l7.6-41.8H148.4z"></path>
		</g>
		<g>
			<path class="st2" d="M362.1,85.4h26.3l3.7-20.8h12.4c22.9,0,35.6-12.4,35.6-35c0-15.9-11-25-30.3-25h-33.6L362.1,85.4z      M403.1,44.7h-7.6l3.5-19h7.1c5.8,0,8.9,2.9,8.9,8.2C415,40.6,410.5,44.7,403.1,44.7z"></path>
		</g>
		<g>
			<path class="st2" d="M491.2,2.9c-28.6,0-45.7,18.5-45.7,49.6c0,21.6,13.6,34.6,36.4,34.6c28.6,0,45.7-18.5,45.7-49.6     C527.7,15.8,514.1,2.9,491.2,2.9z M483.5,66.2c-6.8,0-10.8-4.9-10.8-13.1c0-3.1,0.3-5.8,0.9-8.9c2-10.6,5.4-20.3,16.1-20.3     c6.8,0,10.8,4.9,10.8,13.1c0,3.1-0.3,5.8-0.9,8.9C497.6,56.4,494.2,66.2,483.5,66.2z"></path>
		</g>
		<g>
			<path class="st2" d="M619.1,85.4h26.3l10.4-59.1h19.8l3.8-21.6h-67.3l-3.8,21.6h21L619.1,85.4z"></path>
		</g>
		<path class="st2" d="M575.5,34.5c-8.3-2.6-11.7-3.7-11.7-7.6c0-2.6,1.7-5.7,6.7-5.7c3.7,0,6.8,2.1,8,5.2l23.2-6.3    c-2.7-11.4-12.5-17.2-29.4-17.2c-31.8,0-34.3,21.3-34.3,27.8c0,13.2,7,21.3,22.2,25.5c4,1.1,8.6,2.3,8.6,6.7    c0,3.5-2.6,5.7-7.1,5.7c-4.1,0-8.5-2.4-10-6.6l-22.9,6.2c2.5,12,13.8,18.8,31.5,18.8c13.5,0,36.1-3.6,36.1-28    C596.5,46.8,589.6,38.8,575.5,34.5z"></path>
		<g>
			<path class="st3" d="M29.3,85.4L43.5,4.7H0v80.7L29.3,85.4z"></path>
		</g>
		<g>
			<path class="st3" d="M690.7,4.7l-14.1,80.7l43.5,0V4.7L690.7,4.7z"></path>
		</g>
		<polygon class="st2" points="238.1,4.6 224,85.4 250.3,85.4 254.5,61.6 276.6,61.6 280.2,41.3 258.1,41.3 260.6,26.3 291.5,26.3     295.3,4.6   "></polygon>
		<polygon class="st2" points="307.1,4.6 293,85.4 319.3,85.4 323.4,61.6 345.6,61.6 349.2,41.3 327.1,41.3 329.6,26.3 360.5,26.3     364.3,4.6   "></polygon>
	</g>
</g>
</svg>

</div>
<div class="inline-newsletter__content">
<div class="inline-newsletter__title">Subscribe to Breaking Alerts.</div>
<div class="inline-newsletter__description">Don’t miss out — be the first to know all the latest news.</div>
</div>
<div class="inline-newsletter__signup">
<form class="newsletter-signup__form" id="inline-newsletter-form">
<input name="newsletter_source" type="hidden" value="Inline Newsletter new-school desktop">
<input name="newsletter_provider" type="hidden" value="cm">
<input name="newsletter_id" type="hidden" value="0eb60568a730719cfead15d5782df96b">
<input name="newsletter_name" type="hidden" value="Breaking Alerts">
<input name="newsletter_display_name" type="hidden" value="Breaking Alerts">
<input name="newsletter_entry_list_tag" type="hidden" value="">
<div class="newsletter-signup__success">
<div class="newsletter-signup__success-check-icon"></div>
</div>
<div class="newsletter-signup__progress-bar">
<div class="newsletter-signup__progress-bar-first-half"></div>
<div class="newsletter-signup__progress-bar-second-half"></div>
</div>
<div class="newsletter-signup__email">
<div class="newsletter-signup__email-input-container">
<input class="newsletter-signup__email-input" name="email_address" placeholder="address@email.com" title="Enter your email" type="text">
</div>
<div class="newsletter-signup__submit-container">
<input class="newsletter-signup__submit bn-signup-submit bn-clickable" type="submit" value="SUBSCRIBE" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;subscribe&quot;}}" data-beacon-parsed="true">
</div>
</div>
<div class="newsletter-signup__message"></div>
</form>
<div class="newsletter-signup__success-message hidden"></div>

</div>
</div>




<div class="advertisement right_rail_300x250_lower">
<div id="ad_right_rail_300x250_lower" class="ad_wrapper_"><div class="adtech-adspot ad-right_rail_300x250_lower" id="adsDiv2"><iframe itemscope="" itemtype="https://schema.org/WPAdBlock" width="300" height="250" title="Ad" marginwidth="0" marginheight="0" allowtransparency="true" frameborder="0" scrolling="no" id="atwAdFrame2" banid="18017865|109970436" class="uac_300x250" data-integralas-id-74ade3ac-375e-6d63-d12c-4da9e4d1d18a="" data-integralas-id-22f8d178-7f16-ba3e-4f72-c6be1f415a4d=""></iframe><div id="atwAdFrame2EAN" style="width: 300px; height: 15px; top: 0px; left: 0px; margin: 0px auto; position: relative;"><a id="atwAdFrame2EANA" style="position:absolute;right:0px;z-index:4999;margin:0 auto;width:77px;height:15px" href="http://adinfo.aol.com/" target="_blank"><img src="//o.aolcdn.com/ads/adchoices.png" style="border:none;width:77px;height:15px;vertical-align:top" alt="AdChoices"></a></div></div>
<script>
(function() {
  var adsDiv = document.querySelector('#adtech_right_rail_300x250_lower');
  adsDiv.removeAttribute('id');
  window.modulousQueue.add(function(){
    
      
        waitForGlobal('htmlAdWH', function() {
          adsDiv.reloadCount = 0;
          var recoveryId = '';
          var needsRecovery = recoveryId && recoveryId.length && kraken__adBlock && kraken__adBlock.active;
          var adId = needsRecovery ? recoveryId : '93432943';
          try { adId = io_c3sd.ads["right_rail_300x250_lower"] || adId; } catch(e) {} 
          
          // Override adtech_call_type
          var delivery_type = "" || "";
          var adtech_call_type = "";
          switch (delivery_type) {
            case "by_request": adtech_call_type = "ajax"; break;
            case "text": adtech_call_type = "text"; break;
            case "iframe": adtech_call_type = "iframe"; break;
            case "iframe_proxy": adtech_call_type = "f"; break;
            case "collapsed": adtech_call_type = "fc"; break;
            default: adtech_call_type = "f"; break;
          }

          adSetInView('D');
          
          
          var args = ['RR', 'RR'
          ];
          
          

          if(needsRecovery) {
            waitForGlobal('readyToRecover', function() {
              adsDiv = document.querySelector('.right_rail_300x250_lower');
              //htmlAdWH(adId, args[0], args[1], "fileless", adsDiv);
            });
          } else {
            htmlAdWH(adId, args[0], args[1], adtech_call_type, adsDiv);
          }
          
          if (typeof debugadcode == 'function') { debugadcode('AdTech', {placement: 'right_rail_300x250_lower', mnid: adId}, adsDiv); }
          
          jQuery(document).on('reloadAd', function(ev, elem) {
            if (elem && !elem.contains(adsDiv)) { return; }
            var refreshRate = 1;  
            adsDiv.reloadCount++;
            if (adsDiv.reloadCount % refreshRate === 0 && typeof adsReloadAd === 'function') { adsReloadAd(adsDiv); }
          });
          
          adSetInView('0');
        }); // wFG:htmlAdWH
      
    
  });
}());
</script></div>
</div>


</div>

</div>

<script class="taboola-ad-script-template" type="text/template">

</script>
</article>


<div class="bn-nav-sticky js-nav-sticky nav-sticky nav-sticky--b-page" data-mobilepath="/us/entry/6805902" data-sharingimage="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" data-sharingtitle="Florida Deputy Shoots Unarmed Man While Serving Warrant" data-sharingtweetname="HuffPostCrime" data-sharingtweettext="Florida+Deputy+Shoots+Unarmed+Man+While+Serving+Warrant" data-sharingurl="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;sticky-nav&quot;}}" data-beacon-parsed="true">
<div class="nav-sticky__bar">
<div class="nav-sticky__item nav-sticky__hamburger bn-hamburger-menu" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;hamburger_menu&quot;}}" data-beacon-parsed="true">
<div class="nav-sticky__hamburger-link bn-hamburger-button js-hamburger-link-sticky-b bn-clickable" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;open&quot;}}" data-beacon-parsed="true" data-beacon-toggle-type="lnid" data-beacon-toggle-value="close"></div>
</div>
<a class="nav-sticky__item nav-sticky__logo bn-logo-link" href="/" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;logo&quot;}}" data-beacon-parsed="true">
<img alt="The Huffington Post" class="nav-sticky__logo-img bn-logo-image bn-clickable" src="http://s.m.huffpost.com/assets/H-icon-72fd5782b36a1a42ab82a4af65cbc6dd6ddc4d4a71190a91c5cfaab7c66fd836.svg" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;homepage&quot;}}" data-beacon-parsed="true">
</a>
<div class="nav-sticky__item nav-sticky__title">Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant</div>
<div class="nav-sticky__item nav-sticky__sharebar">
<div class="js-sharebar share-bar share-bar--nav-sticky" data-mobilepath="/us/entry/6805902" data-sharingimage="http://i.huffpost.com/gen/2684130/images/o-DEREK-CRUICE-facebook.jpg" data-sharingtitle="Florida Deputy Shoots Unarmed Man While Serving Warrant" data-sharingtweetname="HuffPostCrime" data-sharingtweettext="Florida+Deputy+Shoots+Unarmed+Man+While+Serving+Warrant" data-sharingurl="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html">
<ul class="share-bar__list bn-share" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;share&quot;}}" data-beacon-parsed="true">
<div class="share-bar__social-counts">
<span class="share-bar__social-counts__number">1.4k</span>
</div>
<li class="bn-share-item facebook js-sharebar-facebook share-bar__item bn-clickable" data-share-name="facebook" data-type="facebook" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;facebook&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="facebook share-bar__item-link bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item js-sharebar-twitter share-bar__item twitter bn-clickable" data-share-name="twitter" data-type="twitter" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;twitter&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="share-bar__item-link twitter bn-clickable" href="#" target="_blank" data-beacon-click-disabled="true"></a>
</li>
<li class="bn-share-item email js-sharebar-email share-bar__item bn-clickable" data-share-name="email" data-type="email" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;email&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<a class="email share-bar__item-link bn-clickable" href="#" target="" data-beacon-click-disabled="true"></a>
</li>
<div class="bn-share-item js-sharebar-comments share-bar__comments with-count bn-clickable" data-share-name="comments_open" data-type="comments" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;comments_open&quot;}}" data-beacon-parsed="true" data-beacon-click-disabled="true">
<div class="share-bar__comments__icon with-count bn-clickable" data-beacon-click-disabled="true"></div>
<div class="share-bar__comments__count with-count bn-clickable" data-beacon-click-disabled="true">390</div>
</div>
</ul>
</div>

</div>
</div>
<div class="nav-sticky__progress-bar js-progress-bar">
<div class="nav-sticky__progress-bar__fill js-progress-fill" style="width: 3.35396%;"></div>
</div>
</div>
<script>
  window.HP.modules.smarten.selector('.js-nav-sticky');
</script>


<div class="bn-below-entry" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;below_entry&quot;}}" data-beacon-parsed="true">
<div class="js-bottom-most-shared bn-suggested-trending" id="bottom-most-shared" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;suggested-trending&quot;}}" data-beacon-parsed="true"><section class="bn-recirc component js-scroll loaded recirc recirc--most-shared recirc--horizontal recirc--rows-of-five recirc--page" data-recirc-name="" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;&quot;}}" data-beacon-parsed="true">
  <h3 class="recirc__title">
  <span class="recirc__title__text">MOST SHARED</span>
</h3>
<div class="recirc__content">
  <div class="card recirc__item bn-recirc-item   " id="us_5954e6c2e4b0da2c73217669" data-recirc-link="us_5954e6c2e4b0da2c73217669" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:1,&quot;plid&quot;:&quot;us_5954e6c2e4b0da2c73217669&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    POLITICS
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_5954e6c2e4b0da2c73217669" href="http://www.huffingtonpost.com/entry/this-nra-recruitment-video-is-so-divisive-even-gun-owners-are-angry_us_5954e6c2e4b0da2c73217669" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    This NRA Recruitment Video Is So Divisive, Even Gun Owners Are Angry</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/this-nra-recruitment-video-is-so-divisive-even-gun-owners-are-angry_us_5954e6c2e4b0da2c73217669" target="" data-entry-id="us_5954e6c2e4b0da2c73217669" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/59550bcc1800003500672125.png?cache=l3bz3lnq2n);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59550156e4b0da2c732198a0" data-recirc-link="us_59550156e4b0da2c732198a0" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:2,&quot;plid&quot;:&quot;us_59550156e4b0da2c732198a0&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    WEDDINGS
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59550156e4b0da2c732198a0" href="http://www.huffingtonpost.com/entry/how-did-serena-williams-meet-her-fiance-alexis-ohanian_us_59550156e4b0da2c732198a0" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Serena Williams’ First Words To Her Fiancé Will Make You Cringe</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/how-did-serena-williams-meet-her-fiance-alexis-ohanian_us_59550156e4b0da2c732198a0" target="" data-entry-id="us_59550156e4b0da2c732198a0" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(https://img.huffingtonpost.com/asset/59550e091500002000bfb62c.jpeg?cache=sxorb1y7y6&ops=432_243,quality_75);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_5954f351e4b02734df305df7" data-recirc-link="us_5954f351e4b02734df305df7" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:3,&quot;plid&quot;:&quot;us_5954f351e4b02734df305df7&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    ENTERTAINMENT
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_5954f351e4b02734df305df7" href="http://www.huffingtonpost.com/entry/chrissy-teigen-donald-trump-grow-up_us_5954f351e4b02734df305df7" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Bless Chrissy Teigen For Instructing Donald Trump To ‘Grow. The F**k. Up.’</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/chrissy-teigen-donald-trump-grow-up_us_5954f351e4b02734df305df7" target="" data-entry-id="us_5954f351e4b02734df305df7" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/5954f54b2100003700fc4a44.jpeg?cache=nwryvs15tv);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59551869e4b05c37bb7c9535" data-recirc-link="us_59551869e4b05c37bb7c9535" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:4,&quot;plid&quot;:&quot;us_59551869e4b05c37bb7c9535&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    MEDIA
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59551869e4b05c37bb7c9535" href="http://www.huffingtonpost.com/entry/sean-hannity-trump-tinder_us_59551869e4b05c37bb7c9535" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Tinder Users Are Swiping Left On Trump Fans And Sean Hannity Can’t Deal</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/sean-hannity-trump-tinder_us_59551869e4b05c37bb7c9535" target="" data-entry-id="us_59551869e4b05c37bb7c9535" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/595518ac1800003500672154.jpeg?cache=k69a1o7fx2);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59550905e4b0da2c7321a54f" data-recirc-link="us_59550905e4b0da2c7321a54f" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:5,&quot;plid&quot;:&quot;us_59550905e4b0da2c7321a54f&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    ARTS & CULTURE
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59550905e4b0da2c7321a54f" href="http://www.huffingtonpost.com/entry/jk-rowling-tweets-quote-about-character-and-power-to-trump_us_59550905e4b0da2c7321a54f" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    J.K. Rowling Puts Trump’s Latest Sexist Tweet Into Chilling Perspective</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/jk-rowling-tweets-quote-about-character-and-power-to-trump_us_59550905e4b0da2c7321a54f" target="" data-entry-id="us_59550905e4b0da2c7321a54f" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/595511bd1500002000bfb63f.png);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
</div>
</section></div>
<div class="bottom-trending js-bottom-trending">
<div class="bn-bottom-trending" id="bottom-trending" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;bottom-trending&quot;}}" data-beacon-parsed="true"><section class="bn-recirc component js-scroll loaded recirc recirc--bottom-trending recirc--horizontal recirc--rows-of-four recirc--page" data-recirc-name="" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;&quot;}}" data-beacon-parsed="true">
  <h3 class="recirc__title">
  <span class="recirc__title__text">WHAT’S HOT</span>
</h3>
<div class="recirc__content">
  <div class="card recirc__item bn-recirc-item   " id="us_5954ebe0e4b02734df30538e" data-recirc-link="us_5954ebe0e4b02734df30538e" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:1,&quot;plid&quot;:&quot;us_5954ebe0e4b02734df30538e&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    POLITICS
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_5954ebe0e4b02734df30538e" href="http://www.huffingtonpost.com/entry/marco-rubio-fool-bible-twitter_us_5954ebe0e4b02734df30538e" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Twitterverse Trolls Marco Rubio Over His ‘Fool’ Bible Verse Tweet</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/marco-rubio-fool-bible-twitter_us_5954ebe0e4b02734df30538e" target="" data-entry-id="us_5954ebe0e4b02734df30538e" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(https://img.huffingtonpost.com/asset/5954fe3218000035006720f7.jpeg?cache=dybpiruzon&ops=432_243,quality_75);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59553280e4b02734df30c0c8" data-recirc-link="us_59553280e4b02734df30c0c8" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:2,&quot;plid&quot;:&quot;us_59553280e4b02734df30c0c8&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    QUEER VOICES
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59553280e4b02734df30c0c8" href="http://www.huffingtonpost.com/entry/heterosexualprideday-trending-twitter_us_59553280e4b02734df30c0c8" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Here Are Some Of The Best Responses To #HeterosexualPrideDay</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/heterosexualprideday-trending-twitter_us_59553280e4b02734df30c0c8" target="" data-entry-id="us_59553280e4b02734df30c0c8" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/59554085180000380067220f.png);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_595515bce4b0da2c7321b7f3" data-recirc-link="us_595515bce4b0da2c7321b7f3" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:3,&quot;plid&quot;:&quot;us_595515bce4b0da2c7321b7f3&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    BLACK VOICES
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_595515bce4b0da2c7321b7f3" href="http://www.huffingtonpost.com/entry/jordan-edwards-police-shooting_us_595515bce4b0da2c7321b7f3" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Nobody Was Drinking At The Party Where Police Killed Jordan Edwards</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/jordan-edwards-police-shooting_us_595515bce4b0da2c7321b7f3" target="" data-entry-id="us_595515bce4b0da2c7321b7f3" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/5908a8e3260000bd9ac48e86.jpeg?cache=gnvo7kiwrt);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59556750e4b0da2c732237fa" data-recirc-link="us_59556750e4b0da2c732237fa" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:4,&quot;plid&quot;:&quot;us_59556750e4b0da2c732237fa&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    POLITICS
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59556750e4b0da2c732237fa" href="http://www.huffingtonpost.com/entry/medicaid-congressional-budget-office_us_59556750e4b0da2c732237fa" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Cuts To Medicaid In GOP Health Bill Would Get Even Bigger Over Time, CBO Says</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/medicaid-congressional-budget-office_us_59556750e4b0da2c732237fa" target="" data-entry-id="us_59556750e4b0da2c732237fa" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/595567c51500002000bfb7d9.jpeg?cache=erww4i4fvv);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59551705e4b05c37bb7c93d5" data-recirc-link="us_59551705e4b05c37bb7c93d5" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:5,&quot;plid&quot;:&quot;us_59551705e4b05c37bb7c93d5&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    WOMEN
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59551705e4b05c37bb7c93d5" href="http://www.huffingtonpost.com/entry/another-day-another-tweet-from-your-president-about-how-much-he-hates-women_us_59551705e4b05c37bb7c93d5" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Another Day, Another Tweet From Your President About How Much He Hates Women</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/another-day-another-tweet-from-your-president-about-how-much-he-hates-women_us_59551705e4b05c37bb7c93d5" target="" data-entry-id="us_59551705e4b05c37bb7c93d5" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/595544d01800003800672226.png?cache=fnjyflrj5k);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59552255e4b0da2c7321cacb" data-recirc-link="us_59552255e4b0da2c7321cacb" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:6,&quot;plid&quot;:&quot;us_59552255e4b0da2c7321cacb&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    HOME
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59552255e4b0da2c7321cacb" href="http://www.huffingtonpost.com/entry/bad-dog-haircuts_us_59552255e4b0da2c7321cacb" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    11 Unfortunate Doggy Haircuts That Will Make You Giggle Endlessly</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/bad-dog-haircuts_us_59552255e4b0da2c7321cacb" target="" data-entry-id="us_59552255e4b0da2c7321cacb" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/59555c2e180000350067228f.jpeg?cache=kubeywyhhb);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_5954c94de4b05c37bb7c1cca" data-recirc-link="us_5954c94de4b05c37bb7c1cca" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:7,&quot;plid&quot;:&quot;us_5954c94de4b05c37bb7c1cca&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    SPORTS
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_5954c94de4b05c37bb7c1cca" href="http://www.huffingtonpost.com/entry/cubs-albert-almora-jr-outfielder-denies-flipping-off-donald-trump-at-white-house_us_5954c94de4b05c37bb7c1cca" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    Cubs’ Albert Almora Jr. Denies Flipping Off Donald Trump At White House</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/cubs-albert-almora-jr-outfielder-denies-flipping-off-donald-trump-at-white-house_us_5954c94de4b05c37bb7c1cca" target="" data-entry-id="us_5954c94de4b05c37bb7c1cca" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/5954cc431500002000bfb57c.png?cache=m3kmilikzv);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
  <div class="card recirc__item bn-recirc-item   " id="us_59548411e4b02734df2fe1d7" data-recirc-link="us_59548411e4b02734df2fe1d7" data-beacon="{&quot;p&quot;:{&quot;mpid&quot;:8,&quot;plid&quot;:&quot;us_59548411e4b02734df2fe1d7&quot;}}" data-beacon-parsed="true">
  <div class="card__content">
        <div class="card__label">
  <span class="card__label__text">
    POLITICS
  </span>
</div>
<div class="card__headlines">
  <h2 class="card__headline js-card-headline">
    <a class="card__link bn-card-headline bn-clickable" data-entry-id="us_59548411e4b02734df2fe1d7" href="http://www.huffingtonpost.com/entry/mcconnell-offers-extra-opioid-money-to-snag-votes-just-as-expected_us_59548411e4b02734df2fe1d7" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;headline&quot;}}" data-beacon-parsed="true">    McConnell Offers Extra Opioid Money To Snag Votes, Just As Expected</a>
  </h2>
</div>
    <a class="card__image js-card-image bn-card-image card__link  bn-clickable" href="http://www.huffingtonpost.com/entry/mcconnell-offers-extra-opioid-money-to-snag-votes-just-as-expected_us_59548411e4b02734df2fe1d7" target="" data-entry-id="us_59548411e4b02734df2fe1d7" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;image&quot;}}" data-beacon-parsed="true">
      <div class="card__image__wrapper null bn-clickable">
        <div class="sigil-icon bn-clickable"></div>
        <div class="card__image__cover-bg bn-clickable">
          <div class="card__image__src bn-clickable" style="background-image: url(http://img.huffingtonpost.com/asset/432_243,quality_75/595484a91500004500bfb547.jpeg?cache=naecsgsgqg);"></div>
        </div>
      </div>
    </a>
  </div>
</div>
</div>
</section></div>

</div>
</div>
<div class="entry__comments-pane js-comment-pane bn-comments-pane hide" id="comment-pane_6805902" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;comments-pane&quot;}}" data-beacon-parsed="true">
<div class="bn-comments-container" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;comments&quot;}}" data-beacon-parsed="true">
<div class="entry__comments-pane__close js-comment-close bn-close bn-clickable" id="comment-close_6805902" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;close&quot;}}" data-beacon-parsed="true"></div>
</div>
<h2 class="below-entry-recirc__conversations-header">CONVERSATIONS</h2>
<div class="comments" id="comments">
<script>
  (function(d, s, id){
    var js, fjs = d.getElementsByTagName(s)[0];
    if (d.getElementById(id)) {return;}
    js = d.createElement(s); js.id = id; js.async = true;
    js.src = "https://connect.facebook.net/en_US/sdk.js#xfbml=1&version=v2.4";
    fjs.parentNode.insertBefore(js, fjs);
  }(document, 'script', 'facebook-jssdk'));
</script>

<div class="fb-comments fb_iframe_widget fb_iframe_widget_fluid" data-colorscheme="light" data-href="http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html" data-numposts="10" data-width="100%" fb-xfbml-state="rendered"><span style="height: 1800px;"><iframe id="fa9d6377b6f5c" name="f25e296335346e8" scrolling="no" title="Facebook Social Plugin" class="fb_ltr" src="https://www.facebook.com/plugins/comments.php?api_key=&channel_url=http%3A%2F%2Fstaticxx.facebook.com%2Fconnect%2Fxd_arbiter%2Fr%2FXBwzv5Yrm_1.js%3Fversion%3D42%23cb%3Df33a3b672b7f5d8%26domain%3Dwww.huffingtonpost.com%26origin%3Dhttp%253A%252F%252Fwww.huffingtonpost.com%252Ff1a7ccb946473ac%26relation%3Dparent.parent&colorscheme=light&href=http%3A%2F%2Fwww.huffingtonpost.com%2F2015%2F03%2F05%2Fderek-cruice-shot-in-face_n_6805902.html&locale=en_US&numposts=10&sdk=joey&skin=light&version=v2.4&width=100%25" style="border: none; overflow: hidden; height: 1800px; width: 100%;"></iframe></span></div>
</div>

</div>


</div>
<script>
  HP.modules.sharing = {
    buttons: [""]
  };
</script>
<!-- Footer -->
<footer class="footer bn-footer" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;footer&quot;}}" data-beacon-parsed="true">
<script async="" src="https://ovp.iris.tv/plugins/vidible/v2/iris-vidible.adaptive.min.js" type="text/javascript"></script>
<script async="" src="https://ovp.iris.tv/libs/adaptive/v2/iris.adaptive.js" type="text/javascript"></script>


<script src="//dc8xl0ndzn2cb.cloudfront.net/js/huffingtonpost/v0/keywee.min.js" type="text/javascript" async=""></script><script async="" src="http://plugin.mediavoice.com/mediaconductor/mc.js"></script>
<script>
window.mediaconductor = window.mediaconductor || function(){(mediaconductor.q=mediaconductor.q||[]).push(arguments);};
mediaconductor("init", "d05ce97492f5478d864f1b629b02a278");
mediaconductor("exec");
</script><!-- AD highlights module tracking -->
<script type="text/javascript">
HPAds = typeof HPAds !== "undefined" ? HPAds : {};
HPAds.highlightstracker = {
	exec : function(){
		var img = document.createElement("img");img.width=1;img.height=1;img.alt="";img.border=0;img.style.display="none";
		img.src = "http://at.atwola.com/adserv/3.0/5113.1/2583061/1/16/AdId=3495516;BnId=1";
		document.getElementById("ad_bottom_1x1").appendChild(img);
	},
	init : function(){
		if(document.getElementById("ad_bottom_1x1") && document.getElementById("huffhighlights"))
		{
			this.exec();
		}
	}
}
HPAds.highlightstracker.init();
</script><script>
(function(d, s, id) {
  var src = 'http://cdn.pubexchange.com/modules/partner/huffpost_crime';
  if (window._HPJS && _HPJS.loadJSLater) {
    _HPJS.loadJSLater(src);
  } else {
    var js, pjs = d.getElementsByTagName(s)[0];
    if (d.getElementById(id)) { return; }
    js = d.createElement(s); js.id = id; js.async = true; js.src = src;
    pjs.parentNode.insertBefore(js, pjs);
  }
}(document, 'script', 'pubexchange-jssdk'));
</script><script src="http://ar.voicefive.com/bmx3/node.pli?pub=aol2"></script>
<script>
// interfering with omniture
//_HPJS.loadJSLater('http://ar.voicefive.com/bmx3/node.pli?pub=aol2');
</script>

<script>
window.addEventListener('load', function() {
  var ref = document.getElementsByTagName("script")[0];
  var script = document.createElement("script");
  script.src = 'http://s.skimresources.com/js/38395X1559799.skimlinks.js';
  script.async = true;
  ref.parentNode.insertBefore(script, ref);
}, false);
</script>

<script async="" type="text/javascript">
    var assetname = "crime";
    var _nolParams = {
        nol_sfcode: 'dcr',
        nol_ci: 'us-500136',              
        nol_apid: 'P6987AC99-4DBA-45C3-BFE4-9CD649BA8C46',   
        nol_assetname: assetname,
        nol_apn: 'huffingtonpostCOM',     
        nol_vc: 'c29',
        nol_segA: 'HPMG News',
        nol_segB: '',
        nol_segC:''
    };

    (function (params) {
        var url = 'http://cdn-gl.imrworldwide.com/novms/js/2/nlsDcrLite510.min.js#';
        var query = [];
        for (var param in params) {query.push(param + '=' + params[param]);}
        url += query.join('&');
        var script = window.document.createElement("script");
        script.async = true;
        script.src = url;
        window.document.getElementsByTagName('head')[0].appendChild(script);
    })(_nolParams);

</script>
<script>
    window._taboola = window._taboola || [];
    _taboola.push({flush:true});
</script>
<div class="footer__top">
<div class="footer__logo">
<a class="bn-logo-link" href="/" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;logo&quot;}}" data-beacon-parsed="true">
<img alt="HuffPost" class="footer__img bn-logo-image bn-clickable" src="http://s.m.huffpost.com/assets/HuffPostHero-d783f680fb974e80346bdfa145131d058e596e1d32f791981875aa481702c770.svg" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;homepage&quot;}}" data-beacon-parsed="true">
</a>
</div>
<div class="footer__lists bn-footer-links" data-beacon="{&quot;p&quot;:{&quot;mnid&quot;:&quot;corp-links&quot;}}" data-beacon-parsed="true">
<ul class="footer__list">
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/p/huffington-post.html" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;ABOUT US&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/p/huffington-post.html&quot;}}" data-beacon-parsed="true">ABOUT US</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://advertising.aol.com/properties#huffington-post" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;ADVERTISE&quot;,&quot;plid&quot;:&quot;http://advertising.aol.com/properties#huffington-post&quot;}}" data-beacon-parsed="true">ADVERTISE</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://adinfo.aol.com/about-our-ads/" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;About Our Ads&quot;,&quot;plid&quot;:&quot;http://adinfo.aol.com/about-our-ads/&quot;}}" data-beacon-parsed="true">About Our Ads</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/contact/" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Contact Us&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/contact/&quot;}}" data-beacon-parsed="true">Contact Us</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/syndication/" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;RSS&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/syndication/&quot;}}" data-beacon-parsed="true">RSS</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/p/frequently-asked-question.html" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;FAQ&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/p/frequently-asked-question.html&quot;}}" data-beacon-parsed="true">FAQ</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="https://careers.smartrecruiters.com/Aol/huffington-post/" target="_blank" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Careers&quot;,&quot;plid&quot;:&quot;https://careers.smartrecruiters.com/Aol/huffington-post/&quot;}}" data-beacon-parsed="true">Careers</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/archive/" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Archive&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/archive/&quot;}}" data-beacon-parsed="true">Archive</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/terms.html" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;User Agreement&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/terms.html&quot;}}" data-beacon-parsed="true">User Agreement</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://privacy.aol.com" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Privacy Policy&quot;,&quot;plid&quot;:&quot;http://privacy.aol.com&quot;}}" data-beacon-parsed="true">Privacy Policy</a>
</li>
<li class="footer__list__item">
<a class="footer__link bn-footer-link-item bn-clickable" href="http://www.huffingtonpost.com/comment/policy/" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;Comment Policy&quot;,&quot;plid&quot;:&quot;http://www.huffingtonpost.com/comment/policy/&quot;}}" data-beacon-parsed="true">Comment Policy</a>
</li>
</ul>
</div>
</div>
<div class="footer__bottom">
<a class="footer__bottom__left" href="http://oath.com">
<img class="footer__bottom__left__image" src="http://s.m.huffpost.com/assets/oath-footer-white-4f6d0a4bbc98c395c8c4e2c7536851b43338ff822be8e91cc77cfbcf7e5efdd1.svg">
</a>
<div class="footer__bottom__right">
<div class="footer__copyright">©2017 Oath Inc. All rights reserved.</div>
<div class="footer__rights"> Part of HuffPost • HPMG News</div>
</div>
</div>
<script>
  // if we don't need ad recovery, we need to define this
  // global variable because something is waiting for it
  window.kraken__adBlock = { active: false };
</script>

<script>
  //sp bootstrap
  !function(e){function t(i){if(n[i])return n[i].exports;var r=n[i]={exports:{},id:i,loaded:!1};return e[i].call(r.exports,r,r.exports,t),r.loaded=!0,r.exports}var n={};return t.m=e,t.c=n,t.p="",t(0)}({0:function(e,t,n){"use strict";function i(e){return e&&e.__esModule?e:{"default":e}}function r(){if(!M){M=!0,window._sp_=window._sp_||{},O=window._sp_.config=window._sp_.config||{},I=O.content_control_callback||window._sp_lock,R=O.account_id||O.client_id||window.sp_cid;for(var e in O)L[e]=O[e]}}function o(e,t){var n=arguments.length>2&&void 0!==arguments[2]?arguments[2]:"",i=new Image;i.src="//"+x+"/cct?v="+encodeURIComponent(b)+"&ct=1&cid="+encodeURIComponent(R)+"&l="+encodeURIComponent(e.toString())+"&rc="+encodeURIComponent(t)+"&d0="+encodeURIComponent(n)}function a(e,t,n,i){var r=arguments.length>4&&void 0!==arguments[4]?arguments[4]:{},o=arguments.length>5&&void 0!==arguments[5]&&arguments[5],s=function(){return a(e,t,n,i,r,!0)},c=o?n:s;e(t,c,i,r)}function s(e,t,n,i){var r=document.createElement("script");r.src=e,r.onload=function(){try{if(window._sp_&&window._sp_.bootstrap===p&&window.spBootstrap===d&&window._sp_.config===O){for(var e in L)if(L[e]!==O[e]){y&&y();break}}else y&&y()}catch(e){y&&y()}i.onLoad&&i.onLoad()},r.onerror=function(){function r(e,t){n(e,t),i.onError&&i.onError(t)}function o(e){return a.indexOf(e)>-1}var a=(navigator.userAgent||navigator.vendor||window.opera).toLowerCase();o("mobi")||o("ipad")||o("android")||o("iphone")?r(P):o("exabot")?r(j):o("bingbot")||o("bingpreview")?r(A):o("googlebot")||o("adsbot-google")||o("mediapartners-google")?r(k):o("googleweblight")?r(S):window.location.host===v?r(C):window.location.host.indexOf(w)>-1?r(N):o("msie 10")||o("msie 9")||o("msie 8")?r(E):c(e,t,r)},document.head.appendChild(r),document.querySelector('script[src="'+e+'"]')||window.location.host.indexOf(w)!==-1||t(H,e)}function c(e,t,n){(0,g["default"])(e,function(i,r,o,a){var s=i||r?t:n;s(a,e+"::"+o)})}function l(e,t){return D?void u($+"::"+e,t):(o(T,e,t),void(I&&setTimeout(function(){I()},250)))}function u(e,t){o(_,e,t)}function d(e,t){r(),t&&(I=window._sp_lock=t),a(s,e,l,u)}function p(e,t){r(),a(s,e,l,u,t)}var f=n(381),h=i(f),m=n(102),g=i(m),v=["w","e","b","c","a","c","h","e",".","g","o","o","g","l","e","u","s","e","r","c","o","n","t","e","n","t",".","c","o","m"].join(""),w=["o","p","t","i","m","i","z","e","l","y","p","r","e","v","i","e","w",".","c","o","m"].join(""),y=void 0;try{y=window._sp_&&window._sp_.config&&window._sp_.config.spid_control_callback}catch(e){}var x=h["default"].be.join(""),b=h["default"].bv,_=0,T=1,H="s",P="m",k="g",C="gw",S="gl",E="i",A="b",j="e",$="n",N="o",L={},M=!1,D=!1,O=void 0,I=void 0,R=void 0;window.addEventListener("beforeunload",function(){D=!0});try{window._sp_=window._sp_||{},window._sp_.bootstrap=p,window.spBootstrap=d,window._sp_.bootstrap===p&&window.spBootstrap===d||y&&y()}catch(e){y&&y()}},102:function(e,t){"use strict";function n(e,t){function n(e){try{return e&&e.timeStamp?e.timeStamp:window.performance.now()}catch(e){return Date.now()}}var c=document.createElement("a");if(c.href=e,"https:"===window.location.protocol&&c.protocol!==window.location.protocol)return void t(!1,!1,c.protocol,s);var l=new XMLHttpRequest;try{l.open("GET",e)}catch(e){return void t(!0,!1,e.toString(),o,l)}var u=void 0,d=void 0;l.onloadstart=function(e){u=n(e)},l.onreadystatechange=function(e){if(4===this.readyState){d=n(e)-u;var o=0===this.status,a="2"===this.status.toString()[0],s=a?r:i;return void t(o,a,this.status+"::"+d,s,l)}};try{l.send()}catch(e){return void t(!0,!1,e.toString(),a,l)}}Object.defineProperty(t,"__esModule",{value:!0}),t["default"]=n;var i=t.NETWORK_FAILURE="n",r=t.XHR_SUCCEEDED="nx",o=t.OPEN_BLOCKED="xo",a=t.SEND_BLOCKED="xs",s=t.PROTOCOL_MISMATCH="p"},381:function(e){"use strict";e.exports={be:["w","w","w",".","r","o","o","s","t","e","r","f","i","r","e","w","o","r","k",".","c","o","m"],bv:"1.7.620"}}});
  //sp messaging config
  (function() {
    window._sp_ = window._sp_ || {};
    window._sp_.config = window._sp_.config || {};
    window._sp_.config.account_id =  313;
    window._sp_.config.enable_rid = true;
    window._sp_.config.mms_domain =  'mms.huffingtonpost.com';
    window._sp_.config.content_control_callback = function() {
      //window.location.href = '/static/sp-block';
    };
    // messaging-specific configuration  
    window._sp_.mms = window._sp_.mms || {};
    window._sp_.mms.cmd = window._sp_.mms.cmd || [];
    // add a K/V pair for targeting purposes
    window._sp_.mms.cmd.push(function() { 
      window._sp_.mms.setTargeting("x", "y");
    });
    // tell the messaging library that all params are set and it can proceed
    window._sp_.mms.cmd.push(function() {
      window._sp_.mms.startMsg();
    });
    window._sp_.config.mms_client_data_callback = function(dataObject) {
      // do something useful with the dataObject
    };
    // magic points these to sourcepoint hosted files
    window._sp_.bootstrap('/cr3wc89sw3-m.js');
    window._sp_.bootstrap('/cr3wc89sw3-c.js');
  }());
</script>

</footer>

<link rel="stylesheet" href="http://s.m.huffpost.com/assets/desktop-46d987e2de0aa3f015f9cf90a20052b95e342552b66c8e5eeaeb193744cd1947.css" media="all"><script>
  HP.functions.loadCSS('http://s.m.huffpost.com/assets/desktop-46d987e2de0aa3f015f9cf90a20052b95e342552b66c8e5eeaeb193744cd1947.css');
</script>
<div class="tracking-data">
<script>
  window.comscore_data = {"c1":2,"c2":6723616,"c3":"","c4":"/us/entry/6805902","c5":"crime","c6":"","c15":"","options":{"url_append":"comscorekw=crime"}};
  window.comscore_data.c4 = location.protocol+"//"+location.host+window.comscore_data.c4;
</script>

<script>
  var s_265_account ="aolhuffpostcrime,aolsvc";
  var s_265_config = {"pfxID":"hpo","channel":"us.hpmgcrime","linkInternalFilters":"javascript:,huffingtonpost.com,m.huffpost.com/us","prop1":"slides_ajax","prop2":"","prop6custom":"","prop9":"hpo:6805902","prop16":"news","prop17":"","prop18":"","prop19":"","prop21":"","prop22":"andy-campbell","prop23":"03-05-2015","prop53":"","prop54":"huffpost","prop59":"","prop62":"video_novideo","prop63":"","prop65":"original_content","prop66":"","prop69":"","prop70":"","prop74":"","pageName":"Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant"};
  waitForGlobal(function() {
    return !!window.extendOmniVars && !!window.s_265;
  }, function() {
    window.omnitureVarsExtended = true;
    extendOmniVars(s_265_config);
  });
</script>

<script>
  bN_cfg = {
    h: location.hostname,
    b: "b.huffingtonpost.com",
    p: {"t":"Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant","dL_ch":"us.hpmgcrime","dL_dpt":"slides_ajax","dL_sDpt":"","dL_flid":"_","dL_cmsID":"hpo:6805902","dL_author":"Andy Campbell","author-id":"andy-campbell","dL_pubts":"1425542265000","dL_blogID":"2","dL_tags":"derek-cruice,derek-cruice-dead,todd-raible,volusia-county-sheriff,todd-raible-volusia-county-sheriff,cop-shoots-pot-dealer-in-face,deputy-shoots-man-in-face-serving-warrant,video,police,@health_adhd,@health_depression,@health_gad,@health_hiv,@health_ht,@health_ibs,@health_ost,@health_pain,@health_erectile,@health_models","dL_url":"http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html","dL_vid":"video_novideo","cms_src":"","cobrand":"HuffPost","edition":"us","publishEdition":"us","platform":"desktop","sectionId":"559333aee4b0569b63402a11","sectionName":"Crime","renderingEngine":"kraken","publishSectionId":"559333aee4b0569b63402a11","publishSectionName":"Crime","a_type":"","a_value":"","pageVersion":"","engagement":"25","ad_presence":"[\"bottom_1x1\", \"slideshow_end_card\", \"bottom_article_text\", \"slideshow_300x250_req\", \"below_entry_recirc\", \"leaderboard_flex\", \"video_page_left\", \"sourcepoint\", \"pub_exchange_json\", \"asset_inserts\", \"skimlinks\", \"ab_config\", \"right_rail_300x250_lower\", \"right_rail_flex\", \"commercial_video\", \"slideshow_728x90_req\"]"},
    view: 0,
    click: 0
  };
  if (typeof(io_c3sd) != 'undefined' && typeof(io_c3sd.omniture) != 'undefined') {
    if (typeof(io_c3sd.omniture.s_account) != 'undefined') {
      s_265_account = io_c3sd.omniture.s_account;
    }
    if (typeof(io_c3sd.omniture.prop17) != 'undefined') {
      bN_cfg['p']['Custom02'] = io_c3sd.omniture.prop17;
    }
    if (typeof(io_c3sd.omniture.prop21) != 'undefined') {
      bN_cfg['p']['Custom03'] = io_c3sd.omniture.prop21;
    }
  }
  waitForGlobal(function() {
    return !!window.HP.modules.Tracky;
  }, function() {
    window.HP.modules.Tracky.bN.set(bN_cfg.p);
  });
</script>







<script>
  window.pubExchangeData = {"partnerList":{"uuid":"e7e0a839-b20c-4558-ab4a-f10b75152c4d","partners":{"investigationdiscovery.com":"c8d9a681-5baf-4e1b-9cbf-3d93523cccd5","barstoolsports.com":"86243412-cc68-4853-be37-1cac0eae8a58","bustedcoverage.com":"0391536f-f975-4b37-b5e9-b4351e3ef937","crimefeed.com":"e541b4c4-f530-44e8-9747-02937ba92501","brobible.com":"e735e28a-6872-4346-adaa-806bac11ad75","cracked.com":"fcd72009-6121-4bb0-ab3b-91d0ac52cd59","newser.com":"f9b2e782-68a3-42d8-9bc0-e25418369d42","oddee.com":"63dc06ba-64ca-440c-a858-146cf6f371aa","coed.com":"c0407d5c-c100-418c-906e-a230b17f1d78"}},"url":"http://traffic.pubexchange.com/a/"};
</script>


</div>
<div class="tracking">
<script>
  waitForGlobal(function() {
    return window.PARSELY && window.PARSELY.beacon;
  }, function() {
    PARSELY.beacon.trackPageView({
      url: window.location.href,
      urlref: HP.params.original_url,
      js: 1
    });
  });
</script>

<script>
  waitForGlobal(function() {
    return !!window.HP.modules.Tracky;
  }, function() {
    window.HP.modules.Tracky.comscore(window.comscore_data);
  })
</script>

<script>
  waitForGlobal(function() {
    return !!window.HP.modules && !!window.HP.modules.Tracky && !!window.extendOmniVars && !!window.s_265 && !!window.omnitureVarsExtended;
  }, function() {
    window.HP.modules.Tracky.omniture();
  });
</script>

<script>
  waitForGlobal(function() {
    return !!window.HP.modules.Tracky;
  }, function() {
    window.HP.modules.Tracky.bN.view();
  });
</script>










<script src="http://b.scorecardresearch.com/beacon.js" async=""></script><script async="" src="http://o.aolcdn.com/os/aol/omniture.min.js"></script><script async="" src="http://o.aolcdn.com/os/aol/beacon.min.js"></script></div>
<script>
  HP.modules.ads.slideshowLeft = "\n<div class=\"advertisement slideshow_300x250_req\">\n<div id=\"adtech_slideshow_300x250_req\" class=\"adtech-adspot ad-slideshow_300x250_req\"><\/div>\n<script>\n(function() {\n  var adsDiv = document.querySelector(\'#adtech_slideshow_300x250_req\');\n  adsDiv.removeAttribute(\'id\');\n  window.modulousQueue.add(function(){\n    \n      \n        waitForGlobal(\'htmlAdWH\', function() {\n          adsDiv.reloadCount = 0;\n          var recoveryId = \'\';\n          var needsRecovery = recoveryId && recoveryId.length && kraken__adBlock && kraken__adBlock.active;\n          var adId = needsRecovery ? recoveryId : \'93317341\';\n          try { adId = io_c3sd.ads[\"slideshow_300x250_req\"] || adId; } catch(e) {} \n          \n          // Override adtech_call_type\n          var delivery_type = \"\" || \"\";\n          var adtech_call_type = \"\";\n          switch (delivery_type) {\n            case \"by_request\": adtech_call_type = \"ajax\"; break;\n            case \"text\": adtech_call_type = \"text\"; break;\n            case \"iframe\": adtech_call_type = \"iframe\"; break;\n            case \"iframe_proxy\": adtech_call_type = \"f\"; break;\n            case \"collapsed\": adtech_call_type = \"fc\"; break;\n            default: adtech_call_type = \"ajax\"; break;\n          }\n\n          \n          \n          \n          var args = [\'300\', \'250\'];\n          \n          \n\n          if(needsRecovery) {\n            waitForGlobal(\'readyToRecover\', function() {\n              adsDiv = document.querySelector(\'.slideshow_300x250_req\');\n              //htmlAdWH(adId, args[0], args[1], \"fileless\", adsDiv);\n            });\n          } else {\n            htmlAdWH(adId, args[0], args[1], adtech_call_type, adsDiv);\n          }\n          \n          if (typeof debugadcode == \'function\') { debugadcode(\'AdTech\', {placement: \'slideshow_300x250_req\', mnid: adId}, adsDiv); }\n          \n          jQuery(document).on(\'reloadAd\', function(ev, elem) {\n            if (elem && !elem.contains(adsDiv)) { return; }\n            var refreshRate = 1;  \n            adsDiv.reloadCount++;\n            if (adsDiv.reloadCount % refreshRate === 0 && typeof adsReloadAd === \'function\') { adsReloadAd(adsDiv); }\n          });\n          \n          adSetInView(\'0\');\n        }); // wFG:htmlAdWH\n      \n    \n  });\n}());\n<\/script>\n<\/div>\n";
  HP.modules.ads.slideshowTop = "\n<div class=\"advertisement slideshow_728x90_req\">\n<div id=\"adtech_slideshow_728x90_req\" class=\"adtech-adspot ad-slideshow_728x90_req\"><\/div>\n<script>\n(function() {\n  var adsDiv = document.querySelector(\'#adtech_slideshow_728x90_req\');\n  adsDiv.removeAttribute(\'id\');\n  window.modulousQueue.add(function(){\n    \n      \n        waitForGlobal(\'htmlAdWH\', function() {\n          adsDiv.reloadCount = 0;\n          var recoveryId = \'\';\n          var needsRecovery = recoveryId && recoveryId.length && kraken__adBlock && kraken__adBlock.active;\n          var adId = needsRecovery ? recoveryId : \'93407314\';\n          try { adId = io_c3sd.ads[\"slideshow_728x90_req\"] || adId; } catch(e) {} \n          \n          // Override adtech_call_type\n          var delivery_type = \"\" || \"\";\n          var adtech_call_type = \"\";\n          switch (delivery_type) {\n            case \"by_request\": adtech_call_type = \"ajax\"; break;\n            case \"text\": adtech_call_type = \"text\"; break;\n            case \"iframe\": adtech_call_type = \"iframe\"; break;\n            case \"iframe_proxy\": adtech_call_type = \"f\"; break;\n            case \"collapsed\": adtech_call_type = \"fc\"; break;\n            default: adtech_call_type = \"ajax\"; break;\n          }\n\n          \n          \n          \n          var args = [\'728\', \'90\'];\n          \n          \n\n          if(needsRecovery) {\n            waitForGlobal(\'readyToRecover\', function() {\n              adsDiv = document.querySelector(\'.slideshow_728x90_req\');\n              //htmlAdWH(adId, args[0], args[1], \"fileless\", adsDiv);\n            });\n          } else {\n            htmlAdWH(adId, args[0], args[1], adtech_call_type, adsDiv);\n          }\n          \n          if (typeof debugadcode == \'function\') { debugadcode(\'AdTech\', {placement: \'slideshow_728x90_req\', mnid: adId}, adsDiv); }\n          \n          jQuery(document).on(\'reloadAd\', function(ev, elem) {\n            if (elem && !elem.contains(adsDiv)) { return; }\n            var refreshRate = 1;  \n            adsDiv.reloadCount++;\n            if (adsDiv.reloadCount % refreshRate === 0 && typeof adsReloadAd === \'function\') { adsReloadAd(adsDiv); }\n          });\n          \n          adSetInView(\'0\');\n        }); // wFG:htmlAdWH\n      \n    \n  });\n}());\n<\/script>\n<\/div>\n";
  HP.modules.ads.slideshowEndCard = "\n<div class=\"advertisement slideshow_end_card\">\n<div id=\"taboola-endslate-thumbnails\" class=\"taboola hp-entry-you_may_like\"><\/div>\n<div id=\"hp-more-gravity\"><\/div>\n<script>\n    (function(){\n        var c = document.getElementById(\'taboola-endslate-thumbnails\');\n        c.id += \'-\' + Math.round(Math.random()*1e16);\n        \n        var taboolaParams = {\n            loader: \"//cdn.taboola.com/libtrc/aol-huffingtonpost/loader.js\",\n            mode: \"thumbnails-b\",\n            container: c.id,\n            placement: \"Endslate Thumbnails\",\n            target_type: \"mix\"\n        };\n        \n        \n        window.modulousQueue.add(function(){ doTaboola(taboolaParams); });\n        \n    }());\n<\/script>\n<\/div>\n";
  HP.modules.ads.rightRailRecircCards = [];
  HP.modules.ads.rightRailVideoCards = [];
  HP.modules.ads.categoriesBottom = "";
</script>
<script src="http://s.m.huffpost.com/assets/desktop-f2218b4a61994248531c22c6118ddbe13bbf3fb80e5ba3eb6b8db1c9d737a572.js"></script>
<script>
  (function() {
    window.HP.params.entry = {
      id: "6805902",
      tags: ["derek-cruice", "derek-cruice-dead", "todd-raible", "volusia-county-sheriff", "todd-raible-volusia-county-sheriff", "cop-shoots-pot-dealer-in-face", "deputy-shoots-man-in-face-serving-warrant", "video", "police", "@health_adhd", "@health_depression", "@health_gad", "@health_hiv", "@health_ht", "@health_ibs", "@health_ost", "@health_pain", "@health_erectile", "@health_models"],
      title: "Florida Deputy Shoots Unarmed Man In Face While Serving Pot Warrant",
      excerpt: "A Volusia County Sheriff's deputy shot an unarmed man in the face while serving a drug warrant on Wednesday. Deputies say Derek Cruice, 26, resiste...",
      url: "http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html",
      author: "Andy Campbell",
      edition: "us"
    };
    window.HP.params.preview = ("" === "true");
    window.HP.params.isWide = ("false" === "true");
    window.HP.params.isAthena = ("false" === "true");
    var isLong = 450 <= ($('.js-entry-body').height() / 2.5);
    window.HP.events.publish("entry_opened", {entry: document.querySelector('article'), isLong: isLong});
  }());
</script>

<script>
  (function(d, s, id){
    var js, fjs = d.getElementsByTagName(s)[0];
    if (d.getElementById(id)) {return;}
    js = d.createElement(s); js.id = id; js.async = true;
    js.src = "https://connect.facebook.net/en_US/sdk.js#xfbml=1&version=v2.4";
    fjs.parentNode.insertBefore(js, fjs);
  }(document, 'script', 'facebook-jssdk'));
</script>

<script>
  window.twttr = (function(d, s, id) {
    var js, fjs = d.getElementsByTagName(s)[0],
      t = window.twttr || {};
    if (d.getElementById(id)) return t;
    js = d.createElement(s);
    js.id = id; js.async = true;
    js.src = "https://platform.twitter.com/widgets.js";
    fjs.parentNode.insertBefore(js, fjs);
  
    t._e = [];
    t.ready = function(f) {
      t._e.push(f);
    };
  
    return t;
  }(document, "script", "twitter-wjs"));
</script>

<script async="" src="https://apis.google.com/js/platform.js" gapi_processed="true"></script>

<div class="tracking-init">
<div id="parsely-root" style="display: none">
<div data-parsely-site="huffingtonpost.com" id="parsely-cfg"></div>
<script id="parsely-script" async="" src="http://static.parsely.com/p.js"></script></div>
<script>
  (function(){
    window.PARSELY = {autotrack: false};
    (function(s, p, d) {
      var h=d.location.protocol, i=p+"-"+s,
          e=d.getElementById(i), r=d.getElementById(p+"-root"),
          u=h==="https:"?"d1z2jf7jlzjs58.cloudfront.net"
          :"static."+p+".com";
      if (e) return;
      e = d.createElement(s); e.id = i; e.async = true;
      e.src = h+"//"+u+"/p.js"; r.appendChild(e);
    })("script", "parsely", document);
  }());
</script>

<div id="vanitydiv" style="visibility:hidden; height: 0px;"></div>
<!-- Begin comScore Tag -->
<script>
  var prefix = (document.location.protocol == "https:" ? "https://sb" : "http://b");
  var script = document.createElement("script");
  script.src = prefix + ".scorecardresearch.com/beacon.js";
  script.async = true;
  script.onload = function() {
    window.HP.events.publish('tracking_loaded_comscore');
  };
  document.querySelector('.tracking').appendChild(script);
</script>
<noscript>
&lt;img alt="" height="1" src="http://b.scorecardresearch.com/p?c1=2&amp;c2=6723616&amp;c3=&amp;c4=&amp;c5=&amp;c6=&amp;c15=&amp;cj=1" width="1"&gt;
</noscript>
<!-- End comScore Tag -->

<script>
  (function(d){
    var s = d.createElement('script');
    s.async = true;
    s.src = "http://o.aolcdn.com/os/aol/omniture.min.js";
    s.onload = function() {
      window.HP.events.publish('tracking_loaded_omniture');
    };
    d.querySelector('.tracking').appendChild(s);
  })(document);
</script>

<script>
  (function(d){
    var s = d.createElement('script');
    s.async = true;
    s.src = "http://o.aolcdn.com/os/aol/beacon.min.js";
    s.onload = function() {
      window.HP.events.publish('tracking_loaded_datalayer');
    };
    d.querySelector('.tracking').appendChild(s);
  })(document);
</script>

<script>
  
  window.extendOmniVars=function(e){""!==window.HP.functions.getCookie("huffpost_user_id")&&(e.eVar17="huffpost_"+window.HP.functions.getCookie("huffpost_user_id")),e.prop12=document.location.origin+document.location.pathname,"undefined"!=typeof io_c3sd&&"undefined"!=typeof io_c3sd.omniture&&("undefined"!=typeof io_c3sd.omniture.prop17&&(e.prop17=io_c3sd.omniture.prop17),"undefined"!=typeof io_c3sd.omniture.prop21&&(e.prop21=io_c3sd.omniture.prop21)),waitForGlobal("s_265",function(){Object.keys(e).forEach(function(t){s_265[t]=e[t]})})};
  !function(){var e=document.createElement("script");e.type="text/javascript",e.async=!0,e.src=location.protocol+"//ak.sail-horizon.com/horizon/v1.js";var t=document.getElementsByTagName("script")[0];t.parentNode.insertBefore(e,t),waitForGlobal(function(){return window.Sailthru&&"function"==typeof window.Sailthru.setup},function(){console.log("Sailthru setup call for horizon"),Sailthru.setup({domain:"horizon.huffingtonpost.com",useStoredTags:!1})})}();
</script>




<!-- Init Standard Include -->
<!-- START Nielsen Online SiteCensus V6.0 -->
<!-- COPYRIGHT 2012 Nielsen Online -->
<script>
  modulousQueue.add(function () {
    var d = new Image(1, 1);
    d.onerror = d.onload = function () {
      d.onerror = d.onload = null;
    };
    d.src = ["//secure-us.imrworldwide.com/cgi-bin/m?ci=us-703240h&cg=0&cc=1&si=", escape(window.location.href), "&rp=", escape(document.referrer), "&ts=compact&rnd=", (new Date()).getTime()].join('');
  });
</script>
<noscript>
&lt;div&gt;
&lt;img alt="" height="1" src="//secure-us.imrworldwide.com/cgi-bin/m?ci=us-703240h&amp;cg=0&amp;cc=1&amp;ts=noscript" width="1"&gt;
&lt;/div&gt;
</noscript>
<!-- END Nielsen Online SiteCensus V6.0 -->





<!-- Facebook Pixel Code -->
<script>
  !function(f,b,e,v,n,t,s){if(f.fbq)return;n=f.fbq=function(){n.callMethod?
  n.callMethod.apply(n,arguments):n.queue.push(arguments)};if(!f._fbq)f._fbq=n;
  n.push=n;n.loaded=!0;n.version='2.0';n.queue=[];t=b.createElement(e);t.async=!0;
  t.src=v;s=b.getElementsByTagName(e)[0];
  s.parentNode.insertBefore(t,s)}(window,document,'script','https://connect.facebook.net/en_US/fbevents.js');
  fbq('init', '1621685564716533'); // Edition specific
  fbq('init', '1043018625788392'); // Partner Studio
  fbq('track', "PageView");
  fbq('trackCustom', 'EntryPage', {"section_name":"Crime","tags":["derek-cruice","derek-cruice-dead","todd-raible","volusia-county-sheriff","todd-raible-volusia-county-sheriff","cop-shoots-pot-dealer-in-face","deputy-shoots-man-in-face-serving-warrant","video","police","@health_adhd","@health_depression","@health_gad","@health_hiv","@health_ht","@health_ibs","@health_ost","@health_pain","@health_erectile","@health_models"],"ncid":null,"environment":"desktop","render_type":"web"} );
  waitForGlobal(function() {
    return HP.modules.Tracky;
  }, function() {
    $('body').on('click', function(event) {
      HP.modules.Tracky.reportClick(event, function(data) {
        fbq('trackCustom', "Click", data);
      });
    });
  });
</script>
<noscript>
&lt;img height="1" src="https://www.facebook.com/tr?id=1621685564716533&amp;ev=PageView&amp;noscript=1" style="display:none" width="1"&gt;
&lt;img height="1" src="https://www.facebook.com/tr?id=10153394098876130&amp;ev=PageView&amp;noscript=1" style="display:none" width="1"&gt;
</noscript>
<!-- End Facebook Pixel Code -->

<script>
  (function() {
    RegExp.escape = function(s) {
      return s.replace(/[-\/\\^$*+?.()|[\]{}]/g, '\\$&');
    };
  
    var data = window.pubExchangeData;
    if (data) {
      var PUBX = {};
      PUBX.linksUrl = data.url;
      PUBX.publicationUuid = data.partnerList.uuid;
      PUBX.publicationPartners = {};
      for (var key in data.partnerList.partners) {
        PUBX.publicationPartners[key] = data.partnerList.partners[key];
      }
  
      var partnerRegex = Object.keys(PUBX.publicationPartners).map(function(key) {
        return RegExp.escape(key);
      }).join("|");
      var linkRegex = RegExp("^(?:https?:)?//(?:[^/]+\\.)?("+partnerRegex+")", "i");
  
      $('body').on('mousedown', 'a:not(.peskip)', function(event) {
        var link = event.currentTarget;
        if(!link.hasAttribute("onmousedown")) {
          var match = link.href.match(linkRegex);
          if (match && match.length) {
            var partner = match[1]
            link.setAttribute("data-url", link.href);
            link.href = PUBX.linksUrl + PUBX.publicationUuid + "/" + PUBX.publicationPartners[partner] + "/" + encodeURIComponent(link.href);
          }
        }
      });
    }
  })();
</script>


</div>
<script>
  HP.functions.init();
  window.addEventListener('load', modulousQueue.runAll);
</script>
<!--[if IE]>
<link rel="stylesheet" media="screen" href="http://s.m.huffpost.com/assets/ie-bddc3f1b9fb2f2d2c7c30f4b176a3829825fd1b1e93b16e5edac4c4745e150f5.css" />
<![endif]-->
</div>


<iframe id="rufous-sandbox" scrolling="no" frameborder="0" allowtransparency="true" allowfullscreen="true" style="position: absolute; visibility: hidden; display: none; width: 0px; height: 0px; padding: 0px; border: none;" title="Twitter analytics iframe"></iframe><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
  console.log("AOL BUSDEV BF - Instrumentation Tag");
  (function() {

    window.PolarAOL = window.PolarAOL || {};

    window.PolarAOL.getBusDevBackfillPromoFeedIDForAllocation = function(allocation) {
      var keys = Object.keys(allocation);
      var sum = 0;
      keys.forEach(function(key) {
        sum += allocation[key];
      });

      var random_value = Math.random(0, sum)*sum;
      // console.log("random value: ", random_value);
      var feed = "";


      var running_sum = 0;
      for (var key in allocation) {
        // normalize weight to be between 0 and 1
        if (random_value < allocation[key] + running_sum) {
          feed = key;
          break;
        } else {
          running_sum += allocation[key];
        }

      }

      return feed;

    }
    
    
    window.PolarAOL.getBusDevBackfillPromoFeedIDForClientAllocation = function(feed) {
//          "promofeed_8a3082c90e1d466c93e41c1060e25b09": 11767, // Target           
//          "promofeed_1f858ae05c5246d8bdd614929f40d043": 70000, // RHM_Media_Mental Health_Desktop from HuffPo account
//          "promofeed_81acbcf847fa431f848d573a3ce2e2f7": 23333, // RHM_Custom_Xeljanz_RA_Mobile from huffpo account      
      
      var feed_allocation = {};
      if (feed === "conde_feed") {
        feed_allocation = {
          "promofeed_9df74ea741794b899d0473ff02adbafb": 34000, // Ebay
          "promofeed_af0f54390799462b98bcd7b2b110af90": 47800 // Alfa Romeo
        }
      }
      else if (feed === "remedy_desktop_feed") {
        feed_allocation = {
          "promofeed_cbc76b7a9fe64a848ab19f3462b6087c": 6000, // RHM_Custom_Cosentyx_Psoriasis_Desktop
          "promofeed_7fc926cda12a40e282a6624438fc514e": 8000, // RHM_Custom_Otezla_Psoriasis_Desktop
          "promofeed_f047f8bb1b064f7b863534c7a9469f70": 17000, // RHM_Custom_Xolair_Skincare_Chronic Hives_Desktop
          "promofeed_6ba4734fb81a42f2ae3943640feff353": 3000, // RHM_Custom_Humira Derm_Psoriasis_Desktop
          "promofeed_4a307914529340dba863343b33e3e41c": 2000, // RHM_Custom_Humira_RA_Desktop
          "promofeed_6cacbee0018f483a8547af16f96b8708": 11667, // RHM_Custom_Xeljanz_RA_Desktop
          "promofeed_8573b2699b88492abea378722631489e": 3000, // RHM_TheBody_Starting Treatment RC_Desktop ID: 8573b2699b88492abea378722631489e
          "promofeed_5963a6fa1eb449609ad809acd45028e5": 2000, // RHM_TheBody_Transgender RC_Desktop ID: 8573b2699b88492abea378722631489e
          "promofeed_9adde5d4cb5f4fbb9a6098cbd1bb572e": 2000, // RHM_TheBody_Aging RC_Desktop ID: 9adde5d4cb5f4fbb9a6098cbd1bb572e
          "promofeed_b7c7e7bb8a244379806f769f8aee47ab": 75000, // RHM_Media_Skincare_Desktop
          "promofeed_4642c956a28f417ea6dd9c3e3a452412": 70000, // RHM_Media_Mental Health_Desktop from Remedy acccount
          "promofeed_6925e98b128946848ab09acc269ec191": 50000, // RHM_Media_Afib_Desktop
          "promofeed_4b0eb9d5de9d43fbb25b2b480cd245fd": 30000, // RHM_Media_Chronic Pain_Desktop
          "promofeed_79d279e44f2c48f791580a46d546e262": 20000 // RHM_Media_Menopause_Desktop
        }
      }
      else if (feed === "remedy_mobile_feed") {
        feed_allocation = {
          "promofeed_29a1b5016cd343058f43185d1104dde3": 2500, // RHM_Custom_Cosentyx_Psoriasis_Mobile
          "promofeed_dfff1bfda7734faeab6e641b5f89d662": 4000, // RHM_Custom_Humira_RA_Mobile
          "promofeed_09c09a1d9bdd4c5e9097fce3d5dd0625": 23333, // RHM_Custom_Xeljanz_RA_Mobile  from remedy account
          "promofeed_26b32ea737d549e381266d2066df91f7": 2000 // RHM_Custom_Digestive_Humira_Crohn's_Mobile        
        }
      }
      else if (feed === "healthination_desktop_feed") {
        feed_allocation = {
          "promofeed_e3c4ada2cc6d4c02b53664f13bb5c2c9": 50000, // June 2017 insomnia_spotlight
          "promofeed_380e125e37c942e4ab53f7466636e0e2": 30000, // June 2017 womenshealth
          "promofeed_a92a9d8be89341029399b63365d0377b": 45000, // June 2017 diabetescontent
          "promofeed_81157ec57de046ed997c125b3e62c8c1": 12000, // June 2017 hiv_treatmentcenter
          "promofeed_19b4566f7e73476aad2d58d7fa9e2935": 30000, // June 2017 menshealth
          "promofeed_62aecba4672743c0bd78c37995e7ae74": 27000, // June 2017 psoriasiscontent
          "promofeed_96005f92d707498a852562a9a8dc8868": 25000, // June 2017 hs_spotlight
          "promofeed_d9837a206a39402db7429f608396912c": 50000, // June 2017 psaspotlight
          "promofeed_7c1bc9d2c19d40fbb9bb8ddf4be69026": 5000 // June 2017 asthmaspotlight
        }
      }
      else {
        return feed;
      }
      
      return window.PolarAOL.getBusDevBackfillPromoFeedIDForAllocation(feed_allocation);
    }

    window.PolarAOL.getBusDevBackfillPromoFeedID = function(tagName) {

      // Allocations for position
      var topleft_desktop_allocation = {
        "promofeed_6f6440bf27624cd68a7c43bf0c385cae": 0.3, // Brand Forum
        "promofeed_85011f7ea9764b58ad5394701c9e3425": 0.6, // CBS CNET
        "promofeed_d3cc846eaed044269a364a99affad768": 0.6, // CBS TV Guide Desktop        
        "promofeed_0c290c477cdd484f9469911aec78247d": 0.2, // AP HealthGrades
        "promofeed_d08c1294f5eb432cb8d8f8743e26b93b": 0.1, // BusDev Desktop (will be used as all)
        "promofeed_18017672b4744a099891780a85cffda1": 0.3, // 24/7 WallSt
        conde_feed: 0.2,
        remedy_desktop_feed: 0.3,
        healthination_desktop_feed: 0.8

      };

      var topleft_mobile_allocation = {
        "promofeed_6f6440bf27624cd68a7c43bf0c385cae": 0.2, // Brand Forum
        "promofeed_0c290c477cdd484f9469911aec78247d": 0.1, // AP HealthGrades
        "promofeed_5e91a41adc924536bbdddb86ee676f8c": 1.0, // LowerMyBills Mobile
        conde_feed: 0.1,
        remedy_mobile_feed: 0.5
      };


      var mobile_allocation = {
        "promofeed_5e91a41adc924536bbdddb86ee676f8c": 2.0, // LowerMyBills Mobile
        "promofeed_79c9fb2a4a1d4f80841f55292d89d0eb": 0.1, // NextAdvisor Mobile
        "promofeed_e3144291c588447790982f5254420544": 0.6, // Bankrate
        "promofeed_a5fff0fc2a6747a89aa471bdeeabc76e": 0.0, // BizDev backfill mobile,
        "promofeed_7dbcf43f83664737b7c783fac4daa278": 0.4, // LendingTree
        conde_feed: 0.1,
        remedy_mobile_feed: 0.3
      };

      var desktop_allocation = {
        "promofeed_71e8c0d0c98c42f28aa127ee4f93982b": 1.2, // LowerMyBills Desktop
        "promofeed_85011f7ea9764b58ad5394701c9e3425": 0.3, // CBS CNET
        "promofeed_d3cc846eaed044269a364a99affad768": 0.3, // CBS TV Guide Desktop
        "promofeed_8109c95e7d87415db0634dbf1232403f": 0.3, // NextAdvisor Desktop
        "promofeed_0c290c477cdd484f9469911aec78247d": 0.2, // AP HealthGrades
        "promofeed_82439077298e47259b791f4d4fdeb628": 0.7, // Bankrate
        "promofeed_d08c1294f5eb432cb8d8f8743e26b93b": 0.1, // BizDev backfill desktop
        "promofeed_7dbcf43f83664737b7c783fac4daa278": 0.4, // LendingTree
        "promofeed_18017672b4744a099891780a85cffda1": 0.1, // 24/7 WallSt
        conde_feed: 0.1,
        remedy_desktop_feed: 0.4,
        healthination_desktop_feed: 0.8
        
      };

      var feed = "";
      //Desktop and Mobile - Top Left
      if (tagName.toLowerCase().indexOf("top left") > -1) {

        /// desktop top left allocation
        if (tagName.toLowerCase().indexOf("desktop") > -1) {
          feed = window.PolarAOL.getBusDevBackfillPromoFeedIDForAllocation(topleft_desktop_allocation);
        }

        /// mobile top left allocation
        else {
          feed = window.PolarAOL.getBusDevBackfillPromoFeedIDForAllocation(topleft_mobile_allocation);
        }

      } else if (tagName.toLowerCase().indexOf("desktop") > -1) {
        // Desktop
        feed = window.PolarAOL.getBusDevBackfillPromoFeedIDForAllocation(desktop_allocation);
      } else {
        // Mobile
        feed = window.PolarAOL.getBusDevBackfillPromoFeedIDForAllocation(mobile_allocation);
      }
      
      feed = window.PolarAOL.getBusDevBackfillPromoFeedIDForClientAllocation(feed);
      console.log("PolarAOL.getBusDevBackfillPromoFeedID - Tag: " + tagName + " PF ID: " + feed);
      return feed;

    }




  })();
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
(function(){
  console.log("[MediaVoice]: Instrumentation Tag Run!");
  
  window.NATIVEADS = window.NATIVEADS || {};
  window.NATIVEADS_QUEUE = window.NATIVEADS_QUEUE || [];
  
  window.NATIVEADS_QUEUE.push(["setSecondaryPageURL", ""]);
  window.NATIVEADS_QUEUE.push(["configureSecondaryPage", {
    binding: {},
    onFill: function(data) { },
    onRender: function() { },
    onError: function(error) { },
    track: function() {
      if (window.tacProp && window.tacProp.tags && window.tacProp.tags.indexOf('@sponsor:') != -1) {
        return "inbound";
      }
    }
  }]);
})();

//Helper Functions for Placements  
var pad = function(value, digits) {
	var missing, padding;
	missing = digits - String(value).length;
	if (missing > 0) {
	  padding = (new Array(missing + 1)).join('0');
	} else {
	  padding = '';
	}
	return "" + padding + value;
};

var hpRegex = /_(\d+).html$/;
var convertToMobile = function(link) {
  //UPDATE: April 11th 2017 - Do not rewrite due to click-tracking issues
  //Destkop URLs will automatically 302 to mobile
  return link; 
  
  //var match = hpRegex.exec(link);
  //return match && match[1] ? ("/us/entry/" + match[1]) : link;
}


//Repeat on 3 scroll events for Mobile
var insertCount = 0;
var sub;
if (window.MW !== undefined && window.MW.events !== undefined) {
  console.log("[MediaVoice]: Subscribed to Recirc Page Load Events");
  sub = MW.events.subscribe("append_recirc_page", getPolar);
  //Repeat for First 3 Scroll Events
  function getPolar() {
     console.log("[MediaVoice]: Get Polar Function - insertCount= " + insertCount);
     if (insertCount < 3) {
        mediaconductor("exec");
     } else {
        console.log("[MediaVoice]: Get Polar Function - removing Sub");
        sub.remove();
     }
  }
}
  
//360 Logic
var load360 = function($element) {
  /*----------  Inject Dependencies  ----------*/

  // pannellum
  var pnlm_js = document.createElement('script');
  pnlm_js.src = "https://static.polarcdn.com/vendor/pannellum.js";
  document.getElementsByTagName('head')[0].appendChild(pnlm_js);

  var pnlm_css = document.createElement('link');
  pnlm_css.href = "https://static.polarcdn.com/vendor/pannellum.css";
  pnlm_css.rel = "stylesheet";
  document.getElementsByTagName('head')[0].appendChild(pnlm_css);

  /*----------  Loop until dependencies load  ----------*/

  (function threesixty() {
    // short circuit until the dependencies load
    if (typeof pannellum === "undefined") {
      setTimeout(threesixty, 1000);
      return;
    }

    // Instantiate panellum 
    var img_url = $element.find(".pnlm-img-url").text();
    var img_preview_url = $element.find(".preview-img-url").text();

    pannellum.viewer($element.find(".plr-pnlm-wrapper")[0], {
      "type": "equirectangular",
      "panorama": img_url,
      "preview": img_preview_url,
      "autoLoad": true,
      "autoRotate": true,
      "autoRotateInactivityDelay": 1000,
      "hfov": 80
    });
  })();
};
 
  

//Download native-ads plugin
(function(d, s, id) {
  var js, fjs = d.getElementsByTagName(s)[0], p = d.location.protocol;
  if (d.getElementById(id)) {return;}
  js = d.createElement(s);
  js.id = id; js.type = "text/javascript"; js.async = true;
  js.src = ((p == "https:") ? p : "http:") + "//plugin.mediavoice.com/plugin.js";
  fjs.parentNode.insertBefore(js, fjs);
})(document, "script", "nativeads-plugin");
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
console.log("USA - Setup Instrumentation Tag");
(function(){
  window.NATIVEADS = window.NATIVEADS || {};
  window.NATIVEADS_QUEUE = window.NATIVEADS_QUEUE || [];
  window.NATIVEADS_QUEUE.push(["setPropertyID", "NA-HUFFPOST-11235841"]);
})();
</script>

<script>
(function(d, s, id) {
  var js, fjs = d.getElementsByTagName(s)[0], p = d.location.protocol;
  if (d.getElementById(id)) {return;}
  js = d.createElement(s);
  js.id = id; js.type = "text/javascript"; js.async = true;
  js.src = ((p == "https:") ? p : "http:") + "//plugin.mediavoice.com/plugin.js";
  fjs.parentNode.insertBefore(js, fjs);
})(document, "script", "nativeads-plugin");
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script type="text/javascript">
  window._taboola = window._taboola || [];
  _taboola.push({flush: true});
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
  console.log("USA - Template - Top Stories Instrumentation Tag");
(function(){

  window.PolarAOL = window.PolarAOL || {};
  window.PolarAOL.templates = window.PolarAOL.templates || {};
  window.PolarAOL.css = window.PolarAOL.css || {};
    
/* HUFFINGTON POST US TOP STORIES 2 & 7 PLACEMENT TEMPLATE & CSS*/ 
/*

<div class="plr-card-2 noskim">
    <div class="plr-card__content-2"> 
        <div class="plr-card__headlines-2"> 
            <h2 class="plr-card__headline-2"> 
                <a class="plr-card__link-2" href="" target="_blank"></a>
            </h2>
            <span class="plr-card__pill-text-2"></span> 
        </div>
        <a class="plr-card__image-2 plr-card__link-2" href="" target="_blank"> 
            <div class="plr-card__image__wrapper-2"> 
                <div class="plr-card__image__cover-bg"> 
                    <div class="plr-card__image__src-2 plr-bg_found" style="background-image: url()"></div>
                </div>
            </div>
        </a>
    </div>
</div>

*/


  window.PolarAOL.templates["news"] = function (Handlebars,depth0,helpers,partials,data) {  this.compilerInfo = [4,'>= 1.0.0'];helpers = this.merge(helpers, Handlebars.helpers); data = data || {};  var buffer = "", stack1, stack2, options, functionType="function", escapeExpression=this.escapeExpression, helperMissing=helpers.helperMissing;  buffer += "<div class=\"plr-card-2 noskim\">\n    <div class=\"plr-card__content-2\"> \n        <div class=\"plr-card__headlines-2\"> \n            <h2 class=\"plr-card__headline-2\"> \n                <a class=\"plr-card__link-2\" href=\"";  if (stack1 = helpers.link) { stack1 = stack1.call(depth0, {hash:{},data:data}); }  else { stack1 = depth0.link; stack1 = typeof stack1 === functionType ? stack1.apply(depth0) : stack1; }  buffer += escapeExpression(stack1)    + "\" target=\"_blank\">";  if (stack1 = helpers.title) { stack1 = stack1.call(depth0, {hash:{},data:data}); }  else { stack1 = depth0.title; stack1 = typeof stack1 === functionType ? stack1.apply(depth0) : stack1; }  buffer += escapeExpression(stack1)    + "</a>\n            </h2>\n            <span class=\"plr-card__pill-text-2\">"    + escapeExpression(((stack1 = ((stack1 = depth0.sponsor),stack1 == null || stack1 === false ? stack1 : stack1.name)),typeof stack1 === functionType ? stack1.apply(depth0) : stack1))    + "</span> \n        </div>\n        <a class=\"plr-card__image-2 plr-card__link-2\" href=\"";  if (stack2 = helpers.link) { stack2 = stack2.call(depth0, {hash:{},data:data}); }  else { stack2 = depth0.link; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }  buffer += escapeExpression(stack2)    + "\" target=\"_blank\"> \n            <div class=\"plr-card__image__wrapper-2\"> \n                <div class=\"plr-card__image__cover-bg\"> \n                    <div class=\"plr-card__image__src-2 plr-bg_found\" style=\"background-image: url(";  options = {hash:{    'width': (601),    'height': (401)  },data:data};  buffer += escapeExpression(((stack1 = helpers.getThumbHref || depth0.getThumbHref),stack1 ? stack1.call(depth0, options) : helperMissing.call(depth0, "getThumbHref", options)))    + ")\"></div>\n                </div>\n            </div>\n        </a>\n    </div>\n</div>\n\n";  return buffer;  };
  window.PolarAOL.css["news"] = ["",
                                        ".plr-card-2{",
                                        "    display: block;",
                                        "    padding: 0 15px;",
                                        "    width: 100%;",
                                        "}",
                                        "",
                                        ".plr-card__content-2{",
                                        "    padding-bottom: 25px;",
                                        "    margin-bottom: 25px;",
                                        "    border-bottom: 1px solid #dbdbdb;",
                                        "    min-height: 120px;",
                                        "    max-width: 800px;",
                                        "    display: flex;",
                                        "    -webkit-box-orient: horizontal;",
                                        "    -webkit-box-direction: normal;",
                                        "    flex-direction: row;",
                                        "    flex-wrap: wrap;",
                                        "}",
                                        "",
                                        ".plr-card__headlines-2{",
                                        "    display: flex;",
                                        "    flex-direction: column;",
                                        "    order: 2;",
                                        "    flex: 1 0;",
                                        "    max-width: 100%;",
                                        "}",
                                        "",
                                        ".plr-card__label__text-2{ text-transform: uppercase; letter-spacing: 1.4px; }",
                                        "",
                                        ".plr-card__headline-2{",
                                        "    font-family: \"ProximaNovaCond-Bold\";",
                                        "    font-size: 1.75rem;",
                                        "    line-height: 1.875rem;",
                                        "    letter-spacing: 0.01em;",
                                        "    display: flex;",
                                        "    flex-flow: row wrap;",
                                        "    color: #000000;",
                                        "    padding-right: 5px;",
                                        "}",
                                        "",
                                        ".plr-card__link-2{",
                                        "    color: inherit;",
                                        "    text-decoration: none;",
                                        "}",
                                        "",
                                        ".plr-card__link-2:hover{",
                                        "    text-decoration: underline;",
                                        "}",
                                        ".plr-card__pill-text-2{",
                                        "    font-family: \"NotoNashkArabic\", \"ProximaNova\", \"Helvetica Neue\", \"Helvetica\", Roboto, Arial, sans-serif;",
                                        "    font-size: 0.8125rem;",
                                        "    margin-top: 8px;",
                                        "    display: block;",
                                        "    color: #f83371;",
                                        "}",
                                        "",
                                        ".plr-card__image-2{",
                                        "    order: 1;",
                                        "    flex: 0 0 212px;",
                                        "    margin-right: 15px;",
                                        "    display: block;",
                                        "    position: relative;",
                                        "}",
                                        "",
                                        ".plr-card__image__wrapper-2{",
                                        "    line-height: 0;",
                                        "    position: relative;",
                                        "    overflow: hidden;",
                                        "}",
                                        "",
                                        "",
                                        ".plr-card__image__cover-bg{",
                                        "    width: 100%;",
                                        "    height: 0;",
                                        "    position: relative;",
                                        "    padding-bottom: 56.26%;",
                                        "    display: block;",
                                        "}",
                                        "",
                                        ".plr-card__image__src-2{",
                                        "    width: 100%;",
                                        "    background-size: cover;",
                                        "    background-position: 50% 50%;",
                                        "    height: 100%;",
                                        "    position: absolute;",
                                        "}",
                                        "@media only screen and (max-width: 767px) {",
                                        "",
                                        ".plr-card-2{",
                                        "   max-width: 580px;",
                                        "   margin-right: auto;",
                                        "   margin-left: auto;",
                                        "}",
                                        "",
                                        ".plr-card__headline-2{",
                                        "   font-size: 1.25rem;",
                                        "   line-height: 1.375rem;",
                                        "}",
                                        ".plr-card__image-2{ flex: 0 0 80px; }",
                                        "",
                                        ".plr-card__image__cover-bg{ height: 80px; }",
                                        "",
                                        ".plr-card__content-2{",
                                        "   padding-bottom: 15px;",
                                        "   margin-bottom: 15px;",
                                        "   min-height: 10px;",
                                        "}",
                                        ".plr-card__link-2:hover{",
                                        "    text-decoration: none;",
                                        "}",
                                        "",
                                        "}",
                                        "",
                                        ""].join("\n")

})();
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
  console.log("USA - Template - In The News- Instrumentation Tag");
(function(){

  window.PolarAOL = window.PolarAOL || {};
  window.PolarAOL.templates = window.PolarAOL.templates || {};
  window.PolarAOL.css = window.PolarAOL.css || {};

/* HUFFINGTON POST IN THE NEWS PLACEMENT TEMPLATE & CSS */
/*
<div class="plr-card noskim" >
    <div class="plr-card__content">
        <a class="plr-card__image plr-card__link" href=""  target="_blank">
            <div class="plr-card__image__wrapper" style="background-image: url()">
         
            </div>
        </a>
        <div class="plr-card__headlines">
            <h2 class="plr-card__headline">
                <a class="plr-card__link" href=""  target="_blank"></a>
            </h2>
            <span class="plr-card__pill-text"></span>
        </div>
    </div>
</div>

*/

  window.PolarAOL.templates["top_left"] = function (Handlebars,depth0,helpers,partials,data) {  this.compilerInfo = [4,'>= 1.0.0'];helpers = this.merge(helpers, Handlebars.helpers); data = data || {};  var buffer = "", stack1, stack2, options, functionType="function", escapeExpression=this.escapeExpression, helperMissing=helpers.helperMissing;  buffer += "<div class=\"plr-card noskim\" >\n    <div class=\"plr-card__content\">\n        <a class=\"plr-card__image plr-card__link\" href=\"";  if (stack1 = helpers.link) { stack1 = stack1.call(depth0, {hash:{},data:data}); }  else { stack1 = depth0.link; stack1 = typeof stack1 === functionType ? stack1.apply(depth0) : stack1; }  buffer += escapeExpression(stack1)    + "\"  target=\"_blank\">\n            <div class=\"plr-card__image__wrapper\" style=\"background-image: url(";  options = {hash:{    'width': (124),    'height': (124)  },data:data};  buffer += escapeExpression(((stack1 = helpers.getThumbHref || depth0.getThumbHref),stack1 ? stack1.call(depth0, options) : helperMissing.call(depth0, "getThumbHref", options)))    + ")\">\n         \n            </div>\n        </a>\n        <div class=\"plr-card__headlines\">\n            <h2 class=\"plr-card__headline\">\n                <a class=\"plr-card__link\" href=\"";  if (stack2 = helpers.link) { stack2 = stack2.call(depth0, {hash:{},data:data}); }  else { stack2 = depth0.link; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }  buffer += escapeExpression(stack2)    + "\"  target=\"_blank\">";  if (stack2 = helpers.title) { stack2 = stack2.call(depth0, {hash:{},data:data}); }  else { stack2 = depth0.title; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }  buffer += escapeExpression(stack2)    + "</a>\n            </h2>\n            <span class=\"plr-card__pill-text\">"    + escapeExpression(((stack1 = ((stack1 = depth0.sponsor),stack1 == null || stack1 === false ? stack1 : stack1.name)),typeof stack1 === functionType ? stack1.apply(depth0) : stack1))    + "</span>\n        </div>\n    </div>\n</div>\n\n\n";  return buffer;  };

  window.PolarAOL.css["top_left"] =["",
                                      ".plr-card{",
                                      "    display: block;",
                                      "    margin-bottom: 20px;",
                                      "    width: 100%;",
                                      "    padding: 0 15px;",
                                      "}",
                                      "",
                                      ".plr-card__content{",
                                      "    padding-bottom: 20px;",
                                      "}",
                                      "",
                                      ".plr-card__headlines{",
                                      "    max-width: 100%;",
                                      "    min-height: 64px;",
                                      "    overflow: hidden;",
                                      "}",
                                      "",
                                      ".plr-card__headline{",
                                      "    margin-top: 0;",
                                      "    letter-spacing: 0.25px;",
                                      "    font-family: \"ProximaNovaCond-Medium\", \"Helvetica Neue\", \"Helvetica\", Roboto, Arial, sans-serif;",
                                      "    font-size: 1.125rem;",
                                      "    line-height: 1.25rem;",
                                      "}",
                                      "",
                                      ".plr-card__link{",
                                      "    color: inherit;",
                                      "    text-decoration: none;",
                                      "}",
                                      "",
                                      ".plr-card__link:hover{",
                                      "    text-decoration: underline;",
                                      "}",
                                      "",
                                      ".plr-card__image{",
                                      "    width: 64px;",
                                      "    max-width: 64px;",
                                      "    float: left;",
                                      "    margin-right: 15px;",
                                      "}",
                                      "",
                                      ".plr-card__image__wrapper{",
                                      "    line-height: 0;",
                                      "    position: relative;",
                                      "    overflow: hidden;",
                                      "    background-size: cover;",
                                      "    background-position: center center;",
                                      "    height: 64px;",
                                      "}",
                                      "",
                                      ".plr-card__pill-text{",
                                      "    font-size: 0.75rem;",
                                      "    line-height: 1.125rem;",
                                      "    letter-spacing: -0.01em;",
                                      "    color: #f83371;",
                                      "}",
                                      "@media only screen and (max-width: 767px) {",
                                      "",
                                      ".plr-card{",
                                      "   max-width: 580px;",
                                      "   margin-right: auto;",
                                      "   margin-left: auto;",
                                      "}",
                                      "",
                                      ".plr-card__headline{",
                                      "   font-size: 1.25rem;",
                                      "   line-height: 1.375rem;",
                                      "   margin-bottom: 2px;",
                                      "}",
                                      ".plr-card__pill-text{",
                                      "   font-size: 0.8125rem;",
                                      "   line-height: 1rem;",
                                      "}",
                                      "",
                                      ".plr-card__link:hover{",
                                      "    text-decoration: none;",
                                      "}",
                                      ".plr-card__image{",
                                      "    width: 80px;",
                                      "    max-width: 80px;",
                                      "    float: left;",
                                      "    margin-right: 15px;",
                                      "}",
                                      "",
                                      ".plr-card__image__wrapper{ height: 80px; width:80px; }",
                                      "",
                                      "}",
                                      ""].join("\n")

  


})();
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
 console.log("AOL CORE - Instrumentation Tag");
(function(){
  
  window.PolarAOL = window.PolarAOL || {};
  window.PolarAOL.templates = window.PolarAOL.templates || {};
  window.PolarAOL.css = window.PolarAOL.css || {};
  
  window.PolarAOL.inject = function(name, config) {
  
    var nonce = config.nonce;
    var unit = config.unit;
    var siteName = config.siteName;
    var placementName = config.placementName;
    var tagName = config.tagName;
    

    
    var template = window.PolarAOL.templates[name];
    var css = window.PolarAOL.css[name];
    
    var originCountry = "US";
    if (config.originCountry) {
    	originCountry = config.originCountry;
    }
  
    //For International tags on US Edition site
    if (siteName.indexOf('USA') > -1) { //Site name is 'HuffPost - USA'
      switch(originCountry) {
          case "CA":
              window.NATIVEADS_QUEUE.push(["setPropertyID", "NA-HUFFPOSTUSEDIT-11238860"]);
              break;
          case "UK":
              window.NATIVEADS_QUEUE.push(["setPropertyID", "NA-HUFFPOSTUSEDIT-11238859"]);
              break;
          case "BR":
              window.NATIVEADS_QUEUE.push(["setPropertyID", "NA-HUFFPOSTBRAZ-11237943"]);
              break;
          case "SA":
              window.NATIVEADS_QUEUE.push(["setPropertyID", "NA-HUFFPOSTSOUTAFRI-11238692"]);
              break;
          case "AU":
              window.NATIVEADS_QUEUE.push(["setPropertyID", "NA-HUFFPOSTAUSTUSEDITTRAF-11238906"]);
              break;
          default:
              break;
      }
    }

    if ((typeof template === "undefined" || template === null) ||
        (typeof css      === "undefined" || css      === null)) {
      
      // Looking up the Handlebars template or associated CSS failed.
 	  if (typeof console !== "undefined" && console !== null) {
  	    if (typeof console.log === "function") {
		  console.log("PolarAOL: Skipping injection: unknown name: ", name);
	    }
      }
      return;

    }      
                
    window.NATIVEADS_QUEUE.push(["insertPreview", {
      label: placementName + " [" + tagName + "]",
      unit: unit,
      location: "#" + nonce,
      onRender: function($element, data) {
        polartag("partner:fill", true, nonce);
      },
      onFill: function(data) {
        if (data.sponsor && originCountry == "US" && data.custom.content_type != "Storybook") {
          //Do Not add Sponsor label if Brand Forum creative
          if (data.custom.content_type  != "Brand Forum") {    
              
      		var sponsor = data.sponsor.name.toLowerCase();
        	if(sponsor.indexOf('sponsor') > -1 || sponsor.indexOf('present') > -1) {
        		//Do Nothing
       		 } else {
           		data.sponsor.name = "Sponsored by " + data.sponsor.name;
        	}
            
          }
        }
        
        //Lightbox CSS for Storybook format
        if (data.custom.content_type === "Storybook") {

           window.NATIVEADS_QUEUE.push(["injectCSS", ["",
              ".polar-deck-content {",
              "    height: 100%",
              "}",
              "@media screen and (min-width: 768px) {",
              ".polar-deck-dialog {",
                  "height: 700px;",
                  "width: 450px;",
              "}",
              "",
              ".polar-deck-share-btns {",
              		"display: none;",
              "}",
              "",
    		].join("\n"), "head"]);
        }
      },
      onError: function(error) {
        console.log("OnError:  " + placementName + " / " + tagName);                           
        polartag("partner:fill", false, nonce);
      },
      template: template,
    }]);

    window.NATIVEADS_QUEUE.push(["injectCSS", css, "head"]);

  }
  
})();
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
  console.log("USA - Template - In The News- Instrumentation Tag");
(function(){

  window.PolarAOL = window.PolarAOL || {};
  window.PolarAOL.templates = window.PolarAOL.templates || {};
  window.PolarAOL.css = window.PolarAOL.css || {};

/* HUFFINGTON POST RIGHT RAIL UNIT BESIDES POLITICS(subject to change) */
/*
<div class="plr-elevator noskim">
  <div class="plr-ad">
     <a href="" target="_blank">
       <div class="plr-rr-sponsor">
          
       </div>
       <div class="plr-image-wrapper" style="background: url('') no-repeat center center/cover">
       </div> 
        <div class="plr-content">
            <div class= "plr-title-rr">
               
            </div>
       </div>
    </a>
  </div>
</div>

*/

  window.PolarAOL.templates["right_rail"] = function (Handlebars,depth0,helpers,partials,data) {  this.compilerInfo = [4,'>= 1.0.0'];helpers = this.merge(helpers, Handlebars.helpers); data = data || {};  var buffer = "", stack1, stack2, options, functionType="function", escapeExpression=this.escapeExpression, helperMissing=helpers.helperMissing;  buffer += "<div class=\"plr-elevator noskim\">\n  <div class=\"plr-ad\">\n     <a href=\"";  if (stack1 = helpers.link) { stack1 = stack1.call(depth0, {hash:{},data:data}); }  else { stack1 = depth0.link; stack1 = typeof stack1 === functionType ? stack1.apply(depth0) : stack1; }  buffer += escapeExpression(stack1)    + "\" target=\"_blank\">\n       <div class=\"plr-rr-sponsor\">\n          "    + escapeExpression(((stack1 = ((stack1 = depth0.sponsor),stack1 == null || stack1 === false ? stack1 : stack1.name)),typeof stack1 === functionType ? stack1.apply(depth0) : stack1))    + "\n       </div>\n       <div class=\"plr-image-wrapper\" style=\"background: url('";  options = {hash:{    'width': (300)  },data:data};  buffer += escapeExpression(((stack1 = helpers.getThumbHref || depth0.getThumbHref),stack1 ? stack1.call(depth0, options) : helperMissing.call(depth0, "getThumbHref", options)))    + "') no-repeat center center/cover\">\n       </div> \n        <div class=\"plr-content\">\n            <div class= \"plr-title-rr\">\n               ";  if (stack2 = helpers.title) { stack2 = stack2.call(depth0, {hash:{},data:data}); }  else { stack2 = depth0.title; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }  buffer += escapeExpression(stack2)    + "\n            </div>\n       </div>\n    </a>\n  </div>\n</div>\n\n";  return buffer;  };
 
  window.PolarAOL.css["right_rail"] =["",
                                      ".plr-elevator{ display: none; }",
                                      "",
                                      ".plr-ad {",
                                      "  width: 300px;",
                                      "  max-width: 100%;",
                                      "  height: auto;",
                                      "  position: relative;",
                                      "  margin: 0 auto;",
                                      "  margin-bottom: 10px;",
                                      "}",
                                      "",
                                      ".plr-ad a {",
                                      "  text-decoration: none;",
                                      "}",
                                      "",
                                      ".plr-image-wrapper {",
                                      "  width: 100%;",
                                      "  height: 200px;",
                                      "}",
                                      "",
                                      ".plr-content {",
                                      "  position: relative;",
                                      "  width: 100%;",
                                      "  height: auto;",
                                      "  background-color: white;",
                                      "  min-height: 10%;",
                                      "}",
                                      "",
                                      ".plr-title-rr {",
                                      "  color: #333333;",
                                      "  font-family: \"ProximaNovaCond-Medium\", \"Helvetica Neue\", \"Helvetica\", Roboto, Arial, sans-serif;",
                                      "  margin-top: 10px;",
                                      "  font-size: 20px;",
                                      "  line-height: 24px;",
                                      "  letter-spacing: 0.25px;",
                                      "}",
                                      "",
                                      ".plr-rr-sponsor{",
                                      "    font-family: \"ProximaNovaCond-Bold\";",
                                      "    text-transform: uppercase;",
                                      "    font-size: 14px;",
                                      "    margin-bottom: 8px;",
                                      "    color: #444444;",
                                      "    letter-spacing: 0.25px;",
                                      "}",
                                      "",
                                      "@media screen and (min-width: 1024px) {",
                                      "    .plr-elevator {",
                                      "    display: block;",
                                      "    position: absolute;",
                                      "    right: -330px;",
                                      "    top: 0;",
                                      "    width: 330px;",
                                      "    }",
                                      "}",
                                      "",
                                      ""].join("\n")


})();
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><div class="mediaconductor-itag-begin polar-itag-begin" style="display: none;"> </div> <script>
  console.log("USA - UK BACKFILL - TRENDING");
(function(){

  window.PolarAOL = window.PolarAOL || {};
  window.PolarAOL.templates = window.PolarAOL.templates || {};
  window.PolarAOL.css = window.PolarAOL.css || {};

/* HUFFINGTON POST TRENDING LAST UNIT on ARTICLE PAGE */
/*

<div class="plr-card-t noskim" >
    <div class="plr-card__content-t">
        <a class="plr-card__image-t plr-card__link-t" href=""  target="_blank">
            <div class="plr-card__image__wrapper-t" style="background-image: url()">
         
            </div>
        </a>
        <div class="plr-card__headlines-t">
            <h2 class="plr-card__headline-t">
                <a class="plr-card__link-t" href=""  target="_blank"></a>
            </h2>
            <span class="plr-card__pill-text-t"></span>
        </div>
    </div>
</div>

*/

  window.PolarAOL.templates["trending"] = function (Handlebars,depth0,helpers,partials,data) {  this.compilerInfo = [4,'>= 1.0.0'];helpers = this.merge(helpers, Handlebars.helpers); data = data || {};  var buffer = "", stack1, stack2, options, functionType="function", escapeExpression=this.escapeExpression, helperMissing=helpers.helperMissing;  buffer += "<div class=\"plr-card-t noskim\" >\n    <div class=\"plr-card__content-t\">\n        <a class=\"plr-card__image-t plr-card__link-t\" href=\"";  if (stack1 = helpers.link) { stack1 = stack1.call(depth0, {hash:{},data:data}); }  else { stack1 = depth0.link; stack1 = typeof stack1 === functionType ? stack1.apply(depth0) : stack1; }  buffer += escapeExpression(stack1)    + "\"  target=\"_blank\">\n            <div class=\"plr-card__image__wrapper-t\" style=\"background-image: url(";  options = {hash:{    'width': (124),    'height': (124)  },data:data};  buffer += escapeExpression(((stack1 = helpers.getThumbHref || depth0.getThumbHref),stack1 ? stack1.call(depth0, options) : helperMissing.call(depth0, "getThumbHref", options)))    + ")\">\n         \n            </div>\n        </a>\n        <div class=\"plr-card__headlines-t\">\n            <h2 class=\"plr-card__headline-t\">\n                <a class=\"plr-card__link-t\" href=\"";  if (stack2 = helpers.link) { stack2 = stack2.call(depth0, {hash:{},data:data}); }  else { stack2 = depth0.link; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }  buffer += escapeExpression(stack2)    + "\"  target=\"_blank\">";  if (stack2 = helpers.title) { stack2 = stack2.call(depth0, {hash:{},data:data}); }  else { stack2 = depth0.title; stack2 = typeof stack2 === functionType ? stack2.apply(depth0) : stack2; }  buffer += escapeExpression(stack2)    + "</a>\n            </h2>\n            <span class=\"plr-card__pill-text-t\">"    + escapeExpression(((stack1 = ((stack1 = depth0.sponsor),stack1 == null || stack1 === false ? stack1 : stack1.name)),typeof stack1 === functionType ? stack1.apply(depth0) : stack1))    + "</span>\n        </div>\n    </div>\n</div>\n\n";  return buffer;  };
 
  window.PolarAOL.css["trending"] =["",
                                    ".plr-card-t{",
                                    "    display: block;",
                                    "    margin-bottom: 20px;",
                                    "    width: 100%;",
                                    "}",
                                    "",
                                    ".plr-card__content-t{",
                                    "    padding-bottom: 20px;",
                                    "}",
                                    "",
                                    ".plr-card__headlines-t{",
                                    "    max-width: 100%;",
                                    "    min-height: 64px;",
                                    "    overflow: hidden;",
                                    "}",
                                    "",
                                    ".plr-card__headline-t{",
                                    "    margin-top: 0;",
                                    "    letter-spacing: 0.25px;",
                                    "    font-family: \"ProximaNovaCond-Medium\", \"Helvetica Neue\", \"Helvetica\", Roboto, Arial, sans-serif;",
                                    "    font-size: 1.125rem;",
                                    "    line-height: 1.25rem;",
                                    "}",
                                    "",
                                    ".plr-card__link-t{",
                                    "    color: inherit;",
                                    "    text-decoration: none;",
                                    "}",
                                    "",
                                    ".plr-card__link-t:hover{",
                                    "    text-decoration: underline;",
                                    "}",
                                    "",
                                    ".plr-card__image-t{",
                                    "    width: 64px;",
                                    "    max-width: 64px;",
                                    "    float: left;",
                                    "    margin-right: 15px;",
                                    "}",
                                    "",
                                    ".plr-card__image__wrapper-t{",
                                    "    line-height: 0;",
                                    "    position: relative;",
                                    "    overflow: hidden;",
                                    "    background-size: cover;",
                                    "    background-position: center center;",
                                    "    height: 64px;",
                                    "}",
                                    "",
                                    ".plr-card__pill-text-t{",
                                    "    font-size: 0.75rem;",
                                    "    line-height: 1.125rem;",
                                    "    letter-spacing: -0.01em;",
                                    "    color: #f83371;",
                                    "}",
                                    ""].join("\n")


})();
</script><div class="mediaconductor-itag-end polar-itag-end" style="display: none;"></div><iframe sandbox="allow-scripts allow-same-origin" title="Adobe ID Syncing iFrame" id="destination_publishing_iframe_aol_undefined" src="http://fast.aol.demdex.net/dest5.html?d_nsid=undefined#http%3A%2F%2Fwww.huffingtonpost.com%2F2015%2F03%2F05%2Fderek-cruice-shot-in-face_n_6805902.html" class="aamIframeLoaded" style="display: none; width: 0px; height: 0px;"></iframe><div id="fb-root" class=" fb_reset"><div style="position: absolute; top: -10000px; height: 0px; width: 0px;"><div><iframe name="fb_xdm_frame_http" frameborder="0" allowtransparency="true" allowfullscreen="true" scrolling="no" id="fb_xdm_frame_http" aria-hidden="true" title="Facebook Cross Domain Communication Frame" tabindex="-1" src="http://staticxx.facebook.com/connect/xd_arbiter/r/XBwzv5Yrm_1.js?version=42#channel=f1a7ccb946473ac&origin=http%3A%2F%2Fwww.huffingtonpost.com" style="border: none;"></iframe><iframe name="fb_xdm_frame_https" frameborder="0" allowtransparency="true" allowfullscreen="true" scrolling="no" id="fb_xdm_frame_https" aria-hidden="true" title="Facebook Cross Domain Communication Frame" tabindex="-1" src="https://staticxx.facebook.com/connect/xd_arbiter/r/XBwzv5Yrm_1.js?version=42#channel=f1a7ccb946473ac&origin=http%3A%2F%2Fwww.huffingtonpost.com" style="border: none;"></iframe></div></div><div style="position: absolute; top: -10000px; height: 0px; width: 0px;"><div></div></div></div><script id="PIN_1498785571564.f.callback[0]" type="text/javascript" charset="utf-8" src="https://log.pinterest.com/?guid=_A2PCmxz6_S4&tv=2017042001&type=pidget&sub=uk&button_count=0&follow_count=0&pin_count=0&profile_count=0&board_count=0&lang=en&via=http%3A%2F%2Fwww.huffingtonpost.com%2F2015%2F03%2F05%2Fderek-cruice-shot-in-face_n_6805902.html&callback=PIN_1498785571564.f.callback[0]"></script><span><iframe class="trc-hidden" id="trc-pixel-iframe-3235" name="trc-pixel-iframe-3235" width="0" height="0" style="display:none"></iframe></span><div id="tbl-aug-ipsash" class="trc_popover_aug_container"><div id="tbl-aug-o1omwa" class="trc_popover_aug_container"><div id="tbl-aug-63lmh2" class="trc_popover_aug_container"><div class=" trc_popover trc_popover_fade trc_bottom "><div class=" trc_popover_arrow "></div><iframe frameborder="0" scrolling="no" src="javascript:void(0)" style="width: 100%;"></iframe></div></div></div></div><form class="trc-hidden trc-transport-form-element" target="tb-trc-transportFrame-4323" method="post" action="http://trc.taboola.com/aol-huffingtonpost/log/3/available" style="display: none;"><input name="ri" type="hidden" value="15279135fd5768d9291487aa58d6b84b"><input name="sd" type="hidden" value="v2_516e1885ff39641d955b7dada665b013_821f5062-de8f-4a5d-a2fa-dfbe0e831f8e_1498785590_1498785590_CNawjgYQisE_GKS81LTPKyABKAM"><input name="ui" type="hidden" value="821f5062-de8f-4a5d-a2fa-dfbe0e831f8e"><input name="pi" type="hidden" value="/2015/03/05/derek-cruice-shot-in-face_n_6805902.html"><input name="wi" type="hidden" value="4433759080567544826"><input name="pt" type="hidden" value="text"><input name="vi" type="hidden" value="1498785586724"><input name="li" type="hidden" value="rbox-t2m"><input name="utm" type="hidden" value="3,15026,15062,18680"><input name="df" type="hidden" value="1"><input name="ppb" type="hidden" value="CJAH"><input name="cpb" type="hidden" value="Eg8yNjAtMTg3LVJFTEVBU0UYgA0gnP__________ASoZYW0udGFib29sYXN5bmRpY2F0aW9uLmNvbTIId2F0ZXI0NTU"><input name="tim" type="hidden" value="02:19:50.466"><input name="id" type="hidden" value="4323"><input name="llvl" type="hidden" value="1"><input name="cv" type="hidden" value="260-187-RELEASE"><input name="fil" type="hidden" value="[{&quot;tii&quot;:&quot;~~V1~~-3855892779090245063~~gzb9aGhgIDA3l5GmWEus-8KkJbiEquim0vePrRQTYNWLWDEoSQwuhc4ZvLOUkuMCftZYEZdxVIARG00h6_pFjzHLaeWSs14Edab5ZbVOdayOGk-YHaGh0F8ZOTIP3n7goR9_aMALbfUvo7eNuAaR8g&quot;,&quot;tipt&quot;:&quot;SP&quot;,&quot;tit&quot;:&quot;photo&quot;,&quot;tids&quot;:&quot;a&quot;},{&quot;tii&quot;:&quot;~~V1~~4048768638394217214~~hkx9Ui5VaCAMGi1d0NRXE75JXqzlx-HayPNr0iUxi7MSO_fFkG-bpaW2-qyqBDD6hbvTJ9f5uoHZ9n0f0Rxsj9Z3uhcZH3NvLIMM0XLNNnpULF2ZqOy-MWoLYDSbniS78kPHpmKzIb6f9PDGye4U9Q&quot;,&quot;tipt&quot;:&quot;SP&quot;,&quot;tit&quot;:&quot;photo&quot;,&quot;tids&quot;:&quot;a&quot;},{&quot;tii&quot;:&quot;~~V1~~3272264891947756139~~xpg5r0G6o29pJAbh9aA-afO9gIl_ly0gxu0Ud1XMNYASO_fFkG-bpaW2-qyqBDD6boj4nsjC6tw9ul26EUANANZ3uhcZH3NvLIMM0XLNNnofk0yu0wfvbKEwkxpNYM1AYFTpzOxp7jLKs_zraWu8Hg&quot;,&quot;tipt&quot;:&quot;SP&quot;,&quot;tit&quot;:&quot;text&quot;,&quot;tids&quot;:&quot;a&quot;},{&quot;tii&quot;:&quot;~~V1~~-7527002819135068892~~BVr-GE0AjYrTn-s26BYuVbQ1Nv3LsHyVDTVYtlw2MeYSO_fFkG-bpaW2-qyqBDD6zU5Xh5Yvqkieis1PbDO3DtZ3uhcZH3NvLIMM0XLNNnof2GMNlgL-5SUL08r2w9648kPHpmKzIb6f9PDGye4U9Q&quot;,&quot;tipt&quot;:&quot;SP&quot;,&quot;tit&quot;:&quot;video&quot;,&quot;tids&quot;:&quot;a&quot;},{&quot;tii&quot;:&quot;~~V1~~-5946300570125137099~~EeuF_EpbdbysgjL8a8oIRnGQ8r7wkoWe0iuV1BIEwtESO_fFkG-bpaW2-qyqBDD699Qc-gAVgcBVSl_mVDmD5tZ3uhcZH3NvLIMM0XLNNnrZKF83No0MlKdnOn7S8GqNYFTpzOxp7jLKs_zraWu8Hg&quot;,&quot;tipt&quot;:&quot;SP&quot;,&quot;tit&quot;:&quot;text&quot;,&quot;tids&quot;:&quot;a&quot;},{&quot;tii&quot;:&quot;~~V1~~-3361922757362746208~~9OUD6FhUvxlEkTgqaFzUqmAhdCjzvnL4PhEsKziBUTwSO_fFkG-bpaW2-qyqBDD6ElAJaEr7rU9vx-bTiwZGodZ3uhcZH3NvLIMM0XLNNnoiTgRPaTSavZq6snjrOR1A1mTO636rCKHciJUY36Chsg&quot;,&quot;tipt&quot;:&quot;SP&quot;,&quot;tit&quot;:&quot;photo&quot;,&quot;tids&quot;:&quot;a&quot;}]"></form><span class="trc-transport-form-element"><iframe class="trc-hidden" id="tb-trc-transportFrame-4323" name="tb-trc-transportFrame-4323" width="0" height="0" style="display:none"></iframe></span><iframe id="localStorage" itemscope="" itemtype="https://schema.org/WPAdBlock" src="//cdn.at.atwola.com/_media/uac/guid.html" style="display: none; width: 0px; height: 0px;"></iframe><iframe itemscope="" itemtype="https://schema.org/WPAdBlock" id="adTacFr" src="//cdn.at.atwola.com/_media/uac/tcode3.html#&tags=kvpagetype=atf:bpage:news:news:slideshow;kvtopic=taser;kventryid=6805902;kvcmsid=hpo:6805902;kveditags=@health_ht:@health_gad:police:@health_pain:todd-raible-volusia-county-sheriff:deputy-shoots-man-in-face-serving-warrant:@health_depression:@health_adhd:@health_models:todd-raible:@health_hiv:@health_erectile:@health_ibs:volusia-county-sheriff:video:derek-cruice:cop-shoots-pot-dealer-in-face:@health_ost:derek-cruice-dead;kvvert=crime;kvblogger=andy-campbell;kvrefd=http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html&kvpagetype=atf%3Abpage%3Anews%3Anews%3Aslideshow&kvtopic=taser&kventryid=6805902&kvcmsid=hpo%3A6805902&kveditags=%40health_ht%3A%40health_gad%3Apolice%3A%40health_pain%3Atodd-raible-volusia-county-sheriff%3Adeputy-shoots-man-in-face-serving-warrant%3A%40health_depression%3A%40health_adhd%3A%40health_models%3Atodd-raible%3A%40health_hiv%3A%40health_erectile%3A%40health_ibs%3Avolusia-county-sheriff%3Avideo%3Aderek-cruice%3Acop-shoots-pot-dealer-in-face%3A%40health_ost%3Aderek-cruice-dead&kvvert=crime&kvblogger=andy-campbell&kvrefd=http%3A%2F%2Fwww.huffingtonpost.com%2F2015%2F03%2F05%2Fderek-cruice-shot-in-face_n_6805902.html&" style="display: none; width: 0px; height: 0px;"></iframe><script src="http://ar.voicefive.com/bmx3/../b/node_rcAll.pli?func=COMSCORE.BMX.Buddy.run&1498785614758"></script><script src="http://ar.voicefive.com/bmx3/builder.pli?pid=p289261914&InvConfigNum=aol"></script></body></html>""")

In [ ]:
data_with.loc[20,'source_2'] = missing

Afetr creating the sql table (let's say it's name is lol) we can import data from a cav file with:
- COPY lol FROM '/path/to/csv/LOL.txt' DELIMITER ',' CSV
- or \COPY
- and we can specify the columns to read lol(col1,col3)

In [ ]:
def for_profiler(): # use just code inside the function when not using the profiler
    for source_col in ['source_0','source_1','source_2','source_3','source_4']:
        # this is just on 5 first entries, as a test.
        df2015_with_info.loc[:5,source_col] = df2015_with_info.loc[:5,source_col].apply(scraper)
    return

%lprun -f scraper for_profiler()